In [ ]:
# ==============================
# SYSTEM CHECK & DEPENDENCIES
# ==============================

import sys
import subprocess
import platform
import os

def run(cmd, check=True):
    """Run a command and optionally check for errors."""
    try:
        subprocess.check_call([sys.executable, "-m"] + cmd,
                             stdout=subprocess.DEVNULL if not check else None,
                             stderr=subprocess.DEVNULL if not check else None)
        return True
    except subprocess.CalledProcessError as e:
        if check:
            print(f"⚠️  Warning: Command failed: {' '.join(cmd)}")
        return False

def check_gpu_available():
    """Check if GPU is available in the system."""
    try:
        # Check for NVIDIA GPU via nvidia-smi
        result = subprocess.run(['nvidia-smi'],
                              capture_output=True,
                              text=True,
                              timeout=5)
        if result.returncode == 0:
            return True
    except (FileNotFoundError, subprocess.TimeoutExpired):
        pass
    return False

# ===== System Information =====
print("=" * 50)
print("SYSTEM CHECK")
print("=" * 50)
print(f"OS: {platform.system()} {platform.release()}")
print(f"Python: {platform.python_version()}")
print(f"Architecture: {platform.machine()}")

# Check for Colab
try:
    import google.colab
    print("Environment: Google Colab")
    IS_COLAB = True
except ImportError:
    print("Environment: Local / Other")
    IS_COLAB = False

# Check GPU availability
gpu_available = check_gpu_available()
if gpu_available:
    print("GPU: NVIDIA GPU detected (nvidia-smi)")
else:
    print("GPU: No GPU detected or nvidia-smi not available")

print()

# ===== PyTorch Check & Installation =====
print("=" * 50)
print("PYTORCH CHECK")
print("=" * 50)

try:
    import torch
    print("✅ PyTorch already installed")
    print(f"   Version: {torch.__version__}")

    # Check CUDA availability
    cuda_available = torch.cuda.is_available()
    if cuda_available:
        print(f"✅ CUDA available")
        print(f"   CUDA Version: {torch.version.cuda}")
        print(f"   cuDNN Version: {torch.backends.cudnn.version()}")
        print(f"   GPU Device: {torch.cuda.get_device_name(0)}")
        print(f"   GPU Count: {torch.cuda.device_count()}")

        # Memory info
        if torch.cuda.is_available():
            for i in range(torch.cuda.device_count()):
                props = torch.cuda.get_device_properties(i)
                total_mem = props.total_memory / (1024**3)  # GB
                print(f"   GPU {i} Memory: {total_mem:.1f} GB")
    else:
        print("⚠️  CUDA NOT available")
        if gpu_available:
            print("   → GPU detected but PyTorch CPU version installed")
            print("   → Consider installing PyTorch with CUDA support")
        else:
            print("   → Running on CPU (expected if no GPU)")

except ImportError:
    print("❌ PyTorch NOT installed")
    print("Installing PyTorch...")

    # Determine which version to install
    if IS_COLAB:
        # Colab usually has GPU, install CUDA version
        print("   Detected Colab environment")
        if gpu_available:
            print("   Installing PyTorch with CUDA support...")
            # Use pip index URL for CUDA 11.8 or 12.1 (Colab default)
            run(["pip", "install", "torch", "torchvision", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cu118"])
        else:
            print("   Installing PyTorch (CPU version)...")
            run(["pip", "install", "torch", "torchvision", "torchaudio"])
    else:
        # Local installation - check OS
        if platform.system() == "Windows":
            print("   Installing PyTorch (CPU version for Windows)...")
            run(["pip", "install", "torch", "torchvision", "torchaudio"])
        else:
            # Linux/Mac - try to detect GPU
            if gpu_available:
                print("   GPU detected - installing PyTorch with CUDA support...")
                print("   (You may need to specify CUDA version manually)")
                run(["pip", "install", "torch", "torchvision", "torchaudio"])
            else:
                print("   Installing PyTorch (CPU version)...")
                run(["pip", "install", "torch", "torchvision", "torchaudio"])

    print("✅ PyTorch installation completed")

    # Re-import and verify
    import torch
    print(f"   Installed version: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"   ✅ CUDA is now available!")
    else:
        print(f"   ⚠️  CUDA not available (CPU mode)")

# ===== Final Verification =====
print()
print("=" * 50)
print("FINAL VERIFICATION")
print("=" * 50)

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ Ready for GPU acceleration")
    DEVICE_TYPE = "cuda"
else:
    print(f"ℹ️  Running in CPU mode")
    DEVICE_TYPE = "cpu"

# Store device type for later use (will be overridden in Cell 2, but useful here)
print()
print("=" * 50)
print("✅ System check complete!")
print("=" * 50)

SYSTEM CHECK
OS: Linux 6.6.105+
Python: 3.12.12
Architecture: x86_64
Environment: Google Colab
GPU: NVIDIA GPU detected (nvidia-smi)

PYTORCH CHECK
✅ PyTorch already installed
   Version: 2.9.0+cu126
✅ CUDA available
   CUDA Version: 12.6
   cuDNN Version: 91002
   GPU Device: Tesla T4
   GPU Count: 1
   GPU 0 Memory: 14.7 GB

FINAL VERIFICATION
PyTorch: 2.9.0+cu126
CUDA Available: True
✅ Ready for GPU acceleration

✅ System check complete!


In [ ]:
# ==============================
# GOOGLE DRIVE MOUNT (COLAB)
# ==============================

import os
from pathlib import Path

# Configuration: Set to True if you need to force remount (e.g., after auth issues)
FORCE_REMOUNT = False

def is_colab_environment():
    """Detect if running in Google Colab."""
    try:
        import google.colab
        return True
    except ImportError:
        # Fallback check via environment variables
        return "COLAB_GPU" in os.environ or "google.colab" in str(os.environ)

def mount_drive_if_colab(force_remount: bool = False):
    """
    Mount Google Drive if running in Colab environment.

    Args:
        force_remount: If True, force remount even if already mounted

    Returns:
        bool: True if Colab and mount successful/already mounted, False otherwise
    """
    if not is_colab_environment():
        print("ℹ️  Not running in Google Colab — skipping Drive mount")
        return False

    try:
        from google.colab import drive

        mount_point = Path("/content/drive")

        # Check if already mounted
        if mount_point.exists() and mount_point.is_dir():
            if force_remount:
                print("🔄 Force remounting Google Drive...")
                drive.mount(str(mount_point), force_remount=True)
                print("✅ Google Drive remounted at /content/drive")
            else:
                print("✅ Google Drive already mounted at /content/drive")
                # Verify it's accessible
                test_path = mount_point / "MyDrive"
                if test_path.exists():
                    print(f"   Verified: MyDrive accessible")
                else:
                    print(f"   ⚠️  Warning: MyDrive not found at {test_path}")
                return True
        else:
            print("📁 Mounting Google Drive...")
            drive.mount(str(mount_point))
            print("✅ Google Drive mounted at /content/drive")

        # Verify mount point is accessible
        if not mount_point.exists():
            raise RuntimeError(f"Mount point {mount_point} does not exist after mounting")

        return True

    except ImportError:
        print("❌ Error: google.colab.drive not available")
        print("   This should not happen if running in Colab. Check your environment.")
        return False
    except Exception as e:
        print(f"❌ Error mounting Google Drive: {e}")
        print("   Troubleshooting:")
        print("   1. Make sure you're running in Google Colab")
        print("   2. Check your internet connection")
        print("   3. Try setting FORCE_REMOUNT = True and re-run this cell")
        return False

# Mount drive
IS_COLAB = mount_drive_if_colab(force_remount=FORCE_REMOUNT)

# Print summary
if IS_COLAB:
    drive_path = Path("/content/drive/MyDrive")
    if drive_path.exists():
        print(f"\n📂 Drive root: {drive_path}")
    else:
        print(f"\n⚠️  Warning: MyDrive not found at {drive_path}")
else:
    print("\n💡 Running locally - using local file system")


📁 Mounting Google Drive...
Mounted at /content/drive
✅ Google Drive mounted at /content/drive

📂 Drive root: /content/drive/MyDrive


In [ ]:
# ==============================
# VIDEO SYNCHRONIZATION CONFIGURATION
# ==============================

from pathlib import Path
import os

print("=" * 50)
print("VIDEO SYNCHRONIZATION CONFIGURATION")
print("=" * 50)

# -------- Detect environment --------
# Use IS_COLAB from Cell 2 if available, otherwise detect
try:
    if 'IS_COLAB' not in globals():
        try:
            import google.colab
            IS_COLAB = True
        except ImportError:
            IS_COLAB = False
except NameError:
    try:
        import google.colab
        IS_COLAB = True
    except ImportError:
        IS_COLAB = False

# -------- Project root (will be used for paths) --------
if IS_COLAB:
    # Project root in Drive (will be confirmed in Cell 3)
    PROJECT_ROOT_DRIVE = Path("/content/drive/MyDrive/football/final")
else:
    # Local project root (current directory)
    PROJECT_ROOT_DRIVE = Path(".").resolve()

# -------- Video Synchronization Paths --------
# Path to Videos directory in Drive (source videos)
VIDEOS_DIR = PROJECT_ROOT_DRIVE / "videos"

# Path to input directory (where synced videos will be saved)
# Note: This will be confirmed/created in Cell 3, but we define it here
INPUT_DIR_SYNC = PROJECT_ROOT_DRIVE / "input"

# -------- Synchronization Parameters --------
# Maximum allowed time difference between videos (in seconds)
# Videos with larger time differences will trigger a warning
MAX_TIME_DIFFERENCE_SECONDS = 5 * 60  # 5 minutes default

# Minimum video duration after trimming (in seconds)
# If synchronized duration is below this, will warn user
MIN_SYNC_DURATION_SECONDS = 30  # 30 seconds default

# Minimum overlap required for synchronization (in seconds)
# Videos must overlap by at least this duration to be synchronized
MIN_OVERLAP_SECONDS = 10  # 10 seconds default

# -------- Processing Options --------
# Skip synchronization if synced videos already exist
SKIP_IF_SYNCED_EXISTS = True

# Output format for synced videos (mp4 recommended for compatibility)
SYNC_OUTPUT_FORMAT = "mp4"  # Options: "mp4", "mov", or "same" (keep original format)

# Use fast lossless trimming (stream copy) - recommended for speed
USE_LOSSLESS_TRIMMING = True

# -------- Performance Optimization Options --------
# Enable parallel processing for metadata extraction and trimming
# Note: Parallel processing uses more CPU/memory but is faster
ENABLE_PARALLEL_PROCESSING = False  # Set to True for faster processing (uses multiprocessing)

# Maximum number of parallel workers (None = auto-detect based on CPU count)
MAX_PARALLEL_WORKERS = None  # None = use all available CPUs, or set specific number (e.g., 2)

# Enable metadata caching to avoid re-extraction
ENABLE_METADATA_CACHE = True

# Cache directory for storing metadata
METADATA_CACHE_DIR = PROJECT_ROOT_DRIVE / "debug" / "metadata_cache"

# Enhanced skip check: Compare timestamps of existing synced videos
# If timestamps match original videos, skip re-synchronization
ENHANCED_SKIP_CHECK = True

# -------- Edge Case Handling Options --------
# Allow manual override for large time differences
ALLOW_LARGE_TIME_DIFF_OVERRIDE = True

# Maximum time difference before requiring manual confirmation (in hours)
LARGE_TIME_DIFF_THRESHOLD_HOURS = 1.0  # 1 hour

# Allow proceeding with insufficient overlap (with warning)
ALLOW_INSUFFICIENT_OVERLAP = False  # Set to True to allow proceeding anyway

# Minimum overlap threshold for warning (in seconds)
INSUFFICIENT_OVERLAP_WARNING_SECONDS = 5  # Warn if overlap is less than this

# Handle corrupted metadata gracefully
SKIP_CORRUPTED_METADATA = True  # Continue with videos that have valid metadata

# -------- Display Configuration --------
print(f"\n📁 Path Configuration:")
print(f"   Environment: {'Google Colab' if IS_COLAB else 'Local'}")
print(f"   Project root: {PROJECT_ROOT_DRIVE}")
print(f"   Videos directory: {VIDEOS_DIR}")
print(f"   Input directory (output): {INPUT_DIR_SYNC}")

print(f"\n⚙️  Synchronization Parameters:")
print(f"   Max time difference: {MAX_TIME_DIFFERENCE_SECONDS / 60:.1f} minutes")
print(f"   Min sync duration: {MIN_SYNC_DURATION_SECONDS} seconds")
print(f"   Min overlap required: {MIN_OVERLAP_SECONDS} seconds")

print(f"\n🔧 Processing Options:")
print(f"   Skip if synced exists: {'✅' if SKIP_IF_SYNCED_EXISTS else '❌'}")
print(f"   Enhanced skip check: {'✅' if ENHANCED_SKIP_CHECK else '❌'}")
print(f"   Output format: {SYNC_OUTPUT_FORMAT}")
print(f"   Lossless trimming: {'✅' if USE_LOSSLESS_TRIMMING else '❌'}")

print(f"\n⚡ Performance Options:")
print(f"   Parallel processing: {'✅' if ENABLE_PARALLEL_PROCESSING else '❌'}")
if ENABLE_PARALLEL_PROCESSING:
    import multiprocessing
    cpu_count = multiprocessing.cpu_count()
    max_workers = MAX_PARALLEL_WORKERS if MAX_PARALLEL_WORKERS else cpu_count
    print(f"   Max workers: {max_workers} (available CPUs: {cpu_count})")
print(f"   Metadata caching: {'✅' if ENABLE_METADATA_CACHE else '❌'}")

print(f"\n🛡️  Edge Case Handling:")
print(f"   Allow large time diff override: {'✅' if ALLOW_LARGE_TIME_DIFF_OVERRIDE else '❌'}")
print(f"   Allow insufficient overlap: {'✅' if ALLOW_INSUFFICIENT_OVERLAP else '❌'}")
print(f"   Skip corrupted metadata: {'✅' if SKIP_CORRUPTED_METADATA else '❌'}")

# -------- Validation --------
print(f"\n🔍 Validating paths...")

# Check if Videos directory exists
if VIDEOS_DIR.exists():
    print(f"   ✅ Videos directory exists: {VIDEOS_DIR}")
    try:
        video_count = len(list(VIDEOS_DIR.glob("*")))
        print(f"      Found {video_count} items")
    except Exception as e:
        print(f"      ⚠️  Could not scan directory: {e}")
else:
    print(f"   ⚠️  Videos directory not found: {VIDEOS_DIR}")
    print(f"      Will be created if needed, or check path configuration")

# Check/create input directory
if INPUT_DIR_SYNC.exists():
    print(f"   ✅ Input directory exists: {INPUT_DIR_SYNC}")
else:
    print(f"   ℹ️  Input directory will be created: {INPUT_DIR_SYNC}")
    INPUT_DIR_SYNC.mkdir(parents=True, exist_ok=True)
    print(f"      ✅ Created input directory")

# -------- Video Discovery --------
print(f"\n" + "=" * 50)
print("VIDEO DISCOVERY")
print("=" * 50)

# Supported video formats (case-insensitive)
SUPPORTED_VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".MP4", ".MOV", ".AVI", ".MKV"}

# Ensure Videos directory exists
if not VIDEOS_DIR.exists():
    print(f"\n⚠️  Videos directory does not exist: {VIDEOS_DIR}")
    print(f"   Creating directory...")
    try:
        VIDEOS_DIR.mkdir(parents=True, exist_ok=True)
        print(f"   ✅ Created Videos directory")
    except Exception as e:
        error_msg = (
            f"❌ Could not create Videos directory: {e}\n"
            f"   Please create the directory manually or check permissions.\n"
            f"   Expected path: {VIDEOS_DIR}"
        )
        print(f"\n{error_msg}")
        raise RuntimeError(error_msg)

if not VIDEOS_DIR.is_dir():
    error_msg = (
        f"❌ Videos path is not a directory: {VIDEOS_DIR}\n"
        f"   Please check the path configuration."
    )
    print(f"\n{error_msg}")
    raise ValueError(error_msg)

# Check if directory is accessible
try:
    items = list(VIDEOS_DIR.iterdir())
except PermissionError:
    error_msg = (
        f"❌ Permission denied accessing Videos directory: {VIDEOS_DIR}\n"
        f"   Please check directory permissions."
    )
    print(f"\n{error_msg}")
    raise PermissionError(error_msg)
except Exception as e:
    error_msg = f"❌ Error scanning Videos directory: {e}"
    print(f"\n{error_msg}")
    raise RuntimeError(error_msg)

# Discover video files
print(f"\n📂 Scanning Videos directory: {VIDEOS_DIR}")
print(f"   Supported formats: {', '.join(sorted(SUPPORTED_VIDEO_EXTS))}")

VIDEO_FILES = []
skipped_files = []

for item in sorted(VIDEOS_DIR.iterdir()):
    if item.is_file():
        if item.suffix in SUPPORTED_VIDEO_EXTS:
            VIDEO_FILES.append(item)
        else:
            skipped_files.append(item.suffix)

# Display discovery results
print(f"\n📹 Video Discovery Results:")
print(f"   Found: {len(VIDEO_FILES)} video file(s)")

if len(VIDEO_FILES) > 0:
    print(f"\n   Video Files:")
    total_size = 0
    for i, video in enumerate(VIDEO_FILES):
        try:
            size_bytes = video.stat().st_size
            size_mb = size_bytes / (1024 * 1024)
            total_size += size_bytes
            print(f"   [{i:2d}] {video.name:40s} ({size_mb:7.2f} MB)")
        except Exception as e:
            print(f"   [{i:2d}] {video.name:40s} (size unknown: {e})")

    total_size_gb = total_size / (1024 ** 3)
    print(f"\n   Total size: {total_size_gb:.2f} GB ({len(VIDEO_FILES)} files)")
else:
    print(f"\n   ⚠️  No video files found!")

# Show skipped file types if any
if skipped_files:
    unique_skipped = set(skipped_files)
    if len(unique_skipped) > 0:
        print(f"\n   ℹ️  Skipped file types: {', '.join(sorted(unique_skipped))}")
        print(f"      (Supported: {', '.join(sorted(SUPPORTED_VIDEO_EXTS))})")

# -------- Validation --------
print(f"\n🔍 Validating video discovery...")

# Error: No videos found
if len(VIDEO_FILES) == 0:
    error_msg = (
        f"\n❌ ERROR: No supported video files found in Videos directory!\n"
        f"   Directory: {VIDEOS_DIR}\n"
        f"   Supported extensions: {', '.join(sorted(SUPPORTED_VIDEO_EXTS))}\n"
        f"\n   Please ensure:\n"
        f"   1. Video files are placed in: {VIDEOS_DIR}\n"
        f"   2. Files have one of the supported extensions\n"
        f"   3. Files are not corrupted or empty"
    )
    print(error_msg)
    raise RuntimeError(error_msg)

# Warning: Only 1 video found
if len(VIDEO_FILES) == 1:
    warning_msg = (
        f"\n⚠️  WARNING: Only 1 video file found!\n"
        f"   Video: {VIDEO_FILES[0].name}\n"
        f"   Synchronization requires at least 2 videos from different cameras.\n"
        f"   Continuing anyway, but synchronization may not be needed."
    )
    print(warning_msg)
    # Don't raise error, just warn - user might want to process single video

# Success: 2+ videos found
if len(VIDEO_FILES) >= 2:
    print(f"   ✅ Found {len(VIDEO_FILES)} videos - sufficient for multi-camera synchronization")

# Store video files list for later use
print(f"\n" + "=" * 50)
print(f"✅ Video discovery complete!")
print("=" * 50)
print(f"\n📝 Summary:")
print(f"   Videos directory: {VIDEOS_DIR}")
print(f"   Videos found: {len(VIDEO_FILES)}")
print(f"   Ready for metadata extraction: ✅")

# ==============================
# METADATA EXTRACTION
# ==============================

import json
import subprocess
import sys
import shutil
import hashlib
import pickle
from datetime import datetime, timedelta
from typing import Dict, Any, Optional
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed

# -------- Helper Functions --------
def format_duration_display(seconds: float) -> str:
    """Format seconds to HR:MIN:SEC format for display (e.g., 1:23:45 or 0:05:30)."""
    if seconds is None or seconds < 0:
        return "N/A"
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours}:{minutes:02d}:{secs:02d}"

print(f"\n" + "=" * 50)
print("METADATA EXTRACTION")
print("=" * 50)

# -------- Metadata Caching Setup --------
if ENABLE_METADATA_CACHE:
    METADATA_CACHE_DIR.mkdir(parents=True, exist_ok=True)
    print(f"\n💾 Metadata caching enabled: {METADATA_CACHE_DIR}")

def get_video_cache_key(video_path: Path) -> str:
    """Generate cache key based on file path and modification time."""
    try:
        mtime = video_path.stat().st_mtime
        size = video_path.stat().st_size
        key_string = f"{str(video_path)}_{mtime}_{size}"
        return hashlib.md5(key_string.encode()).hexdigest()
    except Exception:
        return hashlib.md5(str(video_path).encode()).hexdigest()

def load_metadata_cache(video_path: Path) -> Optional[Dict[str, Any]]:
    """Load cached metadata if available."""
    if not ENABLE_METADATA_CACHE:
        return None

    try:
        cache_key = get_video_cache_key(video_path)
        cache_file = METADATA_CACHE_DIR / f"{cache_key}.pkl"

        if cache_file.exists():
            with open(cache_file, 'rb') as f:
                cached_data = pickle.load(f)
                # Verify file still exists and matches
                if Path(cached_data.get('path', '')).exists():
                    return cached_data
    except Exception:
        pass

    return None

def save_metadata_cache(video_path: Path, metadata: Dict[str, Any]):
    """Save metadata to cache."""
    if not ENABLE_METADATA_CACHE:
        return

    try:
        cache_key = get_video_cache_key(video_path)
        cache_file = METADATA_CACHE_DIR / f"{cache_key}.pkl"

        # Make metadata JSON-serializable for caching
        cache_data = metadata.copy()
        if 'creation_time_dt' in cache_data and cache_data['creation_time_dt']:
            cache_data['creation_time_dt'] = cache_data['creation_time_dt'].isoformat()
        if 'creation_time_utc_dt' in cache_data and cache_data['creation_time_utc_dt']:
            cache_data['creation_time_utc_dt'] = cache_data['creation_time_utc_dt'].isoformat()

        with open(cache_file, 'wb') as f:
            pickle.dump(cache_data, f)
    except Exception:
        pass  # Fail silently - caching is optional

# -------- FFmpeg Installation Check (Google Colab) --------
print(f"\n🔧 Checking FFmpeg availability...")

if IS_COLAB:
    print(f"   Environment: Google Colab")
    print(f"   ⚠️  Note: Your local machine's FFmpeg installation does NOT affect Google Colab.")
    print(f"      Google Colab runs in a cloud environment separate from your local machine.")

    # Check if ffmpeg is already installed in Colab
    try:
        result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print(f"   ✅ FFmpeg is already installed in this Colab session")
        else:
            raise FileNotFoundError()
    except (FileNotFoundError, subprocess.TimeoutExpired):
        print(f"   📦 FFmpeg not found. Installing...")
        try:
            # Install FFmpeg in Colab
            subprocess.run(['apt-get', 'update'], check=True, capture_output=True)
            subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], check=True, capture_output=True)
            print(f"   ✅ FFmpeg installed successfully!")
        except Exception as e:
            error_msg = (
                f"\n❌ ERROR: Could not install FFmpeg in Google Colab!\n"
                f"   Error: {e}\n"
                f"   Please run this command manually in a Colab cell:\n"
                f"   !apt-get update && apt-get install -y ffmpeg"
            )
            print(error_msg)
            raise RuntimeError(error_msg)
else:
    print(f"   Environment: Local machine")
    print(f"   Checking for FFmpeg installation...")

# -------- Helper Functions (from extract_metadata.py) --------

def find_ffprobe(ffprobe_path: Optional[str] = None) -> str:
    """Find ffprobe executable path."""
    if ffprobe_path:
        custom_path = Path(ffprobe_path)
        if custom_path.exists() and custom_path.is_file():
            return str(custom_path.resolve())
        raise FileNotFoundError(f"ffprobe not found at specified path: {ffprobe_path}")

    # Try to find in PATH || Prefer PATH lookup (works on Colab, Linux, macOS)
    ffprobe_exe = shutil.which('ffprobe')
    if ffprobe_exe:
        return ffprobe_exe

    # Search common Windows installation locations
    if sys.platform == 'win32':
        common_paths = [
            Path('C:/ffmpeg/bin/ffprobe.exe'),
            Path('C:/Program Files/ffmpeg/bin/ffprobe.exe'),
            Path('C:/Program Files (x86)/ffmpeg/bin/ffprobe.exe'),
            Path.home() / 'ffmpeg/bin/ffprobe.exe',
            Path('C:/tools/ffmpeg/bin/ffprobe.exe'),
        ]

        for path in common_paths:
            if path.exists() and path.is_file():
                return str(path.resolve())

    raise FileNotFoundError(
        "ffprobe not found. Please ensure FFmpeg is installed and available in PATH.\n"
        "Download FFmpeg from: https://ffmpeg.org/download.html"
    )

def run_ffprobe(video_path: str, ffprobe_path: Optional[str] = None) -> Dict[str, Any]:
    """Run ffprobe on a video file and return parsed JSON metadata."""
    ffprobe_exe = find_ffprobe(ffprobe_path)
    video_path = str(Path(video_path).resolve())

    cmd = [
        ffprobe_exe,
        '-v', 'quiet',
        '-print_format', 'json',
        '-show_format',
        '-show_streams',
        video_path
    ]

    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            check=True,
            creationflags=subprocess.CREATE_NO_WINDOW if sys.platform == 'win32' else 0
        )

        # Careful validation: Check if output is empty or invalid
        if not result.stdout or not result.stdout.strip():
            raise ValueError(f"ffprobe returned empty output for: {video_path}")

        # Parse JSON with careful error handling
        try:
            ffprobe_data = json.loads(result.stdout)
        except json.JSONDecodeError as e:
            # Log the problematic output for debugging
            error_msg = f"Failed to parse ffprobe JSON output: {e}"
            if len(result.stdout) > 500:
                error_msg += f"\nOutput preview: {result.stdout[:500]}..."
            else:
                error_msg += f"\nOutput: {result.stdout}"
            raise ValueError(error_msg)

        # Careful validation: Ensure ffprobe_data is a dictionary
        if not isinstance(ffprobe_data, dict):
            raise ValueError(f"ffprobe output is not a dictionary: {type(ffprobe_data)}")

        # Careful validation: Check for required structure
        if 'format' not in ffprobe_data:
            raise ValueError(f"ffprobe output missing 'format' key for: {video_path}")

        if not isinstance(ffprobe_data['format'], dict):
            raise ValueError(f"ffprobe 'format' is not a dictionary: {type(ffprobe_data['format'])}")

        # Validate that we have at least some useful data
        if 'streams' not in ffprobe_data:
            # This is a warning, not an error - some files might not have streams info
            pass

        return ffprobe_data

    except FileNotFoundError:
        raise FileNotFoundError("ffprobe not found. Please ensure FFmpeg is installed.")
    except subprocess.CalledProcessError as e:
        error_details = e.stderr if e.stderr else "No error details available"
        raise RuntimeError(f"ffprobe execution failed for {video_path}: {error_details}")
    except (ValueError, RuntimeError) as e:
        # Re-raise validation errors
        raise
    except Exception as e:
        raise RuntimeError(f"Unexpected error running ffprobe on {video_path}: {e}")

def parse_timestamp(timestamp_str: Optional[str]) -> Optional[datetime]:
    """Parse ISO 8601 timestamp string to datetime object."""
    if not timestamp_str:
        return None

    # Common timestamp formats
    formats = [
        "%Y-%m-%dT%H:%M:%S%z",      # 2026-01-05T20:04:20+0000
        "%Y-%m-%dT%H:%M:%S.%f%z",    # 2026-01-05T20:04:20.123+0000
        "%Y-%m-%dT%H:%M:%S",         # 2026-01-05T20:04:20
        "%Y-%m-%d %H:%M:%S",         # 2026-01-05 20:04:20
        "%Y:%m:%d %H:%M:%S",         # 2026:01:05 20:04:20 (EXIF format)
    ]

    for fmt in formats:
        try:
            return datetime.strptime(timestamp_str, fmt)
        except ValueError:
            continue

    # Try parsing with dateutil if available
    try:
        from dateutil import parser
        return parser.parse(timestamp_str)
    except (ImportError, ValueError):
        pass

    return None

def extract_video_metadata_sync(video_path: Path) -> Optional[Dict[str, Any]]:
    """
    Extract key metadata needed for synchronization.
    Returns: dict with path, name, creation_time, duration, fps, metadata_file_path
    """
    try:
        # Careful validation: Check if file exists and is readable
        if not video_path.exists():
            raise FileNotFoundError(f"Video file not found: {video_path}")

        if not video_path.is_file():
            raise ValueError(f"Path is not a file: {video_path}")

        # Check file size (very small files might be corrupted)
        file_size = video_path.stat().st_size
        if file_size < 1024:  # Less than 1KB is suspicious
            raise ValueError(f"Video file is suspiciously small ({file_size} bytes): {video_path}")

        # Run ffprobe with careful error handling
        try:
            ffprobe_data = run_ffprobe(str(video_path))
        except Exception as e:
            raise RuntimeError(f"Failed to run ffprobe on {video_path.name}: {e}")

        # Careful validation: Ensure ffprobe_data structure
        if not isinstance(ffprobe_data, dict):
            raise ValueError(f"Invalid ffprobe output structure for {video_path.name}")

        # Extract duration with careful validation
        duration_seconds = None
        if 'format' in ffprobe_data and 'duration' in ffprobe_data['format']:
            try:
                duration_raw = ffprobe_data['format']['duration']
                if duration_raw is not None:
                    duration_seconds = float(duration_raw)
                    # Validate duration is reasonable (positive, not too large)
                    if duration_seconds <= 0:
                        raise ValueError(f"Invalid duration (non-positive): {duration_seconds}s")
                    if duration_seconds > 86400 * 10:  # More than 10 days is suspicious
                        raise ValueError(f"Suspiciously long duration: {duration_seconds}s ({duration_seconds/3600:.1f} hours)")
            except (ValueError, TypeError) as e:
                # Log but don't fail - duration might be missing
                pass

        # Extract creation timestamp with careful validation
        creation_time_str = None
        creation_time_dt = None
        timestamp_source = None

        # Check format tags first
        if 'format' in ffprobe_data and 'tags' in ffprobe_data['format']:
            tags = ffprobe_data['format']['tags']
            if not isinstance(tags, dict):
                tags = {}

            # Try multiple timestamp fields in priority order
            timestamp_fields = [
                ('com.apple.quicktime.creationdate', 'metadata'),
                ('creation_time', 'metadata'),
                ('date', 'metadata'),
                ('creationdate', 'metadata')
            ]

            for field_name, source in timestamp_fields:
                if field_name in tags and tags[field_name]:
                    creation_time_str = str(tags[field_name]).strip()
                    if creation_time_str:  # Ensure not empty
                        timestamp_source = source
                        break

        # Parse timestamp with careful validation
        if creation_time_str:
            try:
                creation_time_dt = parse_timestamp(creation_time_str)
                # Validate parsed timestamp is reasonable (not too far in past/future)
                if creation_time_dt:
                    now = datetime.now(creation_time_dt.tzinfo if creation_time_dt.tzinfo else None)
                    if creation_time_dt.tzinfo:
                        now = datetime.now(creation_time_dt.tzinfo)
                    else:
                        now = datetime.now()

                    # Check if timestamp is reasonable (between 2000 and 2100)
                    if creation_time_dt.year < 2000 or creation_time_dt.year > 2100:
                        raise ValueError(f"Timestamp year out of reasonable range: {creation_time_dt.year}")

                    # Check if timestamp is not too far in the future (more than 1 day)
                    if creation_time_dt > now:
                        time_diff = (creation_time_dt - now).total_seconds()
                        if time_diff > 86400:  # More than 1 day in future
                            raise ValueError(f"Timestamp is too far in future: {time_diff/3600:.1f} hours")
            except Exception as e:
                # Log parsing error but continue - will use fallback
                creation_time_dt = None
                timestamp_source = None

        # Extract FPS from video stream with careful validation
        fps = None
        if 'streams' in ffprobe_data and isinstance(ffprobe_data['streams'], list):
            for stream in ffprobe_data['streams']:
                if not isinstance(stream, dict):
                    continue
                if stream.get('codec_type') == 'video':
                    avg_frame_rate = stream.get('avg_frame_rate')
                    if avg_frame_rate and isinstance(avg_frame_rate, str):
                        try:
                            # Parse frame rate (format: "num/den")
                            if '/' in avg_frame_rate:
                                num, den = map(int, avg_frame_rate.split('/'))
                                if den > 0:
                                    fps = round(num / den, 3)
                                    # Validate FPS is reasonable (between 1 and 1000)
                                    if fps < 1.0 or fps > 1000.0:
                                        raise ValueError(f"FPS out of reasonable range: {fps}")
                                    break
                        except (ValueError, ZeroDivisionError, AttributeError) as e:
                            # Invalid frame rate format - skip this stream
                            continue

        # Fallback: use file modification time if no creation_time
        if creation_time_dt is None:
            try:
                mtime = video_path.stat().st_mtime
                creation_time_dt = datetime.fromtimestamp(mtime)
                creation_time_str = creation_time_dt.isoformat()
                timestamp_source = 'file_modification_time'
            except Exception as e:
                # If even file modification time fails, leave as None
                timestamp_source = 'none'

        # Final validation: Ensure we have at least duration or timestamp
        if duration_seconds is None and creation_time_dt is None:
            raise ValueError(f"Could not extract duration or timestamp from {video_path.name}")

        # Build metadata dictionary with careful validation
        metadata = {
            'path': str(video_path.resolve()),
            'name': video_path.name,
            'creation_time': creation_time_str,
            'creation_time_dt': creation_time_dt,
            'duration': duration_seconds,
            'fps': fps,
            'timestamp_source': timestamp_source,
            'file_size_bytes': file_size,
            'metadata_file_path': None  # We're not saving separate metadata files
        }

        # Validate metadata completeness
        if duration_seconds is None:
            # Duration is critical - log warning
            pass  # We'll handle this in the calling code

        return metadata

    except Exception as e:
        # Careful error handling: Provide detailed error information
        error_type = type(e).__name__
        error_msg = str(e)

        # Return error metadata with full context
        return {
            'path': str(video_path.resolve()) if video_path.exists() else str(video_path),
            'name': video_path.name,
            'error': error_msg,
            'error_type': error_type,
            'creation_time': None,
            'creation_time_dt': None,
            'duration': None,
            'fps': None,
            'timestamp_source': None,
            'file_size_bytes': None,
            'metadata_file_path': None
        }

def compare_metadata(before_metadata: Dict[str, Any], after_metadata: Dict[str, Any],
                     expected_duration: Optional[float] = None,
                     tolerance_seconds: float = 0.5) -> Dict[str, Any]:
    """
    Carefully compare metadata before and after trimming.

    Args:
        before_metadata: Metadata extracted before trimming
        after_metadata: Metadata extracted after trimming
        expected_duration: Expected duration after trimming (for validation)
        tolerance_seconds: Tolerance for duration comparison

    Returns:
        Dictionary with comparison results and validation status
    """
    comparison = {
        'valid': True,
        'warnings': [],
        'errors': [],
        'duration_match': None,
        'fps_match': None,
        'timestamp_match': None,
        'file_size_reasonable': None
    }

    # Validate inputs
    if not isinstance(before_metadata, dict):
        comparison['errors'].append(f"before_metadata is not a dictionary: {type(before_metadata)}")
        comparison['valid'] = False
        return comparison

    if not isinstance(after_metadata, dict):
        comparison['errors'].append(f"after_metadata is not a dictionary: {type(after_metadata)}")
        comparison['valid'] = False
        return comparison

    # Check for errors in metadata
    if before_metadata.get('error'):
        comparison['errors'].append(f"Before metadata has error: {before_metadata.get('error')}")
        comparison['valid'] = False

    if after_metadata.get('error'):
        comparison['errors'].append(f"After metadata has error: {after_metadata.get('error')}")
        comparison['valid'] = False

    # Compare duration
    before_duration = before_metadata.get('duration')
    after_duration = after_metadata.get('duration')

    if before_duration is not None and after_duration is not None:
        # After duration should be less than or equal to before duration
        if after_duration > before_duration:
            comparison['errors'].append(
                f"After duration ({after_duration:.2f}s) > before duration ({before_duration:.2f}s)"
            )
            comparison['valid'] = False
        else:
            comparison['duration_match'] = True

    # Validate against expected duration if provided
    if expected_duration is not None and after_duration is not None:
        duration_diff = abs(after_duration - expected_duration)
        if duration_diff > tolerance_seconds:
            comparison['warnings'].append(
                f"Duration mismatch: expected {expected_duration:.2f}s, got {after_duration:.2f}s "
                f"(diff: {duration_diff:.2f}s)"
            )
            comparison['duration_match'] = False
        else:
            comparison['duration_match'] = True

    # Compare FPS (should match)
    before_fps = before_metadata.get('fps')
    after_fps = after_metadata.get('fps')

    if before_fps is not None and after_fps is not None:
        fps_diff = abs(before_fps - after_fps)
        if fps_diff > 0.01:  # Allow small floating point differences
            comparison['warnings'].append(
                f"FPS mismatch: before {before_fps:.3f}, after {after_fps:.3f} (diff: {fps_diff:.3f})"
            )
            comparison['fps_match'] = False
        else:
            comparison['fps_match'] = True

    # Compare file sizes (after should be smaller or similar)
    before_size = before_metadata.get('file_size_bytes')
    after_size = after_metadata.get('file_size_bytes')

    if before_size is not None and after_size is not None:
        if after_size > before_size * 1.1:  # Allow 10% tolerance for encoding differences
            comparison['warnings'].append(
                f"After file size ({after_size} bytes) is larger than before ({before_size} bytes)"
            )
            comparison['file_size_reasonable'] = False
        elif after_size < before_size * 0.1:  # Less than 10% of original is suspicious
            comparison['warnings'].append(
                f"After file size ({after_size} bytes) is suspiciously small compared to before ({before_size} bytes)"
            )
            comparison['file_size_reasonable'] = False
        else:
            comparison['file_size_reasonable'] = True

    return comparison

# -------- Extract Metadata from All Videos --------
print(f"\n📥 Extracting metadata from {len(VIDEO_FILES)} video(s)...")
if ENABLE_METADATA_CACHE:
    print(f"   Using metadata cache: {METADATA_CACHE_DIR}")

VIDEO_METADATA = []
failed_extractions = []
cached_count = 0
extracted_count = 0

def extract_metadata_with_cache(video_file: Path) -> Dict[str, Any]:
    """Extract metadata with caching support."""
    # Try loading from cache first
    cached_metadata = load_metadata_cache(video_file)
    if cached_metadata:
        # Restore datetime objects
        if 'creation_time_dt' in cached_metadata and cached_metadata['creation_time_dt']:
            if isinstance(cached_metadata['creation_time_dt'], str):
                cached_metadata['creation_time_dt'] = parse_timestamp(cached_metadata['creation_time_dt'])
        if 'creation_time_utc_dt' in cached_metadata and cached_metadata['creation_time_utc_dt']:
            if isinstance(cached_metadata['creation_time_utc_dt'], str):
                cached_metadata['creation_time_utc_dt'] = parse_timestamp(cached_metadata['creation_time_utc_dt'])
        return cached_metadata

    # Extract fresh metadata
    metadata = extract_video_metadata_sync(video_file)

    # Save to cache if successful
    if not metadata.get('error'):
        save_metadata_cache(video_file, metadata)

    return metadata

# Sequential or parallel processing
if ENABLE_PARALLEL_PROCESSING and len(VIDEO_FILES) > 1:
    print(f"   Using parallel processing...")
    import multiprocessing
    max_workers = MAX_PARALLEL_WORKERS if MAX_PARALLEL_WORKERS else multiprocessing.cpu_count()

    # Note: Parallel processing requires picklable functions
    # For simplicity, we'll use ThreadPoolExecutor which works better with Path objects
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_video = {executor.submit(extract_metadata_with_cache, video_file): video_file
                           for video_file in VIDEO_FILES}

        results = {}
        for future in as_completed(future_to_video):
            video_file = future_to_video[future]
            try:
                metadata = future.result()
                results[video_file] = metadata
            except Exception as e:
                results[video_file] = {
                    'path': str(video_file.resolve()),
                    'name': video_file.name,
                    'error': str(e),
                    'creation_time': None,
                    'creation_time_dt': None,
                    'duration': None,
                    'fps': None,
                    'metadata_file_path': None
                }

    # Process results in original order
    for i, video_file in enumerate(VIDEO_FILES):
        metadata = results[video_file]
        was_cached = load_metadata_cache(video_file) is not None

        print(f"\n   [{i+1}/{len(VIDEO_FILES)}] Processing: {video_file.name}")
        if was_cached:
            print(f"      💾 Using cached metadata")
            cached_count += 1
        else:
            extracted_count += 1

        if metadata.get('error'):
            error_msg = metadata['error']
            print(f"      ❌ Failed: {error_msg}")
            failed_extractions.append({
                'video': video_file.name,
                'error': error_msg
            })
            VIDEO_METADATA.append(metadata)
        else:
            # Ensure format_duration_display is available (safety check for Jupyter notebook execution order)
            if 'format_duration_display' not in globals():
                def format_duration_display(seconds: float) -> str:
                    """Format seconds to HR:MIN:SEC format for display (e.g., 1:23:45 or 0:05:30)."""
                    if seconds is None or seconds < 0:
                        return "N/A"
                    hours = int(seconds // 3600)
                    minutes = int((seconds % 3600) // 60)
                    secs = int(seconds % 60)
                    return f"{hours}:{minutes:02d}:{secs:02d}"
                globals()['format_duration_display'] = format_duration_display
            duration_str = format_duration_display(metadata['duration']) if metadata['duration'] else "unknown"
            duration_seconds_str = f"({metadata['duration']:.1f}s)" if metadata['duration'] else ""
            fps_str = f"{metadata['fps']:.2f} fps" if metadata['fps'] else "unknown"
            time_str = metadata['creation_time'] if metadata['creation_time'] else "not found"

            print(f"      ✅ Duration: {duration_str} {duration_seconds_str}, FPS: {fps_str}")
            print(f"         Timestamp: {time_str}")

            VIDEO_METADATA.append(metadata)
else:
    # Sequential processing
    for i, video_file in enumerate(VIDEO_FILES):
        print(f"\n   [{i+1}/{len(VIDEO_FILES)}] Processing: {video_file.name}")

        try:
            # Check cache first
            cached_metadata = load_metadata_cache(video_file)
            if cached_metadata:
                print(f"      💾 Using cached metadata")
                cached_count += 1
                # Restore datetime objects
                if 'creation_time_dt' in cached_metadata and cached_metadata['creation_time_dt']:
                    if isinstance(cached_metadata['creation_time_dt'], str):
                        cached_metadata['creation_time_dt'] = parse_timestamp(cached_metadata['creation_time_dt'])
                if 'creation_time_utc_dt' in cached_metadata and cached_metadata['creation_time_utc_dt']:
                    if isinstance(cached_metadata['creation_time_utc_dt'], str):
                        cached_metadata['creation_time_utc_dt'] = parse_timestamp(cached_metadata['creation_time_utc_dt'])
                metadata = cached_metadata
            else:
                extracted_count += 1
                metadata = extract_video_metadata_sync(video_file)
                # Save to cache if successful
                if not metadata.get('error'):
                    save_metadata_cache(video_file, metadata)

            if metadata.get('error'):
                error_msg = metadata['error']

                if SKIP_CORRUPTED_METADATA:
                    print(f"      ⚠️  Metadata extraction failed (corrupted?): {error_msg}")
                    print(f"      ⏭️  Skipping this video (SKIP_CORRUPTED_METADATA=True)")
                    failed_extractions.append({
                        'video': video_file.name,
                        'error': error_msg,
                        'skipped': True
                    })
                    # Don't add to VIDEO_METADATA if skipping corrupted
                    continue
                else:
                    print(f"      ❌ Failed: {error_msg}")
                    failed_extractions.append({
                        'video': video_file.name,
                        'error': error_msg,
                        'skipped': False
                    })
                    VIDEO_METADATA.append(metadata)
            else:
                # Success
                                # Ensure format_duration_display is available (safety check for Jupyter notebook execution order)
                if 'format_duration_display' not in globals():
                    def format_duration_display(seconds: float) -> str:
                        """Format seconds to HR:MIN:SEC format for display (e.g., 1:23:45 or 0:05:30)."""
                        if seconds is None or seconds < 0:
                            return "N/A"
                        hours = int(seconds // 3600)
                        minutes = int((seconds % 3600) // 60)
                        secs = int(seconds % 60)
                        return f"{hours}:{minutes:02d}:{secs:02d}"
                    globals()['format_duration_display'] = format_duration_display
                duration_str = format_duration_display(metadata['duration']) if metadata['duration'] else "unknown"
                duration_seconds_str = f"({metadata['duration']:.1f}s)" if metadata['duration'] else ""
                fps_str = f"{metadata['fps']:.2f} fps" if metadata['fps'] else "unknown"
                time_str = metadata['creation_time'] if metadata['creation_time'] else "not found"

                print(f"      ✅ Duration: {duration_str} {duration_seconds_str}, FPS: {fps_str}")
                print(f"         Timestamp: {time_str}")

                VIDEO_METADATA.append(metadata)

        except Exception as e:
            error_msg = f"Unexpected error: {e}"
            print(f"      ❌ {error_msg}")
            failed_extractions.append({
                'video': video_file.name,
                'error': error_msg
            })
            # Add with error
            VIDEO_METADATA.append({
                'path': str(video_file.resolve()),
                'name': video_file.name,
                'error': error_msg,
                'creation_time': None,
                'creation_time_dt': None,
                'duration': None,
                'fps': None,
                'metadata_file_path': None
            })

if ENABLE_METADATA_CACHE:
    print(f"\n   📊 Cache statistics: {cached_count} cached, {extracted_count} extracted")

# -------- Summary of Metadata Extraction --------
print(f"\n" + "=" * 50)
print("METADATA EXTRACTION SUMMARY")
print("=" * 50)

successful = len([m for m in VIDEO_METADATA if 'error' not in m or not m.get('error')])
failed = len(failed_extractions)

print(f"\n📊 Results:")
print(f"   Successful: {successful}/{len(VIDEO_FILES)}")
if failed > 0:
    print(f"   Failed: {failed}/{len(VIDEO_FILES)}")
    print(f"\n   Failed videos:")
    for fail in failed_extractions:
        print(f"      - {fail['video']}: {fail['error']}")

# Check if we have enough valid metadata
valid_metadata = [m for m in VIDEO_METADATA if m.get('creation_time_dt') is not None and m.get('duration') is not None]

if len(valid_metadata) < 2:
    error_msg = (
        f"\n❌ ERROR: Insufficient valid metadata for synchronization!\n"
        f"   Required: At least 2 videos with valid timestamps and duration\n"
        f"   Found: {len(valid_metadata)} videos with valid metadata\n"
        f"   Failed extractions: {failed}"
    )
    print(error_msg)
    if len(valid_metadata) == 0:
        raise RuntimeError(error_msg)
    else:
        print(f"\n⚠️  Warning: Only {len(valid_metadata)} video(s) have valid metadata.")
        print(f"   Synchronization may not work correctly.")

# ==============================
# PARSE AND NORMALIZE TIMESTAMPS
# ==============================

from datetime import timezone

print(f"\n" + "=" * 50)
print("PARSE AND NORMALIZE TIMESTAMPS")
print("=" * 50)

# -------- Enhanced Timestamp Parsing Function --------
def normalize_timestamp_to_utc(dt: Optional[datetime], video_path: Optional[Path] = None) -> Optional[datetime]:
    """
    Normalize a datetime object to UTC timezone.
    If datetime is None, fallback to file modification time.

    Args:
        dt: Datetime object (may be timezone-aware or naive)
        video_path: Path to video file for fallback

    Returns:
        UTC datetime object, or None if unavailable
    """
    # If datetime is provided
    if dt is not None:
        # If timezone-aware, convert to UTC
        if dt.tzinfo is not None:
            return dt.astimezone(timezone.utc)
        else:
            # If naive, assume it's already in UTC (or local time - we'll treat as UTC)
            # For better accuracy, you could add timezone detection here
            return dt.replace(tzinfo=timezone.utc)

    # Fallback: use file modification time
    if video_path is not None:
        try:
            mtime = video_path.stat().st_mtime
            dt_fallback = datetime.fromtimestamp(mtime, tz=timezone.utc)
            return dt_fallback
        except Exception:
            pass

    return None

# -------- Parse and Normalize All Timestamps --------
print(f"\n🕐 Parsing and normalizing timestamps to UTC...")

normalized_count = 0
fallback_count = 0
missing_count = 0

for i, metadata in enumerate(VIDEO_METADATA):
    video_path = Path(metadata['path']) if metadata.get('path') else None

    # Get original timestamp
    original_dt = metadata.get('creation_time_dt')
    original_str = metadata.get('creation_time')

    # Normalize to UTC
    normalized_dt = normalize_timestamp_to_utc(original_dt, video_path)

    if normalized_dt is not None:
        # Store normalized timestamp
        metadata['creation_time_utc'] = normalized_dt.isoformat()
        metadata['creation_time_utc_dt'] = normalized_dt

        # Track if we used fallback
        if original_dt is None:
            fallback_count += 1
            metadata['timestamp_source'] = 'file_modification_time'
            print(f"   [{i+1}/{len(VIDEO_METADATA)}] {metadata['name']:40s} - Fallback to file mtime: {normalized_dt.isoformat()}")
        else:
            normalized_count += 1
            metadata['timestamp_source'] = 'metadata'

            # Show timezone conversion if applicable
            if original_dt.tzinfo is not None:
                tz_name = str(original_dt.tzinfo)
                print(f"   [{i+1}/{len(VIDEO_METADATA)}] {metadata['name']:40s} - UTC: {normalized_dt.isoformat()} (from {tz_name})")
            else:
                print(f"   [{i+1}/{len(VIDEO_METADATA)}] {metadata['name']:40s} - UTC: {normalized_dt.isoformat()}")
    else:
        missing_count += 1
        metadata['creation_time_utc'] = None
        metadata['creation_time_utc_dt'] = None
        metadata['timestamp_source'] = 'none'
        print(f"   [{i+1}/{len(VIDEO_METADATA)}] {metadata['name']:40s} - ⚠️  No timestamp available")

# -------- Calculate Absolute Time Differences --------
print(f"\n📊 Calculating time differences between videos...")

# Get all normalized timestamps
normalized_timestamps = []
for metadata in VIDEO_METADATA:
    if metadata.get('creation_time_utc_dt') is not None:
        normalized_timestamps.append({
            'name': metadata['name'],
            'timestamp': metadata['creation_time_utc_dt'],
            'metadata': metadata
        })

if len(normalized_timestamps) >= 2:
    # Sort by timestamp
    normalized_timestamps.sort(key=lambda x: x['timestamp'])

    # Calculate pairwise differences
    time_differences = []
    earliest_video = normalized_timestamps[0]

    print(f"\n   Reference (earliest): {earliest_video['name']}")
    print(f"   Timestamp: {earliest_video['timestamp'].isoformat()}")

    for i, video_info in enumerate(normalized_timestamps[1:], 1):
        diff_seconds = (video_info['timestamp'] - earliest_video['timestamp']).total_seconds()

        # Format difference
        if abs(diff_seconds) < 60:
            diff_str = f"{diff_seconds:.2f} seconds"
        elif abs(diff_seconds) < 3600:
            diff_str = f"{diff_seconds / 60:.2f} minutes"
        else:
            diff_str = f"{diff_seconds / 3600:.2f} hours"

        # Determine if ahead or behind
        if diff_seconds > 0:
            relation = "ahead"
        elif diff_seconds < 0:
            relation = "behind"
        else:
            relation = "same"

        time_differences.append({
            'video': video_info['name'],
            'difference_seconds': diff_seconds,
            'difference_formatted': diff_str,
            'relation': relation,
            'timestamp': video_info['timestamp']
        })

        print(f"\n   [{i+1}] {video_info['name']:40s}")
        print(f"       Timestamp: {video_info['timestamp'].isoformat()}")
        print(f"       Difference: {abs(diff_seconds):.2f} seconds {relation} reference")

        # Store in metadata
        video_info['metadata']['time_offset_seconds'] = diff_seconds
        video_info['metadata']['time_offset_formatted'] = diff_str
        video_info['metadata']['time_relation'] = relation

    # Calculate statistics
    all_diffs = [abs(td['difference_seconds']) for td in time_differences]
    max_diff = max(all_diffs) if all_diffs else 0
    min_diff = min(all_diffs) if all_diffs else 0
    avg_diff = sum(all_diffs) / len(all_diffs) if all_diffs else 0

    print(f"\n   📈 Time Difference Statistics:")
    print(f"      Maximum difference: {max_diff:.2f} seconds ({max_diff / 60:.2f} minutes)")
    print(f"      Minimum difference: {min_diff:.2f} seconds")
    print(f"      Average difference: {avg_diff:.2f} seconds ({avg_diff / 60:.2f} minutes)")

    # Store statistics
    TIMESTAMP_STATS = {
        'total_videos': len(VIDEO_METADATA),
        'videos_with_timestamps': len(normalized_timestamps),
        'normalized_from_metadata': normalized_count,
        'normalized_from_file_mtime': fallback_count,
        'missing_timestamps': missing_count,
        'earliest_video': earliest_video['name'],
        'earliest_timestamp': earliest_video['timestamp'].isoformat(),
        'max_difference_seconds': max_diff,
        'min_difference_seconds': min_diff,
        'avg_difference_seconds': avg_diff,
        'time_differences': time_differences
    }

elif len(normalized_timestamps) == 1:
    print(f"\n   ⚠️  Only 1 video has a valid timestamp")
    print(f"   Cannot calculate time differences")
    TIMESTAMP_STATS = {
        'total_videos': len(VIDEO_METADATA),
        'videos_with_timestamps': 1,
        'normalized_from_metadata': normalized_count,
        'normalized_from_file_mtime': fallback_count,
        'missing_timestamps': missing_count,
        'warning': 'Only one video has timestamp - cannot calculate differences'
    }
else:
    print(f"\n   ⚠️  No videos have valid timestamps")
    print(f"   Cannot calculate time differences")
    TIMESTAMP_STATS = {
        'total_videos': len(VIDEO_METADATA),
        'videos_with_timestamps': 0,
        'normalized_from_metadata': normalized_count,
        'normalized_from_file_mtime': fallback_count,
        'missing_timestamps': missing_count,
        'error': 'No valid timestamps found'
    }

# -------- Summary --------
print(f"\n" + "=" * 50)
print("TIMESTAMP NORMALIZATION SUMMARY")
print("=" * 50)

print(f"\n📊 Normalization Results:")
print(f"   Total videos: {len(VIDEO_METADATA)}")
print(f"   Normalized from metadata: {normalized_count}")
print(f"   Normalized from file mtime (fallback): {fallback_count}")
print(f"   Missing timestamps: {missing_count}")

if len(normalized_timestamps) >= 2:
    print(f"\n✅ All timestamps normalized to UTC")
    print(f"   Ready for synchronization calculation")
elif len(normalized_timestamps) == 1:
    print(f"\n⚠️  Only 1 video has timestamp - synchronization may not be accurate")
else:
    print(f"\n❌ No valid timestamps found - synchronization cannot proceed")

# Update valid_metadata to use UTC timestamps
valid_metadata = [m for m in VIDEO_METADATA if m.get('creation_time_utc_dt') is not None and m.get('duration') is not None]

# ==============================
# DETERMINE SYNCHRONIZATION STRATEGY
# ==============================

print(f"\n" + "=" * 50)
print("DETERMINE SYNCHRONIZATION STRATEGY")
print("=" * 50)

# Initialize synchronization strategy variables
SYNC_STRATEGY = None
TRIM_PARAMETERS = []

if len(valid_metadata) >= 2:
    print(f"\n📊 Analyzing {len(valid_metadata)} videos for synchronization strategy...")

    # -------- Step 1: Find Earliest Start Time (Reference) --------
    print(f"\n1️⃣ Finding earliest start time (reference point)...")

    # Get all videos with their start times and durations
    video_timeline = []
    for metadata in valid_metadata:
        start_time = metadata['creation_time_utc_dt']
        duration = metadata['duration']
        end_time = start_time + timedelta(seconds=duration)

        video_timeline.append({
            'name': metadata['name'],
            'metadata': metadata,
            'start_time': start_time,
            'duration': duration,
            'end_time': end_time
        })

    # Sort by start time
    video_timeline.sort(key=lambda x: x['start_time'])

    # Find earliest start time (reference)
    earliest_video = video_timeline[0]
    reference_start = earliest_video['start_time']

    print(f"   Reference video: {earliest_video['name']}")
    print(f"   Reference start time: {reference_start.isoformat()}")

    # -------- Step 2: Calculate Time Offsets --------
    print(f"\n2️⃣ Calculating time offsets relative to reference...")

    for video_info in video_timeline:
        offset_seconds = (video_info['start_time'] - reference_start).total_seconds()
        video_info['offset_seconds'] = offset_seconds

        if offset_seconds == 0:
            print(f"   {video_info['name']:40s} - No offset (reference)")
        else:
            offset_str = f"{abs(offset_seconds):.2f}s {'ahead' if offset_seconds < 0 else 'behind'}"
            print(f"   {video_info['name']:40s} - Offset: {offset_str}")

    # -------- Step 3: Determine Common Duration (Overlap Period) --------
    print(f"\n3️⃣ Determining common duration (overlap period)...")

    # Find latest start time (when all videos have started)
    latest_start = max(video['start_time'] for video in video_timeline)
    latest_start_offset = (latest_start - reference_start).total_seconds()

    # Find earliest end time (when first video ends)
    earliest_end = min(video['end_time'] for video in video_timeline)

    # Calculate common time window
    sync_start_time = latest_start  # All videos are recording from this point
    sync_end_time = earliest_end     # All videos are still recording until this point
    common_duration = (sync_end_time - sync_start_time).total_seconds()

    print(f"   Latest start (sync start): {latest_start.isoformat()}")
    print(f"   Earliest end (sync end): {earliest_end.isoformat()}")
    print(f"   Common duration: {format_duration_display(common_duration)} ({common_duration:.2f} seconds, {common_duration / 60:.2f} minutes)")

    # Validate minimum overlap
    if common_duration < MIN_OVERLAP_SECONDS:
        warning_msg = (
            f"\n⚠️  WARNING: Common duration {format_duration_display(common_duration)} ({common_duration:.1f}s) is less than minimum required {format_duration_display(MIN_OVERLAP_SECONDS)} ({MIN_OVERLAP_SECONDS}s)!\n"
            f"   Synchronization may result in very short videos.\n"
            f"   Consider checking video timestamps and durations."
        )
        print(warning_msg)

    if common_duration < MIN_SYNC_DURATION_SECONDS:
        warning_msg = (
            f"\n⚠️  WARNING: Common duration {format_duration_display(common_duration)} ({common_duration:.1f}s) is less than minimum sync duration {format_duration_display(MIN_SYNC_DURATION_SECONDS)} ({MIN_SYNC_DURATION_SECONDS}s)!\n"
            f"   Synchronized videos will be shorter than recommended."
        )
        print(warning_msg)

    # -------- Step 4: Identify Trim Requirements --------
    print(f"\n4️⃣ Identifying trim requirements for each video...")

    # Determine scenario
    start_times = [v['start_time'] for v in video_timeline]
    durations = [v['duration'] for v in video_timeline]

    start_times_equal = all(abs((t - start_times[0]).total_seconds()) < 1.0 for t in start_times)
    durations_equal = all(abs(d - durations[0]) < 1.0 for d in durations)

    if start_times_equal and durations_equal:
        scenario = "All videos have same start time and duration"
        print(f"   Scenario: All videos synchronized (same start time and duration)")
    elif start_times_equal:
        scenario = "B: Same start time, different durations"
        print(f"   Scenario B: Videos start at same time but have different durations")
        print(f"   Solution: Trim longer videos from the end to match shortest duration")
    elif durations_equal:
        scenario = "A: Different start times, same duration"
        print(f"   Scenario A: Videos start at different times but have same duration")
        print(f"   Solution: Trim early-starting videos from beginning to match latest start")
    else:
        scenario = "C: Different start times and durations"
        print(f"   Scenario C: Videos have both different start times and durations")
        print(f"   Solution: Trim from start to align, then trim from end to match shortest common duration")

    # Calculate trim parameters for each video
    print(f"\n   Trim Parameters:")

    for video_info in video_timeline:
        # Calculate start trim (how much to cut from beginning)
        # Videos that started before sync_start_time need to be trimmed
        start_trim_seconds = max(0, (sync_start_time - video_info['start_time']).total_seconds())

        # Calculate end trim (how much to cut from end)
        # Videos that end after sync_end_time need to be trimmed
        end_trim_seconds = max(0, (video_info['end_time'] - sync_end_time).total_seconds())

        # Calculate final duration after trimming
        final_duration = video_info['duration'] - start_trim_seconds - end_trim_seconds

        trim_params = {
            'video_name': video_info['name'],
            'original_duration': video_info['duration'],
            'start_trim_seconds': start_trim_seconds,
            'end_trim_seconds': end_trim_seconds,
            'final_duration': final_duration,
            'start_time_offset': video_info['offset_seconds'],
            'sync_start_time': sync_start_time.isoformat(),
            'sync_end_time': sync_end_time.isoformat()
        }

        TRIM_PARAMETERS.append(trim_params)

        # Display trim info
        trim_info = []
        if start_trim_seconds > 0:
            trim_info.append(f"start: {format_duration_display(start_trim_seconds)} ({start_trim_seconds:.2f}s)")
        if end_trim_seconds > 0:
            trim_info.append(f"end: {format_duration_display(end_trim_seconds)} ({end_trim_seconds:.2f}s)")
        if not trim_info:
            trim_info.append("none")

        print(f"   {video_info['name']:40s} - Trim: {', '.join(trim_info)} | Final: {format_duration_display(final_duration)} ({final_duration:.2f}s)")

        # Store trim parameters in metadata
        video_info['metadata']['trim_start_seconds'] = start_trim_seconds
        video_info['metadata']['trim_end_seconds'] = end_trim_seconds
        video_info['metadata']['final_duration_seconds'] = final_duration
        video_info['metadata']['sync_start_time'] = sync_start_time.isoformat()
        video_info['metadata']['sync_end_time'] = sync_end_time.isoformat()

    # -------- Step 5: Store Synchronization Strategy --------
    SYNC_STRATEGY = {
        'scenario': scenario,
        'reference_video': earliest_video['name'],
        'reference_start_time': reference_start.isoformat(),
        'sync_start_time': sync_start_time.isoformat(),
        'sync_end_time': sync_end_time.isoformat(),
        'common_duration_seconds': common_duration,
        'common_duration_formatted': f"{common_duration / 60:.2f} minutes",
        'latest_start_offset_seconds': latest_start_offset,
        'total_videos': len(video_timeline),
        'trim_parameters': TRIM_PARAMETERS,
        'validation': {
            'meets_min_overlap': common_duration >= MIN_OVERLAP_SECONDS,
            'meets_min_duration': common_duration >= MIN_SYNC_DURATION_SECONDS,
            'min_overlap_required': MIN_OVERLAP_SECONDS,
            'min_duration_required': MIN_SYNC_DURATION_SECONDS
        }
    }

    print(f"\n" + "=" * 50)
    print("SYNCHRONIZATION STRATEGY SUMMARY")
    print("=" * 50)

    print(f"\n📋 Strategy Details:")
    print(f"   Scenario: {scenario}")
    print(f"   Reference video: {earliest_video['name']}")
    print(f"   Synchronized start: {sync_start_time.isoformat()}")
    print(f"   Synchronized end: {sync_end_time.isoformat()}")
    print(f"   Common duration: {format_duration_display(common_duration)} ({common_duration:.2f} seconds, {common_duration / 60:.2f} minutes)")

    print(f"\n✅ Validation:")
    print(f"   Meets minimum overlap ({format_duration_display(MIN_OVERLAP_SECONDS)} / {MIN_OVERLAP_SECONDS}s): {'✅' if common_duration >= MIN_OVERLAP_SECONDS else '❌'}")
    print(f"   Meets minimum duration ({format_duration_display(MIN_SYNC_DURATION_SECONDS)} / {MIN_SYNC_DURATION_SECONDS}s): {'✅' if common_duration >= MIN_SYNC_DURATION_SECONDS else '❌'}")

    if common_duration >= MIN_OVERLAP_SECONDS and common_duration >= MIN_SYNC_DURATION_SECONDS:
        print(f"\n✅ Synchronization strategy determined successfully!")
        print(f"   Ready to proceed with video trimming.")
    else:
        print(f"\n⚠️  Synchronization strategy determined, but warnings exist.")
        print(f"   Review trim parameters before proceeding.")

else:
    print(f"\n⚠️  Cannot determine synchronization strategy:")
    if len(valid_metadata) < 2:
        print(f"   Need at least 2 videos with valid timestamps and duration")
        print(f"   Found: {len(valid_metadata)} videos with valid metadata")
    else:
        print(f"   Insufficient valid metadata")

    SYNC_STRATEGY = {
        'error': 'Insufficient videos for synchronization',
        'valid_metadata_count': len(valid_metadata),
        'required': 2
    }

# ==============================
# VALIDATE SYNCHRONIZATION FEASIBILITY
# ==============================

print(f"\n" + "=" * 50)
print("VALIDATE SYNCHRONIZATION FEASIBILITY")
print("=" * 50)

SYNC_FEASIBLE = False
SYNC_VALIDATION_RESULTS = {
    'overlap_duration_check': None,
    'time_difference_check': None,
    'metadata_validity_check': None,
    'frame_rate_compatibility_check': None,
    'overall_feasible': False,
    'warnings': [],
    'errors': []
}

if SYNC_STRATEGY and 'error' not in SYNC_STRATEGY:
    print(f"\n🔍 Performing feasibility checks...")

    # -------- Check 1: Common Overlap Duration --------
    print(f"\n1️⃣ Checking common overlap duration...")

    if 'common_duration_seconds' in SYNC_STRATEGY:
        common_duration = SYNC_STRATEGY['common_duration_seconds']
        min_overlap = MIN_OVERLAP_SECONDS
        min_duration = MIN_SYNC_DURATION_SECONDS

        if common_duration >= min_duration:
            print(f"   ✅ Common duration {format_duration_display(common_duration)} ({common_duration:.1f}s) meets minimum requirement {format_duration_display(min_duration)} ({min_duration}s)")
            SYNC_VALIDATION_RESULTS['overlap_duration_check'] = {
                'status': 'pass',
                'common_duration': common_duration,
                'min_required': min_duration,
                'meets_requirement': True
            }
        elif common_duration >= min_overlap:
            print(f"   ⚠️  Common duration {format_duration_display(common_duration)} ({common_duration:.1f}s) meets minimum overlap {format_duration_display(min_overlap)} ({min_overlap}s) but is below recommended duration {format_duration_display(min_duration)} ({min_duration}s)")
            warning_msg = f"Common duration {format_duration_display(common_duration)} ({common_duration:.1f}s) is below recommended minimum {format_duration_display(min_duration)} ({min_duration}s)"
            SYNC_VALIDATION_RESULTS['warnings'].append(warning_msg)
            SYNC_VALIDATION_RESULTS['overlap_duration_check'] = {
                'status': 'warning',
                'common_duration': common_duration,
                'min_overlap': min_overlap,
                'min_duration': min_duration,
                'meets_overlap': True,
                'meets_duration': False
            }
        elif common_duration >= INSUFFICIENT_OVERLAP_WARNING_SECONDS:
            # Below minimum overlap but above warning threshold
            if ALLOW_INSUFFICIENT_OVERLAP:
                warning_msg = (
                    f"Common duration ({common_duration:.1f}s) is below minimum overlap ({min_overlap}s) "
                    f"but above warning threshold ({INSUFFICIENT_OVERLAP_WARNING_SECONDS}s). "
                    f"Proceeding with override."
                )
                print(f"   ⚠️  {warning_msg}")
                SYNC_VALIDATION_RESULTS['warnings'].append(warning_msg)
                SYNC_VALIDATION_RESULTS['overlap_duration_check'] = {
                    'status': 'warning',
                    'common_duration': common_duration,
                    'min_overlap': min_overlap,
                    'min_duration': min_duration,
                    'meets_overlap': False,
                    'meets_duration': False,
                    'override_enabled': True
                }
            else:
                error_msg = (
                    f"Common duration ({common_duration:.1f}s) is insufficient (minimum: {min_overlap}s). "
                    f"Set ALLOW_INSUFFICIENT_OVERLAP=True to proceed anyway."
                )
                print(f"   ❌ {error_msg}")
                SYNC_VALIDATION_RESULTS['errors'].append(error_msg)
                SYNC_VALIDATION_RESULTS['overlap_duration_check'] = {
                    'status': 'fail',
                    'common_duration': common_duration,
                    'min_required': min_overlap,
                    'meets_requirement': False,
                    'requires_override': True
                }
        else:
            # Very low overlap
            error_msg = (
                f"Common duration ({common_duration:.1f}s) is critically low (minimum: {min_overlap}s, "
                f"warning threshold: {INSUFFICIENT_OVERLAP_WARNING_SECONDS}s). "
                f"Synchronization may not be meaningful."
            )
            print(f"   ❌ {error_msg}")
            SYNC_VALIDATION_RESULTS['errors'].append(error_msg)
            SYNC_VALIDATION_RESULTS['overlap_duration_check'] = {
                'status': 'fail',
                'common_duration': common_duration,
                'min_required': min_overlap,
                'meets_requirement': False,
                'critically_low': True
            }
    else:
        error_msg = "Common duration not available in synchronization strategy"
        print(f"   ❌ {error_msg}")
        SYNC_VALIDATION_RESULTS['errors'].append(error_msg)
        SYNC_VALIDATION_RESULTS['overlap_duration_check'] = {
            'status': 'fail',
            'error': error_msg
        }

    # -------- Check 2: Time Differences Reasonableness --------
    print(f"\n2️⃣ Checking time differences...")

    if 'trim_parameters' in SYNC_STRATEGY and SYNC_STRATEGY['trim_parameters']:
        # Get maximum time offset (difference between earliest and latest start)
        max_offset = max(abs(tp['start_time_offset']) for tp in SYNC_STRATEGY['trim_parameters'])
        max_offset_minutes = max_offset / 60
        max_offset_hours = max_offset / 3600

        # Reasonable threshold: 5 minutes (configurable via MAX_TIME_DIFFERENCE_SECONDS)
        if max_offset <= MAX_TIME_DIFFERENCE_SECONDS:
            print(f"   ✅ Maximum time difference ({max_offset:.1f}s / {max_offset_minutes:.1f} min) is within reasonable limit ({MAX_TIME_DIFFERENCE_SECONDS / 60:.1f} min)")
            SYNC_VALIDATION_RESULTS['time_difference_check'] = {
                'status': 'pass',
                'max_offset_seconds': max_offset,
                'max_offset_minutes': max_offset_minutes,
                'threshold_seconds': MAX_TIME_DIFFERENCE_SECONDS,
                'within_limit': True
            }
        else:
            max_offset_hours = max_offset / 3600
            is_very_large = max_offset_hours >= LARGE_TIME_DIFF_THRESHOLD_HOURS

            if is_very_large:
                warning_msg = (
                    f"VERY LARGE time difference detected: {max_offset:.1f}s / {max_offset_hours:.2f} hours "
                    f"(exceeds {LARGE_TIME_DIFF_THRESHOLD_HOURS} hour threshold)."
                )
                print(f"   ⚠️  {warning_msg}")

                if ALLOW_LARGE_TIME_DIFF_OVERRIDE:
                    print(f"   💡 Manual override allowed - proceeding with synchronization")
                    print(f"      (Set ALLOW_LARGE_TIME_DIFF_OVERRIDE=False to require manual confirmation)")
                    SYNC_VALIDATION_RESULTS['warnings'].append(warning_msg + " (override enabled)")
                else:
                    error_msg = (
                        f"Time difference too large ({max_offset_hours:.2f} hours). "
                        f"Set ALLOW_LARGE_TIME_DIFF_OVERRIDE=True to proceed anyway."
                    )
                    print(f"   ❌ {error_msg}")
                    SYNC_VALIDATION_RESULTS['errors'].append(error_msg)
                    SYNC_VALIDATION_RESULTS['time_difference_check'] = {
                        'status': 'fail',
                        'max_offset_seconds': max_offset,
                        'max_offset_hours': max_offset_hours,
                        'threshold_hours': LARGE_TIME_DIFF_THRESHOLD_HOURS,
                        'within_limit': False,
                        'requires_override': True
                    }

            else:
                warning_msg = (
                    f"Maximum time difference ({max_offset:.1f}s / {max_offset_minutes:.1f} min) exceeds recommended limit "
                    f"({MAX_TIME_DIFFERENCE_SECONDS / 60:.1f} min). Consider manual review."
                )
                print(f"   ⚠️  {warning_msg}")
                SYNC_VALIDATION_RESULTS['warnings'].append(warning_msg)

            SYNC_VALIDATION_RESULTS['time_difference_check'] = {
                'status': 'warning',
                'max_offset_seconds': max_offset,
                'max_offset_minutes': max_offset_minutes,
                'max_offset_hours': max_offset_hours,
                'threshold_seconds': MAX_TIME_DIFFERENCE_SECONDS,
                'within_limit': False,
                'recommendation': 'manual_review',
                'is_very_large': is_very_large
            }
    else:
        error_msg = "Time offset information not available"
        print(f"   ❌ {error_msg}")
        SYNC_VALIDATION_RESULTS['errors'].append(error_msg)
        SYNC_VALIDATION_RESULTS['time_difference_check'] = {
            'status': 'fail',
            'error': error_msg
        }

    # -------- Check 3: Metadata Validity --------
    print(f"\n3️⃣ Checking metadata validity...")

    videos_with_valid_metadata = len(valid_metadata)
    total_videos = len(VIDEO_FILES)

    if videos_with_valid_metadata == total_videos:
        print(f"   ✅ All {total_videos} videos have valid metadata")
        SYNC_VALIDATION_RESULTS['metadata_validity_check'] = {
            'status': 'pass',
            'total_videos': total_videos,
            'valid_metadata_count': videos_with_valid_metadata,
            'all_valid': True
        }
    elif videos_with_valid_metadata >= 2:
        warning_msg = f"Only {videos_with_valid_metadata}/{total_videos} videos have valid metadata"
        print(f"   ⚠️  {warning_msg}")
        print(f"      Synchronization will proceed with available videos")
        SYNC_VALIDATION_RESULTS['warnings'].append(warning_msg)
        SYNC_VALIDATION_RESULTS['metadata_validity_check'] = {
            'status': 'warning',
            'total_videos': total_videos,
            'valid_metadata_count': videos_with_valid_metadata,
            'all_valid': False,
            'sufficient_for_sync': True
        }
    else:
        error_msg = f"Insufficient valid metadata: {videos_with_valid_metadata}/{total_videos} videos (need at least 2)"
        print(f"   ❌ {error_msg}")
        SYNC_VALIDATION_RESULTS['errors'].append(error_msg)
        SYNC_VALIDATION_RESULTS['metadata_validity_check'] = {
            'status': 'fail',
            'total_videos': total_videos,
            'valid_metadata_count': videos_with_valid_metadata,
            'all_valid': False,
            'sufficient_for_sync': False
        }

    # Check for missing critical fields
    missing_fields = []
    for metadata in valid_metadata:
        if not metadata.get('creation_time_utc_dt'):
            missing_fields.append(f"{metadata['name']}: missing UTC timestamp")
        if not metadata.get('duration'):
            missing_fields.append(f"{metadata['name']}: missing duration")

    if missing_fields:
        warning_msg = f"Some videos missing critical metadata fields: {', '.join(missing_fields)}"
        print(f"   ⚠️  {warning_msg}")
        SYNC_VALIDATION_RESULTS['warnings'].append(warning_msg)

    # -------- Check 4: Frame Rate Compatibility --------
    print(f"\n4️⃣ Checking frame rate compatibility...")

    fps_values = [m['fps'] for m in valid_metadata if m.get('fps') is not None]

    if len(fps_values) >= 2:
        min_fps = min(fps_values)
        max_fps = max(fps_values)
        fps_diff = abs(max_fps - min_fps)
        fps_diff_percent = (fps_diff / min_fps) * 100 if min_fps > 0 else 0

        # Consider frame rates compatible if difference is < 5% or < 1 fps
        fps_compatible = fps_diff_percent < 5.0 or fps_diff < 1.0

        if fps_compatible:
            print(f"   ✅ Frame rates are compatible")
            print(f"      Range: {min_fps:.2f} - {max_fps:.2f} fps (diff: {fps_diff:.2f} fps / {fps_diff_percent:.1f}%)")
            SYNC_VALIDATION_RESULTS['frame_rate_compatibility_check'] = {
                'status': 'pass',
                'min_fps': min_fps,
                'max_fps': max_fps,
                'fps_difference': fps_diff,
                'fps_difference_percent': fps_diff_percent,
                'compatible': True
            }
        else:
            warning_msg = (
                f"Frame rates differ significantly: {min_fps:.2f} - {max_fps:.2f} fps "
                f"(diff: {fps_diff:.2f} fps / {fps_diff_percent:.1f}%). "
                f"This may cause synchronization issues or require frame rate conversion."
            )
            print(f"   ⚠️  {warning_msg}")
            SYNC_VALIDATION_RESULTS['warnings'].append(warning_msg)
            SYNC_VALIDATION_RESULTS['frame_rate_compatibility_check'] = {
                'status': 'warning',
                'min_fps': min_fps,
                'max_fps': max_fps,
                'fps_difference': fps_diff,
                'fps_difference_percent': fps_diff_percent,
                'compatible': False,
                'recommendation': 'consider_frame_rate_conversion'
            }

        # Show individual frame rates
        print(f"\n      Individual frame rates:")
        for metadata in valid_metadata:
            fps = metadata.get('fps')
            if fps is not None:
                print(f"         {metadata['name']:40s} - {fps:.2f} fps")
            else:
                print(f"         {metadata['name']:40s} - fps unknown")
    elif len(fps_values) == 1:
        warning_msg = "Only 1 video has frame rate information - cannot compare compatibility"
        print(f"   ⚠️  {warning_msg}")
        SYNC_VALIDATION_RESULTS['warnings'].append(warning_msg)
        SYNC_VALIDATION_RESULTS['frame_rate_compatibility_check'] = {
            'status': 'warning',
            'fps_values_count': 1,
            'cannot_compare': True
        }
    else:
        warning_msg = "No frame rate information available - cannot verify compatibility"
        print(f"   ⚠️  {warning_msg}")
        SYNC_VALIDATION_RESULTS['warnings'].append(warning_msg)
        SYNC_VALIDATION_RESULTS['frame_rate_compatibility_check'] = {
            'status': 'warning',
            'fps_values_count': 0,
            'no_data': True
        }

    # -------- Overall Feasibility Assessment --------
    print(f"\n" + "=" * 50)
    print("FEASIBILITY ASSESSMENT")
    print("=" * 50)

    # Determine overall feasibility
    has_errors = len(SYNC_VALIDATION_RESULTS['errors']) > 0
    has_critical_warnings = (
        SYNC_VALIDATION_RESULTS.get('overlap_duration_check', {}).get('status') == 'fail' or
        SYNC_VALIDATION_RESULTS.get('metadata_validity_check', {}).get('sufficient_for_sync') == False
    )

    if has_errors or has_critical_warnings:
        SYNC_FEASIBLE = False
        print(f"\n❌ Synchronization is NOT feasible")
        if has_errors:
            print(f"\n   Errors found:")
            for error in SYNC_VALIDATION_RESULTS['errors']:
                print(f"      - {error}")
        if has_critical_warnings:
            print(f"\n   Critical warnings:")
            for warning in SYNC_VALIDATION_RESULTS['warnings']:
                if 'insufficient' in warning.lower() or 'missing' in warning.lower():
                    print(f"      - {warning}")
    else:
        SYNC_FEASIBLE = True
        print(f"\n✅ Synchronization is FEASIBLE")

        if len(SYNC_VALIDATION_RESULTS['warnings']) > 0:
            print(f"\n   Warnings (non-critical):")
            for warning in SYNC_VALIDATION_RESULTS['warnings']:
                print(f"      ⚠️  {warning}")
            print(f"\n   Note: Synchronization can proceed, but review warnings above.")
        else:
            print(f"\n   All checks passed - ready to proceed with synchronization!")

    SYNC_VALIDATION_RESULTS['overall_feasible'] = SYNC_FEASIBLE

    # Update sync strategy with validation results
    if SYNC_STRATEGY:
        SYNC_STRATEGY['validation_results'] = SYNC_VALIDATION_RESULTS
        SYNC_STRATEGY['feasible'] = SYNC_FEASIBLE

else:
    print(f"\n⚠️  Cannot validate feasibility - synchronization strategy not available")
    if SYNC_STRATEGY and 'error' in SYNC_STRATEGY:
        print(f"   Reason: {SYNC_STRATEGY['error']}")
    SYNC_VALIDATION_RESULTS['overall_feasible'] = False
    SYNC_VALIDATION_RESULTS['errors'].append("Synchronization strategy not available")

# -------- Check if Videos are Already Synchronized --------
print(f"\n🔍 Checking timestamp alignment (using normalized UTC timestamps)...")

if len(valid_metadata) >= 2:
    # Get all normalized UTC timestamps
    timestamps_utc = [m['creation_time_utc_dt'] for m in valid_metadata if m.get('creation_time_utc_dt')]

    if len(timestamps_utc) >= 2:
        # Check if all timestamps are the same (within 1 second tolerance)
        timestamps_sorted = sorted(timestamps_utc)
        time_diffs = []

        for i in range(len(timestamps_sorted) - 1):
            diff = abs((timestamps_sorted[i+1] - timestamps_sorted[i]).total_seconds())
            time_diffs.append(diff)

        max_diff = max(time_diffs) if time_diffs else 0

        print(f"   UTC timestamp differences: {[f'{d:.1f}s' for d in time_diffs]}")
        print(f"   Maximum difference: {max_diff:.1f} seconds")

        # If all timestamps are within 1 second, consider them synchronized
        if max_diff <= 1.0:
            print(f"\n✅ All videos have the same timestamp (within 1 second tolerance)")
            print(f"   Videos appear to be already synchronized!")

            # Check if durations are also similar (within 5 seconds)
            durations = [m['duration'] for m in valid_metadata if m.get('duration')]
            if durations:
                max_duration = max(durations)
                min_duration = min(durations)
                duration_diff = max_duration - min_duration

                if duration_diff <= 5.0:
                    print(f"   Durations are also similar (diff: {duration_diff:.1f}s)")
                    print(f"\n💡 Skipping video synchronization step.")
                    print(f"   Videos will be used directly from Videos directory.")
                    SKIP_SYNC = True
                else:
                    print(f"   Durations differ significantly (diff: {duration_diff:.1f}s)")
                    print(f"   Synchronization strategy will trim videos to common duration.")
                    SKIP_SYNC = False
            else:
                SKIP_SYNC = False

            if SKIP_SYNC:
                # Copy videos to input directory (or create symlinks/references)
                print(f"\n📋 Preparing videos for camera switching...")
                print(f"   Note: Videos will be referenced from: {VIDEOS_DIR}")
                print(f"   Input directory: {INPUT_DIR_SYNC}")

        else:
            print(f"\n⚠️  Videos have different timestamps (max diff: {max_diff:.1f}s)")
            if SYNC_STRATEGY and 'common_duration_seconds' in SYNC_STRATEGY:
                print(f"   Synchronization strategy determined: {SYNC_STRATEGY['scenario']}")
                print(f"   Common duration: {format_duration_display(SYNC_STRATEGY['common_duration_seconds'])} ({SYNC_STRATEGY['common_duration_seconds']:.1f}s)")

            # Check feasibility before proceeding
            if 'SYNC_FEASIBLE' in globals() and SYNC_FEASIBLE:
                print(f"   ✅ Synchronization is feasible - will proceed")
                SKIP_SYNC = False
            elif 'SYNC_FEASIBLE' in globals() and not SYNC_FEASIBLE:
                print(f"   ❌ Synchronization is NOT feasible - check validation errors above")
                SKIP_SYNC = True  # Skip sync if not feasible
            else:
                print(f"   Synchronization will be attempted.")
                SKIP_SYNC = False
    else:
        print(f"   ⚠️  Not enough timestamps to check alignment")
        SKIP_SYNC = False
else:
    print(f"   ⚠️  Not enough valid metadata to check alignment")
    SKIP_SYNC = False

# ==============================
# GENERATE TRIMMED/SYNCHRONIZED VIDEOS
# ==============================

SYNCED_VIDEOS = []
TRIMMING_RESULTS = []

if not SKIP_SYNC and SYNC_FEASIBLE and SYNC_STRATEGY and 'trim_parameters' in SYNC_STRATEGY:
    print(f"\n" + "=" * 50)
    print("GENERATE TRIMMED/SYNCHRONIZED VIDEOS")
    print("=" * 50)

    # -------- Integrate trim_video.py functionality --------

    def find_ffmpeg(ffmpeg_path: Optional[str] = None) -> str:
        """Find FFmpeg executable path."""
        if ffmpeg_path:
            custom_path = Path(ffmpeg_path)
            if custom_path.exists() and custom_path.is_file():
                return str(custom_path.resolve())
            raise FileNotFoundError(f"FFmpeg not found at specified path: {ffmpeg_path}")

        ffmpeg_exe = shutil.which('ffmpeg')
        if ffmpeg_exe:
            return ffmpeg_exe

        if sys.platform == 'win32':
            common_paths = [
                Path('C:/ffmpeg/bin/ffmpeg.exe'),
                Path('C:/Program Files/ffmpeg/bin/ffmpeg.exe'),
                Path('C:/Program Files (x86)/ffmpeg/bin/ffmpeg.exe'),
                Path.home() / 'ffmpeg/bin/ffmpeg.exe',
                Path('C:/tools/ffmpeg/bin/ffmpeg.exe'),
            ]

            for path in common_paths:
                if path.exists() and path.is_file():
                    return str(path.resolve())

        raise FileNotFoundError(
            "FFmpeg not found. Please ensure FFmpeg is installed and available in PATH.\n"
            "Download FFmpeg from: https://ffmpeg.org/download.html"
        )

    def format_time_for_ffmpeg(seconds: float) -> str:
        """Format seconds to HH:MM:SS.mmm format for FFmpeg."""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = seconds % 60
        return f"{hours:02d}:{minutes:02d}:{secs:06.3f}"

    def trim_video_sync(
        input_file: Path,
        output_file: Path,
        start_trim_seconds: float,
        duration_seconds: float,
        use_lossless: bool = True,
        overwrite: bool = False
    ) -> bool:
        """
        Trim video using FFmpeg (integrated from trim_video.py).

        Args:
            input_file: Path to input video
            output_file: Path to output video
            start_trim_seconds: Seconds to skip from start
            duration_seconds: Duration to keep
            use_lossless: Use stream copy (fast, lossless) vs re-encoding
            overwrite: Overwrite output if exists

        Returns:
            True if successful, False otherwise
        """
        try:
                        # Validate input
            if not input_file.exists():
                raise FileNotFoundError(f"Input video not found: {input_file}")

            # Stricter validation: Get video duration and validate trim parameters
            try:
                # Get video metadata to validate trim parameters
                ffprobe_data = run_ffprobe(str(input_file))
                video_duration = None
                if 'format' in ffprobe_data and 'duration' in ffprobe_data['format']:
                    video_duration = float(ffprobe_data['format']['duration'])

                if video_duration is not None:
                    # Validate start_trim_seconds
                    if start_trim_seconds < 0:
                        raise ValueError(f"Start trim cannot be negative: {start_trim_seconds}")
                    if start_trim_seconds >= video_duration:
                        raise ValueError(
                            f"Start trim ({start_trim_seconds:.2f}s) must be less than video duration "
                            f"({video_duration:.2f}s)"
                        )

                    # Validate duration_seconds
                    if duration_seconds <= 0:
                        raise ValueError(f"Duration must be positive: {duration_seconds}")
                    if duration_seconds > video_duration:
                        raise ValueError(
                            f"Duration ({duration_seconds:.2f}s) cannot exceed video duration "
                            f"({video_duration:.2f}s)"
                        )

                    # Validate that start_trim + duration doesn't exceed video duration
                    end_time = start_trim_seconds + duration_seconds
                    if end_time > video_duration:
                        raise ValueError(
                            f"Trim range exceeds video: start ({start_trim_seconds:.2f}s) + duration "
                            f"({duration_seconds:.2f}s) = {end_time:.2f}s > video duration "
                            f"({video_duration:.2f}s)"
                        )

                    # Additional strict check: ensure minimum duration
                    if duration_seconds < 1.0:
                        raise ValueError(f"Duration too short: {duration_seconds:.2f}s (minimum: 1.0s)")
            except ValueError:
                # Re-raise validation errors
                raise
            except Exception as e:
                # If ffprobe fails, log warning but continue (less strict)
                if 'ENABLE_ORCHESTRATOR_LOGGING' in globals() and ENABLE_ORCHESTRATOR_LOGGING:
                    print(f"      ⚠️  Warning: Could not validate trim parameters (ffprobe failed): {e}")

            # Create output directory if needed
            output_file.parent.mkdir(parents=True, exist_ok=True)

            # Check if output exists
            if output_file.exists() and not overwrite:
                if SKIP_IF_SYNCED_EXISTS:
                    print(f"      ⏭️  Skipping (already exists): {output_file.name}")
                    return True
                else:
                    raise FileExistsError(f"Output file exists: {output_file}")

            # Find FFmpeg
            ffmpeg_exe = find_ffmpeg()

            # Format times for FFmpeg
            start_time_str = format_time_for_ffmpeg(start_trim_seconds)
            duration_str = format_time_for_ffmpeg(duration_seconds)

            # Build FFmpeg command
            cmd = [
                ffmpeg_exe,
                '-i', str(input_file.resolve()),
                '-ss', start_time_str,
                '-t', duration_str,
            ]

            # Add codec options
            if use_lossless:
                cmd.extend(['-c', 'copy'])  # Stream copy - fast, lossless
            else:
                cmd.extend([
                    '-c:v', 'libx264',
                    '-preset', 'medium',
                    '-crf', '18',
                    '-c:a', 'aac',
                    '-b:a', '192k',
                ])

            # Overwrite flag
            cmd.append('-y' if overwrite else '-n')

            # Output file
            cmd.append(str(output_file.resolve()))

            # Run FFmpeg
            creation_flags = subprocess.CREATE_NO_WINDOW if sys.platform == 'win32' else 0
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                check=True,
                creationflags=creation_flags
            )

            return True

        except FileNotFoundError as e:
            raise FileNotFoundError(f"FFmpeg or input file not found: {e}")
        except subprocess.CalledProcessError as e:
            error_msg = e.stderr if e.stderr else e.stdout if e.stdout else "Unknown error"
            raise RuntimeError(f"FFmpeg execution failed: {error_msg}")
        except Exception as e:
            raise RuntimeError(f"Trimming failed: {e}")

    # -------- Process Videos --------
    print(f"\n🎬 Starting video trimming process...")
    print(f"   Mode: {'Lossless (stream copy)' if USE_LOSSLESS_TRIMMING else 'Re-encode (frame-accurate)'}")
    print(f"   Output directory: {INPUT_DIR_SYNC}")
    print(f"   Output format: {SYNC_OUTPUT_FORMAT}")

    trim_params = SYNC_STRATEGY['trim_parameters']
    total_videos = len(trim_params)

    print(f"\n   Processing {total_videos} video(s)...")

    for i, trim_param in enumerate(trim_params, 1):
        video_name = trim_param['video_name']
        start_trim = trim_param['start_trim_seconds']
        end_trim = trim_param['end_trim_seconds']
        final_duration = trim_param['final_duration']

        print(f"\n   [{i}/{total_videos}] Processing: {video_name}")
        print(f"      Start trim: {format_duration_display(start_trim)} ({start_trim:.2f}s)")
        print(f"      End trim: {format_duration_display(end_trim)} ({end_trim:.2f}s)")
        print(f"      Final duration: {format_duration_display(final_duration)} ({final_duration:.2f}s)")

        # Find input video file
        input_video = None
        for video_file in VIDEO_FILES:
            if video_file.name == video_name:
                input_video = video_file
                break

        if not input_video:
            error_msg = f"Input video not found: {video_name}"
            print(f"      ❌ {error_msg}")
            TRIMMING_RESULTS.append({
                'video_name': video_name,
                'success': False,
                'error': error_msg
            })
            continue

        # Generate output filename
        if SYNC_OUTPUT_FORMAT == "same":
            # Keep original extension
            output_ext = input_video.suffix
        else:
            output_ext = f".{SYNC_OUTPUT_FORMAT}"

        # Create output filename: original_name_synced.ext
        output_name = f"{input_video.stem}_synced{output_ext}"
        output_path = INPUT_DIR_SYNC / output_name

        # Enhanced skip check: Compare timestamps if enabled
        should_skip = False
        skip_reason = None

        if output_path.exists() and SKIP_IF_SYNCED_EXISTS:
            if ENHANCED_SKIP_CHECK:
                # Compare timestamps of synced video with original
                try:
                    # Get original video metadata
                    original_metadata = next((m for m in VIDEO_METADATA if m['name'] == video_name), None)

                    if original_metadata and original_metadata.get('creation_time_utc_dt'):
                        # Get synced video metadata
                        synced_ffprobe = run_ffprobe(str(output_path))
                        synced_tags = synced_ffprobe.get('format', {}).get('tags', {})
                        synced_timestamp_str = (
                            synced_tags.get('com.apple.quicktime.creationdate') or
                            synced_tags.get('creation_time') or
                            synced_tags.get('date')
                        )

                        if synced_timestamp_str:
                            synced_timestamp = parse_timestamp(synced_timestamp_str)
                            if synced_timestamp:
                                synced_timestamp_utc = normalize_timestamp_to_utc(synced_timestamp, output_path)
                                original_timestamp_utc = original_metadata.get('creation_time_utc_dt')

                                if synced_timestamp_utc and original_timestamp_utc:
                                    time_diff = abs((synced_timestamp_utc - original_timestamp_utc).total_seconds())
                                    if time_diff <= 1.0:  # Timestamps match within 1 second
                                        should_skip = True
                                        skip_reason = f"timestamp match (diff: {time_diff:.2f}s)"
                                    else:
                                        skip_reason = f"timestamp mismatch (diff: {time_diff:.2f}s) - will re-sync"
                except Exception as e:
                    # If enhanced check fails, fall back to simple existence check
                    should_skip = True
                    skip_reason = "exists (enhanced check failed)"
            else:
                # Simple existence check
                should_skip = True
                skip_reason = "already exists"

        if should_skip:
            print(f"      ⏭️  Skipping: {skip_reason}")
            TRIMMING_RESULTS.append({
                'video_name': video_name,
                'success': True,
                'skipped': True,
                'skip_reason': skip_reason,
                'output_path': str(output_path),
                'output_name': output_name
            })
            SYNCED_VIDEOS.append({
                'original_name': video_name,
                'synced_name': output_name,
                'path': str(output_path),
                'start_trim': start_trim,
                'end_trim': end_trim,
                'final_duration': final_duration
            })
            continue

        # -------- Save Metadata BEFORE Trimming --------
        try:
            print(f"      💾 Saving metadata BEFORE trimming...")

            # Extract metadata from original video
            before_metadata = extract_video_metadata_sync(input_video)

            # Check if extraction was successful
            if before_metadata.get('error'):
                print(f"         ⚠️  Warning: Metadata extraction had error: {before_metadata.get('error')}")

            # Create debug directory if it doesn't exist
            if 'DEBUG_DIR_SYNC' not in globals():
                DEBUG_DIR_SYNC = PROJECT_ROOT_DRIVE / "debug"
            DEBUG_DIR_SYNC.mkdir(parents=True, exist_ok=True)

            # Create metadata filename
            before_metadata_file = DEBUG_DIR_SYNC / f"{input_video.stem}_before_trim_metadata.json"

            # Serialize metadata (convert datetime to ISO string for JSON)
            before_metadata_serialized = {
                'path': before_metadata.get('path'),
                'name': before_metadata.get('name'),
                'creation_time': before_metadata.get('creation_time'),
                'creation_time_dt': before_metadata.get('creation_time_dt').isoformat() if before_metadata.get('creation_time_dt') else None,
                'duration': before_metadata.get('duration'),
                'fps': before_metadata.get('fps'),
                'timestamp_source': before_metadata.get('timestamp_source'),
                'file_size_bytes': before_metadata.get('file_size_bytes'),
                'file_size_mb': (before_metadata.get('file_size_bytes') / (1024 * 1024)) if before_metadata.get('file_size_bytes') else None,
                'trim_parameters': {
                    'start_trim_seconds': start_trim,
                    'end_trim_seconds': end_trim,
                    'final_duration_seconds': final_duration,
                    'start_trim_formatted': format_duration_display(start_trim),
                    'end_trim_formatted': format_duration_display(end_trim),
                    'final_duration_formatted': format_duration_display(final_duration)
                },
                'extraction_timestamp': datetime.now().isoformat(),
                'error': before_metadata.get('error'),
                'error_type': before_metadata.get('error_type')
            }

            # Save to JSON file
            with open(before_metadata_file, 'w', encoding='utf-8') as f:
                json.dump(before_metadata_serialized, f, indent=2, ensure_ascii=False)

            print(f"         ✅ Saved: {before_metadata_file.name}")

            # Display key metadata info
            if before_metadata.get('duration'):
                print(f"         Duration: {format_duration_display(before_metadata['duration'])} ({before_metadata['duration']:.2f}s)")
            if before_metadata.get('fps'):
                print(f"         FPS: {before_metadata['fps']:.2f}")
            if before_metadata.get('creation_time'):
                print(f"         Timestamp: {before_metadata['creation_time']}")

        except Exception as e:
            print(f"         ⚠️  Warning: Could not save before-trim metadata: {e}")
            # Continue with trimming even if metadata save fails

        # -------- Stricter Pre-Trimming Validation --------
        # Validate trim parameters against actual video metadata
        try:
            # Get video metadata for validation
            video_metadata = next((m for m in VIDEO_METADATA if m['name'] == video_name), None)

            if video_metadata and video_metadata.get('duration'):
                original_duration = video_metadata['duration']

                # Strict validation checks
                validation_errors = []

                # Check 1: Start trim must be non-negative
                if start_trim < 0:
                    validation_errors.append(f"Start trim is negative: {start_trim:.2f}s")

                # Check 2: Start trim must be less than video duration
                if start_trim >= original_duration:
                    validation_errors.append(
                        f"Start trim ({start_trim:.2f}s) >= video duration ({original_duration:.2f}s)"
                    )

                # Check 3: Final duration must be positive
                if final_duration <= 0:
                    validation_errors.append(f"Final duration is not positive: {final_duration:.2f}s")

                # Check 4: Final duration must be <= original duration
                if final_duration > original_duration:
                    validation_errors.append(
                        f"Final duration ({final_duration:.2f}s) > original duration ({original_duration:.2f}s)"
                    )

                # Check 5: Start trim + final duration must not exceed original duration
                calculated_end = start_trim + final_duration
                if calculated_end > original_duration:
                    validation_errors.append(
                        f"Trim range exceeds video: {start_trim:.2f}s + {final_duration:.2f}s = "
                        f"{calculated_end:.2f}s > {original_duration:.2f}s"
                    )

                # Check 6: Minimum duration requirement
                if final_duration < MIN_SYNC_DURATION_SECONDS:
                    validation_errors.append(
                        f"Final duration ({final_duration:.2f}s) < minimum required "
                        f"({MIN_SYNC_DURATION_SECONDS}s)"
                    )

                # Check 7: End trim validation (start + duration + end should equal original)
                expected_total = start_trim + final_duration + end_trim
                total_diff = abs(expected_total - original_duration)
                if total_diff > 0.1:  # Allow 0.1s tolerance for floating point
                    validation_errors.append(
                        f"Trim calculation mismatch: start ({start_trim:.2f}s) + duration "
                        f"({final_duration:.2f}s) + end ({end_trim:.2f}s) = {expected_total:.2f}s, "
                        f"expected {original_duration:.2f}s (diff: {total_diff:.2f}s)"
                    )

                if validation_errors:
                    error_msg = "Strict validation failed:\n" + "\n".join(f"   - {e}" for e in validation_errors)
                    print(f"      ❌ {error_msg}")
                    raise ValueError(error_msg)
                else:
                    print(f"      ✅ Strict validation passed")

        except ValueError as e:
            # Re-raise validation errors
            raise
        except Exception as e:
            print(f"      ⚠️  Warning: Pre-trimming validation check failed: {e}")
            # Continue anyway (less strict mode)

        # Perform trimming
        try:
            print(f"      🔄 Trimming...")
            start_time = datetime.now()

            success = trim_video_sync(
                input_file=input_video,
                output_file=output_path,
                start_trim_seconds=start_trim,
                duration_seconds=final_duration,
                use_lossless=USE_LOSSLESS_TRIMMING,
                overwrite=not SKIP_IF_SYNCED_EXISTS
            )

            if success:
                elapsed = (datetime.now() - start_time).total_seconds()

                # Get output file size
                if output_path.exists():
                    output_size = output_path.stat().st_size
                    output_size_mb = output_size / (1024 * 1024)
                    print(f"      ✅ Success! ({elapsed:.1f}s)")
                    print(f"         Output: {output_name}")
                    print(f"         Size: {output_size_mb:.2f} MB")

                    # -------- Save Metadata AFTER Trimming --------
                    try:
                        print(f"      💾 Saving metadata AFTER trimming...")

                        # Extract metadata from trimmed video
                        after_metadata = extract_video_metadata_sync(output_path)

                        # Check if extraction was successful
                        if after_metadata.get('error'):
                            print(f"         ⚠️  Warning: Metadata extraction had error: {after_metadata.get('error')}")

                        # Ensure debug directory exists
                        if 'DEBUG_DIR_SYNC' not in globals():
                            DEBUG_DIR_SYNC = PROJECT_ROOT_DRIVE / "debug"
                        DEBUG_DIR_SYNC.mkdir(parents=True, exist_ok=True)

                        # Create metadata filename
                        after_metadata_file = DEBUG_DIR_SYNC / f"{input_video.stem}_after_trim_metadata.json"

                        # Serialize metadata
                        after_metadata_serialized = {
                            'path': after_metadata.get('path'),
                            'name': after_metadata.get('name'),
                            'creation_time': after_metadata.get('creation_time'),
                            'creation_time_dt': after_metadata.get('creation_time_dt').isoformat() if after_metadata.get('creation_time_dt') else None,
                            'duration': after_metadata.get('duration'),
                            'fps': after_metadata.get('fps'),
                            'timestamp_source': after_metadata.get('timestamp_source'),
                            'file_size_bytes': output_size,
                            'file_size_mb': output_size_mb,
                            'trim_parameters': {
                                'start_trim_seconds': start_trim,
                                'end_trim_seconds': end_trim,
                                'final_duration_seconds': final_duration,
                                'expected_duration': final_duration,
                                'start_trim_formatted': format_duration_display(start_trim),
                                'end_trim_formatted': format_duration_display(end_trim),
                                'final_duration_formatted': format_duration_display(final_duration)
                            },
                            'processing_info': {
                                'processing_time_seconds': elapsed,
                                'trim_mode': 'lossless' if USE_LOSSLESS_TRIMMING else 're-encode'
                            },
                            'extraction_timestamp': datetime.now().isoformat(),
                            'error': after_metadata.get('error'),
                            'error_type': after_metadata.get('error_type')
                        }

                        # Save to JSON file
                        with open(after_metadata_file, 'w', encoding='utf-8') as f:
                            json.dump(after_metadata_serialized, f, indent=2, ensure_ascii=False)

                        print(f"         ✅ Saved: {after_metadata_file.name}")

                        # Verify duration matches expected
                        if after_metadata.get('duration'):
                            actual_duration = after_metadata['duration']
                            duration_diff = abs(actual_duration - final_duration)

                            print(f"         Duration: {format_duration_display(actual_duration)} ({actual_duration:.2f}s)")

                            if duration_diff > 0.5:  # More than 0.5s difference
                                print(f"         ⚠️  Duration mismatch: expected {format_duration_display(final_duration)}, "
                                      f"got {format_duration_display(actual_duration)} (diff: {duration_diff:.2f}s)")
                            else:
                                print(f"         ✅ Duration verified: {format_duration_display(actual_duration)} "
                                      f"(expected: {format_duration_display(final_duration)}, diff: {duration_diff:.2f}s)")
                        else:
                            print(f"         ⚠️  Warning: Could not extract duration from trimmed video")

                        # Display other metadata
                        if after_metadata.get('fps'):
                            print(f"         FPS: {after_metadata['fps']:.2f}")
                        if after_metadata.get('creation_time'):
                            print(f"         Timestamp: {after_metadata['creation_time']}")

                        # Try to compare with before metadata if available
                        try:
                            before_metadata_file = DEBUG_DIR_SYNC / f"{input_video.stem}_before_trim_metadata.json"
                            if before_metadata_file.exists():
                                with open(before_metadata_file, 'r', encoding='utf-8') as f:
                                    before_metadata_loaded = json.load(f)

                                # Reconstruct before_metadata dict for comparison
                                before_metadata_for_compare = {
                                    'duration': before_metadata_loaded.get('duration'),
                                    'fps': before_metadata_loaded.get('fps'),
                                    'file_size_bytes': before_metadata_loaded.get('file_size_bytes'),
                                    'creation_time_dt': before_metadata_loaded.get('creation_time_dt'),
                                    'error': before_metadata_loaded.get('error')
                                }

                                # Perform comparison
                                comparison = compare_metadata(
                                    before_metadata_for_compare,
                                    after_metadata,
                                    expected_duration=final_duration,
                                    tolerance_seconds=0.5
                                )

                                # Report comparison results
                                if comparison.get('valid'):
                                    if comparison.get('warnings'):
                                        print(f"         ⚠️  Comparison warnings:")
                                        for warning in comparison['warnings']:
                                            print(f"            - {warning}")
                                    else:
                                        print(f"         ✅ Metadata comparison passed")
                                else:
                                    print(f"         ❌ Metadata comparison failed:")
                                    for error in comparison.get('errors', []):
                                        print(f"            - {error}")
                        except Exception as e:
                            # Comparison is optional - don't fail if it doesn't work
                            pass

                    except Exception as e:
                        print(f"         ⚠️  Warning: Could not save after-trim metadata: {e}")
                        # Continue - metadata save failure shouldn't stop the process

                    TRIMMING_RESULTS.append({
                        'video_name': video_name,
                        'success': True,
                        'output_path': str(output_path),
                        'output_name': output_name,
                        'output_size_mb': output_size_mb,
                        'processing_time_seconds': elapsed,
                        'start_trim': start_trim,
                        'end_trim': end_trim,
                        'final_duration': final_duration
                    })

                    SYNCED_VIDEOS.append({
                        'original_name': video_name,
                        'synced_name': output_name,
                        'path': str(output_path),
                        'start_trim': start_trim,
                        'end_trim': end_trim,
                        'final_duration': final_duration,
                        'size_mb': output_size_mb
                    })
                else:
                    error_msg = "Output file not created"
                    print(f"      ❌ {error_msg}")
                    TRIMMING_RESULTS.append({
                        'video_name': video_name,
                        'success': False,
                        'error': error_msg
                    })

        except Exception as e:
            error_msg = str(e)
            print(f"      ❌ Failed: {error_msg}")
            TRIMMING_RESULTS.append({
                'video_name': video_name,
                'success': False,
                'error': error_msg
            })
            # Continue with next video
            continue

    # -------- Summary --------
    print(f"\n" + "=" * 50)
    print("TRIMMING SUMMARY")
    print("=" * 50)

    successful = len([r for r in TRIMMING_RESULTS if r.get('success')])
    failed = len([r for r in TRIMMING_RESULTS if not r.get('success')])
    skipped = len([r for r in TRIMMING_RESULTS if r.get('skipped')])

    print(f"\n📊 Results:")
    print(f"   Successful: {successful}/{total_videos}")
    if skipped > 0:
        print(f"   Skipped (already exists): {skipped}/{total_videos}")
    if failed > 0:
        print(f"   Failed: {failed}/{total_videos}")
        print(f"\n   Failed videos:")
        for result in TRIMMING_RESULTS:
            if not result.get('success'):
                print(f"      - {result['video_name']}: {result.get('error', 'Unknown error')}")

    if successful > 0:
        total_size = sum(v.get('size_mb', 0) for v in SYNCED_VIDEOS if 'size_mb' in v)
        print(f"\n   Total output size: {total_size:.2f} MB")
        print(f"   Output directory: {INPUT_DIR_SYNC}")

        print(f"\n   Synced videos:")
        for video in SYNCED_VIDEOS:
            print(f"      ✅ {video['synced_name']}")

    if successful == total_videos:
        print(f"\n✅ All videos trimmed successfully!")
        print(f"   Ready for camera switching pipeline.")
    elif successful > 0:
        print(f"\n⚠️  Some videos failed to trim. Review errors above.")
    else:
        print(f"\n❌ No videos were trimmed successfully.")
        print(f"   Check errors above and verify FFmpeg installation.")

elif SKIP_SYNC:
    print(f"\n" + "=" * 50)
    print("VIDEO SYNCHRONIZATION")
    print("=" * 50)
    print(f"\n💡 Skipping video trimming - videos are already synchronized")
    print(f"   Videos will be used directly from: {VIDEOS_DIR}")

    # Create references to original videos
    for video_file in VIDEO_FILES:
        SYNCED_VIDEOS.append({
            'original_name': video_file.name,
            'synced_name': video_file.name,
            'path': str(video_file),
            'start_trim': 0,
            'end_trim': 0,
            'final_duration': None
        })

elif not SYNC_FEASIBLE:
    print(f"\n" + "=" * 50)
    print("VIDEO SYNCHRONIZATION")
    print("=" * 50)
    print(f"\n❌ Cannot proceed with video trimming - synchronization is not feasible")
    print(f"   Review validation errors above.")

else:
    print(f"\n" + "=" * 50)
    print("VIDEO SYNCHRONIZATION")
    print("=" * 50)
    print(f"\n⚠️  Cannot trim videos - synchronization strategy not available")

# ==============================
# VERIFY SYNCHRONIZED VIDEOS
# ==============================

VERIFICATION_RESULTS = []

if 'SYNCED_VIDEOS' in globals() and SYNCED_VIDEOS and not SKIP_SYNC:
    print(f"\n" + "=" * 50)
    print("VERIFY SYNCHRONIZED VIDEOS")
    print("=" * 50)

    print(f"\n🔍 Verifying synchronized videos...")
    print(f"   Checking {len(SYNCED_VIDEOS)} video(s)...")

    # Minimum file size threshold (1 MB - videos smaller than this are suspicious)
    MIN_FILE_SIZE_MB = 1.0
    MIN_FILE_SIZE_BYTES = MIN_FILE_SIZE_MB * 1024 * 1024

    # Duration tolerance (videos should match within 1 second)
    DURATION_TOLERANCE_SECONDS = 1.0

    verified_count = 0
    failed_count = 0
    warning_count = 0

    synced_durations = []

    for i, synced_video in enumerate(SYNCED_VIDEOS, 1):
        video_name = synced_video.get('synced_name', synced_video.get('original_name', 'unknown'))
        video_path = Path(synced_video.get('path', ''))
        expected_duration = synced_video.get('final_duration')

        verification_result = {
            'video_name': video_name,
            'video_path': str(video_path),
            'file_exists': False,
            'file_size_valid': False,
            'duration_verified': False,
            'warnings': [],
            'errors': []
        }

        print(f"\n   [{i}/{len(SYNCED_VIDEOS)}] Verifying: {video_name}")

        # -------- Check 1: File Existence --------
        if not video_path.exists():
            error_msg = f"File not found: {video_path}"
            print(f"      ❌ {error_msg}")
            verification_result['errors'].append(error_msg)
            failed_count += 1
            VERIFICATION_RESULTS.append(verification_result)
            continue

        verification_result['file_exists'] = True
        print(f"      ✅ File exists")

        # -------- Check 2: File Size --------
        try:
            file_size_bytes = video_path.stat().st_size
            file_size_mb = file_size_bytes / (1024 * 1024)
            verification_result['file_size_bytes'] = file_size_bytes
            verification_result['file_size_mb'] = file_size_mb

            if file_size_bytes == 0:
                error_msg = "File is empty (0 bytes)"
                print(f"      ❌ {error_msg}")
                verification_result['errors'].append(error_msg)
                failed_count += 1
            elif file_size_bytes < MIN_FILE_SIZE_BYTES:
                warning_msg = f"File size is suspiciously small: {file_size_mb:.2f} MB (minimum expected: {MIN_FILE_SIZE_MB} MB)"
                print(f"      ⚠️  {warning_msg}")
                verification_result['warnings'].append(warning_msg)
                verification_result['file_size_valid'] = False
                warning_count += 1
            else:
                print(f"      ✅ File size: {file_size_mb:.2f} MB")
                verification_result['file_size_valid'] = True
        except Exception as e:
            error_msg = f"Could not check file size: {e}"
            print(f"      ❌ {error_msg}")
            verification_result['errors'].append(error_msg)
            failed_count += 1

        # -------- Check 3: Duration Verification (if expected duration available) --------
        if expected_duration is not None and verification_result['file_exists']:
            try:
                # Use existing run_ffprobe function to get duration
                ffprobe_data = run_ffprobe(str(video_path))

                if 'format' in ffprobe_data and 'duration' in ffprobe_data['format']:
                    actual_duration = float(ffprobe_data['format']['duration'])
                    duration_diff = abs(actual_duration - expected_duration)

                    verification_result['expected_duration'] = expected_duration
                    verification_result['actual_duration'] = actual_duration
                    verification_result['duration_difference'] = duration_diff

                    if duration_diff <= DURATION_TOLERANCE_SECONDS:
                        print(f"      ✅ Duration matches: {format_duration_display(actual_duration)} ({actual_duration:.2f}s) - expected: {format_duration_display(expected_duration)} ({expected_duration:.2f}s)")
                        verification_result['duration_verified'] = True
                        synced_durations.append(actual_duration)
                    else:
                        warning_msg = (
                            f"Duration mismatch: {format_duration_display(actual_duration)} ({actual_duration:.2f}s) - "
                            f"expected: {format_duration_display(expected_duration)} ({expected_duration:.2f}s), "
                            f"diff: {duration_diff:.2f}s"
                        )
                        print(f"      ⚠️  {warning_msg}")
                        verification_result['warnings'].append(warning_msg)
                        verification_result['duration_verified'] = False
                        warning_count += 1
                        # Still add to durations list for comparison
                        synced_durations.append(actual_duration)
                else:
                    warning_msg = "Could not extract duration from video metadata"
                    print(f"      ⚠️  {warning_msg}")
                    verification_result['warnings'].append(warning_msg)
                    warning_count += 1

            except Exception as e:
                warning_msg = f"Could not verify duration: {e}"
                print(f"      ⚠️  {warning_msg}")
                verification_result['warnings'].append(warning_msg)
                warning_count += 1

        # Determine overall status
        if len(verification_result['errors']) == 0:
            if len(verification_result['warnings']) == 0:
                verification_result['status'] = 'verified'
                verified_count += 1
            else:
                verification_result['status'] = 'warning'
        else:
            verification_result['status'] = 'failed'

        VERIFICATION_RESULTS.append(verification_result)

    # -------- Check 4: Duration Consistency Across Videos --------
    if len(synced_durations) >= 2:
        print(f"\n   📊 Checking duration consistency across videos...")

        min_duration = min(synced_durations)
        max_duration = max(synced_durations)
        duration_range = max_duration - min_duration

        if duration_range <= DURATION_TOLERANCE_SECONDS:
            print(f"      ✅ All videos have consistent duration")
            print(f"         Range: {format_duration_display(min_duration)} ({min_duration:.2f}s) - {format_duration_display(max_duration)} ({max_duration:.2f}s) - diff: {format_duration_display(duration_range)} ({duration_range:.2f}s)")
        else:
            warning_msg = (
                f"Duration inconsistency detected: "
                f"Range: {format_duration_display(min_duration)} ({min_duration:.2f}s) - "
                f"{format_duration_display(max_duration)} ({max_duration:.2f}s) - "
                f"diff: {format_duration_display(duration_range)} ({duration_range:.2f}s)"
            )
            print(f"      ⚠️  {warning_msg}")
            warning_count += 1

            # Show individual durations
            print(f"\n      Individual durations:")
            for result in VERIFICATION_RESULTS:
                if 'actual_duration' in result:
                    print(f"         {result['video_name']:40s} - {format_duration_display(result['actual_duration'])} ({result['actual_duration']:.2f}s)")

    # -------- Check 5: Comprehensive Synchronization Verification (All Videos) --------
    print(f"\n   🔄 Comprehensive Synchronization Check (All Videos)...")

    # Collect metadata for all verified videos
    video_metadata_list = []
    for result in VERIFICATION_RESULTS:
        if result.get('file_exists') and 'actual_duration' in result:
            video_path = Path(result['video_path'])
            try:
                # Extract full metadata including timestamps
                ffprobe_data = run_ffprobe(str(video_path))

                # Get creation timestamp
                creation_time_str = None
                creation_time_dt = None
                if 'format' in ffprobe_data and 'tags' in ffprobe_data['format']:
                    tags = ffprobe_data['format']['tags']
                    creation_time_str = (
                        tags.get('com.apple.quicktime.creationdate') or
                        tags.get('creation_time') or
                        tags.get('date') or
                        tags.get('creationdate')
                    )

                if creation_time_str:
                    creation_time_dt = parse_timestamp(creation_time_str)
                    creation_time_dt = normalize_timestamp_to_utc(creation_time_dt, video_path)

                # Get FPS
                fps = None
                if 'streams' in ffprobe_data:
                    for stream in ffprobe_data['streams']:
                        if stream.get('codec_type') == 'video':
                            avg_frame_rate = stream.get('avg_frame_rate')
                            if avg_frame_rate:
                                try:
                                    num, den = map(int, avg_frame_rate.split('/'))
                                    if den > 0:
                                        fps = round(num / den, 3)
                                        break
                                except (ValueError, ZeroDivisionError):
                                    pass

                video_metadata_list.append({
                    'name': result['video_name'],
                    'path': result['video_path'],
                    'duration': result.get('actual_duration'),
                    'creation_time_utc': creation_time_dt,
                    'fps': fps
                })
            except Exception as e:
                print(f"      ⚠️  Could not extract metadata for {result['video_name']}: {e}")

    # Verify synchronization across all videos
    if len(video_metadata_list) >= 2:
        print(f"\n      Analyzing {len(video_metadata_list)} video(s) for synchronization...")

        # Check 5a: Start Time Synchronization
        timestamps_utc = [v['creation_time_utc'] for v in video_metadata_list if v.get('creation_time_utc')]

        if len(timestamps_utc) >= 2:
            timestamps_sorted = sorted(timestamps_utc)
            time_diffs = []
            for i in range(len(timestamps_sorted) - 1):
                diff = abs((timestamps_sorted[i+1] - timestamps_sorted[i]).total_seconds())
                time_diffs.append(diff)

            max_time_diff = max(time_diffs) if time_diffs else 0

            if max_time_diff <= 1.0:
                print(f"      ✅ Start times synchronized (max diff: {max_time_diff:.2f}s)")
            else:
                warning_msg = f"Start times not synchronized (max diff: {max_time_diff:.2f}s)"
                print(f"      ⚠️  {warning_msg}")
                warning_count += 1
                print(f"\n         Individual start times:")
                for v in video_metadata_list:
                    if v.get('creation_time_utc'):
                        print(f"            {v['name']:40s} - {v['creation_time_utc'].isoformat()}")
        else:
            print(f"      ⚠️  Cannot verify start time synchronization (insufficient timestamps)")
            warning_count += 1

        # Check 5b: Duration Synchronization
        durations = [v['duration'] for v in video_metadata_list if v.get('duration')]
        if len(durations) >= 2:
            min_dur = min(durations)
            max_dur = max(durations)
            dur_diff = max_dur - min_dur

            if dur_diff <= 0.1:  # Very strict tolerance for exact match
                print(f"      ✅ Durations match exactly (all: {min_dur:.2f}s)")
            elif dur_diff <= DURATION_TOLERANCE_SECONDS:
                print(f"      ✅ Durations synchronized (diff: {dur_diff:.2f}s, tolerance: {DURATION_TOLERANCE_SECONDS}s)")
            else:
                error_msg = f"Durations do not match (diff: {dur_diff:.2f}s, tolerance: {DURATION_TOLERANCE_SECONDS}s)"
                print(f"      ❌ {error_msg}")
                failed_count += 1
                print(f"\n         Individual durations:")
                for v in video_metadata_list:
                    if v.get('duration'):
                        print(f"            {v['name']:40s} - {format_duration_display(v['duration'])} ({v['duration']:.2f}s)")
        else:
            print(f"      ⚠️  Cannot verify duration synchronization (insufficient data)")
            warning_count += 1

        # Check 5c: Frame Rate Compatibility
        fps_values = [v['fps'] for v in video_metadata_list if v.get('fps') is not None]
        if len(fps_values) >= 2:
            min_fps = min(fps_values)
            max_fps = max(fps_values)
            fps_diff = abs(max_fps - min_fps)
            fps_diff_percent = (fps_diff / min_fps) * 100 if min_fps > 0 else 0

            if fps_diff < 0.1:  # Very close frame rates
                print(f"      ✅ Frame rates match exactly (all: {min_fps:.2f} fps)")
            elif fps_diff_percent < 1.0:  # Less than 1% difference
                print(f"      ✅ Frame rates compatible (range: {min_fps:.2f} - {max_fps:.2f} fps, diff: {fps_diff:.2f} fps / {fps_diff_percent:.1f}%)")
            elif fps_diff_percent < 5.0:  # Less than 5% difference
                warning_msg = f"Frame rates differ slightly (range: {min_fps:.2f} - {max_fps:.2f} fps, diff: {fps_diff:.2f} fps / {fps_diff_percent:.1f}%)"
                print(f"      ⚠️  {warning_msg}")
                warning_count += 1
            else:
                error_msg = f"Frame rates incompatible (range: {min_fps:.2f} - {max_fps:.2f} fps, diff: {fps_diff:.2f} fps / {fps_diff_percent:.1f}%)"
                print(f"      ❌ {error_msg}")
                print(f"         Consider frame rate conversion for synchronization")
                failed_count += 1

            print(f"\n         Individual frame rates:")
            for v in video_metadata_list:
                if v.get('fps'):
                    print(f"            {v['name']:40s} - {v['fps']:.2f} fps")
        else:
            print(f"      ⚠️  Cannot verify frame rate compatibility (insufficient data)")
            warning_count += 1

        # Overall synchronization status
        print(f"\n      📋 Synchronization Summary:")
        sync_checks_passed = 0
        sync_checks_total = 0

        if len(timestamps_utc) >= 2:
            sync_checks_total += 1
            if max_time_diff <= 1.0:
                sync_checks_passed += 1
                print(f"         ✅ Start times: Synchronized")
            else:
                print(f"         ❌ Start times: Not synchronized (diff: {max_time_diff:.2f}s)")

        if len(durations) >= 2:
            sync_checks_total += 1
            if dur_diff <= 0.1:
                sync_checks_passed += 1
                print(f"         ✅ Durations: Match exactly")
            elif dur_diff <= DURATION_TOLERANCE_SECONDS:
                sync_checks_passed += 1
                print(f"         ✅ Durations: Synchronized (diff: {dur_diff:.2f}s)")
            else:
                print(f"         ❌ Durations: Do not match (diff: {dur_diff:.2f}s)")

        if len(fps_values) >= 2:
            sync_checks_total += 1
            if fps_diff < 0.1:
                sync_checks_passed += 1
                print(f"         ✅ Frame rates: Match exactly")
            elif fps_diff_percent < 5.0:
                sync_checks_passed += 1
                print(f"         ✅ Frame rates: Compatible (diff: {fps_diff:.2f} fps)")
            else:
                print(f"         ❌ Frame rates: Incompatible (diff: {fps_diff:.2f} fps)")

        if sync_checks_passed == sync_checks_total and sync_checks_total > 0:
            print(f"\n      ✅ All synchronization checks passed! ({sync_checks_passed}/{sync_checks_total})")
            print(f"         Videos are ready for multi-camera switching")
        elif sync_checks_passed > 0:
            print(f"\n      ⚠️  Partial synchronization ({sync_checks_passed}/{sync_checks_total} checks passed)")
            print(f"         Review warnings above before proceeding")
        else:
            print(f"\n      ❌ Synchronization checks failed ({sync_checks_passed}/{sync_checks_total})")
            print(f"         Videos may not be properly synchronized")
    else:
        print(f"      ⚠️  Cannot perform comprehensive check (need at least 2 videos with metadata)")

    # -------- Summary --------
    print(f"\n" + "=" * 50)
    print("VERIFICATION SUMMARY")
    print("=" * 50)

    print(f"\n📊 Results:")
    print(f"   Verified: {verified_count}/{len(SYNCED_VIDEOS)}")
    if warning_count > 0:
        print(f"   Warnings: {warning_count}/{len(SYNCED_VIDEOS)}")
    if failed_count > 0:
        print(f"   Failed: {failed_count}/{len(SYNCED_VIDEOS)}")

    # Show details for videos with issues
    if warning_count > 0 or failed_count > 0:
        print(f"\n   Details:")

        for result in VERIFICATION_RESULTS:
            if result['status'] == 'failed':
                print(f"\n      ❌ {result['video_name']}")
                for error in result['errors']:
                    print(f"         Error: {error}")
            elif result['status'] == 'warning':
                print(f"\n      ⚠️  {result['video_name']}")
                for warning in result['warnings']:
                    print(f"         Warning: {warning}")

    # Overall status
    if verified_count == len(SYNCED_VIDEOS):
        print(f"\n✅ All videos verified successfully!")
        print(f"   Ready for camera switching pipeline.")
    elif verified_count > 0 and failed_count == 0:
        print(f"\n⚠️  Videos verified with warnings")
        print(f"   Review warnings above, but videos should be usable.")
    elif failed_count > 0:
        print(f"\n❌ Some videos failed verification")
        print(f"   Review errors above before proceeding.")
    else:
        print(f"\n⚠️  Verification incomplete")
        print(f"   Check results above.")

elif SKIP_SYNC and 'SYNCED_VIDEOS' in globals() and SYNCED_VIDEOS:
    print(f"\n" + "=" * 50)
    print("VERIFY SYNCHRONIZED VIDEOS")
    print("=" * 50)
    print(f"\n💡 Skipping verification - using original videos (already synchronized)")
    print(f"   Videos: {len(SYNCED_VIDEOS)}")

    # Quick check that original videos exist
    missing_originals = []
    for synced_video in SYNCED_VIDEOS:
        video_path = Path(synced_video.get('path', ''))
        if not video_path.exists():
            missing_originals.append(synced_video.get('original_name', 'unknown'))

    if missing_originals:
        print(f"\n   ⚠️  Warning: Some original videos not found:")
        for name in missing_originals:
            print(f"      - {name}")
    else:
        print(f"   ✅ All original videos found")

elif 'SYNCED_VIDEOS' not in globals() or not SYNCED_VIDEOS:
    print(f"\n" + "=" * 50)
    print("VERIFY SYNCHRONIZED VIDEOS")
    print("=" * 50)
    print(f"\n⚠️  No synchronized videos to verify")
    VERIFICATION_RESULTS = []

# ==============================
# UPDATE CAMERA MAPPING
# ==============================

print(f"\n" + "=" * 50)
print("UPDATE CAMERA MAPPING")
print("=" * 50)

# Initialize camera mapping variables
SYNCED_CAMERA_MAP = {}
SYNCED_CAMERA_NAMES = {}
SYNCED_INPUT_VIDEOS = []

if 'SYNCED_VIDEOS' in globals() and SYNCED_VIDEOS:
    print(f"\n📹 Creating camera mapping from synchronized videos...")
    print(f"   Found {len(SYNCED_VIDEOS)} synchronized video(s)")

    # Sort synced videos by original name for consistent ordering
    sorted_synced = sorted(SYNCED_VIDEOS, key=lambda x: x.get('original_name', ''))

    # Create camera mapping (camera 0 = first video, camera 1 = second video, etc.)
    for cam_id, synced_video in enumerate(sorted_synced):
        video_path = Path(synced_video.get('path', ''))
        synced_name = synced_video.get('synced_name', synced_video.get('original_name', f'video_{cam_id}'))
        original_name = synced_video.get('original_name', 'unknown')

        # Verify file exists
        if video_path.exists():
            SYNCED_CAMERA_MAP[cam_id] = video_path
            SYNCED_INPUT_VIDEOS.append(video_path)

            # Generate camera name based on position
            if cam_id == 0:
                camera_name = "LEFT_CAM"
            elif cam_id == 1:
                camera_name = "RIGHT_CAM"
            else:
                camera_name = f"CAMERA_{cam_id}"

            SYNCED_CAMERA_NAMES[cam_id] = camera_name

            print(f"   Camera {cam_id} ({camera_name}): {synced_name}")
            print(f"      Original: {original_name}")
            print(f"      Path: {video_path}")
        else:
            warning_msg = f"Synced video file not found: {video_path}"
            print(f"   ⚠️  Camera {cam_id}: {warning_msg}")
            print(f"      Skipping from camera map")

    if len(SYNCED_CAMERA_MAP) > 0:
        print(f"\n✅ Camera mapping created successfully!")
        print(f"   Total cameras: {len(SYNCED_CAMERA_MAP)}")
        print(f"\n   Camera Mapping:")
        for cam_id, path in SYNCED_CAMERA_MAP.items():
            camera_name = SYNCED_CAMERA_NAMES.get(cam_id, f"CAMERA_{cam_id}")
            print(f"      {cam_id}: {camera_name} -> {path.name}")

        print(f"\n💡 Note: This mapping will be used in Cell 9 (Orchestrator)")
        print(f"   Update CAMERA_MAP and INPUT_VIDEOS in Cell 9 to use these synced videos")
    else:
        print(f"\n❌ No valid synced videos found for camera mapping")
        print(f"   Check verification results above")

elif SKIP_SYNC and 'VIDEO_FILES' in globals() and VIDEO_FILES:
    print(f"\n📹 Creating camera mapping from original videos (already synchronized)...")
    print(f"   Using original videos from: {VIDEOS_DIR}")

    # Sort original videos for consistent ordering
    sorted_originals = sorted(VIDEO_FILES, key=lambda x: x.name)

    for cam_id, video_file in enumerate(sorted_originals):
        SYNCED_CAMERA_MAP[cam_id] = video_file
        SYNCED_INPUT_VIDEOS.append(video_file)

        # Generate camera name
        if cam_id == 0:
            camera_name = "LEFT_CAM"
        elif cam_id == 1:
            camera_name = "RIGHT_CAM"
        else:
            camera_name = f"CAMERA_{cam_id}"

        SYNCED_CAMERA_NAMES[cam_id] = camera_name

        print(f"   Camera {cam_id} ({camera_name}): {video_file.name}")

    if len(SYNCED_CAMERA_MAP) > 0:
        print(f"\n✅ Camera mapping created from original videos!")
        print(f"   Total cameras: {len(SYNCED_CAMERA_MAP)}")
    else:
        print(f"\n❌ No videos found for camera mapping")

else:
    print(f"\n⚠️  Cannot create camera mapping:")
    print(f"   No synchronized videos available")
    print(f"   Camera mapping will need to be configured manually in Cell 9")

# Store mapping for use in later cells
print(f"\n" + "=" * 50)
print("CAMERA MAPPING READY")
print("=" * 50)

if SYNCED_CAMERA_MAP:
    print(f"\n📋 Summary:")
    print(f"   Cameras configured: {len(SYNCED_CAMERA_MAP)}")
    print(f"   Input videos list: {len(SYNCED_INPUT_VIDEOS)}")
    print(f"\n   To use in Cell 9 (Orchestrator), update:")
    print(f"      CAMERA_MAP = {SYNCED_CAMERA_MAP}")
    print(f"      INPUT_VIDEOS = {SYNCED_INPUT_VIDEOS}")
    print(f"      CAMERA_NAMES = {SYNCED_CAMERA_NAMES}")
else:
    print(f"\n⚠️  No camera mapping available")
    print(f"   Configure manually in Cell 9")

# -------- Save Metadata to JSON File --------
print(f"\n💾 Saving metadata to JSON file...")

# Create debug directory if it doesn't exist
DEBUG_DIR_SYNC = PROJECT_ROOT_DRIVE / "debug"
DEBUG_DIR_SYNC.mkdir(parents=True, exist_ok=True)

# Create JSON-serializable version of metadata (convert datetime to ISO string)
def serialize_metadata(metadata_list):
    """Convert metadata list to JSON-serializable format."""
    serialized = []
    for meta in metadata_list:
        serialized_meta = {
            'path': meta.get('path'),
            'name': meta.get('name'),
            'creation_time': meta.get('creation_time'),  # Original string
            'creation_time_dt': meta['creation_time_dt'].isoformat() if meta.get('creation_time_dt') else None,  # Original datetime
            'creation_time_utc': meta.get('creation_time_utc'),  # Normalized UTC string
            'creation_time_utc_dt': meta['creation_time_utc_dt'].isoformat() if meta.get('creation_time_utc_dt') else None,  # Normalized UTC datetime
            'timestamp_source': meta.get('timestamp_source', 'unknown'),  # 'metadata', 'file_modification_time', or 'none'
            'duration': meta.get('duration'),
            'fps': meta.get('fps'),
            'metadata_file_path': meta.get('metadata_file_path'),
        }
        # Include time offset information if available
        if 'time_offset_seconds' in meta:
            serialized_meta['time_offset_seconds'] = meta.get('time_offset_seconds')
            serialized_meta['time_offset_formatted'] = meta.get('time_offset_formatted')
            serialized_meta['time_relation'] = meta.get('time_relation')
        # Include error if present
        if 'error' in meta:
            serialized_meta['error'] = meta['error']
        serialized.append(serialized_meta)
    return serialized

# Generate filename with timestamp
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
metadata_json_filename = f"video_sync_metadata_{timestamp_str}.json"
metadata_json_path = DEBUG_DIR_SYNC / metadata_json_filename

try:
    # Serialize metadata
    serialized_metadata = serialize_metadata(VIDEO_METADATA)

    # Create summary object
    metadata_summary = {
        'extraction_timestamp': datetime.now().isoformat(),
        'environment': 'Google Colab' if IS_COLAB else 'Local',
        'videos_directory': str(VIDEOS_DIR),
        'total_videos': len(VIDEO_FILES),
        'successful_extractions': successful,
        'failed_extractions': failed,
        'valid_metadata_count': len(valid_metadata),
        'skip_sync': SKIP_SYNC if 'SKIP_SYNC' in globals() else False,
        'timestamp_statistics': TIMESTAMP_STATS if 'TIMESTAMP_STATS' in globals() else None,
        'synchronization_strategy': SYNC_STRATEGY if 'SYNC_STRATEGY' in globals() and SYNC_STRATEGY else None,
        'trim_parameters': TRIM_PARAMETERS if 'TRIM_PARAMETERS' in globals() else [],
        'synchronization_feasibility': SYNC_VALIDATION_RESULTS if 'SYNC_VALIDATION_RESULTS' in globals() else None,
        'sync_feasible': SYNC_FEASIBLE if 'SYNC_FEASIBLE' in globals() else None,
        'metadata': serialized_metadata
    }

    # Write to JSON file
    with open(metadata_json_path, 'w', encoding='utf-8') as f:
        json.dump(metadata_summary, f, indent=2, ensure_ascii=False)

    print(f"   ✅ Metadata saved to: {metadata_json_path}")
    print(f"      File size: {metadata_json_path.stat().st_size / 1024:.1f} KB")

except Exception as e:
    error_msg = f"   ⚠️  Warning: Could not save metadata to JSON: {e}"
    print(error_msg)
    metadata_json_path = None

print(f"\n" + "=" * 50)
print(f"✅ Video Synchronization Pipeline Complete!")
print("=" * 50)
print(f"\n📝 Summary:")
print(f"   Videos processed: {len(VIDEO_FILES)}")
print(f"   Valid metadata: {len(valid_metadata)}")
print(f"   Synchronization needed: {'❌ No' if SKIP_SYNC else '✅ Yes'}")

# Performance statistics
if ENABLE_METADATA_CACHE and 'cached_count' in locals():
    print(f"\n⚡ Performance:")
    print(f"   Metadata cached: {cached_count}")
    print(f"   Metadata extracted: {extracted_count}")
    if ENABLE_PARALLEL_PROCESSING:
        print(f"   Parallel processing: ✅ Enabled")

# Edge cases handled
edge_cases_handled = []
if SKIP_CORRUPTED_METADATA:
    skipped_corrupted = len([f for f in failed_extractions if f.get('skipped')])
    if skipped_corrupted > 0:
        edge_cases_handled.append(f"Skipped {skipped_corrupted} corrupted metadata")
if ALLOW_LARGE_TIME_DIFF_OVERRIDE:
    edge_cases_handled.append("Large time diff override enabled")
if ALLOW_INSUFFICIENT_OVERLAP:
    edge_cases_handled.append("Insufficient overlap override enabled")

if edge_cases_handled:
    print(f"\n🛡️  Edge Cases Handled:")
    for case in edge_cases_handled:
        print(f"   - {case}")

if 'SYNCED_VIDEOS' in globals() and SYNCED_VIDEOS:
    print(f"\n   Synced videos: {len(SYNCED_VIDEOS)}")
    print(f"   Output directory: {INPUT_DIR_SYNC}")
if metadata_json_path:
    print(f"\n   Metadata JSON: {metadata_json_path}")

print(f"\n🎯 Next Steps:")
if 'SYNCED_CAMERA_MAP' in globals() and SYNCED_CAMERA_MAP:
    print(f"   ✅ Camera mapping created: {len(SYNCED_CAMERA_MAP)} camera(s)")
    print(f"   📹 Synchronized videos ready in: {INPUT_DIR_SYNC}")
    print(f"\n   📋 To use in Cell 9 (Orchestrator), update:")
    print(f"      CAMERA_MAP = {{")
    for cam_id, path in SYNCED_CAMERA_MAP.items():
        camera_name = SYNCED_CAMERA_NAMES.get(cam_id, f"CAMERA_{cam_id}")
        print(f"         {cam_id}: Path(\"{path}\"),  # {camera_name}")
    print(f"      }}")
    print(f"\n      INPUT_VIDEOS = {[str(v) for v in SYNCED_INPUT_VIDEOS]}")
    print(f"\n      CAMERA_NAMES = {{")
    for cam_id, name in SYNCED_CAMERA_NAMES.items():
        print(f"         {cam_id}: \"{name}\",")
    print(f"      }}")

    # Show verification status if available
    if 'VERIFICATION_RESULTS' in globals() and VERIFICATION_RESULTS:
        verified = len([r for r in VERIFICATION_RESULTS if r.get('status') == 'verified'])
        total = len(VERIFICATION_RESULTS)
        if verified == total:
            print(f"\n   ✅ All videos verified successfully")
        elif verified > 0:
            print(f"\n   ⚠️  {verified}/{total} videos verified (some warnings)")
        else:
            print(f"\n   ⚠️  Verification completed with issues - review above")
elif 'SYNCED_VIDEOS' in globals() and SYNCED_VIDEOS:
    print(f"   ✅ Synchronized videos are ready in: {INPUT_DIR_SYNC}")
    print(f"   ⚠️  Camera mapping not created - check errors above")
else:
    print(f"   ⚠️  No synchronized videos generated")
    if SKIP_SYNC:
        print(f"      Videos are already synchronized - using original videos")
    else:
        print(f"      Check errors above and verify synchronization feasibility")


VIDEO SYNCHRONIZATION CONFIGURATION

📁 Path Configuration:
   Environment: Google Colab
   Project root: /content/drive/MyDrive/football/final
   Videos directory: /content/drive/MyDrive/football/final/videos
   Input directory (output): /content/drive/MyDrive/football/final/input

⚙️  Synchronization Parameters:
   Max time difference: 5.0 minutes
   Min sync duration: 30 seconds
   Min overlap required: 10 seconds

🔧 Processing Options:
   Skip if synced exists: ✅
   Enhanced skip check: ✅
   Output format: mp4
   Lossless trimming: ✅

⚡ Performance Options:
   Parallel processing: ❌
   Metadata caching: ✅

🛡️  Edge Case Handling:
   Allow large time diff override: ✅
   Allow insufficient overlap: ❌
   Skip corrupted metadata: ✅

🔍 Validating paths...
   ✅ Videos directory exists: /content/drive/MyDrive/football/final/videos
      Found 3 items
   ✅ Input directory exists: /content/drive/MyDrive/football/final/input

VIDEO DISCOVERY

📂 Scanning Videos directory: /content/drive/MyDriv

In [ ]:
# ==============================
# PROJECT CONFIGURATION
# ==============================

from pathlib import Path
import torch
import os

# -------- Detect environment --------
# Try to use IS_COLAB from Cell 1, fallback to detection
try:
    # IS_COLAB should be set in Cell 1
    if 'IS_COLAB' not in globals():
        try:
            import google.colab
            IS_COLAB = True
        except ImportError:
            IS_COLAB = False
except NameError:
    # Not defined yet, detect it
    try:
        import google.colab
        IS_COLAB = True
    except ImportError:
        IS_COLAB = False

# -------- Project root (MATCHES YOUR DRIVE) --------
print("=" * 50)
print("PROJECT CONFIGURATION")
print("=" * 50)

if IS_COLAB:
    PROJECT_ROOT = Path("/content/drive/MyDrive/football/final")
    print(f"Environment: Google Colab")
else:
    PROJECT_ROOT = Path(".").resolve()
    print(f"Environment: Local")

# -------- Folder structure (already exists in Drive) --------
CODE_DIR   = PROJECT_ROOT / "code"
INPUT_DIR  = PROJECT_ROOT / "input"
MODELS_DIR = PROJECT_ROOT / "models"
OUTPUT_DIR = PROJECT_ROOT / "output"
DEBUG_DIR  = PROJECT_ROOT / "debug"
KEYMOMENTS_DIR = PROJECT_ROOT / "keymoments"

# Safety check (won’t recreate if already there)
CODE_DIR.mkdir(exist_ok=True)
INPUT_DIR.mkdir(exist_ok=True)
MODELS_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)
DEBUG_DIR.mkdir(exist_ok=True)
KEYMOMENTS_DIR.mkdir(exist_ok=True)

print(f"\nProject root: {PROJECT_ROOT}")
print("\n📁 Directory Setup:")
dirs_to_create = {
    "Code": CODE_DIR,
    "Input": INPUT_DIR,
    "Models": MODELS_DIR,
    "Output": OUTPUT_DIR,
    "Debug": DEBUG_DIR,
    "KeyMoments": KEYMOMENTS_DIR,
}

for name, dir_path in dirs_to_create.items():
    try:
        if dir_path.exists():
            item_count = len(list(dir_path.iterdir()))
            status = f"exists ({item_count} items)" if item_count > 0 else "exists (empty)"
            print(f"   ✅ {name:8s}: {dir_path} ({status})")
        else:
            print(f"   ⚠️  {name:8s}: {dir_path} (not found)")
    except Exception as e:
        print(f"   ❌ {name:8s}: {dir_path} (ERROR checking: {e})")

# -------- Device --------
print("\n💻 Device Configuration:")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"   Device: {DEVICE}")

if DEVICE == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print(f"   ⚠️  Running on CPU (slower inference)")

# -------- Ball Tracking Model --------
print("\n⚽ Ball Detection Settings:")
# Note: Using YOLO v8 (yolov8n.pt) which downloads automatically if not found locally
# The model name is defined in Cell 4 as BALL_MODEL_NAME = "yolov8n.pt"
# If you have a custom trained model, place it in MODELS_DIR and update Cell 4
BALL_MODEL_PATH = MODELS_DIR / "ball_yolo.pt"   # Optional: path for custom model (if used)
BALL_CONF_THRESH = 0.22  # Increased from 0.15 to 0.20 to reduce false positives while maintaining good detection
BALL_IOU_THRESH = 0.45

print(f"   Model: YOLO v8 (yolov8n.pt) - auto-downloads if needed")
print(f"   Model directory: {MODELS_DIR}")
print(f"   Custom model path (optional): {BALL_MODEL_PATH}")
print(f"   Confidence threshold: {BALL_CONF_THRESH}")
print(f"   IoU threshold: {BALL_IOU_THRESH}")

# Check if custom model exists (optional - not required)
if BALL_MODEL_PATH.exists():
    print(f"   ✅ Custom model found: {BALL_MODEL_PATH}")
    print(f"   ℹ️  Note: Custom model will be used if specified in Cell 4")
else:
    print(f"   ℹ️  No custom model found - using pretrained YOLO v8 (auto-download)")

# -------- Camera Switching Parameters --------
print("\n📹 Camera Switching Settings:")
CAMERA_IDS = [0, 1, 2]          # OR replace with video paths / RTSP URLs
DEFAULT_CAMERA = 0

BALL_LOST_FRAMES = 10           # switch if ball missing N frames
SWITCH_COOLDOWN_FRAMES = 15     # prevent rapid flickering

print(f"   Ball lost frames to switch: {BALL_LOST_FRAMES}")
print(f"   Switch cooldown frames: {SWITCH_COOLDOWN_FRAMES}")
print(f"   Default camera: {DEFAULT_CAMERA}")
print(f"   Note: Actual camera mapping defined in Cell 8")

# -------- Feature Toggles --------
print("\n🎛️  Feature Toggles:")
ENABLE_CAMERA_SWITCHING = True
ENABLE_DEBUG_DRAW = True
ENABLE_FPS_COUNTER = True

features = {
    "Camera Switching": ENABLE_CAMERA_SWITCHING,
    "Debug Drawing": ENABLE_DEBUG_DRAW,
    "FPS Counter": ENABLE_FPS_COUNTER,
}

for feature, enabled in features.items():
    status = "✅ Enabled" if enabled else "❌ Disabled"
    print(f"   {feature:20s}: {status}")

# ===== Configuration Summary =====
print("\n" + "=" * 50)
print("✅ Configuration loaded successfully!")
print("=" * 50)
print(f"\n📝 Quick Reference:")
print(f"   Project: {PROJECT_ROOT}")
print(f"   Device: {DEVICE}")
print(f"   Input videos: {INPUT_DIR}")
print(f"   Output videos: {OUTPUT_DIR}")
print(f"   Models: {MODELS_DIR}")
print(f"   Debug: {DEBUG_DIR}")
print(f"   Key Moments: {KEYMOMENTS_DIR}")


PROJECT CONFIGURATION
Environment: Google Colab

Project root: /content/drive/MyDrive/football/final

📁 Directory Setup:
   ✅ Code    : /content/drive/MyDrive/football/final/code (exists (empty))
   ✅ Input   : /content/drive/MyDrive/football/final/input (exists (2 items))
   ✅ Models  : /content/drive/MyDrive/football/final/models (exists (empty))
   ✅ Output  : /content/drive/MyDrive/football/final/output (exists (13 items))
   ✅ Debug   : /content/drive/MyDrive/football/final/debug (exists (77 items))
   ✅ KeyMoments: /content/drive/MyDrive/football/final/keymoments (exists (empty))

💻 Device Configuration:
   Device: cuda
   GPU: Tesla T4
   CUDA Version: 12.6

⚽ Ball Detection Settings:
   Model: YOLO v8 (yolov8n.pt) - auto-downloads if needed
   Model directory: /content/drive/MyDrive/football/final/models
   Custom model path (optional): /content/drive/MyDrive/football/final/models/ball_yolo.pt
   Confidence threshold: 0.22
   IoU threshold: 0.45
   ℹ️  No custom model found - u

In [ ]:
# ==============================
# DYNAMIC INPUT DISCOVERY
# ==============================

from pathlib import Path
import os

# -------- Supported Video Formats --------
SUPPORTED_VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".MOV", ".MP4", ".AVI", ".MKV"}

print("=" * 50)
print("VIDEO DISCOVERY")
print("=" * 50)

# -------- Verify Input Directory --------
print(f"\n📂 Input Directory: {INPUT_DIR}")

if not INPUT_DIR.exists():
    raise RuntimeError(
        f"❌ Input directory does not exist: {INPUT_DIR}\n"
        f"   Please create the directory or check your PROJECT_ROOT configuration."
    )

if not INPUT_DIR.is_dir():
    raise RuntimeError(
        f"❌ Input path is not a directory: {INPUT_DIR}\n"
        f"   Please check your INPUT_DIR configuration."
    )

# Check if directory is accessible
try:
    list(INPUT_DIR.iterdir())
except PermissionError:
    raise RuntimeError(
        f"❌ Permission denied accessing input directory: {INPUT_DIR}\n"
        f"   Please check directory permissions."
    )

# -------- Discover Videos --------
def discover_videos(input_dir: Path):
    """
    Discover all supported video files in the input directory.

    Returns:
        List of Path objects for discovered video files
    """
    videos = []
    skipped = []

    try:
        for p in sorted(input_dir.iterdir()):
            if p.is_file():
                if p.suffix in SUPPORTED_VIDEO_EXTS:
                    videos.append(p)
                else:
                    skipped.append(p.suffix)
    except Exception as e:
        print(f"⚠️  Warning: Error scanning directory: {e}")

    return videos, skipped

INPUT_VIDEOS, skipped_exts = discover_videos(INPUT_DIR)

# -------- Display Results --------
print(f"\n📹 Video Discovery Results:")
print(f"   Found: {len(INPUT_VIDEOS)} video file(s)")

if len(INPUT_VIDEOS) > 0:
    print(f"\n   Video Files:")
    total_size = 0
    for i, v in enumerate(INPUT_VIDEOS):
        try:
            size_mb = v.stat().st_size / (1024 * 1024)
            total_size += v.stat().st_size
            print(f"   [{i:2d}] {v.name:40s} ({size_mb:7.2f} MB)")
        except Exception as e:
            print(f"   [{i:2d}] {v.name:40s} (size unknown: {e})")

    total_size_gb = total_size / (1024 ** 3)
    print(f"\n   Total size: {total_size_gb:.2f} GB ({len(INPUT_VIDEOS)} files)")
else:
    print(f"\n   ⚠️  No video files found!")

# Show skipped file types if any
if skipped_exts:
    unique_skipped = set(skipped_exts)
    if len(unique_skipped) > 0:
        print(f"\n   ℹ️  Skipped file types: {', '.join(sorted(unique_skipped))}")
        print(f"      (Supported: {', '.join(sorted(SUPPORTED_VIDEO_EXTS))})")

# -------- Safety Check --------
if len(INPUT_VIDEOS) == 0:
    print("\n" + "=" * 50)
    print("❌ ERROR: No supported video files found!")
    print("=" * 50)
    raise RuntimeError(
        f"\nNo supported video files found in {INPUT_DIR}\n"
        f"Supported extensions: {', '.join(sorted(SUPPORTED_VIDEO_EXTS))}\n"
        f"\nPlease ensure:\n"
        f"  1. Video files are placed in: {INPUT_DIR}\n"
        f"  2. Files have one of the supported extensions\n"
        f"  3. Files are not corrupted or empty"
    )

# -------- Summary --------
print("\n" + "=" * 50)
print("✅ Video discovery complete!")
print("=" * 50)
print(f"\n📝 Quick Summary:")
print(f"   Input directory: {INPUT_DIR}")
print(f"   Videos found: {len(INPUT_VIDEOS)}")
print(f"   Ready for processing: ✅")


VIDEO DISCOVERY

📂 Input Directory: /content/drive/MyDrive/football/final/input

📹 Video Discovery Results:
   Found: 2 video file(s)

   Video Files:
   [ 0] IMG_0689 2_synced.mp4                    (6202.46 MB)
   [ 1] IMG_2789_synced.mp4                      (6992.21 MB)

   Total size: 12.89 GB (2 files)

✅ Video discovery complete!

📝 Quick Summary:
   Input directory: /content/drive/MyDrive/football/final/input
   Videos found: 2
   Ready for processing: ✅


In [ ]:
# ==============================
# BALL TRACKING CORE + ENHANCED DEBUG LOGGING
# ==============================

import cv2
import numpy as np
import time
import os
from datetime import datetime
from pathlib import Path
from dataclasses import dataclass
from typing import Optional, Tuple, Dict

# ---- Install/import ultralytics if needed ----
try:
    from ultralytics import YOLO
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ultralytics"])
    from ultralytics import YOLO

print("=" * 50)
print("BALL TRACKING CORE INITIALIZATION")
print("=" * 50)

# ------------------------------
# Debug / Logging Configuration
# ------------------------------
DEBUG_DETECT = True          # print debug logs occasionally
DEBUG_EVERY_N = 30           # log once every N frames (prevents spam)
DEBUG_DRAW_ALL = False       # draw all candidate boxes (not just best)
DEBUG_TIME = True            # measure inference time
MAX_DEBUG_CANDIDATES = 10    # safety cap (keep top-N by confidence)

# Logging to file
ENABLE_FILE_LOGGING = True   # write logs to debug directory
LOG_TO_FILE_EVERY_N = 10     # log to file every N frames (more frequent than console)
LOG_DETAILED_STATS = True     # log detailed statistics

# Ensure debug directory exists
DEBUG_DIR.mkdir(exist_ok=True, parents=True)

# Initialize log file
_log_file = None
_log_file_path = None
if ENABLE_FILE_LOGGING:
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        _log_file_path = DEBUG_DIR / f"ball_detection_{timestamp}.log"
        _log_file = open(_log_file_path, 'w', encoding='utf-8')
        print(f"\n📝 Logging enabled: {_log_file_path}")
    except Exception as e:
        print(f"⚠️  Warning: Could not create log file: {e}")

# ==============================
# IMPROVEMENT: Motion-Consistency & Pitch-Aware Filtering
# Global state for motion-consistency selection
# ==============================
_last_ball_center: Optional[Tuple[float, float]] = None
_last_ball_center_frame: int = 0
_motion_consistency_max_jump_px: float = 150.0  # Maximum jump distance for motion consistency
_motion_consistency_high_conf_threshold: float = 0.7  # High confidence threshold to allow larger jumps

def _create_pitch_mask(frame_bgr: np.ndarray,
                       green_lower: Tuple[int, int, int] = (30, 40, 40),
                       green_upper: Tuple[int, int, int] = (85, 255, 255)) -> np.ndarray:
    """
    Create a pitch mask using HSV color space to detect green field region.

    Args:
        frame_bgr: Input frame in BGR format
        green_lower: Lower HSV bounds for green (default: (30, 40, 40))
        green_upper: Upper HSV bounds for green (default: (85, 255, 255))

    Returns:
        Binary mask where 1 = pitch region, 0 = non-pitch region
    """
    try:
        hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, np.array(green_lower), np.array(green_upper))

        # Morphological operations to clean up the mask
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        return mask
    except Exception as e:
        if ENABLE_FILE_LOGGING:
            _log_to_file(f"Error creating pitch mask: {e}", "WARNING")
        # Return all-ones mask (no filtering) if pitch detection fails
        return np.ones((frame_bgr.shape[0], frame_bgr.shape[1]), dtype=np.uint8) * 255

def _is_on_pitch(center: Tuple[int, int], pitch_mask: np.ndarray,
                 margin_px: int = 10) -> bool:
    """
    Check if a detection center is on the pitch (within green field region).

    Args:
        center: (x, y) center coordinates
        pitch_mask: Binary pitch mask
        margin_px: Margin in pixels to allow near pitch edges

    Returns:
        True if center is on or near pitch, False otherwise
    """
    try:
        cx, cy = int(center[0]), int(center[1])
        h, w = pitch_mask.shape

        # Check bounds
        if cx < 0 or cx >= w or cy < 0 or cy >= h:
            return False

        # Check if center is on pitch (with margin)
        # Check a small region around the center
        y_min = max(0, cy - margin_px)
        y_max = min(h, cy + margin_px)
        x_min = max(0, cx - margin_px)
        x_max = min(w, cx + margin_px)

        region = pitch_mask[y_min:y_max, x_min:x_max]
        pitch_pixels = np.sum(region > 0)
        total_pixels = region.size

        # Consider on pitch if at least 30% of the region is green
        return (pitch_pixels / total_pixels) >= 0.3 if total_pixels > 0 else False
    except Exception:
        return True  # Default to allowing if check fails

def _distance(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
    """Calculate Euclidean distance between two points."""
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

# Statistics tracking
_detection_stats = {
    "total_frames": 0,
    "detections_found": 0,
    "detections_lost": 0,
    "detections_filtered": 0,
    "total_inference_time_ms": 0.0,
    "min_inference_ms": float('inf'),
    "max_inference_ms": 0.0,
    "last_reset_time": time.time()
}

def _log_to_file(message: str, level: str = "INFO"):
    """Write log message to file if logging is enabled."""
    if ENABLE_FILE_LOGGING and _log_file is not None:
        try:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            _log_file.write(f"[{timestamp}] [{level}] {message}\n")
            _log_file.flush()  # Ensure immediate write
        except Exception as e:
            print(f"⚠️  Warning: Failed to write to log file: {e}")

def _update_stats(infer_ms: float, found: bool, filtered: bool = False):
    """Update detection statistics."""
    _detection_stats["total_frames"] += 1
    _detection_stats["total_inference_time_ms"] += infer_ms
    _detection_stats["min_inference_ms"] = min(_detection_stats["min_inference_ms"], infer_ms)
    _detection_stats["max_inference_ms"] = max(_detection_stats["max_inference_ms"], infer_ms)

    if found:
        _detection_stats["detections_found"] += 1
    else:
        _detection_stats["detections_lost"] += 1

    if filtered:
        _detection_stats["detections_filtered"] += 1

def get_detection_stats() -> Dict:
    """Get current detection statistics."""
    stats = _detection_stats.copy()
    if stats["total_frames"] > 0:
        stats["avg_inference_ms"] = stats["total_inference_time_ms"] / stats["total_frames"]
        stats["detection_rate"] = stats["detections_found"] / stats["total_frames"]
    else:
        stats["avg_inference_ms"] = 0.0
        stats["detection_rate"] = 0.0
    return stats

def reset_stats():
    """Reset detection statistics."""
    global _detection_stats
    _detection_stats = {
        "total_frames": 0,
        "detections_found": 0,
        "detections_lost": 0,
        "detections_filtered": 0,
        "total_inference_time_ms": 0.0,
        "min_inference_ms": float('inf'),
        "max_inference_ms": 0.0,
        "last_reset_time": time.time()
    }
    if ENABLE_FILE_LOGGING:
        _log_to_file("Statistics reset", "INFO")


# Filtering (helps reduce false positives)
MIN_BOX_AREA_FRAC = 0.00001   # min bbox area relative to frame area (increased to filter tiny false positives)
MAX_BOX_AREA_FRAC = 0.03     # max bbox area relative to frame area (reduced to filter large false positives like heads)
MIN_ASPECT = 0.5             # ball bbox aspect ratio constraints (w/h) - tightened to filter non-circular objects
MAX_ASPECT = 2.0             # tightened to filter elongated false positives


# ------------------------------
# Output type
# ------------------------------
@dataclass
class BallDet:
    bbox: Optional[Tuple[int, int, int, int]]  # (x1,y1,x2,y2) or None
    center: Optional[Tuple[int, int]]          # (cx,cy) or None
    conf: float                                # 0 if not found
    cls: Optional[int] = None
    meta: Optional[Dict] = None                # debug metadata


# ------------------------------
# Model Configuration
# ------------------------------
# Using YOLO v8 nano model (yolov8n.pt) - automatically downloads if not found locally
# Alternative models: yolov8s.pt (small), yolov8m.pt (medium), yolov8l.pt (large), yolov8x.pt (xlarge)
# For custom trained models, use: BALL_MODEL_NAME = str(BALL_MODEL_PATH) where BALL_MODEL_PATH is from Cell 2
BALL_MODEL_NAME = "yolov8n.pt"   # YOLO v8 nano - fast + sufficient for ball detection
BALL_CLASS_ID = 32               # COCO dataset class ID: sports ball

print("\n🤖 Model Configuration:")
print(f"   Model: {BALL_MODEL_NAME} (YOLO v8)")
print(f"   Note: Model will auto-download if not found locally")
print(f"   Target class: {BALL_CLASS_ID} (sports ball)")
print(f"   Device: {DEVICE}")

# Load model
try:
    print(f"\n📥 Loading YOLO v8 model...")
    print(f"   (First run will download ~6.2MB model file)")
    ball_model = YOLO(BALL_MODEL_NAME)
    print(f"✅ Model loaded successfully")

    # Verify class exists
    if BALL_CLASS_ID not in ball_model.names:
        print(f"⚠️  Warning: Class ID {BALL_CLASS_ID} not found in model classes")
    else:
        print(f"   Class name: '{ball_model.names[BALL_CLASS_ID]}'")

    if ENABLE_FILE_LOGGING:
        _log_to_file(f"Model loaded: {BALL_MODEL_NAME}, Device: {DEVICE}, Class ID: {BALL_CLASS_ID}", "INFO")

except Exception as e:
    error_msg = f"❌ Failed to load model: {e}"
    print(error_msg)
    if ENABLE_FILE_LOGGING:
        _log_to_file(f"Model loading failed: {e}", "ERROR")
    raise

# Internal counter for debug throttling
_frame_counter = 0


def _valid_box(x1, y1, x2, y2, w, h):
    # Basic sanity
    if x2 <= x1 or y2 <= y1:
        return False
    # Clip already done outside
    bw = max(1,x2 - x1)
    bh = max(1,y2 - y1)
    area = bw * bh
    frame_area = w * h
    area_frac = area / float(frame_area + 1e-9)

    # Area filters: ball usually small
    if area_frac < MIN_BOX_AREA_FRAC or area_frac > MAX_BOX_AREA_FRAC:
        return False

    # Aspect ratio filter
    aspect = bw / float(bh + 1e-9)
    if aspect < MIN_ASPECT or aspect > MAX_ASPECT:
        return False

    return True


# def detect_ball(frame_bgr,
#                 conf_thres: float = BALL_CONF_THRESH,
#                 iou_thres: float = BALL_IOU_THRESH,
#                 imgsz: int = 1280) -> BallDet:
#     """
#     Runs ball detection on a single BGR frame.
#     Returns best ball detection or None if missing.

#     Args:
#         frame_bgr: Input frame in BGR format
#         conf_thres: Confidence threshold for detections
#         iou_thres: IoU threshold for NMS

#     Returns:
#         BallDet object with detection results and metadata
#     """
#     global _frame_counter
#     _frame_counter += 1

#     # Validate input
#     if frame_bgr is None or frame_bgr.size == 0:
#         if ENABLE_FILE_LOGGING:
#             _log_to_file(f"Frame {_frame_counter}: Invalid input frame", "WARNING")
#         return BallDet(bbox=None, center=None, conf=0.0, cls=None,
#                        meta={"n": 0, "infer_ms": 0.0, "error": "invalid_frame"})

#     h, w = frame_bgr.shape[:2]
#     t0 = time.time()

#     # Run inference
#     try:
#         results = ball_model.predict(
#             source=frame_bgr,
#             conf=conf_thres,
#             iou=iou_thres,
#             classes=[BALL_CLASS_ID],
#             imgsz=imgsz,
#             device=0 if DEVICE == "cuda" else "cpu",
#             verbose=False
#         )
#     except Exception as e:
#         error_msg = f"Inference error on frame {_frame_counter}: {e}"
#         if ENABLE_FILE_LOGGING:
#             _log_to_file(error_msg, "ERROR")
#         if DEBUG_DETECT:
#             print(f"❌ {error_msg}")
#         return BallDet(bbox=None, center=None, conf=0.0, cls=None,
#                        meta={"n": 0, "infer_ms": 0.0, "error": str(e)})

#     infer_ms = (time.time() - t0) * 1000.0

#     # No detections
#     if not results or len(results[0].boxes) == 0:
#         _update_stats(infer_ms, found=False)

#         if DEBUG_DETECT and (_frame_counter % DEBUG_EVERY_N == 0):
#             msg = f"[detect_ball] frame={_frame_counter} dets=0 infer={infer_ms:.1f}ms"
#             print(msg)
#             if ENABLE_FILE_LOGGING and (_frame_counter % LOG_TO_FILE_EVERY_N == 0):
#                 _log_to_file(msg, "DEBUG")

#         return BallDet(bbox=None, center=None, conf=0.0, cls=None,
#                        meta={"n": 0, "infer_ms": infer_ms})

#     boxes = results[0].boxes
#     candidates = []  # list of dicts for debugging
#     best = None  # (conf, cls, xyxy)
#     filtered_count = 0

#     # Process all detections (FIXED: don't return early on first invalid box)
#     for b in boxes:
#         try:
#             cls_id = int(b.cls.item())
#             conf = float(b.conf.item())
#             xyxy = b.xyxy[0].cpu().numpy().astype(int)
#             x1, y1, x2, y2 = xyxy

#             # Clip to frame
#             x1, y1 = max(0, x1), max(0, y1)
#             x2, y2 = min(w - 1, x2), min(h - 1, y2)

#             # Check class ID
#             if BALL_CLASS_ID is not None and cls_id != BALL_CLASS_ID:
#                 continue

#             # Validate box
#             ok = _valid_box(x1, y1, x2, y2, w, h)

#             candidates.append({
#                 "cls": cls_id,
#                 "conf": conf,
#                 "bbox": (x1, y1, x2, y2),
#                 "valid": ok
#             })

#             if not ok:
#                 filtered_count += 1
#                 continue

#             # Update best detection
#             if (best is None) or (conf > best[0]):
#                 best = (conf, cls_id, (x1, y1, x2, y2))
#         except Exception as e:
#             if ENABLE_FILE_LOGGING:
#                 _log_to_file(f"Error processing detection box: {e}", "WARNING")
#             continue

#     # No valid detections after filtering
#     if best is None:
#         _update_stats(infer_ms, found=False, filtered=(filtered_count > 0))

#         if DEBUG_DETECT and (_frame_counter % DEBUG_EVERY_N == 0):
#             msg = (f"[detect_ball] frame={_frame_counter} dets={len(candidates)} "
#                    f"valid=0 filtered={filtered_count} infer={infer_ms:.1f}ms conf_th={conf_thres}")
#             print(msg)
#             if ENABLE_FILE_LOGGING and (_frame_counter % LOG_TO_FILE_EVERY_N == 0):
#                 _log_to_file(msg, "DEBUG")

#         candidates_sorted = sorted(
#             candidates,
#             key=lambda c: c["conf"],
#             reverse=True
#         )[:MAX_DEBUG_CANDIDATES]

#         return BallDet(bbox=None, center=None, conf=0.0, cls=None,
#                        meta={
#                            "n": len(candidates),
#                            "filtered": filtered_count,
#                            "infer_ms": infer_ms,
#                            "candidates": candidates_sorted
#                        })

#     # Found valid detection
#     conf, cls_id, (x1, y1, x2, y2) = best
#     cx = int((x1 + x2) / 2)
#     cy = int((y1 + y2) / 2)

#     _update_stats(infer_ms, found=True)

#     if DEBUG_DETECT and (_frame_counter % DEBUG_EVERY_N == 0):
#         valid_count = sum(1 for c in candidates if c["valid"])
#         msg = (f"[detect_ball] frame={_frame_counter} dets={len(candidates)} "
#                f"valid={valid_count} best_conf={conf:.2f} center=({cx},{cy}) infer={infer_ms:.1f}ms")
#         print(msg)
#         if ENABLE_FILE_LOGGING and (_frame_counter % LOG_TO_FILE_EVERY_N == 0):
#             _log_to_file(msg, "DEBUG")

#     candidates_sorted = sorted(
#         candidates,
#         key=lambda c: c["conf"],
#         reverse=True
#     )[:MAX_DEBUG_CANDIDATES]

#     return BallDet(
#         bbox=(x1, y1, x2, y2),
#         center=(cx, cy),
#         conf=conf,
#         cls=cls_id,
#         meta={
#             "n": len(candidates),
#             "valid": valid_count if DEBUG_DETECT else None,
#             "infer_ms": infer_ms,
#             "candidates": candidates_sorted
#         }
#     )

def detect_ball(frame_bgr,
                conf_thres: float = BALL_CONF_THRESH,
                iou_thres: float = BALL_IOU_THRESH,
                imgsz: int = 1280) -> BallDet:
    """
    Runs ball detection on a single BGR frame.
    Returns best ball detection or None if missing.
    Simplified version matching test cell for better tracking.
    """
    global _frame_counter
    _frame_counter += 1

    # Validate input
    if frame_bgr is None or frame_bgr.size == 0:
        if ENABLE_FILE_LOGGING:
            _log_to_file(f"Frame {_frame_counter}: Invalid input frame", "WARNING")
        return BallDet(bbox=None, center=None, conf=0.0, cls=None,
                       meta={"n": 0, "infer_ms": 0.0, "error": "invalid_frame"})

    h, w = frame_bgr.shape[:2]
    t0 = time.time()

    # Run inference
    try:
        results = ball_model.predict(
            source=frame_bgr,
            conf=conf_thres,
            iou=iou_thres,
            classes=[BALL_CLASS_ID],
            imgsz=imgsz,
            device=0 if DEVICE == "cuda" else "cpu",
            verbose=False
        )
    except Exception as e:
        error_msg = f"Inference error on frame {_frame_counter}: {e}"
        if ENABLE_FILE_LOGGING:
            _log_to_file(error_msg, "ERROR")
        if DEBUG_DETECT:
            print(f"❌ {error_msg}")
        return BallDet(bbox=None, center=None, conf=0.0, cls=None,
                       meta={"n": 0, "infer_ms": 0.0, "error": str(e)})

    infer_ms = (time.time() - t0) * 1000.0

    # No detections
    if not results or len(results[0].boxes) == 0:
        _update_stats(infer_ms, found=False)

        if DEBUG_DETECT and (_frame_counter % DEBUG_EVERY_N == 0):
            msg = f"[detect_ball] frame={_frame_counter} dets=0 infer={infer_ms:.1f}ms conf_thresh={conf_thres}"
            print(msg)
            if ENABLE_FILE_LOGGING and (_frame_counter % LOG_TO_FILE_EVERY_N == 0):
                _log_to_file(msg, "DEBUG")

        return BallDet(bbox=None, center=None, conf=0.0, cls=None,
                       meta={"n": 0, "infer_ms": infer_ms, "conf_thresh_used": conf_thres})

    # ==============================
    # IMPROVEMENT: Motion-Consistency Selection + Pitch-Aware Filtering
    # ==============================
    boxes = results[0].boxes
    global _last_ball_center, _last_ball_center_frame

    # Configuration flags (can be disabled if causing issues)
    # DISABLED BY DEFAULT: Motion consistency can filter out valid detections if ball jumps or camera switches
    ENABLE_MOTION_CONSISTENCY = False  # Set to True to enable motion-consistency filtering (may be too aggressive)
    ENABLE_PITCH_AWARE = False  # Set to True to enable pitch-aware filtering (may be too aggressive)

    # Create pitch mask for pitch-aware filtering (only if enabled)
    pitch_mask = None
    if ENABLE_PITCH_AWARE:
        try:
            pitch_mask = _create_pitch_mask(frame_bgr)
        except Exception as e:
            if ENABLE_FILE_LOGGING:
                _log_to_file(f"Pitch mask creation failed, disabling pitch-aware filtering: {e}", "WARNING")
            ENABLE_PITCH_AWARE = False
            pitch_mask = None

    # Evaluate all candidates with motion-consistency and pitch-aware scoring
    candidates = []
    for box in boxes:
        conf = float(box.conf.item())
        cls_id = int(box.cls.item())
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

        # Clip to frame
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w - 1, x2), min(h - 1, y2)
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
        center = (cx, cy)

        # Calculate motion-consistency score
        motion_score = 1.0  # Default: no penalty
        if ENABLE_MOTION_CONSISTENCY and _last_ball_center is not None and (_frame_counter - _last_ball_center_frame) <= 10:
            # Only apply motion consistency if we have recent ball position
            try:
                distance = _distance(center, _last_ball_center)

                # Penalize large jumps unless confidence is very high
                if distance > _motion_consistency_max_jump_px:
                    if conf < _motion_consistency_high_conf_threshold:
                        # Large jump + low confidence = moderate penalty (less aggressive)
                        motion_score = max(0.3, 1.0 - (distance - _motion_consistency_max_jump_px) / 300.0)
                    else:
                        # Large jump + high confidence = small penalty
                        motion_score = max(0.7, 1.0 - (distance - _motion_consistency_max_jump_px) / 500.0)
                else:
                    # Small jump = small bonus (prefer candidates near last position)
                    motion_score = 1.0 + (1.0 - distance / _motion_consistency_max_jump_px) * 0.1
            except Exception:
                motion_score = 1.0  # Fallback to no penalty on error

        # Calculate pitch-aware score
        pitch_score = 1.0  # Default: no penalty
        if ENABLE_PITCH_AWARE and pitch_mask is not None:
            try:
                if _is_on_pitch(center, pitch_mask):
                    pitch_score = 1.0  # On pitch: no penalty
                else:
                    pitch_score = 0.5  # Off pitch: moderate penalty (less aggressive than 0.3)
            except Exception:
                pitch_score = 1.0  # Fallback to no penalty on error

        # Combined score: confidence * motion_score * pitch_score
        # If both filters disabled, combined_score = conf (original behavior)
        combined_score = conf * motion_score * pitch_score

        candidates.append({
            'box': box,
            'conf': conf,
            'cls_id': cls_id,
            'bbox': (x1, y1, x2, y2),
            'center': center,
            'motion_score': motion_score,
            'pitch_score': pitch_score,
            'combined_score': combined_score
        })

    # Select best candidate based on combined score
    # Fallback: if all candidates filtered out, use highest confidence (safety net)
    if not candidates:
        if DEBUG_DETECT and (_frame_counter % DEBUG_EVERY_N == 0):
            print(f"[detect_ball] WARNING: No candidates after processing {len(boxes)} boxes")
        return BallDet(bbox=None, center=None, conf=0.0, cls=None,
                       meta={"n": 0, "infer_ms": infer_ms, "boxes_from_yolo": len(boxes)})

    best_candidate = max(candidates, key=lambda c: c['combined_score'])

    # Safety check: if combined score is too low, fall back to highest confidence
    # This prevents filtering out all valid detections
    # Lowered threshold from 0.05 to 0.01 to be less aggressive
    if best_candidate['combined_score'] < 0.01 and len(candidates) > 0:
        # Use highest confidence candidate as fallback
        best_candidate = max(candidates, key=lambda c: c['conf'])
        if DEBUG_DETECT and (_frame_counter % DEBUG_EVERY_N == 0):
            print(f"[detect_ball] Fallback to highest confidence: conf={best_candidate['conf']:.3f}, combined={best_candidate['combined_score']:.3f}")
        if ENABLE_FILE_LOGGING and (_frame_counter % LOG_TO_FILE_EVERY_N == 0):
            _log_to_file(f"Fallback to highest confidence (combined_score too low: {best_candidate['combined_score']:.3f})", "DEBUG")

    # Use best candidate
    conf = best_candidate['conf']
    cls_id = best_candidate['cls_id']
    x1, y1, x2, y2 = best_candidate['bbox']
    cx, cy = best_candidate['center']

    # Update last known position for motion consistency
    _last_ball_center = (float(cx), float(cy))
    _last_ball_center_frame = _frame_counter

    _update_stats(infer_ms, found=True)

    if DEBUG_DETECT and (_frame_counter % DEBUG_EVERY_N == 0):
        motion_info = f" motion={best_candidate['motion_score']:.2f}" if 'motion_score' in best_candidate else ""
        pitch_info = f" pitch={best_candidate['pitch_score']:.2f}" if 'pitch_score' in best_candidate else ""
        msg = (f"[detect_ball] frame={_frame_counter} dets={len(candidates)} "
               f"best_conf={conf:.2f} combined={best_candidate['combined_score']:.2f}{motion_info}{pitch_info} "
               f"center=({cx},{cy}) infer={infer_ms:.1f}ms")
        print(msg)
        if ENABLE_FILE_LOGGING and (_frame_counter % LOG_TO_FILE_EVERY_N == 0):
            _log_to_file(msg, "DEBUG")

    return BallDet(
        bbox=(x1, y1, x2, y2),
        center=(cx, cy),
        conf=conf,
        cls=cls_id,
        meta={
            "n": len(candidates),
            "infer_ms": infer_ms,
            "motion_score": best_candidate.get('motion_score', 1.0),
            "pitch_score": best_candidate.get('pitch_score', 1.0),
            "combined_score": best_candidate.get('combined_score', conf)
        }
    )

def print_detection_stats():
    """Print current detection statistics in a formatted way."""
    stats = get_detection_stats()

    print("\n" + "=" * 50)
    print("📊 DETECTION STATISTICS")
    print("=" * 50)

    if stats["total_frames"] == 0:
        print("   No frames processed yet.")
        return

    print(f"\n📈 Performance Metrics:")
    print(f"   Total frames processed: {stats['total_frames']}")
    print(f"   Detections found: {stats['detections_found']} ({stats['detection_rate']*100:.1f}%)")
    print(f"   Detections lost: {stats['detections_lost']} ({(1-stats['detection_rate'])*100:.1f}%)")
    print(f"   Detections filtered: {stats['detections_filtered']}")

    print(f"\n⏱️  Inference Timing:")
    print(f"   Average: {stats['avg_inference_ms']:.2f} ms")
    print(f"   Minimum: {stats['min_inference_ms']:.2f} ms")
    print(f"   Maximum: {stats['max_inference_ms']:.2f} ms")

    if stats["total_frames"] > 0:
        total_time = stats["total_inference_time_ms"] / 1000.0
        print(f"   Total inference time: {total_time:.2f} s")

    uptime = time.time() - stats["last_reset_time"]
    print(f"\n⏰ Uptime: {uptime:.1f} seconds")

    print("=" * 50)

    # Also log to file if enabled
    if ENABLE_FILE_LOGGING and LOG_DETAILED_STATS:
        _log_to_file(f"Stats: frames={stats['total_frames']}, found={stats['detections_found']}, "
                    f"rate={stats['detection_rate']*100:.1f}%, avg_ms={stats['avg_inference_ms']:.2f}", "STATS")


def draw_ball_debug(frame_bgr, det: BallDet, pos_history=None):
    """
    Draw best bbox/center/conf. Optionally draw all candidates and trajectory trail.
    Enhanced with better visualization options.

    Args:
        frame_bgr: Input frame in BGR format
        det: BallDet object with detection results
        pos_history: Optional deque of (x, y) normalized positions for trajectory trail
    """
    """
    Draw best bbox/center/conf. Optionally draw all candidates and trajectory trail.
    Enhanced with better visualization options.

    Args:
        frame_bgr: Input frame in BGR format
        det: BallDet object with detection results
        pos_history: Optional deque of (x, y) normalized positions for trajectory trail
    """
    out = frame_bgr.copy()
    h, w = frame_bgr.shape[:2]

    # draw all candidates (if requested and available)
    if DEBUG_DRAW_ALL and det.meta and det.meta.get("candidates"):
        for c in det.meta["candidates"]:
            x1, y1, x2, y2 = c["bbox"]
            color = (0, 255, 255) if c["valid"] else (0, 0, 255)  # Yellow for valid, Red for invalid
            thickness = 2 if c["valid"] else 1
            cv2.rectangle(out, (x1, y1), (x2, y2), color, thickness)
            cv2.putText(out, f"{c['conf']:.2f}", (x1, max(15, y1-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # draw best detection
    if det.bbox is not None:
        x1, y1, x2, y2 = det.bbox
        # Green box for valid detection
        cv2.rectangle(out, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw center point (ensure it's valid and convert to integers)
        if det.center is not None:
            try:
                # Convert center to integers (OpenCV requires int coordinates)
                if isinstance(det.center, (tuple, list)) and len(det.center) >= 2:
                    center_x = int(det.center[0])
                    center_y = int(det.center[1])
                    center = (center_x, center_y)
                    # White center point
                    cv2.circle(out, center, 4, (255, 255, 255), -1)
                    cv2.circle(out, center, 6, (0, 255, 0), 1)
                else:
                    # Fallback: compute center from bbox
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)
                    center = (center_x, center_y)
                    cv2.circle(out, center, 4, (255, 255, 255), -1)
                    cv2.circle(out, center, 6, (0, 255, 0), 1)
            except (TypeError, ValueError, IndexError) as e:
                # If center is invalid, compute from bbox
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                center = (center_x, center_y)
                cv2.circle(out, center, 4, (255, 255, 255), -1)
                cv2.circle(out, center, 6, (0, 255, 0), 1)
        else:
            # No center provided, compute from bbox
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            center = (center_x, center_y)
            cv2.circle(out, center, 4, (255, 255, 255), -1)
            cv2.circle(out, center, 6, (0, 255, 0), 1)

        # Confidence label
        label = f"ball {det.conf:.2f}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        label_y = max(25, y1 - 8)
        # Background for text readability
        cv2.rectangle(out, (x1, label_y - label_size[1] - 4),
                     (x1 + label_size[0] + 4, label_y + 4), (0, 0, 0), -1)
        cv2.putText(out, label, (x1 + 2, label_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Draw trajectory trail from position history (if available)
        if pos_history and len(pos_history) > 1:
            try:
                trail_points = []
                for norm_x, norm_y in pos_history:
                    # Convert normalized coordinates to pixel coordinates
                    px = int(norm_x * w)
                    py = int(norm_y * h)
                    trail_points.append((px, py))

                # Draw trail with fading effect (newer = brighter)
                for i in range(1, len(trail_points)):
                    pt1 = trail_points[i-1]
                    pt2 = trail_points[i]
                    # Fade from green (new) to blue (old)
                    alpha = i / len(trail_points)
                    # Green (0, 255, 0) to Cyan (0, 255, 255) to Blue (255, 0, 0) - but simpler: green to yellow
                    color_intensity = int(255 * (1 - alpha * 0.5))  # Fade from 255 to 127
                    color = (0, color_intensity, 255 - color_intensity)  # Blue to Green gradient
                    thickness = max(1, int(3 * (1 - alpha * 0.7)))  # Thinner for older points
                    cv2.line(out, pt1, pt2, color, thickness)

                # Draw small dots at trail points
                for i, pt in enumerate(trail_points):
                    alpha = i / len(trail_points) if len(trail_points) > 1 else 1.0
                    radius = max(1, int(3 * (1 - alpha * 0.5)))
                    color_intensity = int(255 * (1 - alpha * 0.5))
                    color = (0, color_intensity, 255 - color_intensity)
                    cv2.circle(out, pt, radius, color, -1)
            except Exception as e:
                # Silently fail if trajectory drawing has issues
                pass

    # inference time overlay
    if DEBUG_TIME and det.meta and "infer_ms" in det.meta:
        infer_text = f"infer: {det.meta['infer_ms']:.1f}ms"
        text_size, _ = cv2.getTextSize(infer_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        # Background for text
        cv2.rectangle(out, (8, 52), (12 + text_size[0], 68), (0, 0, 0), -1)
        cv2.putText(out, infer_text, (10, 65),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Draw detection status overlay (always show for visibility)
    status_y = 10
    if det.bbox is not None:
        status_text = f"DETECTED: conf={det.conf:.2f}"
        if det.meta and det.meta.get("sticky"):
            status_text += " [STICKY]"
        color = (0, 255, 0)  # Green for detected
    else:
        status_text = "NO DETECTION"
        color = (0, 0, 255)  # Red for no detection

    text_size, _ = cv2.getTextSize(status_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
    # Background for text
    cv2.rectangle(out, (8, status_y), (12 + text_size[0], status_y + text_size[1] + 8), (0, 0, 0), -1)
    cv2.putText(out, status_text, (10, status_y + text_size[1] + 4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    return out


# ------------------------------
# Final Summary
# ------------------------------
print("\n" + "=" * 50)
print("✅ Ball tracking core initialized successfully!")
print("=" * 50)

print("\n📊 Configuration Summary:")
print(f"   Model: {BALL_MODEL_NAME}")
print(f"   Device: {DEVICE}")
print(f"   Confidence threshold: {BALL_CONF_THRESH}")
print(f"   IoU threshold: {BALL_IOU_THRESH}")
print(f"   Min box area fraction: {MIN_BOX_AREA_FRAC}")
print(f"   Max box area fraction: {MAX_BOX_AREA_FRAC}")
print(f"   Aspect ratio range: {MIN_ASPECT:.1f} - {MAX_ASPECT:.1f}")

print("\n🔧 Debug Settings:")
print(f"   Console debug: {'✅' if DEBUG_DETECT else '❌'} (every {DEBUG_EVERY_N} frames)")
print(f"   File logging: {'✅' if ENABLE_FILE_LOGGING else '❌'}")
if ENABLE_FILE_LOGGING and _log_file_path:
    print(f"   Log file: {_log_file_path}")
print(f"   Draw all candidates: {'✅' if DEBUG_DRAW_ALL else '❌'}")
print(f"   Show inference time: {'✅' if DEBUG_TIME else '❌'}")

print("\n💡 Tips:")
print("   - Tune MIN_BOX_AREA_FRAC/MAX_BOX_AREA_FRAC if ball is missed or false positives occur")
print("   - Adjust BALL_CONF_THRESH for sensitivity (lower = more detections, higher = fewer false positives)")
print("   - Check debug logs in:", DEBUG_DIR)
print("   - Use get_detection_stats() to view performance metrics")

if ENABLE_FILE_LOGGING:
    _log_to_file("Ball tracking core initialization complete", "INFO")
    _log_to_file(f"Configuration: conf_thresh={BALL_CONF_THRESH}, iou_thresh={BALL_IOU_THRESH}, device={DEVICE}", "INFO")

print("\n" + "=" * 50)

BALL TRACKING CORE INITIALIZATION

📝 Logging enabled: /content/drive/MyDrive/football/final/debug/ball_detection_20260115_182242.log

🤖 Model Configuration:
   Model: yolov8n.pt (YOLO v8)
   Note: Model will auto-download if not found locally
   Target class: 32 (sports ball)
   Device: cuda

📥 Loading YOLO v8 model...
   (First run will download ~6.2MB model file)
✅ Model loaded successfully
   Class name: 'sports ball'

✅ Ball tracking core initialized successfully!

📊 Configuration Summary:
   Model: yolov8n.pt
   Device: cuda
   Confidence threshold: 0.22
   IoU threshold: 0.45
   Min box area fraction: 1e-05
   Max box area fraction: 0.03
   Aspect ratio range: 0.5 - 2.0

🔧 Debug Settings:
   Console debug: ✅ (every 30 frames)
   File logging: ✅
   Log file: /content/drive/MyDrive/football/final/debug/ball_detection_20260115_182242.log
   Draw all candidates: ❌
   Show inference time: ✅

💡 Tips:
   - Tune MIN_BOX_AREA_FRAC/MAX_BOX_AREA_FRAC if ball is missed or false positives occ

In [ ]:
# ==============================
# STICKY BALL TRACKER + ENHANCED LOGGING
# (stabilizes ball detection for camera switching)
# ==============================

import math
import time
from collections import deque
from datetime import datetime
from typing import Optional, Tuple, Dict, List, Deque

print("=" * 50)
print("STICKY BALL TRACKER INITIALIZATION")
print("=" * 50)

# ------------------------------
# Configuration Parameters
# ------------------------------
STICKY_USE_SECONDS = True
STICKY_FPS = 30

# Core thresholds (seconds -> frames when enabled)
STICKY_MAX_HOLD_SEC = 0.60
STICKY_MAX_HOLD_FRAMES = max(1, int(round(STICKY_MAX_HOLD_SEC * STICKY_FPS))) if STICKY_USE_SECONDS else 8
STICKY_MAX_JUMP_PX = 120
STICKY_IOU_GATE = 0.03
STICKY_CONF_GATE = 0.15

# Smoothing / history
STICKY_CENTER_EMA_ALPHA = 0.55
STICKY_CONF_EMA_ALPHA = 0.45
STICKY_HISTORY_LEN = 12
STICKY_VEL_FRAMES = 4

# Jump filtering / candidate confirmation
STICKY_JUMP_VEL_SCALE = 1.5
STICKY_SUSPECT_CONFIRM_FRAMES = 2
STICKY_SUSPECT_MAX_DIST_PX = 60
STICKY_CAM_SWITCH_RELAX_FRAMES = 3
STICKY_RESET_ON_CAM_SWITCH = True

# Confidence shaping
CONF_SHAPING_ENABLED = True
CONF_BOOST_CONSISTENT_FRAMES = 3
CONF_BOOST_AMOUNT = 0.05
CONF_DECAY_MISS = 0.90
CONF_DECAY_LOW_CONF = 0.85

# Hold extension for strong detections
STICKY_HIGH_CONF_HOLD_GATE = 0.45
STICKY_EXTRA_HOLD_FRAMES_HIGH_CONF = 6

# Warmup / re-arm after loss
STICKY_WARMUP_FRAMES = 2
STICKY_REARM_AFTER_MISS = 4

# False Positive Filtering: Exclusion Zones and Stationary Object Detection
ENABLE_EXCLUSION_ZONES = True  # Enable exclusion zones to filter known false positive locations
ENABLE_STATIONARY_FILTER = True  # Filter detections that stay in same place (stationary objects)
STATIONARY_THRESHOLD_PX = 20
STATIONARY_SEC = 1.0
STATIONARY_FRAMES_REQUIRED = max(1, int(round(STATIONARY_SEC * STICKY_FPS))) if STICKY_USE_SECONDS else 30
STATIONARY_CONF_MAX = 0.35
STATIONARY_CONF_DECAY = 0.60
STATIONARY_ALLOW_HIGH_CONF = True

# False Positive Filtering: Exclusion Zones and Stationary Object Detection
ENABLE_EXCLUSION_ZONES = True  # Enable exclusion zones to filter known false positive locations
ENABLE_STATIONARY_FILTER = True  # Filter detections that stay in same place (stationary objects)
STATIONARY_THRESHOLD_PX = 20   # Maximum pixel movement to consider as "stationary"
STATIONARY_FRAMES_REQUIRED = 30  # Number of frames detection must stay stationary to be filtered

# Debug helper for finding false positive coordinates
DEBUG_EXCLUSION_COORDS = False  # Set True to print pixel + normalized coordinates for all detections (disable after configuring zones)
DEBUG_EXCLUSION_COORDS_EVERY_N = 10  # Print coordinates every N frames (to avoid spam)

# Exclusion zones: (x1, y1, x2, y2) in normalized coordinates [0.0-1.0]
# Detections in these zones will be rejected (useful for filtering ground objects, static markers, etc.)
# Format: {camera_id: [(x1, y1, x2, y2), ...]}
# How to find coordinates:
#   1. Note the pixel position (cx, cy) of the false positive from logs
#   2. Convert to normalized: x_norm = cx / frame_width, y_norm = cy / frame_height
#   3. Create zone around it: (x_norm - margin, y_norm - margin, x_norm + margin, y_norm + margin)
#   4. Adjust margin (typically 0.05-0.10) to cover the false positive area
EXCLUSION_ZONES: Dict[int, List[Tuple[float, float, float, float]]] = {
    # Example: Left camera (ID 1) - exclude bottom-left corner (ground object near camera)
    # Uncomment and adjust coordinates based on your camera setup
    # To find the right coordinates:
    #   - Check logs for false positive detections: "[detect_ball] center=(cx,cy)"
    #   - For 1920x1080 frame: x_norm = cx/1920, y_norm = cy/1080
    #   - Example: If false positive at pixel (200, 900): x_norm=0.10, y_norm=0.83
    #   - Create zone: (0.05, 0.78, 0.15, 0.88) - covers area around the false positive
    # Camera 1 (LEFT_CAM): Ground object near camera (from logs: pixel (837, 1013) = normalized (0.436, 0.938))
    1: [
        (0.36, 0.86, 0.52, 1.00),  # Bottom-center area - ground object near left camera
    ],
    # Camera 2 (MIDDLE_CAM): Stationary false positives in top-right area
    # From logs: (1178, 289) = (0.614, 0.268) appears MANY times (most common false positive)
    # From logs: (1120, 248) = (0.583, 0.229) also appears frequently
    # Combined into one larger zone covering both areas since they're close together
    2: [
        (0.50, 0.15, 0.70, 0.35),  # Top-right area - covers both (1178,289) and (1120,248) false positives
    ],
    # Camera 0 (RIGHT_CAM): Add zones here if false positives are found
    # 0: [
    #     (0.00, 0.85, 0.10, 1.00),  # Example: Bottom-left corner
    # ],
}

if ENABLE_EXCLUSION_ZONES and EXCLUSION_ZONES:
    print(f"\n🚫 Exclusion zones enabled for {len(EXCLUSION_ZONES)} camera(s):")
    for cam_id, zones in EXCLUSION_ZONES.items():
        print(f"   Camera {cam_id}: {len(zones)} exclusion zone(s)")
        for i, (x1, y1, x2, y2) in enumerate(zones):
            print(f"      Zone {i+1}: ({x1:.2f}, {y1:.2f}) to ({x2:.2f}, {y2:.2f})")
elif ENABLE_EXCLUSION_ZONES:
    print(f"\n⚠️  Exclusion zones enabled but no zones defined. Add zones to EXCLUSION_ZONES dictionary.")

if ENABLE_STATIONARY_FILTER:
    print(f"\n🛑 Stationary object filter enabled:")
    print(f"   Threshold: {STATIONARY_THRESHOLD_PX}px movement")
    print(f"   Required frames: {STATIONARY_FRAMES_REQUIRED} (detections stationary for this long will be filtered)")

if DEBUG_EXCLUSION_COORDS:
    print(f"\n🔍 Exclusion zone coordinate debug enabled:")
    print(f"   Will print pixel + normalized coordinates every {DEBUG_EXCLUSION_COORDS_EVERY_N} frames")
    print(f"   Use this to find false positive locations and configure EXCLUSION_ZONES")
    print(f"   Set DEBUG_EXCLUSION_COORDS = False to disable")

def visualize_exclusion_zone(cam_id: int, zone: Tuple[float, float, float, float], frame_width: int = 1920, frame_height: int = 1080):
    """
    Visualize an exclusion zone on a frame.

    Args:
        cam_id: Camera ID
        zone: (x1, y1, x2, y2) in normalized coordinates
        frame_width: Frame width in pixels (default: 1920)
        frame_height: Frame height in pixels (default: 1080)

    Returns:
        Dictionary with pixel coordinates and visualization info
    """
    x1_norm, y1_norm, x2_norm, y2_norm = zone

    # Convert to pixel coordinates
    x1_px = int(x1_norm * frame_width)
    y1_px = int(y1_norm * frame_height)
    x2_px = int(x2_norm * frame_width)
    y2_px = int(y2_norm * frame_height)

    width_px = x2_px - x1_px
    height_px = y2_px - y1_px
    area_px = width_px * height_px
    area_percent = (area_px / (frame_width * frame_height)) * 100

    print(f"\n📐 Exclusion Zone Visualization - Camera {cam_id}:")
    print(f"   Normalized: ({x1_norm:.3f}, {y1_norm:.3f}) to ({x2_norm:.3f}, {y2_norm:.3f})")
    print(f"   Pixel coords: ({x1_px}, {y1_px}) to ({x2_px}, {y2_px})")
    print(f"   Size: {width_px}x{height_px} pixels ({area_percent:.1f}% of frame)")
    print(f"   Center: ({x1_px + width_px//2}, {y1_px + height_px//2})")

    return {
        "normalized": (x1_norm, y1_norm, x2_norm, y2_norm),
        "pixel": (x1_px, y1_px, x2_px, y2_px),
        "size_px": (width_px, height_px),
        "area_percent": area_percent,
        "center_px": (x1_px + width_px//2, y1_px + height_px//2)
    }

def suggest_exclusion_zone(cx: int, cy: int, frame_width: int, frame_height: int, margin: float = 0.08) -> Tuple[float, float, float, float]:
    """
    Suggest an exclusion zone around a detection point.

    Args:
        cx, cy: Detection center in pixels
        frame_width, frame_height: Frame dimensions
        margin: Normalized margin around detection (default: 0.08 = 8%)

    Returns:
        (x1, y1, x2, y2) normalized coordinates
    """
    x_norm = cx / float(frame_width)
    y_norm = cy / float(frame_height)

    x1 = max(0.0, x_norm - margin)
    y1 = max(0.0, y_norm - margin)
    x2 = min(1.0, x_norm + margin)
    y2 = min(1.0, y_norm + margin)

    return (x1, y1, x2, y2)

# Debug / Logging Configuration
STICKY_DEBUG = False           # set True to print debug logs to console
ENABLE_STICKY_FILE_LOGGING = True  # write logs to debug directory
STICKY_LOG_EVERY_N = 20       # log to file every N frames
STICKY_LOG_DETAILED = True     # log detailed sticky decisions

# Initialize sticky logging (separate from ball detection logs)
_sticky_log_file = None
_sticky_log_file_path = None
if ENABLE_STICKY_FILE_LOGGING:
    try:
        DEBUG_DIR.mkdir(exist_ok=True, parents=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        _sticky_log_file_path = DEBUG_DIR / f"sticky_tracker_{timestamp}.log"
        _sticky_log_file = open(_sticky_log_file_path, 'w', encoding='utf-8')
        print(f"\n📝 Sticky tracker logging enabled: {_sticky_log_file_path}")
    except Exception as e:
        print(f"⚠️  Warning: Could not create sticky log file: {e}")
        _sticky_log_file = None
        ENABLE_STICKY_FILE_LOGGING = False

def _sticky_log(message: str, level: str = "INFO"):
    """Write sticky tracker log message to file if logging is enabled."""
    if ENABLE_STICKY_FILE_LOGGING and _sticky_log_file is not None:
        try:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            _sticky_log_file.write(f"[{timestamp}] [{level}] {message}\n")
            _sticky_log_file.flush()
        except Exception as e:
            print(f"⚠️  Warning: Failed to write to sticky log file: {e}")

# Statistics tracking for sticky tracker
_sticky_stats = {
    "total_updates": 0,
    "accepted": 0,
    "hold_last": 0,
    "jump_rejected": 0,
    "low_conf_hold": 0,
    "low_conf_lost": 0,
    "fully_lost": 0,
    "total_jump_distance": 0.0,
    "max_jump_distance": 0.0,
    "total_iou": 0.0,
    "iou_count": 0,
    "last_reset_time": time.time()
}

def _update_sticky_stats(reason: str, jump_px: float = 0.0, iou_val: float = 0.0):
    """Update sticky tracker statistics."""
    _sticky_stats["total_updates"] += 1

    if reason == "accepted":
        _sticky_stats["accepted"] += 1
    elif reason == "hold_last":
        _sticky_stats["hold_last"] += 1
    elif reason == "jump_rejected":
        _sticky_stats["jump_rejected"] += 1
    elif reason == "low_conf_hold_last":
        _sticky_stats["low_conf_hold"] += 1
    elif reason == "low_conf_lost":
        _sticky_stats["low_conf_lost"] += 1
    elif reason == "lost":
        _sticky_stats["fully_lost"] += 1

    if jump_px > 0:
        _sticky_stats["total_jump_distance"] += jump_px
        _sticky_stats["max_jump_distance"] = max(_sticky_stats["max_jump_distance"], jump_px)

    if iou_val > 0:
        _sticky_stats["total_iou"] += iou_val
        _sticky_stats["iou_count"] += 1

def get_sticky_stats() -> Dict:
    """Get current sticky tracker statistics."""
    stats = _sticky_stats.copy()
    if stats["total_updates"] > 0:
        stats["acceptance_rate"] = stats["accepted"] / stats["total_updates"]
        stats["hold_rate"] = stats["hold_last"] / stats["total_updates"]
        if stats["iou_count"] > 0:
            stats["avg_iou"] = stats["total_iou"] / stats["iou_count"]
        else:
            stats["avg_iou"] = 0.0
        if stats["accepted"] > 0:
            stats["avg_jump_distance"] = stats["total_jump_distance"] / stats["accepted"]
        else:
            stats["avg_jump_distance"] = 0.0
    else:
        stats["acceptance_rate"] = 0.0
        stats["hold_rate"] = 0.0
        stats["avg_iou"] = 0.0
        stats["avg_jump_distance"] = 0.0
    return stats

def reset_sticky_stats():
    """Reset sticky tracker statistics."""
    global _sticky_stats
    _sticky_stats = {
        "total_updates": 0,
        "accepted": 0,
        "hold_last": 0,
        "jump_rejected": 0,
        "low_conf_hold": 0,
        "low_conf_lost": 0,
        "fully_lost": 0,
        "total_jump_distance": 0.0,
        "max_jump_distance": 0.0,
        "total_iou": 0.0,
        "iou_count": 0,
        "last_reset_time": time.time()
    }
    if ENABLE_STICKY_FILE_LOGGING:
        _sticky_log("Statistics reset", "INFO")

def print_sticky_stats():
    """Print current sticky tracker statistics in a formatted way."""
    stats = get_sticky_stats()

    print("\n" + "=" * 50)
    print("📊 STICKY TRACKER STATISTICS")
    print("=" * 50)

    if stats["total_updates"] == 0:
        print("   No updates processed yet.")
        return

    print(f"\n📈 Decision Statistics:")
    print(f"   Total updates: {stats['total_updates']}")
    print(f"   Accepted: {stats['accepted']} ({stats['acceptance_rate']*100:.1f}%)")
    print(f"   Held last: {stats['hold_last']} ({stats['hold_rate']*100:.1f}%)")
    print(f"   Jump rejected: {stats['jump_rejected']}")
    print(f"   Low conf held: {stats['low_conf_hold']}")
    print(f"   Low conf lost: {stats['low_conf_lost']}")
    print(f"   Fully lost: {stats['fully_lost']}")

    if stats["accepted"] > 0:
        print(f"\n📏 Jump Distance Metrics:")
        print(f"   Average jump: {stats['avg_jump_distance']:.1f} px")
        print(f"   Maximum jump: {stats['max_jump_distance']:.1f} px")

    if stats["iou_count"] > 0:
        print(f"\n🔗 IoU Metrics:")
        print(f"   Average IoU: {stats['avg_iou']:.3f}")

    uptime = time.time() - stats["last_reset_time"]
    print(f"\n⏰ Uptime: {uptime:.1f} seconds")

    print("=" * 50)

    if ENABLE_STICKY_FILE_LOGGING and STICKY_LOG_DETAILED:
        _sticky_log(f"Stats: updates={stats['total_updates']}, accepted={stats['accepted']}, "
                   f"accept_rate={stats['acceptance_rate']*100:.1f}%, hold_rate={stats['hold_rate']*100:.1f}%", "STATS")

def _iou_xyxy(a: Tuple[int,int,int,int], b: Tuple[int,int,int,int]) -> float:
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, ix2 - ix1), max(0, iy2 - iy1)
    inter = iw * ih
    a_area = max(0, ax2-ax1) * max(0, ay2-ay1)
    b_area = max(0, bx2-bx1) * max(0, by2-by1)
    union = a_area + b_area - inter + 1e-9
    return inter / union

def _center_xyxy(b: Tuple[int,int,int,int]) -> Tuple[int,int]:
    x1, y1, x2, y2 = b
    return ((x1 + x2) // 2, (y1 + y2) // 2)

def _dist(p: Tuple[int,int], q: Tuple[int,int]) -> float:
    return math.hypot(p[0] - q[0], p[1] - q[1])

def _is_in_exclusion_zone(cam_id: int, x: float, y: float, w: int, h: int) -> bool:
    """
    Check if normalized point (x, y) is in any exclusion zone for the given camera.
    Returns True if point is in an exclusion zone (should be filtered).
    """
    if not ENABLE_EXCLUSION_ZONES:
        return False

    if cam_id not in EXCLUSION_ZONES:
        return False

    # Normalize coordinates
    if w <= 0 or h <= 0:
        return False

    # x and y should already be normalized, but ensure they are
    if x > 1.0 or y > 1.0:
        # Assume pixel coordinates, normalize them
        x = x / float(w)
        y = y / float(h)

    # Check against all exclusion zones for this camera
    for zone_rect in EXCLUSION_ZONES[cam_id]:
        x1, y1, x2, y2 = zone_rect
        if (x1 <= x <= x2) and (y1 <= y <= y2):
            return True

    return False

def _is_stationary(positions: List[Tuple[int, int]], threshold_px: float) -> bool:
    """
    Check if recent positions indicate a stationary object.
    Returns True if all positions are within threshold_px of each other.
    """
    if len(positions) < 2:
        return False

    # Check if all positions are within threshold of the first position
    first_pos = positions[0]
    for pos in positions[1:]:
        dist = _dist(first_pos, pos)
        if dist > threshold_px:
            return False

    return True

class StickyBallTracker:
    """
    Wraps detect_ball(frame) -> BallDet and stabilizes output:
    - Accepts good detections
    - Rejects 'jump' detections that are far from last ball AND don't overlap
    - Holds last good detection for a few frames when YOLO misses

    Enhanced with logging and statistics tracking.
    """
    def __init__(self,
                 max_hold_frames: int = STICKY_MAX_HOLD_FRAMES,
                 max_jump_px: int = STICKY_MAX_JUMP_PX,
                 iou_gate: float = STICKY_IOU_GATE,
                 conf_gate: float = STICKY_CONF_GATE,
                 debug: bool = STICKY_DEBUG):
        self.max_hold_frames = int(max_hold_frames)
        self.max_jump_px = int(max_jump_px)
        self.iou_gate = float(iou_gate)
        self.conf_gate = float(conf_gate)
        self.debug = bool(debug)

        self.last_det: Optional[BallDet] = None
        self.hold_count: int = 0
        self.total_miss_count: int = 0
        self.update_counter: int = 0

        # Stationary object detection tracking
        self.stationary_positions: List[Tuple[int, int]] = []
        self.stationary_frame_count: int = 0
        self.current_cam_id: Optional[int] = None

        # Smoothing / history
        self.center_hist: Deque[Tuple[float, float]] = deque(maxlen=STICKY_HISTORY_LEN)
        self.conf_hist: Deque[float] = deque(maxlen=STICKY_HISTORY_LEN)
        self.ema_center: Optional[Tuple[float, float]] = None
        self.ema_conf: Optional[float] = None
        self.consecutive_hits: int = 0
        self.consecutive_misses: int = 0

        # Jump candidate and camera switch handling
        self._suspect_det: Optional[BallDet] = None
        self._suspect_count: int = 0
        self._cam_relax_until: int = 0
        self._warmup_left: int = 0

        if ENABLE_STICKY_FILE_LOGGING:
            _sticky_log(f"StickyBallTracker initialized: hold={max_hold_frames}, jump_px={max_jump_px}, "
                       f"iou_gate={iou_gate}, conf_gate={conf_gate}", "INFO")

    def reset(self):
        """Reset tracker state (but keep statistics)."""
        old_hold = self.hold_count
        old_miss = self.total_miss_count
        self.last_det = None
        self.hold_count = 0
        self.total_miss_count = 0
        self.stationary_positions = []
        self.stationary_frame_count = 0
        self.center_hist.clear()
        self.conf_hist.clear()
        self.ema_center = None
        self.ema_conf = None
        self.consecutive_hits = 0
        self.consecutive_misses = 0
        self._suspect_det = None
        self._suspect_count = 0
        self._warmup_left = 0

        if ENABLE_STICKY_FILE_LOGGING:
            _sticky_log(f"Tracker reset (was holding={old_hold}, miss_count={old_miss})", "INFO")

        if self.debug:
            print(f"[sticky] Reset tracker (hold={old_hold}, miss={old_miss})")

    def set_camera_id(self, cam_id: int):
        """Set current camera ID for exclusion zone filtering."""
        self.current_cam_id = cam_id

    def set_current_cam_id(self, cam_id: int):
        """Alias for set_camera_id (kept for orchestrator compatibility)."""
        self.set_camera_id(cam_id)

    def _effective_max_hold_frames(self) -> int:
        extra = 0
        if (self.last_det is not None and self.last_det.conf is not None and
                self.last_det.conf >= STICKY_HIGH_CONF_HOLD_GATE):
            extra = STICKY_EXTRA_HOLD_FRAMES_HIGH_CONF
        return int(self.max_hold_frames + max(0, extra))

    def _speed_px(self) -> float:
        if len(self.center_hist) <= STICKY_VEL_FRAMES:
            return 0.0
        x2, y2 = self.center_hist[-1]
        x1, y1 = self.center_hist[-1 - STICKY_VEL_FRAMES]
        return _dist((x2, y2), (x1, y1)) / float(STICKY_VEL_FRAMES)

    def _apply_center_smoothing(self, center: Tuple[float, float]) -> Tuple[float, float]:
        if STICKY_CENTER_EMA_ALPHA <= 0:
            return center
        if self.ema_center is None:
            self.ema_center = center
        else:
            a = STICKY_CENTER_EMA_ALPHA
            self.ema_center = (
                a * center[0] + (1 - a) * self.ema_center[0],
                a * center[1] + (1 - a) * self.ema_center[1],
            )
        return self.ema_center

    def _apply_conf_shaping(self, raw_conf: float, has_bbox: bool) -> float:
        if not CONF_SHAPING_ENABLED:
            return raw_conf

        if has_bbox:
            self.consecutive_hits += 1
            self.consecutive_misses = 0
            if self.ema_conf is None:
                self.ema_conf = raw_conf
            else:
                a = STICKY_CONF_EMA_ALPHA
                self.ema_conf = (a * raw_conf) + ((1 - a) * self.ema_conf)
            shaped = self.ema_conf
            if self.consecutive_hits >= CONF_BOOST_CONSISTENT_FRAMES:
                shaped = min(1.0, shaped + CONF_BOOST_AMOUNT)
        else:
            self.consecutive_misses += 1
            self.consecutive_hits = 0
            if self.ema_conf is not None:
                self.ema_conf *= CONF_DECAY_MISS
            shaped = self.ema_conf if self.ema_conf is not None else 0.0

        if raw_conf < self.conf_gate:
            shaped *= CONF_DECAY_LOW_CONF

        return max(0.0, min(1.0, float(shaped)))

    def _reset_suspect(self):
        self._suspect_det = None
        self._suspect_count = 0

    def update(self, frame_bgr, cam_id: Optional[int] = None) -> BallDet:
        """
        Returns a stabilized BallDet.
        Adds fields in det.meta:
          - sticky: True/False
          - reason: accepted | hold_last | jump_rejected | lost | low_conf_hold_last | low_conf_lost | exclusion_zone | stationary
          - hold_count, total_miss_count
          - jump_px, iou_with_last (when applicable)
        """
        self.update_counter += 1

        cam_switched = False
        if cam_id is not None:
            cam_id = int(cam_id)
            if self.current_cam_id is not None and cam_id != self.current_cam_id:
                cam_switched = True
            self.current_cam_id = cam_id

        if cam_switched:
            if STICKY_RESET_ON_CAM_SWITCH:
                self.reset()
            self._cam_relax_until = self.update_counter + STICKY_CAM_SWITCH_RELAX_FRAMES
            self._warmup_left = max(self._warmup_left, STICKY_WARMUP_FRAMES)

        # Get detection from ball detector
        try:
            det = detect_ball(frame_bgr)
        except Exception as e:
            error_msg = f"Error in detect_ball during sticky update: {e}"
            if ENABLE_STICKY_FILE_LOGGING:
                _sticky_log(error_msg, "ERROR")
            if self.debug:
                print(f"? {error_msg}")
            # Return last detection if available, otherwise empty
            if self.last_det is not None and self.last_det.bbox is not None:
                out = self.last_det
                if out.meta is None:
                    out.meta = {}
                out.meta.update({"sticky": True, "reason": "error_hold_last"})
                return out
            return BallDet(bbox=None, center=None, conf=0.0, cls=None, meta={"reason": "error"})

        # Ensure meta exists
        if det.meta is None:
            det.meta = {}
        det.meta.setdefault("sticky", False)
        det.meta.setdefault("reason", "unknown")

        # Compute center if missing
        if det.bbox is not None and det.center is None:
            x1, y1, x2, y2 = det.bbox
            det.center = ((x1 + x2) / 2.0, (y1 + y2) / 2.0)

        # Get frame dimensions for exclusion zone checking
        h, w = frame_bgr.shape[:2] if frame_bgr is not None else (0, 0)

        # DEBUG: Print detection coordinates (pixel + normalized) to help configure exclusion zones
        if DEBUG_EXCLUSION_COORDS and det.bbox is not None and det.center is not None:
            if self.update_counter % DEBUG_EXCLUSION_COORDS_EVERY_N == 0:
                cx, cy = det.center
                x_norm = cx / float(w) if w > 0 else 0.0
                y_norm = cy / float(h) if h > 0 else 0.0
                cam_name = f"Camera {self.current_cam_id}" if self.current_cam_id is not None else "Unknown"
                suggested_zone = suggest_exclusion_zone(cx, cy, w, h, margin=0.08)
                x1, y1, x2, y2 = suggested_zone

                print(f" ?? [EXCLUSION DEBUG] Frame {self.update_counter} - {cam_name}:")
                print(f"   Pixel coords: ({cx}, {cy}) | Frame size: {w}x{h}")
                print(f"   Normalized: ({x_norm:.3f}, {y_norm:.3f})")
                print(f"   Suggested zone (margin=0.08): ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")
                print(f"   Copy this line to EXCLUSION_ZONES:")
                if self.current_cam_id is not None:
                    print(f"   {self.current_cam_id}: [({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})],")
                    visualize_exclusion_zone(self.current_cam_id, suggested_zone, w, h)

        # FALSE POSITIVE FILTERING: Check exclusion zones
        if det.bbox is not None and det.center is not None and self.current_cam_id is not None:
            cx, cy = det.center
            x_norm = cx / float(w) if w > 0 else 0.0
            y_norm = cy / float(h) if h > 0 else 0.0

            if _is_in_exclusion_zone(self.current_cam_id, x_norm, y_norm, w, h):
                if self.last_det is not None and self.last_det.bbox is not None and self.hold_count < self._effective_max_hold_frames():
                    self.hold_count += 1
                    self.total_miss_count += 1

                    out = self.last_det
                    if out.meta is None:
                        out.meta = {}
                    out.meta.update({
                        "sticky": True,
                        "reason": "exclusion_zone_hold",
                        "hold_count": self.hold_count,
                        "total_miss_count": self.total_miss_count,
                    })

                    _update_sticky_stats("hold_last")

                    if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                        _sticky_log(f"Exclusion zone filter: cam={self.current_cam_id}, pos=({cx},{cy}), holding last", "DEBUG")

                    return out
                else:
                    _update_sticky_stats("lost")
                    if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                        _sticky_log(f"Exclusion zone filter: cam={self.current_cam_id}, pos=({cx},{cy}), no last to hold", "DEBUG")
                    return BallDet(bbox=None, center=None, conf=0.0, cls=None, meta={"reason": "exclusion_zone", "n": 0})

        raw_conf = float(det.conf or 0.0)
        det.meta["raw_conf"] = raw_conf

        has_bbox = det.bbox is not None and det.center is not None
        if has_bbox:
            raw_center = det.center
            smoothed_center = self._apply_center_smoothing(raw_center)
            if raw_center != smoothed_center:
                det.meta["raw_center"] = (float(raw_center[0]), float(raw_center[1]))
                det.meta["smoothed"] = True
                det.center = smoothed_center

        shaped_conf = self._apply_conf_shaping(raw_conf, has_bbox)
        det.meta["conf_ema"] = float(self.ema_conf or 0.0)
        det.meta["shaped_conf"] = float(shaped_conf)
        det.conf = float(shaped_conf)

        if has_bbox:
            self.center_hist.append((float(det.center[0]), float(det.center[1])))
            self.conf_hist.append(float(det.conf))

        speed_px = self._speed_px() if has_bbox else 0.0
        det.meta["speed_px"] = float(speed_px)

        # FALSE POSITIVE FILTERING: Check for stationary objects
        if ENABLE_STATIONARY_FILTER and has_bbox:
            self.stationary_positions.append((int(det.center[0]), int(det.center[1])))
            max_track_frames = STATIONARY_FRAMES_REQUIRED + 10
            if len(self.stationary_positions) > max_track_frames:
                self.stationary_positions.pop(0)

            if len(self.stationary_positions) >= STATIONARY_FRAMES_REQUIRED:
                recent_positions = self.stationary_positions[-STATIONARY_FRAMES_REQUIRED:]
                if _is_stationary(recent_positions, STATIONARY_THRESHOLD_PX):
                    self.stationary_frame_count += 1
                    det.meta["stationary"] = True
                    det.meta["stationary_frames"] = self.stationary_frame_count

                    # ==============================
                    # IMPROVEMENT: Context-Aware Stationary Filter
                    # Only filter if:
                    # 1. In exclusion zone (known false-positive), OR
                    # 2. Confidence is consistently low (not a legitimate high-confidence stationary ball)
                    # ==============================
                    cx, cy = det.center
                    x_norm = cx / float(w) if w > 0 else 0.0
                    y_norm = cy / float(h) if h > 0 else 0.0
                    in_exclusion_zone = (self.current_cam_id is not None and
                                        _is_in_exclusion_zone(self.current_cam_id, x_norm, y_norm, w, h))

                    # Check confidence history - only filter if confidence is consistently low
                    recent_confs = list(self.conf_hist)[-STATIONARY_FRAMES_REQUIRED:] if len(self.conf_hist) >= STATIONARY_FRAMES_REQUIRED else []
                    avg_conf = sum(recent_confs) / len(recent_confs) if recent_confs else raw_conf
                    conf_is_low = avg_conf < STATIONARY_CONF_MAX

                    # Apply filter only if in exclusion zone OR confidence is low
                    # This prevents filtering legitimate stationary balls (set pieces) with high confidence
                    suppress_stationary = in_exclusion_zone or (conf_is_low and not STATIONARY_ALLOW_HIGH_CONF)

                    if suppress_stationary:
                        det.conf *= STATIONARY_CONF_DECAY
                        det.meta["stationary_decay"] = STATIONARY_CONF_DECAY

                        if det.conf < self.conf_gate:
                            if self.last_det is not None and self.last_det.bbox is not None and self.hold_count < self._effective_max_hold_frames():
                                self.hold_count += 1
                                self.total_miss_count += 1

                                out = self.last_det
                                if out.meta is None:
                                    out.meta = {}
                                out.meta.update({
                                    "sticky": True,
                                    "reason": "stationary_hold",
                                    "hold_count": self.hold_count,
                                    "total_miss_count": self.total_miss_count,
                                })

                                _update_sticky_stats("hold_last")

                                if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                                    _sticky_log(f"Stationary filter: pos=({det.center[0]},{det.center[1]}), frames={self.stationary_frame_count}, holding last", "DEBUG")

                                return out
                            else:
                                _update_sticky_stats("lost")
                                if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                                    _sticky_log(f"Stationary filter: pos=({det.center[0]},{det.center[1]}), frames={self.stationary_frame_count}, no last to hold", "DEBUG")
                                return BallDet(bbox=None, center=None, conf=0.0, cls=None, meta={"reason": "stationary", "n": 0})
                else:
                    self.stationary_frame_count = 0
            else:
                self.stationary_frame_count = 0
        else:
            if det.bbox is None:
                self.stationary_positions = []
                self.stationary_frame_count = 0

        # Case A: got a detection bbox with sufficient confidence
        if det.bbox is not None and det.conf >= self.conf_gate:
            cam_relaxed = self.update_counter <= self._cam_relax_until
            det.meta["cam_relax"] = bool(cam_relaxed)

            if self._warmup_left > 0:
                self._warmup_left -= 1
                det.meta["rearmed"] = True
                det.meta["accepted_reason"] = "warmup"
            elif self.last_det is not None and self.last_det.bbox is not None and not cam_relaxed:
                jump_px = _dist(det.center, self.last_det.center)
                iou_val = _iou_xyxy(det.bbox, self.last_det.bbox)
                allowed_jump = self.max_jump_px + (speed_px * STICKY_JUMP_VEL_SCALE)

                det.meta["jump_px"] = float(jump_px)
                det.meta["iou_with_last"] = float(iou_val)
                det.meta["allowed_jump_px"] = float(allowed_jump)

                if jump_px > allowed_jump and iou_val < self.iou_gate:
                    # Candidate confirmation logic for suspicious jumps
                    if STICKY_SUSPECT_CONFIRM_FRAMES > 1:
                        if self._suspect_det is None:
                            self._suspect_det = det
                            self._suspect_count = 1
                        else:
                            dist_px = _dist(det.center, self._suspect_det.center)
                            if dist_px <= STICKY_SUSPECT_MAX_DIST_PX:
                                self._suspect_count += 1
                            else:
                                self._suspect_det = det
                                self._suspect_count = 1
                        if self._suspect_count >= STICKY_SUSPECT_CONFIRM_FRAMES:
                            det.meta["accepted_reason"] = "suspect_confirmed"
                            self._reset_suspect()
                        else:
                            if self.hold_count < self._effective_max_hold_frames():
                                self.hold_count += 1
                                self.total_miss_count += 1

                                out = self.last_det
                                if out.meta is None:
                                    out.meta = {}
                                out.meta.update({
                                    "sticky": True,
                                    "reason": "jump_candidate",
                                    "hold_count": self.hold_count,
                                    "total_miss_count": self.total_miss_count,
                                    "jump_px": float(jump_px),
                                    "iou_with_last": float(iou_val),
                                })

                                _update_sticky_stats("hold_last")

                                if self.debug:
                                    print(f"[sticky] jump_candidate jump={jump_px:.1f}px iou={iou_val:.3f} "
                                          f"hold={self.hold_count}/{self._effective_max_hold_frames()}")

                                if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                                    _sticky_log(f"Jump candidate: {jump_px:.1f}px > {allowed_jump:.1f}px, "
                                               f"IoU={iou_val:.3f} < {self.iou_gate}, holding last", "DEBUG")

                                return out
                    # Reject big jump + low overlap (likely wrong object)
                    if self.hold_count < self._effective_max_hold_frames():
                        self.hold_count += 1
                        self.total_miss_count += 1

                        out = self.last_det
                        if out.meta is None:
                            out.meta = {}
                        out.meta.update({
                            "sticky": True,
                            "reason": "jump_rejected",
                            "hold_count": self.hold_count,
                            "total_miss_count": self.total_miss_count,
                            "jump_px": float(jump_px),
                            "iou_with_last": float(iou_val),
                        })

                        _update_sticky_stats("jump_rejected", jump_px, iou_val)

                        if self.debug:
                            print(f"[sticky] jump_rejected jump={jump_px:.1f}px iou={iou_val:.3f} "
                                  f"hold={self.hold_count}/{self._effective_max_hold_frames()}")

                        if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                            _sticky_log(f"Jump rejected: {jump_px:.1f}px > {allowed_jump:.1f}px, "
                                       f"IoU={iou_val:.3f} < {self.iou_gate}, holding last", "DEBUG")

                        return out

            # Accept this detection
            jump_px = det.meta.get("jump_px", 0.0)
            iou_val = det.meta.get("iou_with_last", 0.0)

            self.last_det = det
            self.hold_count = 0
            self.total_miss_count = 0
            self._reset_suspect()

            _update_sticky_stats("accepted", jump_px, iou_val)

            det.meta.update({
                "sticky": False,
                "reason": "accepted",
                "hold_count": self.hold_count,
                "total_miss_count": self.total_miss_count,
            })

            if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                if jump_px > 0:
                    _sticky_log(f"Accepted: conf={det.conf:.2f}, jump={jump_px:.1f}px, iou={iou_val:.3f}", "DEBUG")
                else:
                    _sticky_log(f"Accepted: conf={det.conf:.2f} (first detection or no previous)", "DEBUG")

            # Reset stationary tracking when detection is accepted
            self.stationary_positions = []
            self.stationary_frame_count = 0

            return det

        # Case B: bbox exists but confidence too low
        if det.bbox is not None and det.conf < self.conf_gate:
            if self.last_det is not None and self.last_det.bbox is not None and self.hold_count < self._effective_max_hold_frames():
                self.hold_count += 1
                self.total_miss_count += 1

                out = self.last_det
                if out.meta is None:
                    out.meta = {}
                out.meta.update({
                    "sticky": True,
                    "reason": "low_conf_hold_last",
                    "hold_count": self.hold_count,
                    "total_miss_count": self.total_miss_count,
                    "low_conf": float(det.conf),
                })

                _update_sticky_stats("low_conf_hold_last")

                if self.debug:
                    print(f"[sticky] low_conf={det.conf:.2f} hold={self.hold_count}/{self._effective_max_hold_frames()}")

                if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                    _sticky_log(f"Low conf hold: conf={det.conf:.2f} < {self.conf_gate}, "
                               f"holding last (hold={self.hold_count}/{self._effective_max_hold_frames()})", "DEBUG")

                return out

            _update_sticky_stats("low_conf_lost")
            det.meta.update({
                "sticky": False,
                "reason": "low_conf_lost",
                "hold_count": self.hold_count,
                "total_miss_count": self.total_miss_count + 1,
            })
            self.total_miss_count += 1

            if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                _sticky_log(f"Low conf lost: conf={det.conf:.2f} < {self.conf_gate}, no last to hold", "DEBUG")

            return det

        # Case C: no detection bbox -> hold last if available
        if self.last_det is not None and self.last_det.bbox is not None and self.hold_count < self._effective_max_hold_frames():
            self.hold_count += 1
            self.total_miss_count += 1

            out = self.last_det
            if out.meta is None:
                out.meta = {}
            out.meta.update({
                "sticky": True,
                "reason": "hold_last",
                "hold_count": self.hold_count,
                "total_miss_count": self.total_miss_count,
            })

            _update_sticky_stats("hold_last")

            if self.debug:
                print(f"[sticky] hold_last {self.hold_count}/{self._effective_max_hold_frames()}")

            if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
                _sticky_log(f"Holding last: no detection, hold={self.hold_count}/{self._effective_max_hold_frames()}", "DEBUG")

            return out

        # Case D: fully lost
        self.total_miss_count += 1
        _update_sticky_stats("lost")

        if self.total_miss_count >= STICKY_REARM_AFTER_MISS:
            self._warmup_left = max(self._warmup_left, STICKY_WARMUP_FRAMES)

        det.meta.update({
            "sticky": False,
            "reason": "lost",
            "hold_count": self.hold_count,
            "total_miss_count": self.total_miss_count,
        })

        if self.debug:
            print(f"[sticky] lost miss={self.total_miss_count}")

        if ENABLE_STICKY_FILE_LOGGING and (self.update_counter % STICKY_LOG_EVERY_N == 0):
            _sticky_log(f"Fully lost: no detection, no last to hold, miss_count={self.total_miss_count}", "DEBUG")

        return det


# ------------------------------
# Instantiate Tracker
# ------------------------------
sticky_tracker = StickyBallTracker()

# ------------------------------
# Final Summary
# ------------------------------
print("\n" + "=" * 50)
print("✅ Sticky Ball Tracker initialized successfully!")
print("=" * 50)

print("\n⚙️  Configuration Parameters:")
print(f"   Max hold frames: {STICKY_MAX_HOLD_FRAMES}")
print(f"   Max jump distance: {STICKY_MAX_JUMP_PX} px")
print(f"   IoU gate: {STICKY_IOU_GATE}")
print(f"   Confidence gate: {STICKY_CONF_GATE}")
print(f"   Hold seconds: {STICKY_MAX_HOLD_SEC}")
print(f"   FPS (sticky): {STICKY_FPS}")
print(f"   Center EMA alpha: {STICKY_CENTER_EMA_ALPHA}")
print(f"   Conf EMA alpha: {STICKY_CONF_EMA_ALPHA}")
print(f"   Jump vel scale: {STICKY_JUMP_VEL_SCALE}")
print(f"   Suspect confirm frames: {STICKY_SUSPECT_CONFIRM_FRAMES}")
print(f"   Cam-switch relax frames: {STICKY_CAM_SWITCH_RELAX_FRAMES}")
print(f"   Warmup frames: {STICKY_WARMUP_FRAMES}")
print(f"   Stationary frames: {STATIONARY_FRAMES_REQUIRED}")
print(f"   Stationary conf max: {STATIONARY_CONF_MAX}")

print("\n🔧 Debug Settings:")
print(f"   Console debug: {'✅' if STICKY_DEBUG else '❌'}")
print(f"   File logging: {'✅' if ENABLE_STICKY_FILE_LOGGING else '❌'}")
if ENABLE_STICKY_FILE_LOGGING and _sticky_log_file_path:
    print(f"   Log file: {_sticky_log_file_path}")
print(f"   Log frequency: Every {STICKY_LOG_EVERY_N} frames")
print(f"   Detailed logging: {'✅' if STICKY_LOG_DETAILED else '❌'}")

print("\n💡 How It Works:")
print("   - Accepts detections with confidence >= conf_gate")
print("   - Rejects 'jump' detections (far from last + low IoU)")
print("   - Holds last good detection for up to max_hold_frames when detection misses")
print("   - Provides stabilized output for smooth camera switching")

print("\n📊 Available Functions:")
print("   - get_sticky_stats() → Get current statistics")
print("   - print_sticky_stats() → Print formatted statistics")
print("   - reset_sticky_stats() → Reset statistics")
print("   - sticky_tracker.reset() → Reset tracker state")

if ENABLE_STICKY_FILE_LOGGING:
    _sticky_log("Sticky tracker initialization complete", "INFO")
    _sticky_log(f"Config: hold={STICKY_MAX_HOLD_FRAMES}, jump_px={STICKY_MAX_JUMP_PX}, "
               f"iou_gate={STICKY_IOU_GATE}, conf_gate={STICKY_CONF_GATE}", "INFO")

print("\n" + "=" * 50)


STICKY BALL TRACKER INITIALIZATION

🚫 Exclusion zones enabled for 2 camera(s):
   Camera 1: 1 exclusion zone(s)
      Zone 1: (0.36, 0.86) to (0.52, 1.00)
   Camera 2: 1 exclusion zone(s)
      Zone 1: (0.50, 0.15) to (0.70, 0.35)

🛑 Stationary object filter enabled:
   Threshold: 20px movement
   Required frames: 30 (detections stationary for this long will be filtered)

📝 Sticky tracker logging enabled: /content/drive/MyDrive/football/final/debug/sticky_tracker_20260115_182245.log

✅ Sticky Ball Tracker initialized successfully!

⚙️  Configuration Parameters:
   Max hold frames: 18
   Max jump distance: 120 px
   IoU gate: 0.03
   Confidence gate: 0.15
   Hold seconds: 0.6
   FPS (sticky): 30
   Center EMA alpha: 0.55
   Conf EMA alpha: 0.45
   Jump vel scale: 1.5
   Suspect confirm frames: 2
   Cam-switch relax frames: 3
   Warmup frames: 2
   Stationary frames: 30
   Stationary conf max: 0.35

🔧 Debug Settings:
   Console debug: ❌
   File logging: ✅
   Log file: /content/drive/MyDr

In [ ]:
# Sticky tracker self-check (no video I/O)
import numpy as np

def _make_det(center, conf=0.5):
    if center is None:
        return BallDet(bbox=None, center=None, conf=0.0, cls=None, meta={})
    cx, cy = center
    bbox = (int(cx - 5), int(cy - 5), int(cx + 5), int(cy + 5))
    return BallDet(bbox=bbox, center=(float(cx), float(cy)), conf=float(conf), cls=None, meta={})

def _run_sequence(seq):
    idx = {"i": 0}
    def _stub(_frame):
        i = idx["i"]
        idx["i"] += 1
        return seq[i] if i < len(seq) else _make_det(None)
    return _stub

dummy_frame = np.zeros((720, 1280, 3), dtype=np.uint8)
orig_detect = globals().get("detect_ball")

results = []
try:
    # Test 1: smoothing reduces jitter
    jitter_centers = [(200 + (-1)**i * 3, 300 + (-1)**i * 2) for i in range(12)]
    seq1 = [_make_det(c, conf=0.6) for c in jitter_centers]
    globals()["detect_ball"] = _run_sequence(seq1)
    tracker = StickyBallTracker()
    out_centers = []
    for _ in seq1:
        out = tracker.update(dummy_frame, cam_id=1)
        out_centers.append(out.center)
    raw_jitter = sum(abs(jitter_centers[i][0] - jitter_centers[i-1][0]) for i in range(1, len(jitter_centers)))
    smooth_jitter = sum(abs(out_centers[i][0] - out_centers[i-1][0]) for i in range(1, len(out_centers)) if out_centers[i])
    results.append(("smoothing", smooth_jitter < raw_jitter))

    # Test 2: jump filtering rejects single-frame spike
    centers2 = [(400, 400)] * 4 + [(900, 900)] + [(400, 400)] * 3
    seq2 = [_make_det(c, conf=0.6) for c in centers2]
    globals()["detect_ball"] = _run_sequence(seq2)
    tracker = StickyBallTracker()
    outs = [tracker.update(dummy_frame, cam_id=1) for _ in seq2]
    spike_out = outs[4]
    results.append(("jump_filter", spike_out.center == outs[3].center))

    # Test 3: stationary suppression decays confidence
    centers3 = [(500, 500)] * (STATIONARY_FRAMES_REQUIRED + 2)
    seq3 = [_make_det(c, conf=0.25) for c in centers3]
    globals()["detect_ball"] = _run_sequence(seq3)
    tracker = StickyBallTracker()
    outs = [tracker.update(dummy_frame, cam_id=1) for _ in seq3]
    stationary_reasons = [o.meta.get("reason") for o in outs if o is not None and o.meta]
    results.append(("stationary", any(r in ("stationary", "stationary_hold") for r in stationary_reasons)))
finally:
    if orig_detect is not None:
        globals()["detect_ball"] = orig_detect

print("Sticky self-check:")
for name, ok in results:
    print("  {}: {}".format(name, "PASS" if ok else "FAIL"))

Sticky self-check:
  smoothing: PASS
  jump_filter: PASS
  stationary: FAIL


In [ ]:
# ==============================
# VIDEO PROCESSING TEST + ENHANCED LOGGING
# Writes annotated output video with ball tracking
# Enhanced with statistics, logging, and better error handling
# ==============================

import cv2
import time
import json
from datetime import datetime
from pathlib import Path

print("=" * 50)
print("VIDEO PROCESSING TEST")
print("=" * 50)

# ------------------------------
# CONFIGURATION (tune here)
# ------------------------------
VIDEO_IDX = 1                 # Index in INPUT_VIDEOS list
SKIP_SECONDS = 0              # Skip first N seconds
MAX_SECONDS_TO_PROCESS = 500  # Process only N seconds after skip (None = process all)
OUT_FPS_FALLBACK = 30

# YOLO tuning for SMALL BALLS
# UPDATED: Match main detection parameters for consistency
BALL_CONF_THRESH_TEST = 0.22  # Updated from 0.15 to match BALL_CONF_THRESH = 0.22
BALL_IOU_THRESH_TEST  = 0.45
YOLO_IMGSZ_TEST = 1280

# Logging and Statistics
ENABLE_TEST_LOGGING = True    # Write logs to debug directory
SAVE_STATISTICS = True        # Save statistics to JSON file
PROGRESS_LOG_EVERY_N_SEC = 5  # Log progress every N seconds

# ------------------------------
# PATHS & VALIDATION
# ------------------------------
if VIDEO_IDX >= len(INPUT_VIDEOS):
    raise ValueError(f"VIDEO_IDX={VIDEO_IDX} is out of range. Available videos: {len(INPUT_VIDEOS)}")

video_path = Path(INPUT_VIDEOS[VIDEO_IDX])
if not video_path.exists():
    raise FileNotFoundError(f"Video file not found: {video_path}")

name = video_path.stem
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
out_path = OUTPUT_DIR / f"{name}_tracked_{timestamp}.mp4"

# Ensure output directory exists
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

print(f"\n📹 Video Configuration:")
print(f"   Input video: {video_path.name}")
print(f"   Input path: {video_path}")
print(f"   Output path: {out_path}")
print(f"   Video index: {VIDEO_IDX}/{len(INPUT_VIDEOS)-1}")

# Initialize test logging
_test_log_file = None
_test_log_file_path = None
if ENABLE_TEST_LOGGING:
    try:
        DEBUG_DIR.mkdir(exist_ok=True, parents=True)
        _test_log_file_path = DEBUG_DIR / f"test_processing_{timestamp}.log"
        _test_log_file = open(_test_log_file_path, 'w', encoding='utf-8')
        print(f"\n📝 Test logging enabled: {_test_log_file_path}")
    except Exception as e:
        print(f"⚠️  Warning: Could not create test log file: {e}")
        _test_log_file = None
        ENABLE_TEST_LOGGING = False

def _test_log(message: str, level: str = "INFO"):
    """Write test log message to file if logging is enabled."""
    if ENABLE_TEST_LOGGING and _test_log_file is not None:
        try:
            log_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            _test_log_file.write(f"[{log_timestamp}] [{level}] {message}\n")
            _test_log_file.flush()
        except Exception as e:
            print(f"⚠️  Warning: Failed to write to test log file: {e}")

if ENABLE_TEST_LOGGING:
    _test_log(f"Starting video processing test: {video_path.name}", "INFO")
    _test_log(f"Configuration: skip={SKIP_SECONDS}s, max_duration={MAX_SECONDS_TO_PROCESS}s", "INFO")
    _test_log(f"YOLO params: conf={BALL_CONF_THRESH_TEST}, iou={BALL_IOU_THRESH_TEST}, imgsz={YOLO_IMGSZ_TEST}", "INFO")

# ------------------------------
# VIDEO OPEN & METADATA
# ------------------------------
print(f"\n📥 Opening video...")
cap = cv2.VideoCapture(str(video_path))
if not cap.isOpened():
    error_msg = f"❌ Could not open video: {video_path}"
    if ENABLE_TEST_LOGGING:
        _test_log(error_msg, "ERROR")
    raise RuntimeError(
        f"{error_msg}\n"
        "If this is HEVC .MOV and fails, convert to H.264 mp4 first."
    )

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
fps = fps if fps and fps > 0 else OUT_FPS_FALLBACK
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration_sec = total_frames / fps if fps > 0 else 0

skip_frames = int(SKIP_SECONDS * fps)
if MAX_SECONDS_TO_PROCESS is not None:
    max_frames = int(MAX_SECONDS_TO_PROCESS * fps)
else:
    max_frames = total_frames - skip_frames

print(f"\n📊 Video Properties:")
print(f"   Resolution: {width}x{height}")
print(f"   FPS: {fps:.2f}")
print(f"   Total frames: {total_frames}")
print(f"   Duration: {duration_sec:.1f} seconds ({duration_sec/60:.1f} minutes)")
print(f"   Skip: {SKIP_SECONDS}s ({skip_frames} frames)")
print(f"   Process: {MAX_SECONDS_TO_PROCESS}s ({max_frames} frames)" if MAX_SECONDS_TO_PROCESS else f"   Process: All remaining frames")

if ENABLE_TEST_LOGGING:
    _test_log(f"Video properties: {width}x{height} @ {fps:.2f}fps, {total_frames} frames, {duration_sec:.1f}s", "INFO")

# Validate skip and max frames
if skip_frames >= total_frames:
    error_msg = f"❌ Skip frames ({skip_frames}) >= total frames ({total_frames})"
    if ENABLE_TEST_LOGGING:
        _test_log(error_msg, "ERROR")
    raise ValueError(error_msg)

# Seek to skip point
try:
    cap.set(cv2.CAP_PROP_POS_FRAMES, skip_frames)
    actual_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)
    if abs(actual_pos - skip_frames) > 1:
        print(f"⚠️  Warning: Requested frame {skip_frames}, got {actual_pos}")
        if ENABLE_TEST_LOGGING:
            _test_log(f"Seek warning: requested={skip_frames}, actual={actual_pos}", "WARNING")
except Exception as e:
    error_msg = f"Error seeking to frame {skip_frames}: {e}"
    if ENABLE_TEST_LOGGING:
        _test_log(error_msg, "ERROR")
    print(f"⚠️  {error_msg}")

# ------------------------------
# RESET TRACKERS & STATISTICS
# ------------------------------
print(f"\n🔄 Resetting trackers and statistics...")
try:
    sticky_tracker.reset()
    reset_stats()
    reset_sticky_stats()
    print("   ✅ Trackers reset")
    if ENABLE_TEST_LOGGING:
        _test_log("Trackers and statistics reset", "INFO")
except Exception as e:
    print(f"   ⚠️  Warning during reset: {e}")
    if ENABLE_TEST_LOGGING:
        _test_log(f"Reset warning: {e}", "WARNING")

# Statistics for this test run
test_stats = {
    "video_name": video_path.name,
    "video_path": str(video_path),
    "output_path": str(out_path),
    "start_time": datetime.now().isoformat(),
    "configuration": {
        "video_idx": VIDEO_IDX,
        "skip_seconds": SKIP_SECONDS,
        "max_seconds": MAX_SECONDS_TO_PROCESS,
        "conf_thresh": BALL_CONF_THRESH_TEST,
        "iou_thresh": BALL_IOU_THRESH_TEST,
        "imgsz": YOLO_IMGSZ_TEST,
    },
    "video_properties": {
        "width": width,
        "height": height,
        "fps": fps,
        "total_frames": total_frames,
        "duration_sec": duration_sec,
    },
    "processing": {
        "frames_processed": 0,
        "frames_found": 0,
        "frames_held": 0,
        "frames_lost": 0,
        "processing_time_sec": 0.0,
        "fps_processing": 0.0,
    }
}

# ------------------------------
# OVERRIDE detect_ball FOR TEST
# (uses test-specific parameters for better detection)
# ------------------------------
_original_detect_ball = detect_ball

def detect_ball_test(frame_bgr):
    """
    Test version of detect_ball with custom parameters.
    UPDATED: Now matches main detect_ball function structure for consistency.
    """
    global _frame_counter
    _frame_counter += 1

    # Validate input
    if frame_bgr is None or frame_bgr.size == 0:
        if ENABLE_TEST_LOGGING:
            _test_log(f"Frame {_frame_counter}: Invalid input frame", "WARNING")
        return BallDet(bbox=None, center=None, conf=0.0, cls=None,
                       meta={"n": 0, "infer_ms": 0.0, "error": "invalid_frame"})

    h, w = frame_bgr.shape[:2]
    t0 = time.time()

    # Run inference
    try:
        results = ball_model.predict(
            source=frame_bgr,
            conf=BALL_CONF_THRESH_TEST,
            iou=BALL_IOU_THRESH_TEST,
            classes=[BALL_CLASS_ID],
            imgsz=YOLO_IMGSZ_TEST,
            device=0 if DEVICE == "cuda" else "cpu",
            verbose=False
        )
    except Exception as e:
        error_msg = f"Inference error on frame {_frame_counter}: {e}"
        if ENABLE_TEST_LOGGING:
            _test_log(error_msg, "ERROR")
        return BallDet(bbox=None, center=None, conf=0.0, cls=None,
                       meta={"n": 0, "infer_ms": 0.0, "error": str(e)})

    infer_ms = (time.time() - t0) * 1000.0

    # No detections
    if not results or len(results[0].boxes) == 0:
        if ENABLE_TEST_LOGGING:
            _test_log(f"Frame {_frame_counter}: No detections found (conf_thresh={BALL_CONF_THRESH_TEST})", "DEBUG")
        return BallDet(bbox=None, center=None, conf=0.0, cls=None,
                       meta={"n": 0, "infer_ms": infer_ms, "conf_thresh_used": BALL_CONF_THRESH_TEST})

    # ==============================
    # IMPROVEMENT: Match main detect_ball structure with candidate evaluation
    # (Motion consistency and pitch awareness disabled by default, but structure matches)
    # ==============================
    boxes = results[0].boxes
    global _last_ball_center, _last_ball_center_frame

    # Configuration flags (matching main detect_ball)
    ENABLE_MOTION_CONSISTENCY = False  # Disabled by default
    ENABLE_PITCH_AWARE = False  # Disabled by default

    # Evaluate all candidates (matching main detect_ball structure)
    candidates = []
    for box in boxes:
        conf = float(box.conf.item())
        cls_id = int(box.cls.item())
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

        # Clip to frame
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w - 1, x2), min(h - 1, y2)
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
        center = (cx, cy)

        # Motion consistency score (disabled by default)
        motion_score = 1.0
        if ENABLE_MOTION_CONSISTENCY and _last_ball_center is not None and (_frame_counter - _last_ball_center_frame) <= 10:
            try:
                distance = _distance(center, _last_ball_center)
                if distance > _motion_consistency_max_jump_px:
                    if conf < _motion_consistency_high_conf_threshold:
                        motion_score = max(0.3, 1.0 - (distance - _motion_consistency_max_jump_px) / 300.0)
                    else:
                        motion_score = max(0.7, 1.0 - (distance - _motion_consistency_max_jump_px) / 500.0)
                else:
                    motion_score = 1.0 + (1.0 - distance / _motion_consistency_max_jump_px) * 0.1
            except Exception:
                motion_score = 1.0

        # Pitch-aware score (disabled by default)
        pitch_score = 1.0

        # Combined score
        combined_score = conf * motion_score * pitch_score

        candidates.append({
            'box': box,
            'conf': conf,
            'cls_id': cls_id,
            'bbox': (x1, y1, x2, y2),
            'center': center,
            'motion_score': motion_score,
            'pitch_score': pitch_score,
            'combined_score': combined_score
        })

    # Select best candidate based on combined score
    if not candidates:
        if ENABLE_TEST_LOGGING:
            _test_log(f"Frame {_frame_counter}: No candidates after processing {len(boxes)} boxes", "WARNING")
        return BallDet(bbox=None, center=None, conf=0.0, cls=None,
                       meta={"n": 0, "infer_ms": infer_ms, "boxes_from_yolo": len(boxes)})

    best_candidate = max(candidates, key=lambda c: c['combined_score'])

    # Safety check: fallback to highest confidence if combined score too low
    if best_candidate['combined_score'] < 0.01 and len(candidates) > 0:
        best_candidate = max(candidates, key=lambda c: c['conf'])
        if ENABLE_TEST_LOGGING:
            _test_log(f"Frame {_frame_counter}: Fallback to highest confidence: conf={best_candidate['conf']:.3f}, combined={best_candidate['combined_score']:.3f}", "DEBUG")

    # Use best candidate
    conf = best_candidate['conf']
    cls_id = best_candidate['cls_id']
    x1, y1, x2, y2 = best_candidate['bbox']
    cx, cy = best_candidate['center']

    # Update last known position for motion consistency
    _last_ball_center = (float(cx), float(cy))
    _last_ball_center_frame = _frame_counter

    # Debug: Log detection found
    if ENABLE_TEST_LOGGING:
        _test_log(f"Frame {_frame_counter}: Found {len(candidates)} detection(s), best conf={conf:.3f}, combined={best_candidate['combined_score']:.3f}", "DEBUG")

    return BallDet(
        bbox=(x1, y1, x2, y2),
        center=(cx, cy),
        conf=conf,
        cls=cls_id,
        meta={
            "n": len(candidates),
            "infer_ms": infer_ms,
            "motion_score": best_candidate.get('motion_score', 1.0),
            "pitch_score": best_candidate.get('pitch_score', 1.0),
            "combined_score": best_candidate.get('combined_score', conf)
        }
    )

detect_ball = detect_ball_test

if ENABLE_TEST_LOGGING:
    _test_log("detect_ball overridden with test parameters", "INFO")

# ------------------------------
# PROCESS + WRITE OUTPUT VIDEO
# ------------------------------
# UPDATED: Initialize active_cam for test tracking (defaults to camera 0)
if 'active_cam' not in locals() and 'active_cam' not in globals():
    active_cam = 0  # Default camera ID for test tracking
    if ENABLE_TEST_LOGGING:
        _test_log(f"active_cam not defined, defaulting to {active_cam}", "INFO")

writer = None
processed = 0
found = 0
held = 0
lost = 0
errors = 0
t0 = time.time()
last_progress_log = time.time()

print(f"\n🎬 Processing video...")
if ENABLE_TEST_LOGGING:
    _test_log("Starting frame processing", "INFO")
    _test_log(f"Using camera ID: {active_cam}", "INFO")

try:
    while processed < max_frames:
        ok, frame = cap.read()
        if not ok:
            if ENABLE_TEST_LOGGING:
                _test_log(f"End of video reached at frame {processed}", "INFO")
            break

        processed += 1

        # Process frame
        try:
            det = sticky_tracker.update(frame, cam_id=active_cam)
            vis = draw_ball_debug(frame, det, pos_history=None)
        except Exception as e:
            errors += 1
            if ENABLE_TEST_LOGGING and errors <= 5:  # Log first 5 errors
                _test_log(f"Error processing frame {processed}: {e}", "ERROR")
            # Use original frame on error
            vis = frame.copy()
            det = BallDet(bbox=None, center=None, conf=0.0, cls=None, meta={"error": str(e)})

        meta = det.meta or {}
        reason = meta.get("reason", "")
        sticky = meta.get("sticky", False)

        # Update statistics
        if det.bbox is not None and not sticky:
            found += 1
        elif sticky:
            held += 1
        else:
            lost += 1

        # Enhanced overlay
        overlay_y = 30
        cv2.putText(vis, f"Frame: {processed}/{max_frames}", (10, overlay_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        overlay_y += 30
        cv2.putText(vis, f"Found: {found} | Held: {held} | Lost: {lost}", (10, overlay_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        overlay_y += 30
        if reason:
            cv2.putText(vis, f"Reason: {reason} | Conf: {det.conf:.2f}", (10, overlay_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2)

        # Initialize writer on first frame
        if writer is None:
            h, w = vis.shape[:2]
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            writer = cv2.VideoWriter(str(out_path), fourcc, fps, (w, h))
            if not writer.isOpened():
                raise RuntimeError(f"Failed to initialize video writer for {out_path}")
            print(f"   ✅ Writer initialized: {w}x{h} @ {fps:.2f}fps")
            if ENABLE_TEST_LOGGING:
                _test_log(f"Video writer initialized: {w}x{h} @ {fps:.2f}fps", "INFO")

        writer.write(vis)

        # Progress logging
        current_time = time.time()
        if current_time - last_progress_log >= PROGRESS_LOG_EVERY_N_SEC:
            elapsed = current_time - t0
            processing_fps = processed / elapsed if elapsed > 0 else 0
            progress_pct = (processed / max_frames * 100) if max_frames > 0 else 0

            print(f"   ⏱️  {elapsed:6.1f}s | {progress_pct:5.1f}% | "
                  f"frames={processed}/{max_frames} | "
                  f"found={found} held={held} lost={lost} | "
                  f"fps={processing_fps:.1f}")

            if ENABLE_TEST_LOGGING:
                _test_log(f"Progress: {processed}/{max_frames} frames ({progress_pct:.1f}%), "
                         f"found={found}, held={held}, lost={lost}, fps={processing_fps:.1f}", "INFO")

            last_progress_log = current_time

except KeyboardInterrupt:
    print(f"\n⚠️  Processing interrupted by user at frame {processed}")
    if ENABLE_TEST_LOGGING:
        _test_log(f"Processing interrupted at frame {processed}", "WARNING")
except Exception as e:
    error_msg = f"❌ Error during processing: {e}"
    print(f"\n{error_msg}")
    if ENABLE_TEST_LOGGING:
        _test_log(error_msg, "ERROR")
    raise
finally:
    # Cleanup
    cap.release()
    if writer:
        writer.release()
        writer = None

# Restore original detect_ball
detect_ball = _original_detect_ball

# ------------------------------
# FINAL STATISTICS & SUMMARY
# ------------------------------
processing_time = time.time() - t0
processing_fps = processed / processing_time if processing_time > 0 else 0

test_stats["processing"]["frames_processed"] = processed
test_stats["processing"]["frames_found"] = found
test_stats["processing"]["frames_held"] = held
test_stats["processing"]["frames_lost"] = lost
test_stats["processing"]["errors"] = errors
test_stats["processing"]["processing_time_sec"] = processing_time
test_stats["processing"]["fps_processing"] = processing_fps
test_stats["end_time"] = datetime.now().isoformat()

# Get detection and sticky stats
detection_stats = get_detection_stats()
sticky_stats = get_sticky_stats()
test_stats["detection_stats"] = detection_stats
test_stats["sticky_stats"] = sticky_stats

# Save statistics to file
if SAVE_STATISTICS:
    try:
        stats_file = DEBUG_DIR / f"test_stats_{timestamp}.json"
        with open(stats_file, 'w', encoding='utf-8') as f:
            json.dump(test_stats, f, indent=2, default=str)
        print(f"\n📊 Statistics saved: {stats_file}")
    except Exception as e:
        print(f"⚠️  Warning: Could not save statistics: {e}")

# Print summary
print("\n" + "=" * 50)
print("✅ PROCESSING COMPLETE")
print("=" * 50)

print(f"\n📈 Processing Summary:")
print(f"   Frames processed: {processed}/{max_frames}")
print(f"   Processing time: {processing_time:.1f} seconds ({processing_time/60:.1f} minutes)")
print(f"   Processing speed: {processing_fps:.1f} fps")
print(f"   Real-time factor: {processing_fps/fps:.2f}x" if fps > 0 else "")

print(f"\n⚽ Detection Summary:")
print(f"   Found: {found} ({found/processed*100:.1f}%)" if processed > 0 else "   Found: 0")
print(f"   Held: {held} ({held/processed*100:.1f}%)" if processed > 0 else "   Held: 0")
print(f"   Lost: {lost} ({lost/processed*100:.1f}%)" if processed > 0 else "   Lost: 0")
if errors > 0:
    print(f"   Errors: {errors}")

print(f"\n📁 Output:")
print(f"   Video: {out_path}")
if SAVE_STATISTICS:
    print(f"   Statistics: {stats_file}")
if ENABLE_TEST_LOGGING and _test_log_file_path:
    print(f"   Log file: {_test_log_file_path}")

if ENABLE_TEST_LOGGING:
    _test_log(f"Processing complete: {processed} frames in {processing_time:.1f}s", "INFO")
    _test_log(f"Results: found={found}, held={held}, lost={lost}, errors={errors}", "INFO")
    if _test_log_file:
        _test_log_file.close()

print("\n" + "=" * 50)

VIDEO PROCESSING TEST

📹 Video Configuration:
   Input video: IMG_2789_synced.mp4
   Input path: /content/drive/MyDrive/football/final/input/IMG_2789_synced.mp4
   Output path: /content/drive/MyDrive/football/final/output/IMG_2789_synced_tracked_20260115_152510.mp4
   Video index: 1/1

📝 Test logging enabled: /content/drive/MyDrive/football/final/debug/test_processing_20260115_152510.log

📥 Opening video...

📊 Video Properties:
   Resolution: 1920x1080
   FPS: 30.00
   Total frames: 102273
   Duration: 3409.4 seconds (56.8 minutes)
   Skip: 0s (0 frames)
   Process: 500s (14998 frames)

🔄 Resetting trackers and statistics...
   ✅ Trackers reset

🎬 Processing video...
   ✅ Writer initialized: 1920x1080 @ 30.00fps
   ⏱️     5.0s |   0.2% | frames=37/14998 | found=3 held=0 lost=34 | fps=7.4
   ⏱️    10.0s |   0.8% | frames=125/14998 | found=62 held=28 lost=35 | fps=12.5
   ⏱️    15.1s |   1.3% | frames=191/14998 | found=64 held=52 lost=75 | fps=12.7
   ⏱️    20.1s |   1.9% | frames=280/14

In [ ]:
# ==============================
# CAMERA SWITCHING LOGIC + ENHANCED LOGGING || Main Version
# Efficient camera switching based on ball position and trajectory
# Enhanced with statistics, file logging, and improved error handling
# ==============================

import time
import json
from collections import deque
from dataclasses import dataclass, asdict
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple, Optional, Deque, Any

print("=" * 50)
print("CAMERA SWITCHING LOGIC INITIALIZATION")
print("=" * 50)


# ------------------------------
# CONFIG (tune here)
# ------------------------------

# ==============================
# IMPROVEMENT: FPS-Scaled Thresholds (Seconds-Based)
# ==============================
# Use seconds-based thresholds that scale with FPS for consistent behavior
SWITCHER_USE_SECONDS = True
SWITCHER_FPS = 30  # Default FPS, will be updated from video metadata

# Switching thresholds (in seconds, converted to frames when needed)
# IMPROVED: Increased thresholds for better accuracy and fewer false positives
BALL_MISS_SEC_TO_SWITCH = 0.10        # switch after this many seconds of consecutive misses (3 frames @ 30fps, more stable)
SWITCH_COOLDOWN_SEC = 0.60            # Cooldown period (seconds) after switching (increased to prevent rapid switching)
MIN_HOLD_SEC = 0.0                    # Minimum hold after switch (seconds, 0 = disabled)
ZONE_ARM_SEC = 0.20                   # consecutive seconds in zone to arm (6 frames @ 30fps, more stable arming)
ZONE_STABLE_SEC = 0.15                # require N seconds before zone changes (4-5 frames @ 30fps, more stable)
ZONE_DISARM_GRACE_SEC = 0.10          # grace period before clearing a zone (3 frames @ 30fps)

# Convert to frames (will be recalculated if FPS changes)
def _switcher_sec_to_frames(seconds: float, fps: float = None) -> int:
    """Convert seconds to frames, using provided FPS or default."""
    if fps is None:
        fps = SWITCHER_FPS
    return max(1, int(round(seconds * fps)))

# Legacy frame-based constants (for backward compatibility, computed from seconds)
BALL_MISS_FRAMES_TO_SWITCH = _switcher_sec_to_frames(BALL_MISS_SEC_TO_SWITCH)
SWITCH_COOLDOWN_FRAMES = _switcher_sec_to_frames(SWITCH_COOLDOWN_SEC)
MIN_HOLD_FRAMES = _switcher_sec_to_frames(MIN_HOLD_SEC)
ZONE_ARM_FRAMES = _switcher_sec_to_frames(ZONE_ARM_SEC)
ZONE_STABLE_FRAMES = _switcher_sec_to_frames(ZONE_STABLE_SEC)
ZONE_DISARM_GRACE_FRAMES = _switcher_sec_to_frames(ZONE_DISARM_GRACE_SEC)

# History / motion
HISTORY_LEN = 12                       # positions stored for velocity/trajectory
VEL_FRAMES = 4                         # velocity computed using last N frames gap
MIN_CONF_FOR_FOUND = 0.18              # "true found" threshold (increased to reduce false positives)

# Zone margins (normalized)
ZONE_IN_MARGIN = 0.01               # inset margin for entering zones (normalized)
ZONE_OUT_MARGIN = 0.02              # outset margin for leaving zones (normalized)

# Arming / miss handling (computed from seconds-based values)
BALL_MISS_FRAMES_IN_ZONE_TO_SWITCH = BALL_MISS_FRAMES_TO_SWITCH
BALL_MISS_FRAMES_NOT_IN_ZONE_TO_SWITCH = BALL_MISS_FRAMES_TO_SWITCH

# Speed gating (normalized per frame)
MIN_SPEED_FOR_EXIT = 0.005             # Increased from 0.002 to reduce false switches when ball is barely moving

# ==============================
# DYNAMIC EXIT ZONES CONFIGURATION
# Automatically configures exit zones based on available cameras
# Supports: Right+Left, Middle+Right, Middle+Left, or all three
# ==============================

# ==============================
# IMPROVEMENT: Explicit Camera Roles Configuration
# ==============================
# Define explicit camera roles instead of inferring from names
# Format: {cam_id: "RIGHT" | "LEFT" | "MIDDLE"}
# If not set, will fall back to name-based inference for backward compatibility
CAMERA_ROLES: Optional[Dict[int, str]] = None  # Set this explicitly if you know camera roles

def get_camera_roles() -> Dict[int, str]:
    """
    Get camera roles, using explicit CAMERA_ROLES if available, otherwise infer from names.

    Returns:
        Dict mapping camera_id -> role ("RIGHT", "LEFT", "MIDDLE")
    """
    global CAMERA_ROLES

    # If explicit roles are set, use them
    if CAMERA_ROLES is not None and isinstance(CAMERA_ROLES, dict):
        return CAMERA_ROLES.copy()

    # Otherwise, infer from camera names (backward compatibility)
    camera_names = {}
    if 'CAMERA_NAMES' in globals() and isinstance(CAMERA_NAMES, dict):
        camera_names = CAMERA_NAMES
    elif 'SYNCED_CAMERA_NAMES' in globals() and isinstance(SYNCED_CAMERA_NAMES, dict):
        camera_names = SYNCED_CAMERA_NAMES

    roles = {}
    for cam_id, name in camera_names.items():
        name_upper = name.upper()
        if 'RIGHT' in name_upper:
            roles[cam_id] = "RIGHT"
        elif 'LEFT' in name_upper:
            roles[cam_id] = "LEFT"
        elif 'MIDDLE' in name_upper or 'CENTER' in name_upper:
            roles[cam_id] = "MIDDLE"

    return roles

def update_switcher_fps(fps: float):
    """
    Update switcher FPS and recalculate frame-based thresholds.
    Call this when video FPS is known.

    Args:
        fps: Video FPS (frames per second)
    """
    global SWITCHER_FPS, BALL_MISS_FRAMES_TO_SWITCH, SWITCH_COOLDOWN_FRAMES
    global MIN_HOLD_FRAMES, ZONE_ARM_FRAMES, ZONE_STABLE_FRAMES, ZONE_DISARM_GRACE_FRAMES
    global BALL_MISS_FRAMES_IN_ZONE_TO_SWITCH, BALL_MISS_FRAMES_NOT_IN_ZONE_TO_SWITCH

    SWITCHER_FPS = float(fps)

    # Recalculate all frame-based thresholds
    BALL_MISS_FRAMES_TO_SWITCH = _switcher_sec_to_frames(BALL_MISS_SEC_TO_SWITCH, fps)
    SWITCH_COOLDOWN_FRAMES = _switcher_sec_to_frames(SWITCH_COOLDOWN_SEC, fps)
    MIN_HOLD_FRAMES = _switcher_sec_to_frames(MIN_HOLD_SEC, fps)
    ZONE_ARM_FRAMES = _switcher_sec_to_frames(ZONE_ARM_SEC, fps)
    ZONE_STABLE_FRAMES = _switcher_sec_to_frames(ZONE_STABLE_SEC, fps)
    ZONE_DISARM_GRACE_FRAMES = _switcher_sec_to_frames(ZONE_DISARM_GRACE_SEC, fps)
    BALL_MISS_FRAMES_IN_ZONE_TO_SWITCH = BALL_MISS_FRAMES_TO_SWITCH
    BALL_MISS_FRAMES_NOT_IN_ZONE_TO_SWITCH = BALL_MISS_FRAMES_TO_SWITCH

    if ENABLE_SWITCHER_LOGGING:
        _switcher_log(f"Updated switcher FPS to {fps:.2f}, thresholds recalculated", "INFO")

def build_exit_zones_dynamic() -> Tuple[Dict[int, Dict[str, Tuple[float, float, float, float]]], Dict[int, Dict[str, int]]]:
    """
    Dynamically build EXIT_ZONES and NEXT_CAMERA_BY_ZONE based on available cameras.
    Uses explicit CAMERA_ROLES if available, otherwise infers from CAMERA_NAMES.
    """
    # Get camera roles (explicit or inferred)
    camera_roles = get_camera_roles()

    # Try to get camera map to determine available camera IDs
    camera_map = {}
    if 'CAMERA_MAP' in globals() and isinstance(CAMERA_MAP, dict):
        camera_map = CAMERA_MAP
    elif 'SYNCED_CAMERA_MAP' in globals() and isinstance(SYNCED_CAMERA_MAP, dict):
        camera_map = SYNCED_CAMERA_MAP

    # Identify camera types from roles
    right_cam_id = None
    left_cam_id = None
    middle_cam_id = None

    for cam_id, role in camera_roles.items():
        if role == "RIGHT":
            right_cam_id = cam_id
        elif role == "LEFT":
            left_cam_id = cam_id
        elif role == "MIDDLE":
            middle_cam_id = cam_id

    # If no camera roles found, use camera map IDs (fallback)
    if not camera_roles and camera_map:
        available_ids = sorted(camera_map.keys())
        if len(available_ids) >= 1:
            right_cam_id = available_ids[0]
        if len(available_ids) >= 2:
            left_cam_id = available_ids[1]
        if len(available_ids) >= 3:
            middle_cam_id = available_ids[2]

    # Build exit zones and next camera mappings
    exit_zones = {}
    next_camera_by_zone = {}

    # Right Camera configuration
    if right_cam_id is not None:
        exit_zones[right_cam_id] = {
            "LEFT":        (0.00, 0.00, 0.08, 1.00),      # Left edge
            "LEFT_TOP":    (0.00, 0.00, 0.06, 0.25),      # Left-top corner (slimmer: was 0.10)
            "LEFT_BOTTOM": (0.00, 0.75, 0.06, 1.00),      # Left-bottom corner (slimmer: was 0.10)
            "BOTTOM":      (0.00, 0.95, 1.00, 1.00),      # Bottom edge (slimmer: was 0.92)
            "TOP":         (0.00, 0.00, 1.00, 0.05),      # Top edge (slimmer: was 0.08)
            "RIGHT":       (0.95, 0.00, 1.00, 1.00),      # Right edge (slimmer: was 0.92)
        }

        next_camera_by_zone[right_cam_id] = {}
        # In 3-camera mode: LEFT zone → Middle Camera (must go through middle, not directly to left)
        # In 2-camera mode: LEFT zone → Left Camera (direct switch allowed)
        if middle_cam_id is not None:
            # 3-camera mode: route through middle
            next_camera_by_zone[right_cam_id]["LEFT"] = middle_cam_id
            next_camera_by_zone[right_cam_id]["LEFT_TOP"] = middle_cam_id
            next_camera_by_zone[right_cam_id]["LEFT_BOTTOM"] = middle_cam_id
            next_camera_by_zone[right_cam_id]["BOTTOM"] = middle_cam_id
            next_camera_by_zone[right_cam_id]["TOP"] = middle_cam_id
            next_camera_by_zone[right_cam_id]["RIGHT"] = middle_cam_id  # Right edge also goes to middle
        elif left_cam_id is not None:
            # 2-camera mode: direct switch to left allowed
            next_camera_by_zone[right_cam_id]["LEFT"] = left_cam_id
            next_camera_by_zone[right_cam_id]["LEFT_TOP"] = left_cam_id
            next_camera_by_zone[right_cam_id]["LEFT_BOTTOM"] = left_cam_id
            next_camera_by_zone[right_cam_id]["BOTTOM"] = left_cam_id
            next_camera_by_zone[right_cam_id]["TOP"] = left_cam_id

    # Left Camera configuration
    if left_cam_id is not None:
        exit_zones[left_cam_id] = {
            "RIGHT":       (0.95, 0.00, 1.00, 1.00),      # Right edge (slimmer: was 0.92)
            "RIGHT_TOP":   (0.94, 0.00, 1.00, 0.25),      # Right-top corner (slimmer: was 0.90)
            "RIGHT_BOTTOM":(0.94, 0.75, 1.00, 1.00),      # Right-bottom corner (slimmer: was 0.90)
            "BOTTOM":      (0.00, 0.95, 1.00, 1.00),      # Bottom edge (slimmer: was 0.92)
            "TOP":         (0.00, 0.00, 1.00, 0.05),      # Top edge (slimmer: was 0.08)
            "LEFT":        (0.00, 0.00, 0.08, 1.00),      # Left edge
        }

        next_camera_by_zone[left_cam_id] = {}
        # In 3-camera mode: RIGHT zone → Middle Camera (must go through middle, not directly to right)
        # In 2-camera mode: RIGHT zone → Right Camera (direct switch allowed)
        if middle_cam_id is not None:
            # 3-camera mode: route through middle
            next_camera_by_zone[left_cam_id]["RIGHT"] = middle_cam_id
            next_camera_by_zone[left_cam_id]["RIGHT_TOP"] = middle_cam_id
            next_camera_by_zone[left_cam_id]["RIGHT_BOTTOM"] = middle_cam_id
            next_camera_by_zone[left_cam_id]["BOTTOM"] = middle_cam_id
            next_camera_by_zone[left_cam_id]["TOP"] = middle_cam_id
            next_camera_by_zone[left_cam_id]["LEFT"] = middle_cam_id  # Left edge also goes to middle
        elif right_cam_id is not None:
            # 2-camera mode: direct switch to right allowed
            next_camera_by_zone[left_cam_id]["RIGHT"] = right_cam_id
            next_camera_by_zone[left_cam_id]["RIGHT_TOP"] = right_cam_id
            next_camera_by_zone[left_cam_id]["RIGHT_BOTTOM"] = right_cam_id
            next_camera_by_zone[left_cam_id]["BOTTOM"] = right_cam_id
            next_camera_by_zone[left_cam_id]["TOP"] = right_cam_id

    # Middle Camera configuration
    # Note: Middle camera may have different field of view coverage
    # Zone thresholds can be adjusted based on actual camera positions
    # Test and calibrate based on actual camera setup
    if middle_cam_id is not None:
        # Configurable zone thresholds for middle camera
        # Adjust these values based on actual camera field of view and position
        # NOTE: middle camera is on the opposite sideline, so we use slightly
        # narrower left/right edge bands and slightly taller top/bottom bands
        # to avoid over-triggering switches when the ball is still comfortably
        # inside the middle camera's field of view.
        MIDDLE_CAM_LEFT_THRESHOLD = 0.04      # Slimmer: was 0.06      # Left edge threshold (adjust if needed)
        MIDDLE_CAM_RIGHT_THRESHOLD = 0.96    # Slimmer: was 0.94    # Right edge threshold (adjust if needed)
        MIDDLE_CAM_TOP_THRESHOLD = 0.05       # Slimmer: was 0.10       # Top edge threshold (adjust if needed)
        MIDDLE_CAM_BOTTOM_THRESHOLD = 0.95    # Slimmer: was 0.90    # Bottom edge threshold (adjust if needed)
        MIDDLE_CAM_CORNER_TOP = 0.25         # Slimmer: was 0.28         # Top corner boundary (adjust if needed)
        MIDDLE_CAM_CORNER_BOTTOM = 0.75      # Slimmer: was 0.72      # Bottom corner boundary (adjust if needed)

        exit_zones[middle_cam_id] = {
            "LEFT":        (0.00, 0.00, MIDDLE_CAM_LEFT_THRESHOLD, 1.00),      # Left edge
            "LEFT_TOP":    (0.00, 0.00, MIDDLE_CAM_LEFT_THRESHOLD + 0.02, MIDDLE_CAM_CORNER_TOP),      # Left-top corner
            "LEFT_BOTTOM": (0.00, MIDDLE_CAM_CORNER_BOTTOM, MIDDLE_CAM_LEFT_THRESHOLD + 0.02, 1.00),      # Left-bottom corner
            "RIGHT":       (MIDDLE_CAM_RIGHT_THRESHOLD, 0.00, 1.00, 1.00),      # Right edge
            "RIGHT_TOP":   (MIDDLE_CAM_RIGHT_THRESHOLD - 0.02, 0.00, 1.00, MIDDLE_CAM_CORNER_TOP),      # Right-top corner
            "RIGHT_BOTTOM":(MIDDLE_CAM_RIGHT_THRESHOLD - 0.02, MIDDLE_CAM_CORNER_BOTTOM, 1.00, 1.00),      # Right-bottom corner
            "BOTTOM":      (0.00, MIDDLE_CAM_BOTTOM_THRESHOLD, 1.00, 1.00),      # Bottom edge
            "TOP":         (0.00, 0.00, 1.00, MIDDLE_CAM_TOP_THRESHOLD),      # Top edge
        }

        next_camera_by_zone[middle_cam_id] = {}
        if left_cam_id is not None:
            next_camera_by_zone[middle_cam_id]["LEFT"] = left_cam_id
            next_camera_by_zone[middle_cam_id]["LEFT_TOP"] = left_cam_id
            next_camera_by_zone[middle_cam_id]["LEFT_BOTTOM"] = left_cam_id
        if right_cam_id is not None:
            next_camera_by_zone[middle_cam_id]["RIGHT"] = right_cam_id
            next_camera_by_zone[middle_cam_id]["RIGHT_TOP"] = right_cam_id
            next_camera_by_zone[middle_cam_id]["RIGHT_BOTTOM"] = right_cam_id
        # TOP and BOTTOM use position-based logic (handled in select_next_camera function)
        # Default to right camera if available, otherwise left
        if right_cam_id is not None:
            next_camera_by_zone[middle_cam_id]["BOTTOM"] = right_cam_id
            next_camera_by_zone[middle_cam_id]["TOP"] = right_cam_id
        elif left_cam_id is not None:
            next_camera_by_zone[middle_cam_id]["BOTTOM"] = left_cam_id
            next_camera_by_zone[middle_cam_id]["TOP"] = left_cam_id

    return exit_zones, next_camera_by_zone

# Build exit zones dynamically
EXIT_ZONES, NEXT_CAMERA_BY_ZONE = build_exit_zones_dynamic()

# Print configuration summary
print("\n📊 Dynamic Exit Zones Configuration:")
# Get camera names for display
_display_camera_names = {}
if 'CAMERA_NAMES' in globals() and isinstance(CAMERA_NAMES, dict):
    _display_camera_names = CAMERA_NAMES
elif 'SYNCED_CAMERA_NAMES' in globals() and isinstance(SYNCED_CAMERA_NAMES, dict):
    _display_camera_names = SYNCED_CAMERA_NAMES

if EXIT_ZONES:
    print(f"   Configured for {len(EXIT_ZONES)} camera(s)")
    for cam_id, zones in EXIT_ZONES.items():
        cam_name = _display_camera_names.get(cam_id, f"Camera {cam_id}")
        print(f"   Camera {cam_id} ({cam_name}): {len(zones)} exit zones")
        if cam_id in NEXT_CAMERA_BY_ZONE:
            print(f"      Next cameras: {NEXT_CAMERA_BY_ZONE[cam_id]}")
else:
    print("   ⚠️  No cameras detected - using fallback configuration")
    print("   Make sure CAMERA_NAMES or SYNCED_CAMERA_NAMES is defined in Cell 9 or Cell 2")

# Optional improvements (safe toggles)
USE_TRAJECTORY = True                   # require ball velocity to point toward exit zone (ENABLED for better accuracy)
USE_EXIT_PROBABILITY = True            # compute a probability score instead of simple boolean
EXIT_PROB_THRESHOLD = 0.50             # switch only if exit_prob >= this (increased to reduce false positives)

# Terminal logging
LOG_HEARTBEAT_EVERY_N_FRAMES = 30       # print compact status once per N frames
LOG_EVENTS = True                      # print state-change logs (FOUND/HELD/LOST, zone changes, switch)
LOG_VERBOSE = False                    # add extra internal numbers to logs

# File logging and statistics
ENABLE_SWITCHER_LOGGING = True          # Write logs to debug directory
SAVE_SWITCHER_STATS = True             # Save statistics to JSON file
SWITCHER_LOG_EVERY_N = 20              # Log to file every N frames


# ------------------------------
# Initialize Logging
# ------------------------------
_switcher_log_file = None
_switcher_log_file_path = None
if ENABLE_SWITCHER_LOGGING:
    try:
        DEBUG_DIR.mkdir(exist_ok=True, parents=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        _switcher_log_file_path = DEBUG_DIR / f"camera_switcher_{timestamp}.log"
        _switcher_log_file = open(_switcher_log_file_path, 'w', encoding='utf-8')
        print(f"\n📝 Switcher logging enabled: {_switcher_log_file_path}")
    except Exception as e:
        print(f"⚠️  Warning: Could not create switcher log file: {e}")
        _switcher_log_file = None
        ENABLE_SWITCHER_LOGGING = False

def _switcher_log(message: str, level: str = "INFO"):
    """Write switcher log message to file if logging is enabled."""
    if ENABLE_SWITCHER_LOGGING and _switcher_log_file is not None:
        try:
            log_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            _switcher_log_file.write(f"[{log_timestamp}] [{level}] {message}\n")
            _switcher_log_file.flush()
        except Exception as e:
            print(f"⚠️  Warning: Failed to write to switcher log file: {e}")

# Statistics tracking
_switcher_stats = {
    "total_updates": 0,
    "switches": 0,
    "stays": 0,
    "state_found": 0,
    "state_held": 0,
    "state_lost": 0,
    "zone_changes": 0,
    "cooldown_blocks": 0,
    "miss_threshold_blocks": 0,
    "no_exit_zone_blocks": 0,
    "trajectory_blocks": 0,
    "prob_threshold_blocks": 0,
    "zones_visited": {},
    "switches_by_zone": {},
    "switches_by_camera": {},
    "last_reset_time": time.time()
}

def _update_switcher_stats(action: str, zone: str, from_cam: int, to_cam: int, reason: str):
    """Update switcher statistics."""
    _switcher_stats["total_updates"] += 1

    if action == "SWITCH":
        _switcher_stats["switches"] += 1
        _switcher_stats["switches_by_zone"][zone] = _switcher_stats["switches_by_zone"].get(zone, 0) + 1
        _switcher_stats["switches_by_camera"][f"{from_cam}->{to_cam}"] = _switcher_stats["switches_by_camera"].get(f"{from_cam}->{to_cam}", 0) + 1
    else:
        _switcher_stats["stays"] += 1
        if "cooldown" in reason:
            _switcher_stats["cooldown_blocks"] += 1
        elif "miss<" in reason:
            _switcher_stats["miss_threshold_blocks"] += 1
        elif "no_exit_zone" in reason:
            _switcher_stats["no_exit_zone_blocks"] += 1
        elif "trajectory" in reason:
            _switcher_stats["trajectory_blocks"] += 1
        elif "exit_prob<" in reason:
            _switcher_stats["prob_threshold_blocks"] += 1

def _update_state_stats(state: str):
    """Update state statistics."""
    if state == "FOUND":
        _switcher_stats["state_found"] += 1
    elif state == "HELD":
        _switcher_stats["state_held"] += 1
    elif state == "LOST":
        _switcher_stats["state_lost"] += 1

def _update_zone_stats(zone: str):
    """Update zone statistics."""
    if zone != "NONE":
        _switcher_stats["zones_visited"][zone] = _switcher_stats["zones_visited"].get(zone, 0) + 1

def get_switcher_stats() -> Dict:
    """Get current switcher statistics."""
    stats = _switcher_stats.copy()
    if stats["total_updates"] > 0:
        stats["switch_rate"] = stats["switches"] / stats["total_updates"]
        stats["stay_rate"] = stats["stays"] / stats["total_updates"]
    else:
        stats["switch_rate"] = 0.0
        stats["stay_rate"] = 0.0
    return stats

def reset_switcher_stats():
    """Reset switcher statistics."""
    global _switcher_stats
    _switcher_stats = {
        "total_updates": 0,
        "switches": 0,
        "stays": 0,
        "state_found": 0,
        "state_held": 0,
        "state_lost": 0,
        "zone_changes": 0,
        "cooldown_blocks": 0,
        "miss_threshold_blocks": 0,
        "no_exit_zone_blocks": 0,
        "trajectory_blocks": 0,
        "prob_threshold_blocks": 0,
        "zones_visited": {},
        "switches_by_zone": {},
        "switches_by_camera": {},
        "last_reset_time": time.time()
    }
    if ENABLE_SWITCHER_LOGGING:
        _switcher_log("Statistics reset", "INFO")

def print_switcher_stats():
    """Print current switcher statistics in a formatted way."""
    stats = get_switcher_stats()

    print("\n" + "=" * 50)
    print("📊 CAMERA SWITCHER STATISTICS")
    print("=" * 50)

    if stats["total_updates"] == 0:
        print("   No updates processed yet.")
        return

    print(f"\n📈 Decision Statistics:")
    print(f"   Total updates: {stats['total_updates']}")
    print(f"   Switches: {stats['switches']} ({stats['switch_rate']*100:.1f}%)")
    print(f"   Stays: {stats['stays']} ({stats['stay_rate']*100:.1f}%)")

    print(f"\n📊 State Statistics:")
    print(f"   FOUND: {stats['state_found']}")
    print(f"   HELD: {stats['state_held']}")
    print(f"   LOST: {stats['state_lost']}")

    print(f"\n🚫 Block Reasons:")
    print(f"   Cooldown blocks: {stats['cooldown_blocks']}")
    print(f"   Miss threshold blocks: {stats['miss_threshold_blocks']}")
    print(f"   No exit zone blocks: {stats['no_exit_zone_blocks']}")
    print(f"   Trajectory blocks: {stats['trajectory_blocks']}")
    print(f"   Probability threshold blocks: {stats['prob_threshold_blocks']}")

    if stats["zones_visited"]:
        print(f"\n📍 Zones Visited:")
        for zone, count in sorted(stats["zones_visited"].items(), key=lambda x: x[1], reverse=True):
            print(f"   {zone}: {count}")

    if stats["switches_by_zone"]:
        print(f"\n🔄 Switches by Zone:")
        for zone, count in sorted(stats["switches_by_zone"].items(), key=lambda x: x[1], reverse=True):
            print(f"   {zone}: {count}")

    if stats["switches_by_camera"]:
        print(f"\n📹 Switches by Camera Pair:")
        for pair, count in sorted(stats["switches_by_camera"].items(), key=lambda x: x[1], reverse=True):
            print(f"   {pair}: {count}")

    uptime = time.time() - stats["last_reset_time"]
    print(f"\n⏰ Uptime: {uptime:.1f} seconds")

    print("=" * 50)

    if ENABLE_SWITCHER_LOGGING:
        _switcher_log(f"Stats: updates={stats['total_updates']}, switches={stats['switches']}, "
                     f"switch_rate={stats['switch_rate']*100:.1f}%", "STATS")

def save_switcher_stats(file_path: Optional[Path] = None) -> Path:
    """Save switcher statistics to JSON file."""
    if file_path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_path = DEBUG_DIR / f"switcher_stats_{timestamp}.json"

    try:
        stats = get_switcher_stats()
        # Convert to JSON-serializable format
        stats_json = json.dumps(stats, indent=2, default=str)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(stats_json)

        if ENABLE_SWITCHER_LOGGING:
            _switcher_log(f"Statistics saved to {file_path}", "INFO")
        return file_path
    except Exception as e:
        error_msg = f"Failed to save statistics: {e}"
        if ENABLE_SWITCHER_LOGGING:
            _switcher_log(error_msg, "ERROR")
        print(f"⚠️  {error_msg}")
        raise

# ------------------------------
# Helpers
# ------------------------------

def _norm_xy(cx: int, cy: int, w: int, h: int) -> Tuple[float, float]:
    """Normalize coordinates to [0, 1] range."""
    if w <= 0 or h <= 0:
        return (0.0, 0.0)
    return (cx / float(w), cy / float(h))

def _in_zone(x: float, y: float, zone: Tuple[float, float, float, float]) -> bool:
    """Check if normalized point (x, y) is within zone rectangle."""
    x1, y1, x2, y2 = zone
    return (x1 <= x <= x2) and (y1 <= y <= y2)

def _rect_with_margin(rect: Tuple[float, float, float, float], margin: float) -> Tuple[float, float, float, float]:
    """Shrink (margin>0) or expand (margin<0) a rect in normalized coords."""
    x1, y1, x2, y2 = rect
    x1m = max(0.0, min(1.0, x1 + margin))
    y1m = max(0.0, min(1.0, y1 + margin))
    x2m = max(0.0, min(1.0, x2 - margin))
    y2m = max(0.0, min(1.0, y2 - margin))
    if x1m > x2m:
        x1m, x2m = x2m, x1m
    if y1m > y2m:
        y1m, y2m = y2m, y1m
    return (x1m, y1m, x2m, y2m)

def _zone_of_point_with_margin(cam_id: int, x: float, y: float, margin: float) -> str:
    """Find which exit zone contains point, using an inset/outset margin."""
    zones = EXIT_ZONES.get(cam_id, {})
    sorted_zones = sorted(zones.items(), key=lambda z: len(z[0]), reverse=True)
    for zn, rect in sorted_zones:
        if _in_zone(x, y, _rect_with_margin(rect, margin)):
            return zn
    return "NONE"

def _zone_of_point(cam_id: int, x: float, y: float) -> str:
    """Find which exit zone (if any) contains the normalized point."""
    zones = EXIT_ZONES.get(cam_id, {})
    # Check zones in order - more specific zones first (e.g., LEFT_BOTTOM before LEFT)
    sorted_zones = sorted(zones.items(), key=lambda z: len(z[0]), reverse=True)
    for zn, rect in sorted_zones:
        if _in_zone(x, y, rect):
            return zn
    return "NONE"

def _vel_from_history(hist: Deque[Tuple[float, float]], gap: int = 4) -> Tuple[float, float]:
    """Estimate velocity (vx, vy) in normalized coords per frame from position history."""
    if len(hist) < gap + 1:
        return (0.0, 0.0)
    try:
        x2, y2 = hist[-1]
        x1, y1 = hist[-1 - gap]
        return ((x2 - x1) / float(gap), (y2 - y1) / float(gap))
    except (IndexError, TypeError, ZeroDivisionError):
        return (0.0, 0.0)

def _toward_zone(vx: float, vy: float, zone_name: str) -> bool:
    """
    Check if velocity vector points toward the exit zone.
    Enhanced to handle compound zone names correctly.
    FIX: Lowered thresholds to be more lenient (0.0005 instead of 0.001)
    """
    if zone_name == "NONE":
        return False

    # FIX: Lowered threshold from 0.001 to 0.0005 for more lenient detection
    VELOCITY_THRESHOLD = 0.0005


    # Handle compound zones (check more specific first)
    if "LEFT_BOTTOM" in zone_name or "BOTTOM_LEFT" in zone_name:
        return vx < -VELOCITY_THRESHOLD and vy > VELOCITY_THRESHOLD
    if "RIGHT_BOTTOM" in zone_name or "BOTTOM_RIGHT" in zone_name:
        return vx > VELOCITY_THRESHOLD and vy > VELOCITY_THRESHOLD
    if "LEFT_TOP" in zone_name or "TOP_LEFT" in zone_name:
        return vx < -VELOCITY_THRESHOLD and vy < -VELOCITY_THRESHOLD
    if "RIGHT_TOP" in zone_name or "TOP_RIGHT" in zone_name:
        return vx > VELOCITY_THRESHOLD and vy < -VELOCITY_THRESHOLD

    # Simple directional checks
    if zone_name.startswith("LEFT"):
        return vx < -VELOCITY_THRESHOLD
    if zone_name.startswith("RIGHT"):
        return vx > VELOCITY_THRESHOLD
    if zone_name.startswith("TOP"):
        return vy < -VELOCITY_THRESHOLD
    if zone_name.startswith("BOTTOM"):
        return vy > VELOCITY_THRESHOLD

    return False

def _exit_probability(zone_name: str, vx: float, vy: float, miss_count: int) -> float:
    """
    Cheap, stable heuristic (0..1):
    - zone != NONE increases prob
    - moving toward that zone increases prob
    - more misses increases prob
    """
    if zone_name == "NONE":
        return 0.0

    base = 0.45
    if _toward_zone(vx, vy, zone_name):
        base += 0.25

    # miss_count contribution
    base += min(0.30, 0.03 * miss_count)

    # clamp
    return max(0.0, min(1.0, base))


# ------------------------------
# Switcher State
# ------------------------------

@dataclass
class SwitchDecision:
    action: str                    # "STAY" or "SWITCH"
    from_cam: int
    to_cam: int
    reason: str
    zone: str
    exit_prob: float
    miss_count: int
    cooldown_left: int


class CameraSwitcher:
    """
    Terminal-debuggable camera switching state machine.
    Call update() once per processed frame of the ACTIVE camera.
    """

    def __init__(self):
        self.active_cam: int = 0
        self.cooldown_left: int = 0

        self.miss_count: int = 0
        self.last_state: str = "INIT"            # FOUND / HELD / LOST
        self.last_zone: str = "NONE"

        self.pos_hist: Deque[Tuple[float, float]] = deque(maxlen=HISTORY_LEN)

        self._last_switch_frame: Optional[int] = None
        self._last_active_set_frame: int = 0  # CRITICAL: Initialize to prevent AttributeError

        # Hybrid switching: Track zone state and velocity when ball was in zone
        self._ball_was_in_zone: bool = False      # Was ball in exit zone last frame?
        self._zone_when_in_zone: str = "NONE"     # Which zone was ball in?
        self._velocity_when_in_zone: Tuple[float, float] = (0.0, 0.0)  # Velocity when in zone

        # Zone arming state (for stable zone detection to avoid jitter)
        self._armed_zone: str = "NONE"            # Currently armed zone
        self._zone_arm_count: int = 0            # Frames ball has been in armed zone
        self._zone_armed_frame: int = -10**9    # Frame when zone was armed
        self._zone_last_seen_frame: int = -10**9 # Last frame ball was seen in zone

        # Miss tracking for zone-based switching
        self._miss_in_zone: int = 0               # Consecutive misses while in zone
        self._miss_not_in_zone: int = 0           # Consecutive misses while not in zone

        # Zone stabilization state
        self._zone_candidate: str = "NONE"        # Current candidate zone (for debouncing)
        self._zone_candidate_count: int = 0       # Frames candidate zone has been stable
        self._stable_zone: str = "NONE"           # Confirmed stable zone

    def reset_switch_state(self, active_cam: int = 0):
        """Reset switcher state to initial values."""
        old_cam = self.active_cam
        self.active_cam = int(active_cam)
        self.cooldown_left = 0
        self.miss_count = 0
        self.last_state = "INIT"
        self.last_zone = "NONE"
        self.pos_hist.clear()
        self._last_switch_frame = None
        self._last_active_set_frame = 0  # CRITICAL: Reset to prevent AttributeError

        # Reset hybrid switching state
        self._ball_was_in_zone = False
        self._zone_when_in_zone = "NONE"
        self._velocity_when_in_zone = (0.0, 0.0)

        # Reset zone arming state
        self._armed_zone = "NONE"
        self._zone_arm_count = 0
        self._zone_armed_frame = -10**9
        self._zone_last_seen_frame = -10**9

        # Reset miss tracking
        self._miss_in_zone = 0
        self._miss_not_in_zone = 0

        # Reset zone stabilization state
        self._zone_candidate = "NONE"
        self._zone_candidate_count = 0
        self._stable_zone = "NONE"

        if ENABLE_SWITCHER_LOGGING:
            _switcher_log(f"Switcher state reset: camera {old_cam} -> {self.active_cam}", "INFO")

    def is_cooldown_active(self) -> bool:
        return self.cooldown_left > 0

    def _enter_cooldown(self):
        self.cooldown_left = int(SWITCH_COOLDOWN_FRAMES)

    def _tick_cooldown(self):
        if self.cooldown_left > 0:
            self.cooldown_left -= 1

    def update_ball_history(self, cam_id: int, det, w: int, h: int):
        """
        Updates history only when we have a real position (FOUND or HELD with bbox).
        """
        if det.bbox is None or det.center is None:
            return

        x, y = _norm_xy(det.center[0], det.center[1], w, h)
        self.pos_hist.append((x, y))

    def estimate_ball_velocity(self) -> Tuple[float, float]:
        return _vel_from_history(self.pos_hist, gap=VEL_FRAMES)

    def detect_exit_zone(self, cam_id: int) -> str:
        if len(self.pos_hist) == 0:
            return "NONE"
        x, y = self.pos_hist[-1]
        return _zone_of_point_with_margin(cam_id, x, y, ZONE_IN_MARGIN)

    def _stabilize_zone(self, cam_id: int, raw_zone: str, frame_idx: int) -> str:
        """Debounce zone changes and apply entry/exit hysteresis."""
        pos = self.pos_hist[-1] if len(self.pos_hist) > 0 else None
        if raw_zone != "NONE":
            self._zone_last_seen_frame = frame_idx

        if raw_zone == self._zone_candidate:
            self._zone_candidate_count += 1
        else:
            self._zone_candidate = raw_zone
            self._zone_candidate_count = 1

        if raw_zone != "NONE" and self._zone_candidate_count >= ZONE_STABLE_FRAMES:
            self._stable_zone = raw_zone

        if raw_zone == "NONE" and self._stable_zone != "NONE":
            if pos is not None:
                x, y = pos
                rect = EXIT_ZONES.get(cam_id, {}).get(self._stable_zone)
                if rect is not None and _in_zone(x, y, _rect_with_margin(rect, -ZONE_OUT_MARGIN)):
                    return self._stable_zone
            if (frame_idx - self._zone_last_seen_frame) <= ZONE_DISARM_GRACE_FRAMES:
                return self._stable_zone
            self._stable_zone = "NONE"
            self._zone_candidate = "NONE"
            self._zone_candidate_count = 0

        return self._stable_zone

    def _update_zone_arming(self, zone: str, state: str, conf: float, frame_idx: int) -> bool:
        """Track arming state for stable zones to avoid jitter-triggered switches."""
        if zone != "NONE" and state in ("FOUND", "HELD") and (state == "HELD" or conf >= MIN_CONF_FOR_FOUND):
            if self._armed_zone == zone:
                self._zone_arm_count += 1
            else:
                self._armed_zone = zone
                self._zone_arm_count = 1
                self._zone_armed_frame = frame_idx
            self._zone_last_seen_frame = frame_idx
        else:
            if self._armed_zone != "NONE" and (frame_idx - self._zone_last_seen_frame) > ZONE_DISARM_GRACE_FRAMES:
                self._armed_zone = "NONE"
                self._zone_arm_count = 0
                self._zone_armed_frame = -10**9

        return (self._armed_zone != "NONE" and self._zone_arm_count >= ZONE_ARM_FRAMES)

    def compute_exit_probability(self, zone: str, vx: float, vy: float) -> float:
        if not USE_EXIT_PROBABILITY:
            return 1.0 if zone != "NONE" else 0.0
        return _exit_probability(zone, vx, vy, self.miss_count)

    def select_next_camera(self, cam_id: int, zone: str) -> int:
        """
        Select next camera based on exit zone.

        Routing rules:
        - In 3-camera mode: Left and Right cameras must route through Middle camera.
          No direct Left ↔ Right switches allowed.
        - In 2-camera mode: Direct Left ↔ Right switches are allowed.
        - For middle camera TOP zone: Uses velocity direction.
          If ball moving right (vx > 0) → switch to Left camera.
          If ball moving left (vx < 0) → switch to Right camera.
          Falls back to X position if velocity is too small.
        - For middle camera BOTTOM zone: Uses ball X position to decide between Left/Right.
        """
        mapping = NEXT_CAMERA_BY_ZONE.get(cam_id, {})
        if zone not in mapping:
            return cam_id

        # Get default camera from mapping
        default_cam = mapping.get(zone, cam_id)

        # Detect if current camera is middle camera and if we have left/right cameras
        camera_names = {}
        if 'CAMERA_NAMES' in globals() and isinstance(CAMERA_NAMES, dict):
            camera_names = CAMERA_NAMES
        elif 'SYNCED_CAMERA_NAMES' in globals() and isinstance(SYNCED_CAMERA_NAMES, dict):
            camera_names = SYNCED_CAMERA_NAMES

        current_cam_name = camera_names.get(cam_id, "").upper()
        is_middle_cam = 'MIDDLE' in current_cam_name or 'CENTER' in current_cam_name

        # Find left, right, and middle camera IDs
        left_cam_id = None
        right_cam_id = None
        middle_cam_id = None
        for cid, name in camera_names.items():
            name_upper = name.upper()
            if 'LEFT' in name_upper:
                left_cam_id = cid
            elif 'RIGHT' in name_upper:
                right_cam_id = cid
            elif 'MIDDLE' in name_upper or 'CENTER' in name_upper:
                middle_cam_id = cid

        # Enforce 3-camera routing rule: In 3-camera mode, must go through middle camera
        # Prevent direct Left ↔ Right switches when middle camera exists
        current_cam_name_upper = current_cam_name.upper()
        is_right_cam = 'RIGHT' in current_cam_name_upper and not is_middle_cam
        is_left_cam = 'LEFT' in current_cam_name_upper and not is_middle_cam

        # If we're in 3-camera mode (middle_cam_id exists), enforce routing through middle
        if middle_cam_id is not None:
            # Right camera: LEFT zone must go to Middle (not directly to Left)
            if is_right_cam and zone in ("LEFT", "LEFT_TOP", "LEFT_BOTTOM"):
                if ENABLE_SWITCHER_LOGGING:
                    _switcher_log(f"3-camera routing: Right->Middle (zone={zone}, must go through middle)", "INFO")
                return middle_cam_id

            # Left camera: RIGHT zone must go to Middle (not directly to Right)
            if is_left_cam and zone in ("RIGHT", "RIGHT_TOP", "RIGHT_BOTTOM"):
                if ENABLE_SWITCHER_LOGGING:
                    _switcher_log(f"3-camera routing: Left->Middle (zone={zone}, must go through middle)", "INFO")
                return middle_cam_id

            # Right camera: RIGHT zone goes to Middle
            if is_right_cam and zone == "RIGHT":
                if ENABLE_SWITCHER_LOGGING:
                    _switcher_log(f"3-camera routing: Right->Middle (zone=RIGHT)", "INFO")
                return middle_cam_id

            # Left camera: LEFT zone goes to Middle
            if is_left_cam and zone == "LEFT":
                if ENABLE_SWITCHER_LOGGING:
                    _switcher_log(f"3-camera routing: Left->Middle (zone=LEFT)", "INFO")
                return middle_cam_id

        # Priority-based switching for middle camera TOP/BOTTOM exits
        # TOP zone: Uses velocity direction (if ball moving right → Left camera, if moving left → Right camera)
        # BOTTOM zone: Uses ball X position to determine next camera
        if is_middle_cam and zone in ("TOP", "BOTTOM"):
            if len(self.pos_hist) > 0:
                last_x, last_y = self.pos_hist[-1]

                # For TOP zone: Use velocity to determine direction
                if zone == "TOP":
                    # Estimate velocity from position history
                    vx, vy = self.estimate_ball_velocity()

                    # Velocity threshold to determine direction (avoid noise from small movements)
                    VELOCITY_THRESHOLD = 0.001  # Minimum velocity to consider as "moving"

                    if abs(vx) >= VELOCITY_THRESHOLD:
                        # Ball is moving horizontally - use velocity direction
                        if vx > 0:  # Moving right (positive X direction)
                            # Ball moving right → switch to Left camera
                            if left_cam_id is not None:
                                if ENABLE_SWITCHER_LOGGING:
                                    _switcher_log(f"Velocity-based switch: Middle->Left (TOP zone, vx={vx:+.4f} > 0, moving right)", "INFO")
                                return left_cam_id
                            elif right_cam_id is not None:
                                if ENABLE_SWITCHER_LOGGING:
                                    _switcher_log(f"Velocity-based switch: Middle->Right (fallback, TOP zone, vx={vx:+.4f})", "INFO")
                                return right_cam_id
                        else:  # vx < 0, moving left (negative X direction)
                            # Ball moving left → switch to Right camera
                            if right_cam_id is not None:
                                if ENABLE_SWITCHER_LOGGING:
                                    _switcher_log(f"Velocity-based switch: Middle->Right (TOP zone, vx={vx:+.4f} < 0, moving left)", "INFO")
                                return right_cam_id
                            elif left_cam_id is not None:
                                if ENABLE_SWITCHER_LOGGING:
                                    _switcher_log(f"Velocity-based switch: Middle->Left (fallback, TOP zone, vx={vx:+.4f})", "INFO")
                                return left_cam_id
                    else:
                        # Velocity too small or zero - fallback to X position
                        if ENABLE_SWITCHER_LOGGING:
                            _switcher_log(f"Velocity-based switch: Using X position fallback (TOP zone, vx={vx:+.4f} too small)", "INFO")
                        # Fall through to X position logic below

                # For BOTTOM zone or TOP zone with insufficient velocity: Use X position
                # If ball X position < 0.5 (left half of frame) → Switch to Left Camera
                # If ball X position >= 0.5 (right half of frame) → Switch to Right Camera
                if last_x < 0.5:
                    if left_cam_id is not None:
                        if ENABLE_SWITCHER_LOGGING:
                            _switcher_log(f"Position-based switch: Middle->Left (X={last_x:.3f} < 0.5, zone={zone})", "INFO")
                        return left_cam_id
                    elif right_cam_id is not None:
                        if ENABLE_SWITCHER_LOGGING:
                            _switcher_log(f"Position-based switch: Middle->Right (fallback, X={last_x:.3f}, zone={zone})", "INFO")
                        return right_cam_id
                else:  # last_x >= 0.5
                    if right_cam_id is not None:
                        if ENABLE_SWITCHER_LOGGING:
                            _switcher_log(f"Position-based switch: Middle->Right (X={last_x:.3f} >= 0.5, zone={zone})", "INFO")
                        return right_cam_id
                    elif left_cam_id is not None:
                        if ENABLE_SWITCHER_LOGGING:
                            _switcher_log(f"Position-based switch: Middle->Left (fallback, X={last_x:.3f}, zone={zone})", "INFO")
                        return left_cam_id
            else:
                # No position history available, use default mapping
                if ENABLE_SWITCHER_LOGGING:
                    _switcher_log(f"Priority switch: Middle->Default (no position history, zone={zone})", "INFO")
                return default_cam

        # For all other cases, use the mapping
        return default_cam

    def should_switch_camera(self, cam_id: int, det, zone: str, vx: float, vy: float, exit_prob: float) -> Tuple[bool, str]:
        """
        Hybrid switching system with arming and hysteresis guards.
        Switch when ball disappears from a stable exit zone (if moving toward exit),
        or when strong exit evidence exists while armed.
        """
        # 1) Cooldown gate - Applied consistently across all cameras
        if self.is_cooldown_active():
            return (False, "cooldown_active")

        armed = (self._armed_zone != "NONE" and self._zone_arm_count >= ZONE_ARM_FRAMES)
        if not armed:
            if self._miss_not_in_zone >= BALL_MISS_FRAMES_NOT_IN_ZONE_TO_SWITCH:
                return (False, "lost_not_in_zone_wait_orchestrator")
            return (False, "not_armed")

        # Track current zone state for hybrid switching
        ball_currently_in_zone = (zone != "NONE")
        ball_just_disappeared_from_zone = (self._ball_was_in_zone and not ball_currently_in_zone)

        if ball_currently_in_zone:
            self._ball_was_in_zone = True
            self._zone_when_in_zone = zone
            self._velocity_when_in_zone = (vx, vy)
        else:
            self._ball_was_in_zone = False

        speed = (vx**2 + vy**2) ** 0.5

        # 2) Ball disappeared from zone while armed
        if ball_just_disappeared_from_zone and self._zone_when_in_zone != "NONE":
            prev_vx, prev_vy = self._velocity_when_in_zone
            was_moving_toward_exit = _toward_zone(prev_vx, prev_vy, self._zone_when_in_zone)
            prev_speed = (prev_vx**2 + prev_vy**2) ** 0.5

            # FIX: Relaxed - allow switch if ball disappeared from zone, even if trajectory was weak
            # Only require trajectory if it was STRONGLY away
            if self._miss_in_zone >= BALL_MISS_FRAMES_IN_ZONE_TO_SWITCH:
                # Check if previous velocity was strongly away
                if USE_TRAJECTORY:
                    TRAJECTORY_AWAY_THRESHOLD = 0.002
                    prev_velocity_strongly_away = False
                    if self._zone_when_in_zone.startswith("LEFT") and prev_vx > TRAJECTORY_AWAY_THRESHOLD:
                        prev_velocity_strongly_away = True
                    elif self._zone_when_in_zone.startswith("RIGHT") and prev_vx < -TRAJECTORY_AWAY_THRESHOLD:
                        prev_velocity_strongly_away = True
                    elif self._zone_when_in_zone.startswith("TOP") and prev_vy > TRAJECTORY_AWAY_THRESHOLD:
                        prev_velocity_strongly_away = True
                    elif self._zone_when_in_zone.startswith("BOTTOM") and prev_vy < -TRAJECTORY_AWAY_THRESHOLD:
                        prev_velocity_strongly_away = True

                    if prev_velocity_strongly_away:
                        return (False, "trajectory_strongly_away_when_in_zone")

                    # Only check speed if velocity was toward exit
                    if was_moving_toward_exit and MIN_SPEED_FOR_EXIT > 0 and prev_speed < MIN_SPEED_FOR_EXIT:
                        return (False, "exit_speed_too_low")
                elif MIN_SPEED_FOR_EXIT > 0:
                    prev_speed = (prev_vx**2 + prev_vy**2) ** 0.5
                    if prev_speed < MIN_SPEED_FOR_EXIT:
                        return (False, "exit_speed_too_low")

                if USE_EXIT_PROBABILITY and exit_prob < EXIT_PROB_THRESHOLD:
                    return (False, f"exit_prob<{EXIT_PROB_THRESHOLD:.2f}")
                return (True, f"ball_exited_zone_{self._zone_when_in_zone}")

        # 3) require exit zone evidence (for non-hybrid cases)
        if zone == "NONE":
            return (False, "no_exit_zone")

        # Detect if we're in 2-camera mode (no middle camera)
        camera_names = {}
        if 'CAMERA_NAMES' in globals() and isinstance(CAMERA_NAMES, dict):
            camera_names = CAMERA_NAMES
        elif 'SYNCED_CAMERA_NAMES' in globals() and isinstance(SYNCED_CAMERA_NAMES, dict):
            camera_names = SYNCED_CAMERA_NAMES

        has_middle_cam = any('MIDDLE' in name.upper() or 'CENTER' in name.upper()
                            for name in camera_names.values())
        is_2_camera_mode = len(camera_names) == 2 and not has_middle_cam

        # 3) For 2-camera mode: Only switch when ball passes through zone (not just enters)
        if is_2_camera_mode and det.bbox is not None and det.conf >= MIN_CONF_FOR_FOUND:
            # Check if ball was previously outside this zone (passing through)
            was_in_zone = (self.last_zone == zone)

            # Get current ball position
            if len(self.pos_hist) > 0:
                x, y = self.pos_hist[-1]

                # Get zone boundaries
                zones = EXIT_ZONES.get(cam_id, {})
                zone_rect = zones.get(zone)

                if zone_rect:
                    x1, y1, x2, y2 = zone_rect

                    # Check if ball is near the edge of the zone (passing through)
                    EDGE_THRESHOLD = 0.03  # 3% of frame size
                    near_left_edge = abs(x - x1) < EDGE_THRESHOLD
                    near_right_edge = abs(x - x2) < EDGE_THRESHOLD
                    near_top_edge = abs(y - y1) < EDGE_THRESHOLD
                    near_bottom_edge = abs(y - y2) < EDGE_THRESHOLD
                    near_edge = near_left_edge or near_right_edge or near_top_edge or near_bottom_edge

                    # Require: ball is near edge AND moving toward exit OR just entered zone
                    velocity_toward_exit = _toward_zone(vx, vy, zone)

                    if was_in_zone:
                        # Ball was already in zone - only switch if at edge and moving out
                        if not (near_edge and velocity_toward_exit):
                            return (False, "ball_staying_in_zone")
                    else:
                        # Ball just entered zone - require moving toward exit
                        if not velocity_toward_exit:
                            return (False, "ball_not_moving_toward_exit")

                    # Additional check: velocity magnitude should be significant
                    speed_threshold = MIN_SPEED_FOR_EXIT
                    if speed_threshold > 0 and speed < speed_threshold:
                        return (False, "ball_moving_too_slow")

            # Check other requirements (relaxed trajectory check)
            if USE_TRAJECTORY:
                # Only block if velocity STRONGLY points away
                TRAJECTORY_AWAY_THRESHOLD = 0.002
                velocity_strongly_away = False
                if zone.startswith("LEFT") and vx > TRAJECTORY_AWAY_THRESHOLD:
                    velocity_strongly_away = True
                elif zone.startswith("RIGHT") and vx < -TRAJECTORY_AWAY_THRESHOLD:
                    velocity_strongly_away = True
                elif zone.startswith("TOP") and vy > TRAJECTORY_AWAY_THRESHOLD:
                    velocity_strongly_away = True
                elif zone.startswith("BOTTOM") and vy < -TRAJECTORY_AWAY_THRESHOLD:
                    velocity_strongly_away = True

                if velocity_strongly_away:
                    return (False, "trajectory_strongly_away_from_zone")

            if USE_EXIT_PROBABILITY and exit_prob < EXIT_PROB_THRESHOLD:
                return (False, f"exit_prob<{EXIT_PROB_THRESHOLD:.2f}")
            return (True, "ball_passing_through_zone")

        # 4) Fallback: If ball is missing and was in zone, check if it disappeared from zone
        if det.bbox is None and self._ball_was_in_zone and self._zone_when_in_zone != "NONE":
            prev_vx, prev_vy = self._velocity_when_in_zone
            was_moving_toward_exit = _toward_zone(prev_vx, prev_vy, self._zone_when_in_zone)

           # FIX: Relaxed trajectory check for missing ball
            if self._miss_in_zone >= BALL_MISS_FRAMES_IN_ZONE_TO_SWITCH:
                if USE_TRAJECTORY:
                    # Only block if previous velocity was STRONGLY away
                    TRAJECTORY_AWAY_THRESHOLD = 0.002
                    prev_velocity_strongly_away = False
                    if self._zone_when_in_zone.startswith("LEFT") and prev_vx > TRAJECTORY_AWAY_THRESHOLD:
                        prev_velocity_strongly_away = True
                    elif self._zone_when_in_zone.startswith("RIGHT") and prev_vx < -TRAJECTORY_AWAY_THRESHOLD:
                        prev_velocity_strongly_away = True
                    elif self._zone_when_in_zone.startswith("TOP") and prev_vy > TRAJECTORY_AWAY_THRESHOLD:
                        prev_velocity_strongly_away = True
                    elif self._zone_when_in_zone.startswith("BOTTOM") and prev_vy < -TRAJECTORY_AWAY_THRESHOLD:
                        prev_velocity_strongly_away = True

                    if prev_velocity_strongly_away:
                        return (False, "trajectory_strongly_away_when_missing")

                    # Only check speed if velocity was toward exit
                    if was_moving_toward_exit and MIN_SPEED_FOR_EXIT > 0:
                        prev_speed = (prev_vx**2 + prev_vy**2) ** 0.5
                        if prev_speed < MIN_SPEED_FOR_EXIT:
                            return (False, "exit_speed_too_low")
                elif MIN_SPEED_FOR_EXIT > 0:
                    prev_speed = (prev_vx**2 + prev_vy**2) ** 0.5
                    if prev_speed < MIN_SPEED_FOR_EXIT:
                        return (False, "exit_speed_too_low")

                if USE_EXIT_PROBABILITY and exit_prob < EXIT_PROB_THRESHOLD:
                    return (False, f"exit_prob<{EXIT_PROB_THRESHOLD:.2f}")
                return (True, f"ball_missing_after_exit_{self._zone_when_in_zone}")

        # 5) For 3-camera mode: If ball is found in exit zone, allow switching (don't require misses)
        # FIX: Relaxed trajectory check - allow switch when ball is in zone unless velocity strongly points away
        if det.bbox is not None and det.conf >= MIN_CONF_FOR_FOUND:
           # Check if velocity strongly points AWAY from zone (more strict threshold)
            velocity_strongly_away = False
            if USE_TRAJECTORY:
                # Only block if velocity STRONGLY points away (not just weak/zero)
                TRAJECTORY_AWAY_THRESHOLD = 0.002  # Stricter threshold for "away" detection
                if zone.startswith("LEFT") and vx > TRAJECTORY_AWAY_THRESHOLD:
                    # Moving right, away from LEFT zone
                    velocity_strongly_away = True
                elif zone.startswith("RIGHT") and vx < -TRAJECTORY_AWAY_THRESHOLD:
                    # Moving left, away from RIGHT zone
                    velocity_strongly_away = True
                elif zone.startswith("TOP") and vy > TRAJECTORY_AWAY_THRESHOLD:
                    # Moving down, away from TOP zone
                    velocity_strongly_away = True
                elif zone.startswith("BOTTOM") and vy < -TRAJECTORY_AWAY_THRESHOLD:
                    # Moving up, away from BOTTOM zone
                    velocity_strongly_away = True

                if velocity_strongly_away:
                    return (False, "trajectory_strongly_away_from_zone")

                # If speed requirement is enabled, check it (but be lenient)
                if MIN_SPEED_FOR_EXIT > 0:
                    # Only require speed if velocity points toward zone
                    if _toward_zone(vx, vy, zone) and speed < MIN_SPEED_FOR_EXIT:
                        return (False, "exit_speed_too_low")
                    # If velocity is neutral/zero, allow switch even with low speed
            else:
                # No trajectory check - only check speed if enabled
                if MIN_SPEED_FOR_EXIT > 0 and speed < MIN_SPEED_FOR_EXIT:
                    return (False, "exit_speed_too_low")

            if USE_EXIT_PROBABILITY and exit_prob < EXIT_PROB_THRESHOLD:
                return (False, f"exit_prob<{EXIT_PROB_THRESHOLD:.2f}")
            return (True, "ball_in_exit_zone")

       # 6) optional: require trajectory toward zone (relaxed - only block if strongly away)
        if USE_TRAJECTORY:
            # Only block if velocity STRONGLY points away from zone
            TRAJECTORY_AWAY_THRESHOLD = 0.002
            velocity_strongly_away = False
            if zone.startswith("LEFT") and vx > TRAJECTORY_AWAY_THRESHOLD:
                velocity_strongly_away = True
            elif zone.startswith("RIGHT") and vx < -TRAJECTORY_AWAY_THRESHOLD:
                velocity_strongly_away = True
            elif zone.startswith("TOP") and vy > TRAJECTORY_AWAY_THRESHOLD:
                velocity_strongly_away = True
            elif zone.startswith("BOTTOM") and vy < -TRAJECTORY_AWAY_THRESHOLD:
                velocity_strongly_away = True

            if velocity_strongly_away:
                return (False, "trajectory_strongly_away_from_zone")

        # 7) probability threshold
        if USE_EXIT_PROBABILITY and exit_prob < EXIT_PROB_THRESHOLD:
            return (False, f"exit_prob<{EXIT_PROB_THRESHOLD:.2f}")

        # 8) If ball is missing, require sustained miss
        if self.miss_count < BALL_MISS_FRAMES_TO_SWITCH:
           return (False, f"miss<{BALL_MISS_FRAMES_TO_SWITCH}")

        return (True, "exit_confirmed")

    def log_heartbeat(self, frame_idx: int, cam_id: int, det, zone: str, vx: float, vy: float, exit_prob: float):
        if LOG_HEARTBEAT_EVERY_N_FRAMES <= 0:
            return
        if frame_idx % LOG_HEARTBEAT_EVERY_N_FRAMES != 0:
            return

        meta = det.meta or {}
        sticky = meta.get("sticky", False)
        reason = meta.get("reason", "")
        conf = det.conf if det.conf is not None else 0.0

        state = "FOUND"
        if det.bbox is None:
            state = "LOST"
        elif sticky:
            state = "HELD"

        pos = self.pos_hist[-1] if len(self.pos_hist) else (None, None)
        pos_str = f"pos=({pos[0]:.3f},{pos[1]:.3f})" if pos[0] is not None else "pos=(NA)"

        print(
            f"[HB] f={frame_idx:06d} cam={cam_id} cd={self.cooldown_left:02d} "
            f"state={state} conf={conf:.2f} miss={self.miss_count:02d} "
            f"{pos_str} zone={zone} v=({vx:+.3f},{vy:+.3f}) exit={exit_prob:.2f} "
            f"sticky={sticky} r={reason}"
        )

    def log_event(self, msg: str):
        if LOG_EVENTS:
            print(msg)

    def update_active_camera(self, to_cam: int, frame_idx: int):
        from_cam = self.active_cam
        self.active_cam = int(to_cam)
        self._enter_cooldown()
        self._last_switch_frame = frame_idx
        self._last_active_set_frame = frame_idx
        self.miss_count = 0
        self.last_zone = "NONE"
        self.pos_hist.clear()
        self.log_event(f"[SWITCH] f={frame_idx:06d} {from_cam} -> {to_cam} cooldown={self.cooldown_left}")

    def update(self, cam_id: int, det, frame_shape: Tuple[int, int], frame_idx: int) -> SwitchDecision:
        """
        Update switcher based on the ACTIVE camera frame's detection.
        Returns a SwitchDecision describing what happened.

        Args:
            cam_id: Active camera ID
            det: BallDet object from sticky_tracker.update(frame)
            frame_shape: (height, width) tuple
            frame_idx: Current frame index

        Returns:
            SwitchDecision object with action and details
        """
        # Input validation
        try:
            h, w = frame_shape
            if h <= 0 or w <= 0:
                raise ValueError(f"Invalid frame shape: {frame_shape}")
            cam_id = int(cam_id)
            frame_idx = int(frame_idx)
        except (ValueError, TypeError) as e:
            error_msg = f"Invalid input to update(): {e}"
            if ENABLE_SWITCHER_LOGGING:
                _switcher_log(error_msg, "ERROR")
            raise ValueError(error_msg)

        self.active_cam = cam_id

        # cooldown tick
        self._tick_cooldown()

        min_hold_active = False
        if MIN_HOLD_FRAMES > 0:
            min_hold_active = (frame_idx - self._last_active_set_frame) < MIN_HOLD_FRAMES

        # Determine state from det
        meta = det.meta or {}
        sticky = bool(meta.get("sticky", False))
        conf = float(det.conf or 0.0)

        # FOUND = bbox present AND not sticky AND conf >= MIN_CONF_FOR_FOUND
        # HELD  = bbox present AND sticky
        # LOST  = bbox missing OR (bbox present but low conf and not sticky)
        if det.bbox is None:
            state = "LOST"
        elif sticky:
            state = "HELD"
        elif conf >= MIN_CONF_FOR_FOUND:
            state = "FOUND"
        else:
            # BUG FIX: Low conf detection without sticky should be treated as LOST, not FOUND
            # This was causing incorrect state tracking
            state = "LOST"

        # Update miss counter:
        # - On FOUND: reset miss
        # - On HELD: increment slightly (it means detector is missing but sticky holds)
        # - On LOST: increment
        if state == "FOUND":
            self.miss_count = 0
        elif state == "HELD":
            self.miss_count += 1
        else:  # LOST
            self.miss_count += 1

        # Update statistics
        _update_state_stats(state)

        # Event logs for state transitions
        if state != self.last_state:
            event_msg = f"[STATE] f={frame_idx:06d} cam={cam_id} {self.last_state} -> {state} miss={self.miss_count}"
            self.log_event(event_msg)
            if ENABLE_SWITCHER_LOGGING and (frame_idx % SWITCHER_LOG_EVERY_N == 0):
                _switcher_log(event_msg, "INFO")
            self.last_state = state

        # Update position history if we have a position (FOUND/HELD)
        self.update_ball_history(cam_id, det, w, h)

        # Exit analysis
        try:
            vx, vy = self.estimate_ball_velocity()
            raw_zone = self.detect_exit_zone(cam_id)
            zone = self._stabilize_zone(cam_id, raw_zone, frame_idx)

            # Debug: Log zone detection issues
            if raw_zone == "NONE" and len(self.pos_hist) > 0:
                last_x, last_y = self.pos_hist[-1]
                zones_available = EXIT_ZONES.get(cam_id, {})
                if not zones_available:
                    if ENABLE_SWITCHER_LOGGING and (frame_idx % SWITCHER_LOG_EVERY_N == 0):
                        _switcher_log(f"WARNING: No exit zones defined for camera {cam_id} (pos=({last_x:.3f},{last_y:.3f}))", "WARNING")
        except Exception as e:
            if ENABLE_SWITCHER_LOGGING:
                _switcher_log(f"Error in exit analysis: {e}", "ERROR")
            vx, vy = (0.0, 0.0)
            raw_zone = "NONE"
            zone = "NONE"

        if zone != self.last_zone:
            _switcher_stats["zone_changes"] += 1
            _update_zone_stats(zone)
            event_msg = f"[ZONE]  f={frame_idx:06d} cam={cam_id} {self.last_zone} -> {zone}"
            self.log_event(event_msg)
            if ENABLE_SWITCHER_LOGGING and (frame_idx % SWITCHER_LOG_EVERY_N == 0):
                _switcher_log(event_msg, "INFO")
            self.last_zone = zone
        else:
            _update_zone_stats(zone)

        armed = self._update_zone_arming(zone, state, conf, frame_idx)

        if state == "LOST":
            if armed:
                self._miss_in_zone += 1
            else:
                self._miss_not_in_zone += 1
        else:
            self._miss_in_zone = 0
            self._miss_not_in_zone = 0

        try:
            exit_prob = self.compute_exit_probability(zone, vx, vy)
        except Exception as e:
            if ENABLE_SWITCHER_LOGGING:
                _switcher_log(f"Error computing exit probability: {e}", "ERROR")
            exit_prob = 0.0

        # Heartbeat log
        self.log_heartbeat(frame_idx, cam_id, det, zone, vx, vy, exit_prob)

        # Decision
        try:
            do_switch, reason = self.should_switch_camera(cam_id, det, zone, vx, vy, exit_prob)
        except Exception as e:
            if ENABLE_SWITCHER_LOGGING:
                _switcher_log(f"Error in should_switch_camera: {e}", "ERROR")
            do_switch, reason = (False, f"error: {str(e)}")

        if min_hold_active:
            do_switch, reason = (False, f"min_hold({MIN_HOLD_FRAMES})")

        if do_switch:
            try:
                to_cam = self.select_next_camera(cam_id, zone)
            except Exception as e:
                if ENABLE_SWITCHER_LOGGING:
                    _switcher_log(f"Error selecting next camera: {e}", "ERROR")
                to_cam = cam_id  # Fallback to staying

            if to_cam == cam_id:
                # no mapping; stay
                decision = SwitchDecision("STAY", cam_id, cam_id, "no_mapping", zone, exit_prob, self.miss_count, self.cooldown_left)
                _update_switcher_stats("STAY", zone, cam_id, cam_id, "no_mapping")
                event_msg = f"[DEC]   f={frame_idx:06d} cam={cam_id} STAY reason=no_mapping zone={zone}"
                self.log_event(event_msg)
                if ENABLE_SWITCHER_LOGGING and (frame_idx % SWITCHER_LOG_EVERY_N == 0):
                    _switcher_log(event_msg, "INFO")
                return decision

            event_msg = (
                f"[DEC]   f={frame_idx:06d} cam={cam_id} SWITCH->{to_cam} reason={reason} "
                f"zone={zone} exit={exit_prob:.2f} miss={self.miss_count} v=({vx:+.3f},{vy:+.3f})"
            )
            self.log_event(event_msg)
            if ENABLE_SWITCHER_LOGGING:
                _switcher_log(event_msg, "INFO")

            self.update_active_camera(to_cam, frame_idx)
            decision = SwitchDecision("SWITCH", cam_id, to_cam, reason, zone, exit_prob, self.miss_count, self.cooldown_left)
            _update_switcher_stats("SWITCH", zone, cam_id, to_cam, reason)
            return decision

        # Stay
        stay_reason = reason
        if self.is_cooldown_active():
            stay_reason = f"cooldown({self.cooldown_left})"
        if LOG_VERBOSE:
            self.log_event(f"[DEC]   f={frame_idx:06d} cam={cam_id} STAY reason={stay_reason}")

        decision = SwitchDecision("STAY", cam_id, cam_id, stay_reason, zone, exit_prob, self.miss_count, self.cooldown_left)
        _update_switcher_stats("STAY", zone, cam_id, cam_id, stay_reason)
        return decision


# ------------------------------
# Configuration Validation
# ------------------------------
print(f"\n🔍 Validating configuration...")

# Validate EXIT_ZONES and NEXT_CAMERA_BY_ZONE consistency
validation_errors = []
# Get camera roles to identify edge zones (RIGHT on right camera, LEFT on left camera)
camera_roles = get_camera_roles()
right_cam_id = None
left_cam_id = None
for cam_id, role in camera_roles.items():
    if role == "RIGHT":
        right_cam_id = cam_id
    elif role == "LEFT":
        left_cam_id = cam_id

for cam_id, zones in EXIT_ZONES.items():
    if cam_id not in NEXT_CAMERA_BY_ZONE:
        validation_errors.append(f"Camera {cam_id} has EXIT_ZONES but no NEXT_CAMERA_BY_ZONE mapping")
    else:
        for zone_name in zones.keys():
            if zone_name not in NEXT_CAMERA_BY_ZONE[cam_id]:
                # Edge zones (RIGHT on right camera, LEFT on left camera) are expected to not have mappings
                # These represent the ball going off-field, not switching to another camera
                is_edge_zone = False
                if cam_id == right_cam_id and zone_name == "RIGHT":
                    is_edge_zone = True
                elif cam_id == left_cam_id and zone_name == "LEFT":
                    is_edge_zone = True

                if not is_edge_zone:
                    validation_errors.append(f"Zone '{zone_name}' for camera {cam_id} has no NEXT_CAMERA_BY_ZONE mapping")

if validation_errors:
    print("⚠️  Configuration warnings:")
    for error in validation_errors:
        print(f"   {error}")
    if ENABLE_SWITCHER_LOGGING:
        for error in validation_errors:
            _switcher_log(f"Config warning: {error}", "WARNING")
else:
    print("   ✅ Configuration validated")

# ------------------------------
# Instantiate (used later by orchestrator)
# ------------------------------
camera_switcher = CameraSwitcher()

# Set your starting camera here - default to camera 1 (RIGHT_CAM) as fallback
START_CAMERA = 1  # Changed to camera 1 (RIGHT_CAM) as default fallback
if START_CAMERA not in EXIT_ZONES:
    # Fallback to first available camera with zones, or camera 1 if available
    if 1 in EXIT_ZONES:
        START_CAMERA = 1
    elif 0 in EXIT_ZONES:
        START_CAMERA = 0
    else:
        START_CAMERA = list(EXIT_ZONES.keys())[0] if EXIT_ZONES else 1
camera_switcher.reset_switch_state(active_cam=START_CAMERA)

if ENABLE_SWITCHER_LOGGING:
    _switcher_log(f"CameraSwitcher initialized with start camera: {START_CAMERA}", "INFO")
    _switcher_log(f"Config: miss_frames={BALL_MISS_FRAMES_TO_SWITCH}, cooldown={SWITCH_COOLDOWN_FRAMES}", "INFO")
    _switcher_log(f"Config: trajectory={USE_TRAJECTORY}, exit_prob={USE_EXIT_PROBABILITY}, threshold={EXIT_PROB_THRESHOLD}", "INFO")

print("\n" + "=" * 50)
print("✅ CAMERA SWITCHER INITIALIZED")
print("=" * 50)

print(f"\n⚙️  Configuration:")
print(f"   Start camera: {camera_switcher.active_cam}")
print(f"   Switch after miss frames: {BALL_MISS_FRAMES_TO_SWITCH}")
print(f"   Cooldown frames: {SWITCH_COOLDOWN_FRAMES}")
print(f"   History length: {HISTORY_LEN}")
print(f"   Velocity frames: {VEL_FRAMES}")
print(f"   Min confidence for found: {MIN_CONF_FOR_FOUND}")
print(f"   Miss in-zone frames: {BALL_MISS_FRAMES_IN_ZONE_TO_SWITCH}")
print(f"   Miss not-in-zone frames: {BALL_MISS_FRAMES_NOT_IN_ZONE_TO_SWITCH}")
print(f"   Min hold frames: {MIN_HOLD_FRAMES}")
print(f"   Zone stable frames: {ZONE_STABLE_FRAMES}")
print(f"   Zone arm frames: {ZONE_ARM_FRAMES}")
print(f"   Zone margins in/out: {ZONE_IN_MARGIN:.3f}/{ZONE_OUT_MARGIN:.3f}")
print(f"   Min speed for exit: {MIN_SPEED_FOR_EXIT}")

print(f"\n🎯 Switching Logic:")
print(f"   Trajectory required: {'✅' if USE_TRAJECTORY else '❌'}")
print(f"   Exit probability required: {'✅' if USE_EXIT_PROBABILITY else '❌'}")
if USE_EXIT_PROBABILITY:
    print(f"   Exit probability threshold: {EXIT_PROB_THRESHOLD}")

print(f"\n📊 Exit Zones Configured:")
for cam_id, zones in EXIT_ZONES.items():
    print(f"   Camera {cam_id}: {len(zones)} zones ({', '.join(zones.keys())})")
    if cam_id in NEXT_CAMERA_BY_ZONE:
        print(f"      Next cameras: {NEXT_CAMERA_BY_ZONE[cam_id]}")

print(f"\n🔧 Debug Settings:")
print(f"   Console heartbeat: Every {LOG_HEARTBEAT_EVERY_N_FRAMES} frames")
print(f"   Console events: {'✅' if LOG_EVENTS else '❌'}")
print(f"   Verbose logging: {'✅' if LOG_VERBOSE else '❌'}")
print(f"   File logging: {'✅' if ENABLE_SWITCHER_LOGGING else '❌'}")
if ENABLE_SWITCHER_LOGGING and _switcher_log_file_path:
    print(f"   Log file: {_switcher_log_file_path}")
print(f"   Statistics tracking: {'✅' if SAVE_SWITCHER_STATS else '❌'}")

print(f"\n💡 Tips:")
print("   - Update EXIT_ZONES and NEXT_CAMERA_BY_ZONE per camera for best results")
print("   - Tune BALL_MISS_FRAMES_TO_SWITCH to adjust sensitivity")
print("   - Adjust EXIT_PROB_THRESHOLD to control switching confidence")
print("   - Use print_switcher_stats() to view performance metrics")
print("   - Check debug logs in:", DEBUG_DIR)

print("\n📊 Available Functions:")
print("   - get_switcher_stats() → Get current statistics")
print("   - print_switcher_stats() → Print formatted statistics")
print("   - reset_switcher_stats() → Reset statistics")
print("   - save_switcher_stats(file_path) → Save statistics to JSON")
print("   - camera_switcher.reset_switch_state(cam_id) → Reset switcher state")

if ENABLE_SWITCHER_LOGGING:
    _switcher_log("Camera switcher initialization complete", "INFO")

print("\n" + "=" * 50)


CAMERA SWITCHING LOGIC INITIALIZATION

📊 Dynamic Exit Zones Configuration:
   Configured for 2 camera(s)
   Camera 0 (RIGHT_CAM): 6 exit zones
      Next cameras: {'LEFT': 1, 'LEFT_TOP': 1, 'LEFT_BOTTOM': 1, 'BOTTOM': 1, 'TOP': 1}
   Camera 1 (LEFT_CAM): 6 exit zones
      Next cameras: {'RIGHT': 0, 'RIGHT_TOP': 0, 'RIGHT_BOTTOM': 0, 'BOTTOM': 0, 'TOP': 0}

📝 Switcher logging enabled: /content/drive/MyDrive/football/final/debug/camera_switcher_20260115_184201.log

🔍 Validating configuration...
   ✅ Configuration validated

✅ CAMERA SWITCHER INITIALIZED

⚙️  Configuration:
   Start camera: 1
   Switch after miss frames: 3
   Cooldown frames: 18
   History length: 12
   Velocity frames: 4
   Min confidence for found: 0.18
   Miss in-zone frames: 3
   Miss not-in-zone frames: 3
   Min hold frames: 1
   Zone stable frames: 4
   Zone arm frames: 6
   Zone margins in/out: 0.010/0.020
   Min speed for exit: 0.005

🎯 Switching Logic:
   Trajectory required: ✅
   Exit probability required: ✅
  

In [ ]:
# CameraSwitcher health check (no video I/O)
_ = (CameraSwitcher, SwitchDecision, EXIT_ZONES, NEXT_CAMERA_BY_ZONE)
zone_counts = {cid: len(zones) for cid, zones in EXIT_ZONES.items()}
all_zone_names = set()
for zones in EXIT_ZONES.values():
    all_zone_names.update(zones.keys())
mapped_zone_names = set()
for cam_id, zones in EXIT_ZONES.items():
    mapping = NEXT_CAMERA_BY_ZONE.get(cam_id, {})
    mapped_zone_names.update(mapping.keys())
coverage_ok = all_zone_names.issubset(mapped_zone_names)
print(f"CameraSwitcher OK | zones_per_cam={zone_counts} | mapping_coverage={coverage_ok}")

CameraSwitcher OK | zones_per_cam={0: 6, 1: 6} | mapping_coverage=True


In [ ]:
# ==============================
# MULTI-CAMERA ORCHESTRATOR + ENHANCED LOGGING
# Phase 0: Startup camera selection
# Phase 1: Continuous camera switching with ball tracking
# Enhanced with statistics, logging, and better error handling // Main Script Cell
# ==============================

import cv2
import time
import json
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional

print("=" * 50)
print("MULTI-CAMERA ORCHESTRATOR INITIALIZATION")
print("=" * 50)

# ------------------------------
# CAMERA SETUP (automatic from synchronized videos or manual fallback)
# ------------------------------
# Automatically use SYNCED_CAMERA_MAP if available, otherwise use manual configuration
if 'SYNCED_CAMERA_MAP' in globals() and SYNCED_CAMERA_MAP:
    # Use synchronized camera mapping (supports 2 or 3 cameras dynamically)
    CAMERA_MAP = {}
    for cam_id, path in SYNCED_CAMERA_MAP.items():
        CAMERA_MAP[cam_id] = Path(path) if not isinstance(path, Path) else path

    # Get camera names from synchronized mapping
    if 'SYNCED_CAMERA_NAMES' in globals() and SYNCED_CAMERA_NAMES:
        CAMERA_NAMES = SYNCED_CAMERA_NAMES.copy()
    else:
        # Generate names based on camera IDs if not available
        CAMERA_NAMES = {}
        available_ids = sorted(CAMERA_MAP.keys())
        if len(available_ids) >= 1:
            CAMERA_NAMES[available_ids[0]] = "RIGHT_CAM"
        if len(available_ids) >= 2:
            CAMERA_NAMES[available_ids[1]] = "LEFT_CAM"
        if len(available_ids) >= 3:
            CAMERA_NAMES[available_ids[2]] = "MIDDLE_CAM"

    print(f"✅ Using synchronized camera mapping: {len(CAMERA_MAP)} camera(s)")
    for cam_id, name in sorted(CAMERA_NAMES.items()):
        print(f"   Camera {cam_id}: {name} -> {CAMERA_MAP[cam_id].name}")
else:
    # Fallback to manual configuration (supports 2 or 3 cameras)
    CAMERA_MAP = {}
    CAMERA_NAMES = {}

    # Check if INPUT_VIDEOS is available
    if 'INPUT_VIDEOS' in globals() and INPUT_VIDEOS:
        num_videos = len(INPUT_VIDEOS)

        if num_videos >= 1:
            CAMERA_MAP[0] = Path(INPUT_VIDEOS[1])
            CAMERA_NAMES[0] = "RIGHT_CAM"

        if num_videos >= 2:
            CAMERA_MAP[1] = Path(INPUT_VIDEOS[0])
            CAMERA_NAMES[1] = "LEFT_CAM"

        if num_videos >= 3:
            CAMERA_MAP[2] = Path(INPUT_VIDEOS[2])
            CAMERA_NAMES[2] = "MIDDLE_CAM"

        print(f"✅  Using manual camera configuration: {len(CAMERA_MAP)} camera(s)")
        for cam_id, name in sorted(CAMERA_NAMES.items()):
            print(f"   Camera {cam_id}: {name} -> {CAMERA_MAP[cam_id].name if CAMERA_MAP[cam_id].exists() else 'not found'}")
    else:
        print(f"❌ ERROR: No camera mapping available!")
        print(f"   Please ensure video synchronization (Cell 2) completed successfully,")
        print(f"   or manually configure CAMERA_MAP and CAMERA_NAMES.")
        raise ValueError("No camera mapping available. Check video synchronization.")

# Verify camera mapping is valid
if not CAMERA_MAP:
    raise ValueError("CAMERA_MAP is empty. Cannot proceed with camera switching.")

# Print final configuration
print(f"\n📹 Final Camera Configuration:")
print(f"   Total cameras: {len(CAMERA_MAP)}")
for cam_id in sorted(CAMERA_MAP.keys()):
    cam_name = CAMERA_NAMES.get(cam_id, f"CAMERA_{cam_id}")
    cam_path = CAMERA_MAP[cam_id]
    exists = "✅" if cam_path.exists() else "❌"
    print(f"   {exists} Camera {cam_id} ({cam_name}): {cam_path.name}")

# ------------------------------
# REBUILD EXIT ZONES (ensure they're built with correct CAMERA_NAMES)
# ------------------------------
# Exit zones may have been built in Cell 6 before CAMERA_NAMES was available
# Rebuild them now to ensure they match the current camera configuration
if 'build_exit_zones_dynamic' in globals():
    print(f"\n⚙️ Rebuilding exit zones with current camera configuration...")
    try:
        EXIT_ZONES, NEXT_CAMERA_BY_ZONE = build_exit_zones_dynamic()
        print(f"   ✅ Exit zones rebuilt successfully")
        print(f"   Configured for {len(EXIT_ZONES)} camera(s)")
        for cam_id, zones in EXIT_ZONES.items():
            cam_name = CAMERA_NAMES.get(cam_id, f"Camera {cam_id}")
            print(f"      Camera {cam_id} ({cam_name}): {len(zones)} exit zones")
            if cam_id in NEXT_CAMERA_BY_ZONE:
                print(f"         Next cameras: {NEXT_CAMERA_BY_ZONE[cam_id]}")
    except Exception as e:
        print(f"   ⚠️  Warning: Could not rebuild exit zones: {e}")
        print(f"   Using existing EXIT_ZONES if available")
        if 'EXIT_ZONES' not in globals() or not EXIT_ZONES:
            print(f"   ❌ ERROR: No exit zones available!")
            raise ValueError("EXIT_ZONES is empty. Cannot proceed with camera switching.")
else:
    print(f"\n⚠️  Warning: build_exit_zones_dynamic function not found")

if 'EXIT_ZONES' not in globals() or not EXIT_ZONES:
    print(f"   ❌ ERROR: No exit zones available!")
    raise ValueError("EXIT_ZONES is empty and cannot be rebuilt. Make sure Cell 6 (Camera Switching Logic) has been executed.")
else:
    print(f"   Using existing EXIT_ZONES ({len(EXIT_ZONES)} cameras)")

# Verify EXIT_ZONES matches CAMERA_MAP
if 'EXIT_ZONES' in globals() and EXIT_ZONES:
    missing_zones = set(CAMERA_MAP.keys()) - set(EXIT_ZONES.keys())
    if missing_zones:
        print(f"\n⚠️  Warning: Exit zones missing for cameras: {missing_zones}")
        print(f"   This may prevent switching for these cameras.")

# ------------------------------
# PHASE 0 CONFIG
# ------------------------------
PHASE0_SCAN_FRAMES = 300          # frames to scan per camera (~30 seconds at 30fps)
PHASE0_MIN_DETECTIONS = 3         # minimum detections to accept camera
PHASE0_CONF_THRESHOLD = 0.12      # confidence to count as detection (reduced for smoother tracking)

# ------------------------------
# PHASE 1 CONFIG
# ------------------------------
MAX_TOTAL_FRAMES = None           # None = run until active video ends
ENABLE_VIDEO_OUTPUT = False       # Set True to write output video
OUTPUT_FPS_FALLBACK = 30          # FPS for output video if not detected

# Logging and Statistics
ENABLE_ORCHESTRATOR_LOGGING = True   # Write logs to debug directory
SAVE_ORCHESTRATOR_STATS = True       # Save statistics to JSON file
PROGRESS_LOG_EVERY_N_SEC = 5         # Log progress every N seconds

# ------------------------------
# Initialize Logging
_orch_log_file = None
_orch_log_file_path = None
if ENABLE_ORCHESTRATOR_LOGGING:
    try:
        DEBUG_DIR.mkdir(exist_ok=True, parents=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        _orch_log_file_path = DEBUG_DIR / f"orchestrator_{timestamp}.log"
        _orch_log_file = open(_orch_log_file_path, "w", encoding="utf-8")
        print(f"\n📝 Orchestrator logging enabled: {_orch_log_file_path}")
    except Exception as e:
        print(f"⚠️  Warning: Could not create orchestrator log file: {e}")
        _orch_log_file = None
        ENABLE_ORCHESTRATOR_LOGGING = False


def _orch_log(message: str, level: str = "INFO"):
    """Write orchestrator log message to file if logging is enabled."""
    if ENABLE_ORCHESTRATOR_LOGGING and _orch_log_file is not None:
        try:
            log_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            _orch_log_file.write(f"[{log_timestamp}] [{level}] {message}\n")
            _orch_log_file.flush()
        except Exception as e:
            print(f"⚠️  Warning: Failed to write to orchestrator log file: {e}")

# Statistics tracking
_orch_stats = {
    "phase0": {
        "start_time": None,
        "end_time": None,
        "cameras_scanned": 0,
        "valid_cameras": 0,
        "selected_camera": None,
    },
    "phase1": {
        "start_time": None,
        "end_time": None,
        "total_frames": 0,
        "switches": 0,
        "errors": 0,
        "camera_usage": {},
        "switch_events": [],
    },
    "last_reset_time": time.time(),
}


def _update_orch_stats(phase: str, **kwargs):
    """Update orchestrator statistics."""
    if phase == "phase0":
        for key, value in kwargs.items():
            if key in _orch_stats["phase0"]:
                _orch_stats["phase0"][key] = value
    elif phase == "phase1":
        for key, value in kwargs.items():
            if key in _orch_stats["phase1"]:
                _orch_stats["phase1"][key] = value


def get_orch_stats() -> Dict:
    """Get current orchestrator statistics."""
    stats = _orch_stats.copy()
    if stats["phase1"]["start_time"] and stats["phase1"]["end_time"]:
        stats["phase1"]["duration_sec"] = (
            stats["phase1"]["end_time"] - stats["phase1"]["start_time"]
        )
    return stats


def reset_orch_stats():
    """Reset orchestrator statistics."""
    global _orch_stats
    _orch_stats = {
        "phase0": {
            "start_time": None,
            "end_time": None,
            "cameras_scanned": 0,
            "valid_cameras": 0,
            "selected_camera": None,
        },
        "phase1": {
            "start_time": None,
            "end_time": None,
            "total_frames": 0,
            "switches": 0,
            "errors": 0,
            "camera_usage": {},
            "switch_events": [],
        },
        "last_reset_time": time.time(),
    }
    if ENABLE_ORCHESTRATOR_LOGGING:
        _orch_log("Statistics reset", "INFO")

# ------------------------------
# OPEN ALL VIDEO STREAMS
# ------------------------------
print(f"\n📹 Opening video streams...")

caps: Dict[int, cv2.VideoCapture] = {}
fps_map: Dict[int, float] = {}
video_properties: Dict[int, Dict] = {}

for cam_id, path in CAMERA_MAP.items():
    if not path.exists():
        error_msg = f"❌ Video file not found for camera {cam_id}: {path}"
        print(error_msg)
        if ENABLE_ORCHESTRATOR_LOGGING:
            _orch_log(error_msg, "ERROR")
        raise FileNotFoundError(error_msg)

    cap = cv2.VideoCapture(str(path))
    if not cap.isOpened():
        error_msg = f"❌ Failed to open camera {cam_id}: {path}"
        print(error_msg)
        if ENABLE_ORCHESTRATOR_LOGGING:
            _orch_log(error_msg, "ERROR")
        raise RuntimeError(error_msg)

    caps[cam_id] = cap
    fps = cap.get(cv2.CAP_PROP_FPS)
    fps = fps if fps and fps > 0 else 30.0
    fps_map[cam_id] = fps

    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration_sec = total_frames / fps if fps > 0 else 0

    video_properties[cam_id] = {
        "width": width,
        "height": height,
        "fps": fps,
        "total_frames": total_frames,
        "duration_sec": duration_sec,
    }

    print(f"   ✅ Camera {cam_id} ({CAMERA_NAMES[cam_id]}): {path.name}")
    print(
        f"      Resolution: {width}x{height} @ {fps:.2f}fps, {total_frames} frames ({duration_sec:.1f}s)"
    )

if ENABLE_ORCHESTRATOR_LOGGING:
    _orch_log(f"Opened {len(caps)} camera streams", "INFO")
    for cam_id, props in video_properties.items():
        _orch_log(
            f"Camera {cam_id}: {props['width']}x{props['height']} @ {props['fps']:.2f}fps, {props['total_frames']} frames",
            "INFO",
        )

# ============================================================
# PHASE 0 — STARTUP CAMERA SELECTION
# ============================================================

print("\n" + "=" * 50)
print("PHASE 0: STARTUP CAMERA SELECTION")
print("=" * 50)

phase0_start = time.time()
_update_orch_stats("phase0", start_time=phase0_start)
_update_orch_stats("phase0", cameras_scanned=len(caps))

if ENABLE_ORCHESTRATOR_LOGGING:
    _orch_log("Starting Phase 0: Startup camera selection", "INFO")
    _orch_log(
        f"Config: scan_frames={PHASE0_SCAN_FRAMES}, min_detections={PHASE0_MIN_DETECTIONS}, conf_threshold={PHASE0_CONF_THRESHOLD}",
        "INFO",
    )

startup_scores: Dict[int, Dict[str, float]] = {}
errors_per_camera: Dict[int, int] = {}

for cam_id, cap in caps.items():
    try:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    except Exception as e:
        error_msg = f"Error seeking camera {cam_id} to start: {e}"
        print(f"⚠️  {error_msg}")
        if ENABLE_ORCHESTRATOR_LOGGING:
            _orch_log(error_msg, "ERROR")
        errors_per_camera[cam_id] = errors_per_camera.get(cam_id, 0) + 1
        continue

    detections = 0
    conf_sum = 0.0
    frames_read = 0

    print(f"\n   Scanning camera {cam_id} ({CAMERA_NAMES[cam_id]})...")

    for i in range(PHASE0_SCAN_FRAMES):
        try:
            ok, frame = cap.read()
            if not ok:
                break

            frames_read += 1

            try:
                det = detect_ball(frame)
                if det.bbox is not None and det.conf >= PHASE0_CONF_THRESHOLD:
                    detections += 1
                    conf_sum += det.conf
            except Exception as e:
                if ENABLE_ORCHESTRATOR_LOGGING and errors_per_camera.get(cam_id, 0) < 3:
                    _orch_log(
                        f"Error detecting ball on camera {cam_id} frame {i}: {e}",
                        "WARNING",
                    )
                errors_per_camera[cam_id] = errors_per_camera.get(cam_id, 0) + 1
                continue
        except Exception as e:
            error_msg = f"Error reading frame {i} from camera {cam_id}: {e}"
            if ENABLE_ORCHESTRATOR_LOGGING:
                _orch_log(error_msg, "ERROR")
            errors_per_camera[cam_id] = errors_per_camera.get(cam_id, 0) + 1
            break

    avg_conf = (conf_sum / detections) if detections > 0 else 0.0
    startup_scores[cam_id] = {
        "detections": detections,
        "avg_conf": avg_conf,
        "frames_read": frames_read,
        "errors": errors_per_camera.get(cam_id, 0),
    }

    status = "✅" if detections >= PHASE0_MIN_DETECTIONS else "❌"
    print(
        f"   {status} Camera {cam_id}: {detections} detections, avg_conf={avg_conf:.2f}, frames_read={frames_read}"
    )

    if ENABLE_ORCHESTRATOR_LOGGING:
        _orch_log(
            f"Camera {cam_id} scan: detections={detections}, avg_conf={avg_conf:.2f}, frames_read={frames_read}",
            "INFO",
        )

# Decide starting camera
valid_cams = [
    cam_id for cam_id, s in startup_scores.items() if s["detections"] >= PHASE0_MIN_DETECTIONS
]

_update_orch_stats("phase0", valid_cameras=len(valid_cams))

if len(valid_cams) == 0:
    # fallback to camera 1 (RIGHT_CAM) as default
    active_cam = 1 if 1 in CAMERA_MAP else (list(CAMERA_MAP.keys())[0] if CAMERA_MAP else 0)
    print(
        f"\n⚠️  No camera detected ball reliably (min={PHASE0_MIN_DETECTIONS} detections)"
    )
    print(
        f"   Falling back to camera {active_cam} ({CAMERA_NAMES.get(active_cam, 'UNKNOWN')})"
    )
    if ENABLE_ORCHESTRATOR_LOGGING:
        _orch_log(
            f"No valid cameras found, falling back to camera {active_cam}",
            "WARNING",
        )
else:
    # choose highest avg confidence
    active_cam = max(valid_cams, key=lambda cid: startup_scores[cid]["avg_conf"])
    print(f"\n✅ Selected starting camera: {active_cam} ({CAMERA_NAMES[active_cam]})")
    print(
        f"   Detections: {startup_scores[active_cam]['detections']}, Avg confidence: {startup_scores[active_cam]['avg_conf']:.2f}"
    )
    if ENABLE_ORCHESTRATOR_LOGGING:
        _orch_log(
            f"Selected camera {active_cam} with {startup_scores[active_cam]['detections']} detections, avg_conf={startup_scores[active_cam]['avg_conf']:.2f}",
            "INFO",
        )

_update_orch_stats("phase0", selected_camera=active_cam)

# Reset all streams to start
print(f"\n🔄 Resetting all streams to start...")
for cam_id, cap in caps.items():
    try:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    except Exception as e:
        print(f"⚠️  Warning: Could not reset camera {cam_id}: {e}")
        if ENABLE_ORCHESTRATOR_LOGGING:
            _orch_log(f"Warning: Could not reset camera {cam_id}: {e}", "WARNING")

# Reset trackers
try:
    sticky_tracker.reset()
    camera_switcher.reset_switch_state(active_cam=active_cam)
    reset_stats()
    reset_sticky_stats()
    reset_switcher_stats()
    print("   ✅ Trackers and statistics reset")
    if ENABLE_ORCHESTRATOR_LOGGING:
        _orch_log("Trackers and statistics reset", "INFO")
except Exception as e:
    print(f"⚠️  Warning during reset: {e}")
    if ENABLE_ORCHESTRATOR_LOGGING:
        _orch_log(f"Reset warning: {e}", "WARNING")

phase0_end = time.time()
_update_orch_stats("phase0", end_time=phase0_end)
phase0_duration = phase0_end - phase0_start

print(f"\n✅ Phase 0 complete in {phase0_duration:.1f} seconds")
print("=" * 50)

if ENABLE_ORCHESTRATOR_LOGGING:
    _orch_log(
        f"Phase 0 complete: selected camera {active_cam}, duration={phase0_duration:.1f}s",
        "INFO",
    )

# ============================================================
# PHASE 1 — CONTINUOUS CAMERA SWITCHING
# ============================================================

# Fallback camera scanning configuration - Phase 1 Enhanced
ENABLE_FALLBACK_SCAN = True           # Enable fallback scanning when ball is lost
FALLBACK_SCAN_TIMEOUT_FRAMES = 90     # 3 seconds at 30fps (increased to reduce false switches in center field)
FALLBACK_SCAN_MIN_CONF = 0.20         # Min conf for normal ball tracking (increased from 0.15)
FALLBACK_SCAN_CONF_THRESHOLD = 0.25   # Threshold during fallback scan (increased from 0.22)
FALLBACK_SCAN_INTERVAL = 3            # Scan every N frames once timeout reached
FALLBACK_SCAN_REQUIRE_CONSECUTIVE = 4 # Require N consecutive detections
FALLBACK_SCAN_MAX_ATTEMPTS = 40       # Max scan attempts before pausing

# Multi-criteria bbox validation for false positive detection
FALLBACK_MIN_BBOX_SIZE = 8
FALLBACK_MAX_BBOX_SIZE = 120
FALLBACK_MIN_BBOX_AREA = 64
FALLBACK_MAX_BBOX_AREA = 14400
FALLBACK_ASPECT_RATIO_MIN = 0.5
FALLBACK_ASPECT_RATIO_MAX = 2.0
FALLBACK_RELATIVE_SIZE_MAX = 0.10

# Phase 2: Context-aware and adaptive retry configuration
FALLBACK_CONSECUTIVE_SAME_CAM = 3
FALLBACK_CONSECUTIVE_ALTERNATING = 4
FALLBACK_CONSECUTIVE_TIME_WINDOW = 60
FALLBACK_CONSECUTIVE_DECAY = 0.5
FALLBACK_BALL_VELOCITY_THRESHOLD = 0.01
FALLBACK_ZONE_PROXIMITY_THRESHOLD = 0.20  # Increased from 0.15 - only scan when ball is near exit zones

# Adaptive retry cycles
FALLBACK_SCAN_PAUSE_AFTER_MAX = 90
FALLBACK_SCAN_MAX_CYCLES = 3

# Round-robin fallback scanning
FALLBACK_SCAN_ONE_CAM_PER_TICK = False

# ------------------------------
# TIMELINE SYNCHRONIZATION HELPERS
# ------------------------------

def _get_frame_pos(cap: cv2.VideoCapture) -> int:
    """Get current frame position from video capture."""
    try:
        pos = cap.get(cv2.CAP_PROP_POS_FRAMES)
        return int(pos) if pos >= 0 else 0
    except Exception:
        return 0


def _hard_sync_cap(cap: cv2.VideoCapture, target_frame: int) -> None:
    """Hard-sync camera to target frame position (fixes time-jumps on switch)."""
    try:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(target_frame))
    except Exception as e:
        if ENABLE_ORCHESTRATOR_LOGGING:
            _orch_log(
                f"Warning: Could not sync camera to frame {target_frame}: {e}",
                "WARNING",
            )

# Select reference camera for timeline synchronization
# Use active camera as reference (will be updated in main loop)
REF_CAM_ID = active_cam if 'active_cam' in locals() else (min(caps.keys()) if caps else 0)

# Initialize timeline tracking
ACTIVE_CAM_TIMELINE = []

print("\n" + "=" * 50)
print("PHASE 1: CONTINUOUS CAMERA SWITCHING")
print("=" * 50)

phase1_start = time.time()
_update_orch_stats("phase1", start_time=phase1_start)

# Initialize video writer if output enabled
writer = None
output_path = None
if ENABLE_VIDEO_OUTPUT:
    try:
        OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = OUTPUT_DIR / f"orchestrator_output_{timestamp}.mp4"
        print(f"\n📹 Video output enabled: {output_path}")
        if ENABLE_ORCHESTRATOR_LOGGING:
            _orch_log(f"Video output enabled: {output_path}", "INFO")
    except Exception as e:
        print(f"⚠️  Warning: Could not set up video output: {e}")
        ENABLE_VIDEO_OUTPUT = False
        if ENABLE_ORCHESTRATOR_LOGGING:
            _orch_log(f"Video output disabled due to error: {e}", "WARNING")

print(f"\n🎬 Starting main orchestration loop...")
if ENABLE_ORCHESTRATOR_LOGGING:
    _orch_log("Starting Phase 1: Continuous camera switching", "INFO")
    _orch_log(f"Starting camera: {active_cam} ({CAMERA_NAMES[active_cam]})", "INFO")
    if MAX_TOTAL_FRAMES:
        _orch_log(f"Max frames limit: {MAX_TOTAL_FRAMES}", "INFO")

global_frame_idx = 0
running = True
last_progress_log = time.time()
errors = 0

# Initialize camera usage tracking
_orch_stats["phase1"]["camera_usage"][active_cam] = 0

# Initialize fallback scanning trackers
last_ball_found_frame = 0
fallback_scan_count = 0
fallback_consecutive_found: Dict[int, int] = {}
fallback_detection_history: Dict[int, List[Dict]] = {}
last_fallback_scan_frame = 0
fallback_cycle_count = 0
fallback_pause_until_frame = 0
fallback_rr_idx = 0

try:
    while running:
        global_frame_idx += 1

        # ✅ TIMELINE TRACKING: Use reference camera frame position as true timeline
        # Update reference camera to active camera for better synchronization
        REF_CAM_ID = active_cam

        _orch_stats["phase1"]["total_frames"] = global_frame_idx
        _orch_stats["phase1"]["camera_usage"][active_cam] = _orch_stats["phase1"]["camera_usage"].get(active_cam, 0) + 1

        # Read frame from active camera
        cap = caps.get(active_cam)
        if cap is None:
            error_msg = f"❌ Camera {active_cam} not found in caps"
            print(error_msg)
            if ENABLE_ORCHESTRATOR_LOGGING:
                _orch_log(error_msg, "ERROR")
            break

        try:
            ok, frame = cap.read()
        except Exception as e:
            errors += 1
            error_msg = f"Error reading frame from camera {active_cam}: {e}"
            if ENABLE_ORCHESTRATOR_LOGGING and errors <= 5:
                _orch_log(error_msg, "ERROR")
            if errors > 10:
                print(f"❌ Too many errors ({errors}). Stopping.")
                break
            continue

        # ✅ TIMELINE TRACKING: Get reference frame position AFTER reading
        ref_cap = caps.get(REF_CAM_ID)
        if ref_cap is None:
            ref_cap = cap
        ref_frame_pos = _get_frame_pos(ref_cap)

        # Track timeline
        ACTIVE_CAM_TIMELINE.append((ref_frame_pos, active_cam))

        if not ok:
            # ✅ Better handling: try switching to another camera instead of stopping
            print(
                f"⚠️  End of video on camera {active_cam} ({CAMERA_NAMES[active_cam]}). Trying other cameras..."
            )
            if ENABLE_ORCHESTRATOR_LOGGING:
                _orch_log(
                    f"End of video on camera {active_cam}, attempting failover",
                    "WARNING",
                )

            # Try switching to another camera
            other_cams = [cid for cid in CAMERA_MAP.keys() if cid != active_cam]
            switched = False
            for other_cam_id in other_cams:
                other_cap = caps.get(other_cam_id)
                if other_cap is None:
                    continue

                # Sync to reference position and try reading
                try:
                    _hard_sync_cap(other_cap, ref_frame_pos)
                    ok_test, frame_test = other_cap.read()
                    if ok_test and frame_test is not None:
                        # Successfully switched to another camera
                        old_cam = active_cam
                        active_cam = other_cam_id
                        cap = other_cap
                        frame = frame_test
                        ok = ok_test
                        switched = True
                        print(
                            f"   ✅ Switched to camera {active_cam} ({CAMERA_NAMES[active_cam]})"
                        )
                        if ENABLE_ORCHESTRATOR_LOGGING:
                            _orch_log(
                                f"Failover switch: {old_cam} -> {active_cam} at ref_frame={ref_frame_pos}",
                                "INFO",
                            )
                        break
                except Exception:
                    continue

            if not switched:
                print(f"📹 No other cameras available. Stopping.")
                break

        # ---- Ball tracking ----
        try:
            det = sticky_tracker.update(frame, cam_id=active_cam)
        except Exception as e:
            errors += 1
            error_msg = f"Error in sticky_tracker.update: {e}"
            if ENABLE_ORCHESTRATOR_LOGGING and errors <= 5:
                _orch_log(error_msg, "ERROR")
            # Continue with empty detection (BallDet should be available from Cell 4)
            try:
                det = BallDet(bbox=None, center=None, conf=0.0, cls=None, meta={"error": str(e)})
            except NameError:
                # Fallback if BallDet not available
                det = type(
                    'BallDet',
                    (),
                    {'bbox': None, 'center': None, 'conf': 0.0, 'cls': None, 'meta': {"error": str(e)}}
                )()

        # Track last frame when ball was found (for fallback scanning)
        ball_found = (det.bbox is not None and det.conf >= FALLBACK_SCAN_MIN_CONF)
        if ball_found:
            last_ball_found_frame = global_frame_idx
        elif 'last_ball_found_frame' not in locals():
            last_ball_found_frame = global_frame_idx  # Initialize on first frame

        # ---- Enhanced Fallback camera scanning - Phase 2 ----
        frames_since_ball_found = global_frame_idx - last_ball_found_frame
        fallback_switch_occurred = False

        # IMPROVEMENT: Check if ball was near exit zone before fallback scanning
        # This prevents false switches when ball is in center field but temporarily occluded
        ball_was_near_zone = False
        if hasattr(camera_switcher, 'pos_hist') and len(camera_switcher.pos_hist) > 0:
            last_pos = camera_switcher.pos_hist[-1]
            if last_pos:
                x, y = last_pos
                zones = EXIT_ZONES.get(active_cam, {})
                for zone_name, (x1, y1, x2, y2) in zones.items():
                    # Check if ball was near any exit zone (with proximity threshold)
                    if (x1 - FALLBACK_ZONE_PROXIMITY_THRESHOLD <= x <= x2 + FALLBACK_ZONE_PROXIMITY_THRESHOLD and
                        y1 - FALLBACK_ZONE_PROXIMITY_THRESHOLD <= y <= y2 + FALLBACK_ZONE_PROXIMITY_THRESHOLD):
                        ball_was_near_zone = True
                        break

        # Check if we're in a pause period (between retry cycles)
        if fallback_pause_until_frame > 0:
            if global_frame_idx < fallback_pause_until_frame:
                # Still in pause, skip scanning
                pass
            else:
                # Pause ended, reset for new cycle
                fallback_pause_until_frame = 0
                fallback_scan_count = 0
                fallback_consecutive_found = {}
                fallback_detection_history = {}
                if ENABLE_ORCHESTRATOR_LOGGING:
                    _orch_log(
                        f"FALLBACK_SCAN: Pause ended, starting cycle {fallback_cycle_count + 1}",
                        "INFO",
                    )
        else:
            # Check if we should perform fallback scan
            # IMPROVEMENT: Only scan if ball was near exit zone (prevents center-field false switches)
            should_scan = (
                ENABLE_FALLBACK_SCAN
                and frames_since_ball_found >= FALLBACK_SCAN_TIMEOUT_FRAMES
                and not camera_switcher.is_cooldown_active()
                and fallback_scan_count < FALLBACK_SCAN_MAX_ATTEMPTS
                and fallback_cycle_count < FALLBACK_SCAN_MAX_CYCLES
                and (global_frame_idx - last_fallback_scan_frame) >= FALLBACK_SCAN_INTERVAL
                and ball_was_near_zone  # Only scan when ball was near exit zone
            )

            if should_scan:
                last_fallback_scan_frame = global_frame_idx
                fallback_scan_count += 1

                # Scan other cameras for ball
                other_cams = [cid for cid in CAMERA_MAP.keys() if cid != active_cam]
                best_other_cam = None
                best_other_conf = 0.0

                # Round-robin fallback scanning
                if FALLBACK_SCAN_ONE_CAM_PER_TICK and len(other_cams) > 0:
                    cam_to_scan = other_cams[fallback_rr_idx % len(other_cams)]
                    fallback_rr_idx += 1
                    cams_to_check = [cam_to_scan]
                else:
                    cams_to_check = other_cams

                # Get ball velocity and position for context-aware logic
                ball_velocity = (0.0, 0.0)
                ball_position = None
                ball_near_zone = False

                try:
                    if hasattr(camera_switcher, 'pos_hist') and len(camera_switcher.pos_hist) > 1:
                        recent_positions = list(camera_switcher.pos_hist)[-4:]
                        if len(recent_positions) >= 2:
                            dx = recent_positions[-1][0] - recent_positions[0][0]
                            dy = recent_positions[-1][1] - recent_positions[0][1]
                            ball_velocity = (dx, dy)
                            ball_position = recent_positions[-1]

                            # Check if ball is near exit zones
                            if ball_position:
                                x, y = ball_position
                                zones = EXIT_ZONES.get(active_cam, {})
                                for zone_name, (x1, y1, x2, y2) in zones.items():
                                    if (
                                        x1 - FALLBACK_ZONE_PROXIMITY_THRESHOLD <= x <= x2 + FALLBACK_ZONE_PROXIMITY_THRESHOLD
                                        and y1 - FALLBACK_ZONE_PROXIMITY_THRESHOLD <= y <= y2 + FALLBACK_ZONE_PROXIMITY_THRESHOLD
                                    ):
                                        ball_near_zone = True
                                        break
                except Exception as e:
                    if ENABLE_ORCHESTRATOR_LOGGING:
                        _orch_log(f"Error getting ball context: {e}", "WARNING")

                # Determine if ball is moving
                velocity_magnitude = (ball_velocity[0] ** 2 + ball_velocity[1] ** 2) ** 0.5
                is_ball_moving = velocity_magnitude > FALLBACK_BALL_VELOCITY_THRESHOLD

                # Determine required consecutive count based on context
                if is_ball_moving and ball_near_zone:
                    required_consecutive = FALLBACK_CONSECUTIVE_ALTERNATING
                    allow_alternating = True
                else:
                    required_consecutive = FALLBACK_CONSECUTIVE_SAME_CAM
                    allow_alternating = False

                # Get reference frame position
                ref_cap = caps.get(REF_CAM_ID)
                if ref_cap is None:
                    ref_cap = cap
                ref_frame_pos = _get_frame_pos(ref_cap)

                for other_cam_id in cams_to_check:
                    other_cap = caps.get(other_cam_id)
                    if other_cap is None:
                        continue

                    # ✅ HARD-SYNC: Sync other camera to reference timeline position
                    try:
                        _hard_sync_cap(other_cap, ref_frame_pos)
                        ok_other, other_frame = other_cap.read()

                        if ok_other and other_frame is not None:
                            # Detect ball in other camera
                            try:
                                other_det = detect_ball(other_frame)

                                # Multi-criteria false positive detection
                                is_valid = False
                                if other_det.bbox is not None and other_det.conf >= FALLBACK_SCAN_CONF_THRESHOLD:
                                    x1, y1, x2, y2 = other_det.bbox
                                    bbox_width = abs(x2 - x1)
                                    bbox_height = abs(y2 - y1)
                                    bbox_area = bbox_width * bbox_height
                                    h, w = other_frame.shape[:2]
                                    frame_area = h * w

                                    if (
                                        FALLBACK_MIN_BBOX_SIZE <= bbox_width <= FALLBACK_MAX_BBOX_SIZE
                                        and FALLBACK_MIN_BBOX_SIZE <= bbox_height <= FALLBACK_MAX_BBOX_SIZE
                                    ):
                                        if (
                                            FALLBACK_MIN_BBOX_AREA <= bbox_area <= FALLBACK_MAX_BBOX_AREA
                                        ):
                                            aspect_ratio_w_h = bbox_width / bbox_height if bbox_height > 0 else 0
                                            aspect_ratio_h_w = bbox_height / bbox_width if bbox_width > 0 else 0

                                            if (
                                                FALLBACK_ASPECT_RATIO_MIN <= aspect_ratio_w_h <= FALLBACK_ASPECT_RATIO_MAX
                                                or FALLBACK_ASPECT_RATIO_MIN <= aspect_ratio_h_w <= FALLBACK_ASPECT_RATIO_MAX
                                            ):
                                                relative_size = bbox_area / frame_area if frame_area > 0 else 0
                                                if relative_size <= FALLBACK_RELATIVE_SIZE_MAX:
                                                    is_valid = True

                                if is_valid:
                                    if other_det.conf > best_other_conf:
                                        best_other_cam = other_cam_id
                                        best_other_conf = other_det.conf
                            except Exception as e:
                                if ENABLE_ORCHESTRATOR_LOGGING:
                                    _orch_log(
                                        f"Error detecting ball in camera {other_cam_id} during fallback scan: {e}",
                                        "WARNING",
                                    )
                    except Exception as e:
                        if ENABLE_ORCHESTRATOR_LOGGING:
                            _orch_log(
                                f"Error reading from camera {other_cam_id} during fallback scan: {e}",
                                "WARNING",
                            )

                # Phase 2: Enhanced false positive filtering with time window and context-aware logic
                if best_other_cam is not None:
                    if best_other_cam not in fallback_detection_history:
                        fallback_detection_history[best_other_cam] = []

                    fallback_detection_history[best_other_cam].append(
                        {'frame': global_frame_idx, 'conf': best_other_conf, 'timestamp': global_frame_idx}
                    )

                    # Clean old detections outside time window
                    current_time = global_frame_idx
                    fallback_detection_history[best_other_cam] = [
                        det for det in fallback_detection_history[best_other_cam]
                        if (current_time - det['timestamp']) <= FALLBACK_CONSECUTIVE_TIME_WINDOW
                    ]

                    # Calculate weighted consecutive count with decay
                    weighted_count = 0.0
                    for det in fallback_detection_history[best_other_cam]:
                        age = current_time - det['timestamp']
                        if age <= FALLBACK_CONSECUTIVE_TIME_WINDOW / 2:
                            weight = 1.0
                        else:
                            age_factor = (age - FALLBACK_CONSECUTIVE_TIME_WINDOW / 2) / (FALLBACK_CONSECUTIVE_TIME_WINDOW / 2)
                            weight = max(0.0, 1.0 - (age_factor * (1.0 - FALLBACK_CONSECUTIVE_DECAY)))
                        weighted_count += weight

                    # Update consecutive counter
                    consecutive_count = len(fallback_detection_history[best_other_cam])
                    fallback_consecutive_found[best_other_cam] = consecutive_count

                    # Reset counters for other cameras
                    for cid in list(fallback_consecutive_found.keys()):
                        if cid != best_other_cam:
                            fallback_consecutive_found[cid] = 0
                            if cid in fallback_detection_history:
                                fallback_detection_history[cid] = []

                    # Context-aware consecutive requirement
                    if allow_alternating:
                        recent_cameras = set()
                        for cid, detections in fallback_detection_history.items():
                            recent_dets = [
                                d for d in detections if (current_time - d['timestamp']) <= FALLBACK_CONSECUTIVE_TIME_WINDOW
                            ]
                            if len(recent_dets) > 0:
                                recent_cameras.add(cid)

                        if len(recent_cameras) > 1:
                            effective_required = FALLBACK_CONSECUTIVE_ALTERNATING
                            pattern_type = "alternating"
                        else:
                            effective_required = FALLBACK_CONSECUTIVE_SAME_CAM
                            pattern_type = "same_camera"
                    else:
                        effective_required = required_consecutive
                        pattern_type = "same_camera"

                    # Only switch if we have enough consecutive detections (using weighted count)
                    if weighted_count >= effective_required:
                        old_cam = active_cam

                        # ✅ HARD-SYNC: Sync target camera to reference timeline before switching
                        ref_cap = caps.get(REF_CAM_ID)
                        if ref_cap is None:
                            ref_cap = cap
                        ref_frame_pos = _get_frame_pos(ref_cap)
                        _hard_sync_cap(caps[best_other_cam], ref_frame_pos)

                        active_cam = best_other_cam
                        _orch_stats["phase1"]["switches"] += 1

                        switch_event = {
                            "frame": global_frame_idx,
                            "from_cam": old_cam,
                            "to_cam": active_cam,
                            "zone": "FALLBACK_SCAN",
                            "exit_prob": best_other_conf,
                            "reason": f"ball_lost_{frames_since_ball_found}_frames_found_in_other_cam_after_{fallback_scan_count}_scans_cycle_{fallback_cycle_count + 1}",
                        }
                        _orch_stats["phase1"]["switch_events"].append(switch_event)

                        context_info = f"moving={is_ball_moving:.3f}, near_zone={ball_near_zone}, pattern={pattern_type}"
                        print(
                            f"\n🔄 FALLBACK SWITCH at frame={global_frame_idx:06d}: {CAMERA_NAMES[old_cam]} -> {CAMERA_NAMES[active_cam]} (ball lost for {frames_since_ball_found} frames, conf={best_other_conf:.2f}, scans={fallback_scan_count}, cycle {fallback_cycle_count + 1}, weighted={weighted_count:.1f}/{effective_required}, {context_info})"
                        )

                        if ENABLE_ORCHESTRATOR_LOGGING:
                            _orch_log(
                                f"FALLBACK_SWITCH: frame={global_frame_idx}, {old_cam}->{active_cam}, lost_for={frames_since_ball_found}frames, conf={best_other_conf:.2f}, scans={fallback_scan_count}, cycle={fallback_cycle_count + 1}, weighted={weighted_count:.1f}/{effective_required}, {context_info}",
                                "INFO",
                            )

                        try:
                            sticky_tracker.reset()
                        except Exception as e:
                            if ENABLE_ORCHESTRATOR_LOGGING:
                                _orch_log(f"Error resetting sticky tracker: {e}", "WARNING")

                        last_ball_found_frame = global_frame_idx
                        fallback_scan_count = 0
                        fallback_consecutive_found = {}
                        fallback_detection_history = {}
                        fallback_cycle_count = 0
                        fallback_switch_occurred = True

                        if active_cam not in _orch_stats["phase1"]["camera_usage"]:
                            _orch_stats["phase1"]["camera_usage"][active_cam] = 0
                    else:
                        if ENABLE_ORCHESTRATOR_LOGGING and fallback_scan_count % 10 == 0:
                            _orch_log(
                                f"FALLBACK_SCAN: Ball detected in camera {best_other_cam} with conf={best_other_conf:.2f}, but need {effective_required} consecutive (weighted: {weighted_count:.1f}, raw: {consecutive_count}, pattern: {pattern_type})",
                                "INFO",
                            )
                else:
                    # No valid detection found, clean old detection history
                    current_time = global_frame_idx
                    for cid in list(fallback_detection_history.keys()):
                        fallback_detection_history[cid] = [
                            det for det in fallback_detection_history[cid]
                            if (current_time - det['timestamp']) <= FALLBACK_CONSECUTIVE_TIME_WINDOW
                        ]
                        if len(fallback_detection_history[cid]) == 0:
                            del fallback_detection_history[cid]

                    fallback_consecutive_found = {}

                    if fallback_scan_count >= FALLBACK_SCAN_MAX_ATTEMPTS:
                        fallback_cycle_count += 1

                        if fallback_cycle_count < FALLBACK_SCAN_MAX_CYCLES:
                            fallback_pause_until_frame = global_frame_idx + FALLBACK_SCAN_PAUSE_AFTER_MAX
                            if ENABLE_ORCHESTRATOR_LOGGING:
                                _orch_log(
                                    f"FALLBACK_SCAN: Cycle {fallback_cycle_count} completed ({FALLBACK_SCAN_MAX_ATTEMPTS} attempts), pausing for {FALLBACK_SCAN_PAUSE_AFTER_MAX} frames before cycle {fallback_cycle_count + 1}",
                                    "WARNING",
                                )
                            print(
                                f"\n⚠️  Fallback scan cycle {fallback_cycle_count} completed: Reached {FALLBACK_SCAN_MAX_ATTEMPTS} attempts without finding ball. Pausing for {FALLBACK_SCAN_PAUSE_AFTER_MAX} frames before retry..."
                            )

                            fallback_scan_count = 0
                            fallback_consecutive_found = {}
                            fallback_detection_history = {}
                        else:
                            if ENABLE_ORCHESTRATOR_LOGGING:
                                _orch_log(
                                    f"FALLBACK_SCAN: All {FALLBACK_SCAN_MAX_CYCLES} cycles exhausted, stopping fallback scan",
                                    "WARNING",
                                )
                            print(
                                f"\n⚠️  Fallback scan stopped: All {FALLBACK_SCAN_MAX_CYCLES} retry cycles exhausted. Relying on normal exit zone switching."
                            )

                            fallback_scan_count = 0
                            fallback_cycle_count = 0
                            fallback_consecutive_found = {}
                            fallback_detection_history = {}
                            fallback_pause_until_frame = 0

        # Reset scan counters if ball is found in current camera
        if ball_found:
            fallback_scan_count = 0
            fallback_cycle_count = 0
            fallback_consecutive_found = {}
            fallback_detection_history = {}
            fallback_pause_until_frame = 0

        # ---- Camera switching decision (only if no fallback switch occurred) ----
        try:
            decision = camera_switcher.update(
                cam_id=active_cam,
                det=det,
                frame_shape=frame.shape[:2],
                frame_idx=global_frame_idx,
            )
        except Exception as e:
            errors += 1
            error_msg = f"Error in camera_switcher.update: {e}"
            if ENABLE_ORCHESTRATOR_LOGGING and errors <= 5:
                _orch_log(error_msg, "ERROR")
            # Continue with stay decision (SwitchDecision should be available from Cell 7)
            try:
                decision = SwitchDecision(
                    "STAY",
                    active_cam,
                    active_cam,
                    f"error: {str(e)}",
                    "NONE",
                    0.0,
                    0,
                    0,
                )
            except NameError:
                decision = type(
                    'SwitchDecision',
                    (),
                    {
                        'action': "STAY",
                        'from_cam': active_cam,
                        'to_cam': active_cam,
                        'reason': f"error: {str(e)}",
                        'zone': "NONE",
                        'exit_prob': 0.0,
                        'miss_count': 0,
                        'cooldown_left': 0,
                    },
                )()

        # ---- Apply decision ----
        if decision.action == "SWITCH":
            old_cam = active_cam

            # ✅ HARD-SYNC: Sync target camera to reference timeline before switching
            ref_cap = caps.get(REF_CAM_ID)
            if ref_cap is None:
                ref_cap = cap
            ref_frame_pos = _get_frame_pos(ref_cap)
            # ==============================
            # IMPROVEMENT: Pre-Switch Camera Readiness Check
            # ==============================
            target_cam = decision.to_cam
            target_cap = caps.get(target_cam)

            if target_cap is None:
                if ENABLE_ORCHESTRATOR_LOGGING:
                    _orch_log(f"Pre-switch check failed: target camera {target_cam} not found in caps", "WARNING")
                # Skip switch, stay on current camera
                continue

            # Verify target camera can read a frame
            try:
                # Save current position
                test_pos = _get_frame_pos(target_cap)
                _hard_sync_cap(target_cap, ref_frame_pos)
                ok_test, frame_test = target_cap.read()

                if not ok_test or frame_test is None:
                    if ENABLE_ORCHESTRATOR_LOGGING:
                        _orch_log(f"Pre-switch check failed: target camera {target_cam} cannot read frame", "WARNING")
                    # Restore position and skip switch
                    _hard_sync_cap(target_cap, test_pos)
                    continue

                # Optional: Quick ball detection check (lightweight verification)
                # This can be enabled for higher confidence switches
                PRE_SWITCH_BALL_CHECK = False  # Set to True to enable
                if PRE_SWITCH_BALL_CHECK:
                    try:
                        test_det = detect_ball(frame_test)
                        if test_det.bbox is None or test_det.conf < 0.1:
                            if ENABLE_ORCHESTRATOR_LOGGING:
                                _orch_log(f"Pre-switch check: target camera {target_cam} has no/low ball detection", "INFO")
                            # Still allow switch, but log it
                    except Exception:
                        pass  # Ignore detection errors in pre-check

                # Restore position (will be synced again before actual switch)
                _hard_sync_cap(target_cap, test_pos)
            except Exception as e:
                if ENABLE_ORCHESTRATOR_LOGGING:
                    _orch_log(f"Pre-switch check error for camera {target_cam}: {e}", "WARNING")
                # Skip switch on error
                continue

            # All checks passed, proceed with switch
            _hard_sync_cap(caps[decision.to_cam], ref_frame_pos)

            active_cam = decision.to_cam
            _orch_stats["phase1"]["switches"] += 1

            switch_event = {
                "frame": global_frame_idx,
                "from_cam": old_cam,
                "to_cam": active_cam,
                "zone": decision.zone,
                "exit_prob": decision.exit_prob,
                "reason": decision.reason,
            }
            _orch_stats["phase1"]["switch_events"].append(switch_event)

            print(
                f"\n🔄 SWITCH at frame={global_frame_idx:06d}: {CAMERA_NAMES[old_cam]} -> {CAMERA_NAMES[active_cam]} (zone={decision.zone}, prob={decision.exit_prob:.2f})"
            )

            if ENABLE_ORCHESTRATOR_LOGGING:
                _orch_log(
                    f"SWITCH: frame={global_frame_idx}, {old_cam}->{active_cam}, zone={decision.zone}, prob={decision.exit_prob:.2f}",
                    "INFO",
                )

            # Reset sticky tracker when switching views
            try:
                sticky_tracker.reset()
            except Exception as e:
                if ENABLE_ORCHESTRATOR_LOGGING:
                    _orch_log(f"Warning: Error resetting sticky tracker: {e}", "WARNING")

            # Initialize camera usage for new camera
            if active_cam not in _orch_stats["phase1"]["camera_usage"]:
                _orch_stats["phase1"]["camera_usage"][active_cam] = 0

        # ---- Optional video output ----
        if ENABLE_VIDEO_OUTPUT:
            try:
                vis = draw_ball_debug(frame, det, pos_history=None)

                overlay_text = f"Frame: {global_frame_idx} | Camera: {CAMERA_NAMES[active_cam]}"
                if decision.action == "SWITCH":
                    overlay_text += " [SWITCHING]"
                cv2.putText(
                    vis,
                    overlay_text,
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,
                    (255, 255, 255),
                    2,
                )

                if writer is None:
                    h, w = vis.shape[:2]
                    out_fps = fps_map.get(active_cam, OUTPUT_FPS_FALLBACK)
                    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
                    writer = cv2.VideoWriter(str(output_path), fourcc, out_fps, (w, h))
                    if not writer.isOpened():
                        raise RuntimeError("Failed to initialize video writer")
                    print(f"   ✅ Video writer initialized: {w}x{h} @ {out_fps:.2f}fps")
                    if ENABLE_ORCHESTRATOR_LOGGING:
                        _orch_log(
                            f"Video writer initialized: {w}x{h} @ {out_fps:.2f}fps",
                            "INFO",
                        )

                writer.write(vis)
            except Exception as e:
                if ENABLE_ORCHESTRATOR_LOGGING and errors <= 5:
                    _orch_log(f"Error writing video frame: {e}", "ERROR")

        # ---- Progress logging ----
        current_time = time.time()
        if current_time - last_progress_log >= PROGRESS_LOG_EVERY_N_SEC:
            elapsed = current_time - phase1_start
            processing_fps = global_frame_idx / elapsed if elapsed > 0 else 0
            progress_info = (
                f"⏱️  {elapsed:6.1f}s | frames={global_frame_idx} | "
                f"cam={active_cam} ({CAMERA_NAMES[active_cam]}) | "
                f"switches={_orch_stats['phase1']['switches']} | "
                f"fps={processing_fps:.1f}"
            )
            print(progress_info)

            if ENABLE_ORCHESTRATOR_LOGGING:
                _orch_log(
                    f"Progress: {global_frame_idx} frames, {_orch_stats['phase1']['switches']} switches, fps={processing_fps:.1f}",
                    "INFO",
                )

            last_progress_log = current_time

        # ---- Stop condition ----
        if MAX_TOTAL_FRAMES is not None and global_frame_idx >= MAX_TOTAL_FRAMES:
            print(f"\n📊 Reached MAX_TOTAL_FRAMES limit ({MAX_TOTAL_FRAMES}). Stopping.")
            if ENABLE_ORCHESTRATOR_LOGGING:
                _orch_log(f"Reached MAX_TOTAL_FRAMES limit: {MAX_TOTAL_FRAMES}", "INFO")
            break

except KeyboardInterrupt:
    print(f"\n⚠️  Processing interrupted by user at frame {global_frame_idx}")
    if ENABLE_ORCHESTRATOR_LOGGING:
        _orch_log(f"Processing interrupted at frame {global_frame_idx}", "WARNING")
except Exception as e:
    errors += 1
    error_msg = f"❌ Error during orchestration: {e}"
    print(f"\n{error_msg}")
    if ENABLE_ORCHESTRATOR_LOGGING:
        _orch_log(error_msg, "ERROR")
    raise
finally:
    phase1_end = time.time()
    _update_orch_stats("phase1", end_time=phase1_end)
    _update_orch_stats("phase1", errors=errors)

    # ------------------------------
    # CLEANUP
    # ------------------------------
    print(f"\n🧹 Cleaning up resources...")

    for cam_id, cap in caps.items():
        try:
            cap.release()
        except Exception as e:
            print(f"⚠️  Warning: Error releasing camera {cam_id}: {e}")

    if writer:
        try:
            writer.release()
            print(f"   ✅ Video writer closed")
        except Exception as e:
            print(f"⚠️  Warning: Error closing video writer: {e}")

    if ENABLE_ORCHESTRATOR_LOGGING and _orch_log_file:
        try:
            _orch_log_file.close()
        except Exception:
            pass

    # ------------------------------
    # FINAL STATISTICS & SUMMARY
    # ------------------------------
    phase1_duration = phase1_end - phase1_start

    print("\n" + "=" * 50)
    print("✅ ORCHESTRATION COMPLETE")
    print("=" * 50)

    print(f"\n📈 Phase 1 Summary:")
    print(f"   Total frames processed: {global_frame_idx}")
    print(f"   Processing time: {phase1_duration:.1f} seconds ({phase1_duration/60:.1f} minutes)")
    if phase1_duration > 0:
        print(f"   Processing speed: {global_frame_idx/phase1_duration:.1f} fps")
    print(f"   Camera switches: {_orch_stats['phase1']['switches']}")
    if errors > 0:
        print(f"   Errors: {errors}")

    print(f"\n📹 Camera Usage:")
    for cid, usage in sorted(_orch_stats["phase1"]["camera_usage"].items()):
        pct = (usage / global_frame_idx * 100) if global_frame_idx > 0 else 0
        print(f"   Camera {cid} ({CAMERA_NAMES[cid]}): {usage} frames ({pct:.1f}%)")

    # ==============================
    # IMPROVEMENT: Camera Dominance Warnings
    # ==============================
    CAMERA_DOMINANCE_THRESHOLD = 90.0  # Warn if any camera > 90% usage
    if global_frame_idx > 0:
        for cid, usage in _orch_stats["phase1"]["camera_usage"].items():
            pct = (usage / global_frame_idx * 100)
            if pct >= CAMERA_DOMINANCE_THRESHOLD:
                print(f"\n⚠️  WARNING: Camera {cid} ({CAMERA_NAMES.get(cid, 'Unknown')}) dominates processing: {pct:.1f}%")
                print(f"   This may indicate switching logic issues or camera availability problems.")

    if _orch_stats["phase1"]["switch_events"]:
        print(f"\n🔄 Switch Events (last 5):")
        for event in _orch_stats["phase1"]["switch_events"][-5:]:
            print(
                f"   Frame {event['frame']}: {event['from_cam']} -> {event['to_cam']} (zone={event['zone']}, prob={event['exit_prob']:.2f})"
            )

    if ENABLE_VIDEO_OUTPUT and output_path:
        print(f"\n📁 Output:")
        print(f"   Video: {output_path}")

    # Save statistics
    if SAVE_ORCHESTRATOR_STATS:
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            stats_file = DEBUG_DIR / f"orchestrator_stats_{timestamp}.json"
            stats = get_orch_stats()
            if 'ACTIVE_CAM_TIMELINE' in globals() and ACTIVE_CAM_TIMELINE:
                stats["timeline_sample_head"] = ACTIVE_CAM_TIMELINE[:50]
                stats["timeline_len"] = len(ACTIVE_CAM_TIMELINE)
            with open(stats_file, 'w', encoding='utf-8') as f:
                json.dump(stats, f, indent=2, default=str)
            print(f"   Statistics: {stats_file}")
        except Exception as e:
            print(f"⚠️  Warning: Could not save statistics: {e}")

    if ENABLE_ORCHESTRATOR_LOGGING and _orch_log_file_path:
        print(f"   Log file: {_orch_log_file_path}")

    if ENABLE_ORCHESTRATOR_LOGGING:
        _orch_log(
            f"Orchestration complete: {global_frame_idx} frames, {_orch_stats['phase1']['switches']} switches, duration={phase1_duration:.1f}s",
            "INFO",
        )

    print("\n" + "=" * 50)

MULTI-CAMERA ORCHESTRATOR INITIALIZATION
✅  Using manual camera configuration: 2 camera(s)
   Camera 0: RIGHT_CAM -> IMG_2789_synced.mp4
   Camera 1: LEFT_CAM -> IMG_0689 2_synced.mp4

📹 Final Camera Configuration:
   Total cameras: 2
   ✅ Camera 0 (RIGHT_CAM): IMG_2789_synced.mp4
   ✅ Camera 1 (LEFT_CAM): IMG_0689 2_synced.mp4

⚙️ Rebuilding exit zones with current camera configuration...
   ✅ Exit zones rebuilt successfully
   Configured for 2 camera(s)
      Camera 0 (RIGHT_CAM): 6 exit zones
         Next cameras: {'LEFT': 1, 'LEFT_TOP': 1, 'LEFT_BOTTOM': 1, 'BOTTOM': 1, 'TOP': 1}
      Camera 1 (LEFT_CAM): 6 exit zones
         Next cameras: {'RIGHT': 0, 'RIGHT_TOP': 0, 'RIGHT_BOTTOM': 0, 'BOTTOM': 0, 'TOP': 0}
   Using existing EXIT_ZONES (2 cameras)

📝 Orchestrator logging enabled: /content/drive/MyDrive/football/final/debug/orchestrator_20260115_184208.log

📹 Opening video streams...
   ✅ Camera 0 (RIGHT_CAM): IMG_2789_synced.mp4
      Resolution: 1920x1080 @ 30.00fps, 102273 f

In [ ]:

# ==============================
# FINAL HIGHLIGHT OUTPUT + ENHANCED LOGGING (Second Last)
# Creates a single continuous output video with camera switching
# Enhanced with statistics, logging, and better error handling
# ==============================

import cv2
import time
import json
from pathlib import Path
from datetime import datetime
from typing import Dict

print("=" * 50)
print("FINAL HIGHLIGHT OUTPUT GENERATION")
print("=" * 50)

# ------------------------------
# OUTPUT CONFIG
# ------------------------------
SKIP_SECONDS = 0          # ✅ Skip first 4 minutes
MAX_DUR = 1000              # seconds of OUTPUT after skip (default 5 minutes)
OUTPUT_FPS_FALLBACK = 30

# Naming options
USE_TIMESTAMP_NAME = True    # True -> Highlight_YYYYMMDD_HHMMSS.mp4
HIGHLIGHT_INDEX = 1          # Used only if USE_TIMESTAMP_NAME=False

# Logging and Statistics
ENABLE_HIGHLIGHT_LOGGING = True   # Write logs to debug directory
SAVE_HIGHLIGHT_STATS = True      # Save statistics to JSON file
PROGRESS_LOG_EVERY_N_SEC = 5     # Log progress every N seconds

# Phase 0 Configuration (Startup Camera Selection)
PHASE0_SCAN_FRAMES = 300          # frames to scan per camera (~30 seconds at 30fps)
PHASE0_MIN_DETECTIONS = 3        # minimum detections to accept camera
PHASE0_CONF_THRESHOLD = 0.12     # confidence to count as detection

# PERFORMANCE OPTIMIZATION: Disable fallback scanning for highlight generation (saves significant time)
ENABLE_FALLBACK_FOR_HIGHLIGHT = False  # Set to True to enable fallback scanning (slower but more robust)

# PERFORMANCE OPTIMIZATION: Faster detection for highlight generation (trade-off: slightly less accurate)
# Reduce image size for faster YOLO inference (640 instead of 1280)
HIGHLIGHT_DETECT_IMGSZ = 1280  # Set to 640 for ~2x faster detection (slightly less accurate)


# Output directory
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
DEBUG_DIR.mkdir(exist_ok=True, parents=True)
# Configuration
ENABLE_ZONE_VISUALIZATION = False  # Set to True to show exit zones on video

# Generate output path
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
if USE_TIMESTAMP_NAME:
    output_path = OUTPUT_DIR / f"Highlight_{timestamp}.mp4"
else:
    output_path = OUTPUT_DIR / f"Highlight_{HIGHLIGHT_INDEX}.mp4"

# ------------------------------
# VALIDATE REQUIRED GLOBALS
# ------------------------------
required_globals = ["CAMERA_MAP", "CAMERA_NAMES", "OUTPUT_DIR"]
missing = [g for g in required_globals if g not in globals()]
if missing:
    raise RuntimeError(f"Missing required variables from previous cells: {missing}")

print(f"\n📹 Output Configuration:")
print(f"   Output path: {output_path}")
print(f"   Skip seconds: {SKIP_SECONDS} ({SKIP_SECONDS/60:.1f} minutes)")
print(f"   Max duration: {MAX_DUR} seconds ({MAX_DUR/60:.1f} minutes)")

# Initialize highlight logging
_highlight_log_file = None
_highlight_log_file_path = None
if "ENABLE_HIGHLIGHT_LOGGING" in globals() and ENABLE_HIGHLIGHT_LOGGING:
    try:
        DEBUG_DIR.mkdir(exist_ok=True, parents=True)
        _highlight_log_file_path = DEBUG_DIR / f"highlight_output_{timestamp}.log"
        _highlight_log_file = open(_highlight_log_file_path, 'w', encoding='utf-8')
        print(f"\n📝 Highlight logging enabled: {_highlight_log_file_path}")
    except Exception as e:
        print(f"⚠️  Warning: Could not create highlight log file: {e}")
        _highlight_log_file = None

def _highlight_log(message: str, level: str = "INFO"):
    """Write highlight log message to file if logging is enabled."""
    if _highlight_log_file is not None:
        try:
            log_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            _highlight_log_file.write(f"[{log_timestamp}] [{level}] {message}\n")
            _highlight_log_file.flush()
        except Exception:
            pass

# Statistics tracking
_highlight_stats = {
    "config": {
        "skip_seconds": SKIP_SECONDS,
        "max_duration_seconds": MAX_DUR,
        "output_path": str(output_path),
        "timestamp": timestamp,
    },
    "processing": {
        "written_frames": 0,
        "switches": 0,
        "errors": 0,
        "start_time": datetime.now().isoformat(),
        "end_time": None,
        "processing_time_sec": None,
        "output_fps": 0.0,
        "camera_usage": {},
        "switch_events": [],
    },
    "errors": [],
}

def _update_highlight_stats(**kwargs):
    """Update highlight statistics."""
    for key, value in kwargs.items():
        if key in _highlight_stats["processing"]:
            _highlight_stats["processing"][key] = value
        elif key == "frames_written":
            _highlight_stats["processing"]["written_frames"] = value

if _highlight_log_file is not None:
    _highlight_log(f"Starting highlight output generation: {output_path.name}", "INFO")
    _highlight_log(f"Configuration: skip={SKIP_SECONDS}s, max_dur={MAX_DUR}s", "INFO")

# ------------------------------
# REOPEN CAMERAS (read-only)
# ------------------------------
print(f"\n📥 Opening camera streams for output...")

caps_out = {}
fps_map_out = {}
skip_frames_map = {}
video_properties_out = {}

for cam_id, path in CAMERA_MAP.items():
    if not path.exists():
        error_msg = f"❌ Video file not found for camera {cam_id}: {path}"
        print(error_msg)
        if ENABLE_HIGHLIGHT_LOGGING:
            _highlight_log(error_msg, "ERROR")
        raise FileNotFoundError(error_msg)

    cap = cv2.VideoCapture(str(path))
    if not cap.isOpened():
        error_msg = f"❌ Failed to open camera {cam_id} for output: {path}"
        print(error_msg)
        if ENABLE_HIGHLIGHT_LOGGING:
            _highlight_log(error_msg, "ERROR")
        raise RuntimeError(error_msg)

    caps_out[cam_id] = cap

    fps = cap.get(cv2.CAP_PROP_FPS)
    fps = fps if fps and fps > 0 else OUTPUT_FPS_FALLBACK
    fps_map_out[cam_id] = fps

    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration_sec = (total_frames / fps) if fps > 0 else 0

    video_properties_out[cam_id] = {
        "path": str(path),
        "fps": float(fps),
        "total_frames": int(total_frames),
        "width": int(width),
        "height": int(height),
        "duration_sec": float(duration_sec),
        "name": CAMERA_NAMES.get(cam_id, f"Cam{cam_id}"),
    }

    skip_frames = int(SKIP_SECONDS * fps)
    skip_frames_map[cam_id] = skip_frames

    print(f"   ✅ Camera {cam_id} ({CAMERA_NAMES.get(cam_id, 'Unknown')})")
    print(f"      File: {path}")
    print(f"      FPS: {fps:.2f}, Frames: {total_frames}, Size: {width}x{height}, Dur: {duration_sec:.1f}s")
    print(f"      Skip: {SKIP_SECONDS}s ({skip_frames} frames)")

if _highlight_log_file is not None:
    _highlight_log(f"Opened {len(caps_out)} cameras for output", "INFO")
    for cam_id, props in video_properties_out.items():
        _highlight_log(
            f"Camera {cam_id}: {props['width']}x{props['height']} @ {props['fps']:.2f}fps, skip={skip_frames_map[cam_id]} frames",
            "INFO",
        )

# ------------------------------
# PREPARE VIDEO WRITER (lazy init)
# ------------------------------
writer = None
start_time = time.time()

# ------------------------------
# SEEK / SKIP FIRST N SECONDS
# ------------------------------
print(f"\n⏩ Seeking to skip point ({SKIP_SECONDS}s)...")
for cam_id, cap in caps_out.items():
    try:
        # Use reference camera frame for synchronization (videos are already synced)
        if cam_id == min(caps_out.keys()):
            # First camera: calculate reference
            reference_cam_id = cam_id
            reference_skip_frames = skip_frames_map[cam_id]
            # Try time-based seeking first
            skip_time_ms = int(SKIP_SECONDS * 1000)
            cap.set(cv2.CAP_PROP_POS_MSEC, skip_time_ms)
            actual_time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
            if abs(actual_time_ms - skip_time_ms) > 50:
                # Fallback to frame-based
                cap.set(cv2.CAP_PROP_POS_FRAMES, reference_skip_frames)
                use_time_based = False
            else:
                use_time_based = True
        else:
            # Other cameras: use same method as reference
            if 'use_time_based' in locals() and use_time_based:
                skip_time_ms = int(SKIP_SECONDS * 1000)
                cap.set(cv2.CAP_PROP_POS_MSEC, skip_time_ms)
            else:
                cap.set(cv2.CAP_PROP_POS_FRAMES, reference_skip_frames)
        actual_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)
        expected_pos = reference_skip_frames if "reference_skip_frames" in locals() else skip_frames_map[cam_id]
        if abs(actual_pos - expected_pos) > 1:
            print(
                f"   ⚠️  Camera {cam_id}: Requested frame {expected_pos}, got {actual_pos} "
                f"(diff: {abs(actual_pos - expected_pos)} frames)"
            )
            if _highlight_log_file is not None:
                _highlight_log(f"Seek warning camera {cam_id}: requested={expected_pos}, actual={actual_pos}", "WARNING")
    except Exception as e:
        error_msg = f"Error seeking camera {cam_id}: {e}"
        print(f"   ⚠️  {error_msg}")
        _highlight_stats["errors"].append(error_msg)
        _highlight_stats["processing"]["errors"] += 1
        if _highlight_log_file is not None:
            _highlight_log(error_msg, "ERROR")

print(f"   ✅ Skipped first {SKIP_SECONDS} seconds (~{SKIP_SECONDS/60:.1f} min) for all cameras")

# Verify all cameras are synchronized after skipping
print(f"\n🔍 Verifying camera synchronization after skip...")
sync_verified = True
for cam_id, cap in caps_out.items():
    actual_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)
    expected_pos = reference_skip_frames  # Use reference frame for all cameras
    fps = fps_map_out[cam_id]
    actual_time = actual_pos / fps if fps > 0 else 0
    expected_time = expected_pos / fps if fps > 0 else 0

    # Allow 1 frame tolerance for seek accuracy
    if abs(actual_pos - expected_pos) > 1:
        sync_verified = False
        # Force-resync this camera to reference frame to avoid drift
        try:
            cap.set(cv2.CAP_PROP_POS_FRAMES, expected_pos)
        except Exception:
            pass

        print(
            f"   ⚠️  Camera {cam_id} ({CAMERA_NAMES.get(cam_id, 'Unknown')}): "
            f"Position mismatch - Expected: {expected_pos} frames ({expected_time:.2f}s), "
            f"Got: {actual_pos} frames ({actual_time:.2f}s)"
        )
        if _highlight_log_file is not None:
            _highlight_log(
                f"Sync mismatch camera {cam_id}: expected={expected_pos} got={actual_pos} (forcing resync)",
                "WARNING",
            )

if sync_verified:
    print("   ✅ All cameras appear synchronized after skip.")
    if _highlight_log_file is not None:
        _highlight_log("All cameras synchronized after skip", "INFO")
else:
    print("   ⚠️  Some cameras were not perfectly synchronized (auto-resync attempted).")
    if _highlight_log_file is not None:
        _highlight_log("Some cameras not perfectly synchronized after skip (auto-resync attempted)", "WARNING")

# ------------------------------
# RESET STATES (important)
# ------------------------------
print(f"\n🔄 Resetting trackers and statistics...")
try:
    sticky_tracker.reset()
    camera_switcher.reset_switch_state(active_cam=camera_switcher.active_cam)
    reset_stats()
    reset_sticky_stats()
    reset_switcher_stats()
    print("   ✅ Trackers and statistics reset")
    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log("Trackers and statistics reset", "INFO")
except Exception as e:
    print(f"   ⚠️  Warning during reset: {e}")
    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log(f"Reset warning: {e}", "WARNING")

# ------------------------------
# PHASE 0: STARTUP CAMERA SELECTION
# ------------------------------
print("\n" + "=" * 50)
print("PHASE 0: STARTUP CAMERA SELECTION")
print("=" * 50)

phase0_start = time.time()

if ENABLE_HIGHLIGHT_LOGGING:
    _highlight_log("Starting Phase 0: Startup camera selection", "INFO")
    _highlight_log(f"Config: scan_frames={PHASE0_SCAN_FRAMES}, min_detections={PHASE0_MIN_DETECTIONS}, "
                 f"conf_threshold={PHASE0_CONF_THRESHOLD}", "INFO")

startup_scores: Dict[int, Dict[str, float]] = {}
errors_per_camera: Dict[int, int] = {}

# Scan cameras from skip position (not from start)
for cam_id, cap in caps_out.items():
    try:
        # Ensure we're at the skip position
        if 'reference_skip_frames' in locals():
            cap.set(cv2.CAP_PROP_POS_FRAMES, reference_skip_frames)
        else:
            cap.set(cv2.CAP_PROP_POS_FRAMES, skip_frames_map[cam_id])
    except Exception as e:
        error_msg = f"Error seeking camera {cam_id} to skip position: {e}"
        print(f"⚠️  {error_msg}")
        if ENABLE_HIGHLIGHT_LOGGING:
            _highlight_log(error_msg, "ERROR")
        errors_per_camera[cam_id] = errors_per_camera.get(cam_id, 0) + 1
        continue

    detections = 0
    conf_sum = 0.0
    frames_read = 0

    print(f"\n   Scanning camera {cam_id} ({CAMERA_NAMES.get(cam_id, 'Unknown')})...")

    for i in range(PHASE0_SCAN_FRAMES):
        try:
            ok, frame = cap.read()
            if not ok:
                break

            frames_read += 1

            try:
                det = detect_ball(frame)
                if det.bbox is not None and det.conf >= PHASE0_CONF_THRESHOLD:
                    detections += 1
                    conf_sum += det.conf
            except Exception as e:
                if ENABLE_HIGHLIGHT_LOGGING and errors_per_camera.get(cam_id, 0) < 3:
                    _highlight_log(f"Error detecting ball on camera {cam_id} frame {i}: {e}", "WARNING")
                errors_per_camera[cam_id] = errors_per_camera.get(cam_id, 0) + 1
                continue
        except Exception as e:
            error_msg = f"Error reading frame {i} from camera {cam_id}: {e}"
            if ENABLE_HIGHLIGHT_LOGGING:
                _highlight_log(error_msg, "ERROR")
            errors_per_camera[cam_id] = errors_per_camera.get(cam_id, 0) + 1
            break

    avg_conf = (conf_sum / detections) if detections > 0 else 0.0
    startup_scores[cam_id] = {
        "detections": detections,
        "avg_conf": avg_conf,
        "frames_read": frames_read,
        "errors": errors_per_camera.get(cam_id, 0)
    }

    status = "✅" if detections >= PHASE0_MIN_DETECTIONS else "❌"
    print(f"   {status} Camera {cam_id}: {detections} detections, avg_conf={avg_conf:.2f}, "
          f"frames_read={frames_read}")

    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log(f"Camera {cam_id} scan: detections={detections}, avg_conf={avg_conf:.2f}, "
                     f"frames_read={frames_read}", "INFO")

# Decide starting camera
valid_cams = [
    cam_id for cam_id, s in startup_scores.items()
    if s["detections"] >= PHASE0_MIN_DETECTIONS
]

if len(valid_cams) == 0:
    # fallback to camera with lowest ID (or first in CAMERA_MAP)
    active_cam = min(caps_out.keys()) if caps_out else (list(CAMERA_MAP.keys())[0] if CAMERA_MAP else 0)
    print(f"\n⚠️  No camera detected ball reliably (min={PHASE0_MIN_DETECTIONS} detections)")
    print(f"   Falling back to camera {active_cam} ({CAMERA_NAMES.get(active_cam, 'UNKNOWN')})")
    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log(f"No valid cameras found, falling back to camera {active_cam}", "WARNING")
else:
    # choose highest avg confidence
    active_cam = max(
        valid_cams,
        key=lambda cid: startup_scores[cid]["avg_conf"]
    )
    print(f"\n✅ Selected starting camera: {active_cam} ({CAMERA_NAMES.get(active_cam, 'Unknown')})")
    print(f"   Detections: {startup_scores[active_cam]['detections']}, "
          f"Avg confidence: {startup_scores[active_cam]['avg_conf']:.2f}")
    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log(f"Selected camera {active_cam} with {startup_scores[active_cam]['detections']} "
                     f"detections, avg_conf={startup_scores[active_cam]['avg_conf']:.2f}", "INFO")

# Reset all streams back to skip position
print(f"\n🔄 Resetting all streams to skip position...")
for cam_id, cap in caps_out.items():
    try:
        if 'reference_skip_frames' in locals():
            cap.set(cv2.CAP_PROP_POS_FRAMES, reference_skip_frames)
        else:
            cap.set(cv2.CAP_PROP_POS_FRAMES, skip_frames_map[cam_id])
    except Exception as e:
        print(f"⚠️  Warning: Could not reset camera {cam_id} to skip position: {e}")
        if ENABLE_HIGHLIGHT_LOGGING:
            _highlight_log(f"Warning: Could not reset camera {cam_id} to skip position: {e}", "WARNING")

# Reset trackers with selected camera
try:
    sticky_tracker.reset()
    camera_switcher.reset_switch_state(active_cam=active_cam)
    reset_stats()
    reset_sticky_stats()
    reset_switcher_stats()
    print("   ✅ Trackers and statistics reset")
    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log("Trackers and statistics reset", "INFO")
except Exception as e:
    print(f"⚠️  Warning during reset: {e}")
    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log(f"Reset warning: {e}", "WARNING")

phase0_end = time.time()
phase0_duration = phase0_end - phase0_start

print(f"\n✅ Phase 0 complete in {phase0_duration:.1f} seconds")
print("=" * 50)

if ENABLE_HIGHLIGHT_LOGGING:
    _highlight_log(f"Phase 0 complete: selected camera {active_cam}, duration={phase0_duration:.1f}s", "INFO")

global_frame_idx = 0
written_frames = 0
errors = 0
last_progress_log = time.time()

# Initialize camera usage tracking (use Phase 0 selected camera)
initial_cam = active_cam  # Set by Phase 0
_highlight_stats["processing"]["camera_usage"][initial_cam] = 0

# ------------------------------
# MAIN OUTPUT LOOP
# ------------------------------
print(f"\n🎬 Writing highlight video...")
if ENABLE_HIGHLIGHT_LOGGING:
    _highlight_log("Starting highlight video generation", "INFO")
    _highlight_log(f"Starting camera: {initial_cam} ({CAMERA_NAMES.get(initial_cam, 'Unknown')})", "INFO")

try:
    while True:
        elapsed = time.time() - start_time
        if elapsed >= MAX_DUR:
            print(f"\n📊 Reached MAX_DUR limit ({MAX_DUR}s). Stopping.")
            if ENABLE_HIGHLIGHT_LOGGING:
                _highlight_log(f"Reached MAX_DUR limit: {MAX_DUR}s", "INFO")
            break

        global_frame_idx += 1
        active_cam = camera_switcher.active_cam
        _highlight_stats["processing"]["camera_usage"][active_cam] = (
            _highlight_stats["processing"]["camera_usage"].get(active_cam, 0) + 1
        )

        # Read frame from active camera
        cap = caps_out.get(active_cam)
        if cap is None:
            error_msg = f"❌ Camera {active_cam} not found in caps_out"
            print(error_msg)
            if ENABLE_HIGHLIGHT_LOGGING:
                _highlight_log(error_msg, "ERROR")
            break

        try:
            ok, frame = cap.read()
        except Exception as e:
            errors += 1
            error_msg = f"Error reading frame from camera {active_cam}: {e}"
            if ENABLE_HIGHLIGHT_LOGGING and errors <= 5:
                _highlight_log(error_msg, "ERROR")
            if errors > 10:
                print(f"❌ Too many errors ({errors}). Stopping.")
                break
            continue

        if not ok:
            print(f"📹 End of video on camera {active_cam} ({CAMERA_NAMES[active_cam]}). Stopping.")
            if ENABLE_HIGHLIGHT_LOGGING:
                _highlight_log(f"End of video on camera {active_cam}", "INFO")
            break

       # PERFORMANCE OPTIMIZATION: Sync cameras less frequently (every 30 frames = ~1 second @ 30fps)
        # This reduces overhead while maintaining synchronization
        SYNC_INTERVAL = 30  # Sync every N frames instead of every frame
        if global_frame_idx % SYNC_INTERVAL == 0:
            try:
                active_frame_pos = _get_frame_pos(cap)
                # Sync all other cameras to the same frame position
                for cam_id, other_cap in caps_out.items():
                    if cam_id != active_cam and other_cap is not None:
                        other_pos = _get_frame_pos(other_cap)
                        # Only sync if there's a significant difference (avoid unnecessary seeks)
                        if abs(other_pos - active_frame_pos) > 1:
                            _hard_sync_cap(other_cap, active_frame_pos)
            except Exception as e:
                # Don't fail on sync errors, but log them
                if ENABLE_HIGHLIGHT_LOGGING and global_frame_idx % 300 == 0:  # Log every 10 seconds @ 30fps
                    _highlight_log(f"Warning: Could not sync cameras after frame read: {e}", "WARNING")

        # ---- Ball tracking ----
        try:
            det = sticky_tracker.update(frame, cam_id=active_cam)
        except Exception as e:
            errors += 1
            error_msg = f"Error in sticky_tracker.update: {e}"
            if ENABLE_HIGHLIGHT_LOGGING and errors <= 5:
                _highlight_log(error_msg, "ERROR")
            # Continue with empty detection
            try:
                det = BallDet(bbox=None, center=None, conf=0.0, cls=None, meta={"error": str(e)})
            except NameError:
                det = type('BallDet', (), {
                    'bbox': None, 'center': None, 'conf': 0.0,
                    'cls': None, 'meta': {"error": str(e)}
                })()

        # Track last frame when ball was found (for fallback scanning)
        ball_found = (det.bbox is not None and det.conf >= FALLBACK_SCAN_MIN_CONF)
        if ball_found:
            last_ball_found_frame = global_frame_idx
        elif 'last_ball_found_frame' not in locals():
            last_ball_found_frame = global_frame_idx  # Initialize on first frame

        # ---- Enhanced Fallback camera scanning (when ball lost for too long) ----
        # Priority: When middle camera loses ball, check side cameras first for better visibility
        frames_since_ball_found = global_frame_idx - last_ball_found_frame
        fallback_switch_occurred = False

        # IMPROVEMENT: Check if ball was near exit zone before fallback scanning
        # This prevents false switches when ball is in center field but temporarily occluded
        ball_was_near_zone = False
        if hasattr(camera_switcher, 'pos_hist') and len(camera_switcher.pos_hist) > 0:
            last_pos = camera_switcher.pos_hist[-1]
            if last_pos:
                x, y = last_pos
                zones = EXIT_ZONES.get(active_cam, {})
                for zone_name, (x1, y1, x2, y2) in zones.items():
                    # Check if ball was near any exit zone (with proximity threshold)
                    if (x1 - FALLBACK_ZONE_PROXIMITY_THRESHOLD <= x <= x2 + FALLBACK_ZONE_PROXIMITY_THRESHOLD and
                        y1 - FALLBACK_ZONE_PROXIMITY_THRESHOLD <= y <= y2 + FALLBACK_ZONE_PROXIMITY_THRESHOLD):
                        ball_was_near_zone = True
                        break

        # PERFORMANCE OPTIMIZATION: Disable fallback scanning for highlight generation
        # Fallback scanning is expensive (detects ball on all cameras) and less critical for highlights
        ENABLE_FALLBACK_FOR_HIGHLIGHT = False  # Set to True to enable fallback scanning in highlights

        if (ENABLE_FALLBACK_SCAN and ENABLE_FALLBACK_FOR_HIGHLIGHT and
            frames_since_ball_found >= FALLBACK_SCAN_TIMEOUT_FRAMES and
            not camera_switcher.is_cooldown_active() and
            ball_was_near_zone):  # Only scan if ball was near exit zone (prevents center-field false switches)

            # Identify camera types for priority-based scanning
            current_cam_name = CAMERA_NAMES.get(active_cam, "").upper()
            is_middle_cam = 'MIDDLE' in current_cam_name or 'CENTER' in current_cam_name

            # Find side cameras (Left and Right) for priority scanning
            side_cam_ids = []
            other_cam_ids = []

            for cid in CAMERA_MAP.keys():
                if cid == active_cam:
                    continue
                cam_name = CAMERA_NAMES.get(cid, "").upper()
                if 'LEFT' in cam_name or 'RIGHT' in cam_name:
                    side_cam_ids.append(cid)
                else:
                    other_cam_ids.append(cid)

            # Priority order: If coming from middle camera, scan side cameras first
            # Otherwise, scan all cameras equally
            if is_middle_cam and side_cam_ids:
                scan_order = side_cam_ids + other_cam_ids
                if ENABLE_ORCHESTRATOR_LOGGING:
                    _orch_log(f"FALLBACK_SCAN: Middle camera lost ball, prioritizing side cameras: {side_cam_ids}", "INFO")
            else:
                scan_order = side_cam_ids + other_cam_ids

            # If no specific order needed, use all other cameras
            if not scan_order:
                scan_order = [cid for cid in CAMERA_MAP.keys() if cid != active_cam]

            best_other_cam = None
            best_other_conf = 0.0
            camera_visibilities = {}  # Track visibility scores for all cameras

            for other_cam_id in scan_order:
                 other_cap = caps_out.get(other_cam_id)  # FIX: Use caps_out for highlight generation
                 if other_cap is None:
                    continue

                # Get current frame position of active camera
                 try:
                    active_frame_pos = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                    # Seek other camera to same relative position
                    other_cap.set(cv2.CAP_PROP_POS_FRAMES, active_frame_pos)
                    ok, other_frame = other_cap.read()

                    if ok and other_frame is not None:
                        # Detect ball in other camera
                        try:
                            other_det = detect_ball(other_frame)
                            if other_det.bbox is not None and other_det.conf >= FALLBACK_SCAN_MIN_CONF:
                                # Track visibility for all cameras
                                camera_visibilities[other_cam_id] = other_det.conf

                                # Update best camera based on confidence
                                # If coming from middle camera, prefer side cameras with same confidence
                                is_side_cam = other_cam_id in side_cam_ids
                                current_best_is_side = best_other_cam in side_cam_ids if best_other_cam is not None else False

                                should_update = False
                                if best_other_cam is None:
                                    should_update = True
                                elif is_middle_cam:
                                    # Priority: Side cameras preferred when coming from middle
                                    if is_side_cam and not current_best_is_side:
                                        should_update = True  # Prefer side camera
                                    elif (is_side_cam == current_best_is_side) and other_det.conf > best_other_conf:
                                        should_update = True  # Same type, higher confidence
                                    elif not is_side_cam and current_best_is_side:
                                        should_update = False  # Don't replace side with non-side
                                    elif not is_side_cam and other_det.conf > best_other_conf + 0.1:
                                        should_update = True  # Non-side only if significantly better
                                else:
                                    # Normal priority: highest confidence wins
                                    if other_det.conf > best_other_conf:
                                        should_update = True

                                if should_update:
                                    best_other_cam = other_cam_id
                                    best_other_conf = other_det.conf
                        except Exception as e:
                            if ENABLE_ORCHESTRATOR_LOGGING:
                                _orch_log(f"Error detecting ball in camera {other_cam_id} during fallback scan: {e}", "WARNING")
                 except Exception as e:
                    if ENABLE_ORCHESTRATOR_LOGGING:
                        _orch_log(f"Error reading from camera {other_cam_id} during fallback scan: {e}", "WARNING")

            # Switch to camera with best ball visibility
            # Prefers side cameras when coming from middle camera
            if best_other_cam is not None:
                old_cam = active_cam
                active_cam = best_other_cam

                # CRITICAL FIX: Update camera_switcher's internal state to match
                # Otherwise, the switcher will reset active_cam on the next frame
                try:
                    camera_switcher.update_active_camera(best_other_cam, global_frame_idx)
                except Exception as e:
                    if ENABLE_HIGHLIGHT_LOGGING:
                        _highlight_log(f"Warning: Error updating camera_switcher state: {e}", "WARNING")

                _orch_stats["phase1"]["switches"] += 1
                _highlight_stats["processing"]["switches"] += 1

                switch_event = {
                    "frame": global_frame_idx,
                    "from_cam": old_cam,
                    "to_cam": active_cam,
                    "zone": "FALLBACK_SCAN",
                    "exit_prob": best_other_conf,
                    "reason": f"ball_lost_{frames_since_ball_found}_frames_found_in_other_cam"
                }
                _orch_stats["phase1"]["switch_events"].append(switch_event)
                _highlight_stats["processing"]["switch_events"].append(switch_event)

                # Log visibility scores if multiple cameras detected ball
                visibility_info = f"conf={best_other_conf:.2f}"
                if len(camera_visibilities) > 1:
                    vis_scores = ", ".join([f"{CAMERA_NAMES.get(cid, cid)}={conf:.2f}"
                                            for cid, conf in sorted(camera_visibilities.items(), key=lambda x: x[1], reverse=True)])
                    visibility_info += f" (all: {vis_scores})"

                print(f"\n🔄 FALLBACK SWITCH at frame={global_frame_idx:06d}: "
                      f"{CAMERA_NAMES[old_cam]} -> {CAMERA_NAMES[active_cam]} "
                      f"(ball lost for {frames_since_ball_found} frames, {visibility_info})")

                if ENABLE_ORCHESTRATOR_LOGGING:
                    _orch_log(f"FALLBACK_SWITCH: frame={global_frame_idx}, {old_cam}->{active_cam}, "
                             f"lost_for={frames_since_ball_found}frames, conf={best_other_conf:.2f}, visibilities={camera_visibilities}", "INFO")
                if ENABLE_HIGHLIGHT_LOGGING:
                    _highlight_log(f"FALLBACK_SWITCH: frame={global_frame_idx}, {old_cam}->{active_cam}, "
                                 f"lost_for={frames_since_ball_found}frames, conf={best_other_conf:.2f}", "INFO")

                # Reset sticky tracker and update last_ball_found_frame
                # Maintain sticky tracking behavior during transitions
                try:
                    sticky_tracker.reset()
                except Exception as e:
                    if ENABLE_ORCHESTRATOR_LOGGING:
                        _orch_log(f"Warning: Error resetting sticky tracker: {e}", "WARNING")

                last_ball_found_frame = global_frame_idx
                fallback_switch_occurred = True

                # Initialize camera usage for new camera
                if active_cam not in _orch_stats["phase1"]["camera_usage"]:
                    _orch_stats["phase1"]["camera_usage"][active_cam] = 0
                if active_cam not in _highlight_stats["processing"]["camera_usage"]:
                    _highlight_stats["processing"]["camera_usage"][active_cam] = 0

                # Skip normal switching decision for this frame (already switched)
                continue

        # ---- Camera switching decision (only if no fallback switch occurred) ----

        try:
            decision = camera_switcher.update(
                cam_id=active_cam,
                det=det,
                frame_shape=frame.shape[:2],
                frame_idx=global_frame_idx
            )
        except Exception as e:
            errors += 1
            error_msg = f"Error in camera_switcher.update: {e}"
            if ENABLE_HIGHLIGHT_LOGGING and errors <= 5:
                _highlight_log(error_msg, "ERROR")
            # Continue with stay decision
            try:
                decision = SwitchDecision("STAY", active_cam, active_cam, f"error: {str(e)}",
                                         "NONE", 0.0, 0, 0)
            except NameError:
                decision = type('SwitchDecision', (), {
                    'action': "STAY", 'from_cam': active_cam, 'to_cam': active_cam,
                    'reason': f"error: {str(e)}", 'zone': "NONE", 'exit_prob': 0.0,
                    'miss_count': 0, 'cooldown_left': 0
                })()

        # ---- Apply switch if needed ----
        if decision.action == "SWITCH":
            old_cam = active_cam
            new_cam = decision.to_cam

            # CRITICAL FIX: Synchronize target camera to current frame position before switching
            # This ensures cameras stay in sync during highlight generation
            try:
                current_cap = caps_out.get(old_cam)
                if current_cap is not None:
                    current_frame_pos = _get_frame_pos(current_cap)
                    target_cap = caps_out.get(new_cam)
                    if target_cap is not None:
                        _hard_sync_cap(target_cap, current_frame_pos)
                        if ENABLE_HIGHLIGHT_LOGGING:
                            _highlight_log(f"Synced camera {new_cam} to frame {current_frame_pos} before switch", "INFO")
            except Exception as e:
                if ENABLE_HIGHLIGHT_LOGGING:
                    _highlight_log(f"Warning: Could not sync camera {new_cam} before switch: {e}", "WARNING")

            # CRITICAL: Update active_cam to match switcher's decision
            # The switcher already updated its internal state in update_active_camera()
            active_cam = new_cam
            _highlight_stats["processing"]["switches"] += 1

            switch_event = {
                "frame": global_frame_idx,
                "time_sec": elapsed,
                "from_cam": old_cam,
                "to_cam": new_cam,
                "zone": decision.zone,
                "exit_prob": decision.exit_prob,
                "reason": decision.reason
            }
            _highlight_stats["processing"]["switch_events"].append(switch_event)

            print(f"\n🔄 SWITCH at t={elapsed:6.1f}s frame={global_frame_idx:06d}: "
                  f"{CAMERA_NAMES[old_cam]} -> {CAMERA_NAMES[new_cam]} "
                  f"(zone={decision.zone}, prob={decision.exit_prob:.2f})")

            if ENABLE_HIGHLIGHT_LOGGING:
                _highlight_log(f"SWITCH: t={elapsed:.1f}s, frame={global_frame_idx}, "
                             f"{old_cam}->{new_cam}, zone={decision.zone}, prob={decision.exit_prob:.2f}", "INFO")

            try:
                sticky_tracker.reset()
            except Exception as e:
                if ENABLE_HIGHLIGHT_LOGGING:
                    _highlight_log(f"Warning: Error resetting sticky tracker: {e}", "WARNING")

            # Initialize camera usage for new camera
            if new_cam not in _highlight_stats["processing"]["camera_usage"]:
                _highlight_stats["processing"]["camera_usage"][new_cam] = 0

            # Continue loop; next frame will come from new camera
            continue

        # ---- Draw overlays ----
        try:
            # Pass position history from camera switcher for trajectory visualization
            pos_history = camera_switcher.pos_hist if hasattr(camera_switcher, 'pos_hist') else None
            vis = draw_ball_debug(frame, det, pos_history=pos_history)
        except Exception as e:
            errors += 1
            if ENABLE_HIGHLIGHT_LOGGING and errors <= 5:
                _highlight_log(f"Error in draw_ball_debug: {e}", "ERROR")
            vis = frame.copy()

        # ---- Draw exit zones for visualization ----
        if ENABLE_ZONE_VISUALIZATION:
           try:
            import numpy as np
            h, w = vis.shape[:2]
            zones = EXIT_ZONES.get(active_cam, {})
            if zones:
                # Create overlay for exit zones
                overlay = vis.copy()
                zone_colors = {
                    "LEFT": (0, 0, 255), "LEFT_TOP": (0, 100, 255), "LEFT_BOTTOM": (0, 150, 255),
                    "RIGHT": (255, 0, 0), "RIGHT_TOP": (255, 100, 0), "RIGHT_BOTTOM": (255, 150, 0),
                    "TOP": (255, 255, 0), "BOTTOM": (255, 0, 255),
                }
                for zone_name, (x_min, y_min, x_max, y_max) in zones.items():
                    x1, y1 = int(x_min * w), int(y_min * h)
                    x2, y2 = int(x_max * w), int(y_max * h)
                    color = zone_colors.get(zone_name, (128, 128, 128))
                    cv2.rectangle(overlay, (x1, y1), (x2, y2), color, -1)
                    cv2.rectangle(overlay, (x1, y1), (x2, y2), color, 2)
                    label = zone_name
                    label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                    label_x, label_y = x1 + 5, y1 + label_size[1] + 5
                    if label_y < h and label_x + label_size[0] < w:
                        cv2.rectangle(overlay, (label_x - 2, label_y - label_size[1] - 2),
                                     (label_x + label_size[0] + 2, label_y + 2), (0, 0, 0), -1)
                        cv2.putText(overlay, label, (label_x, label_y),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                cv2.addWeighted(overlay, 0.25, vis, 0.75, 0, vis)
           except Exception as e:
            if ENABLE_HIGHLIGHT_LOGGING and errors <= 5:
                _highlight_log(f"Error drawing exit zones: {e}", "WARNING")

        # Enhanced overlay with better formatting
        overlay_y = 30
        overlay_texts = [
            f"Frame: {global_frame_idx} | Camera: {CAMERA_NAMES[active_cam]}",
            f"Time: {elapsed:5.1f}s / {MAX_DUR:.0f}s | Skip: {SKIP_SECONDS/60:.1f}m"
        ]

        for i, text in enumerate(overlay_texts):
            # Background for text readability
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            cv2.rectangle(vis, (8, overlay_y - text_size[1] - 4),
                         (12 + text_size[0], overlay_y + 4), (0, 0, 0), -1)
            cv2.putText(vis, text, (10, overlay_y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            overlay_y += 35

        # Show switch indicator if just switched
        if decision.action == "SWITCH" or (global_frame_idx < 10 and _highlight_stats["processing"]["switches"] > 0):
            cv2.putText(vis, "[SWITCHING]", (10, overlay_y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        # ---- Initialize writer lazily ----
        if writer is None:
            h, w = vis.shape[:2]
            out_fps = fps_map_out.get(active_cam, OUTPUT_FPS_FALLBACK)
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            writer = cv2.VideoWriter(str(output_path), fourcc, out_fps, (w, h))
            if not writer.isOpened():
                raise RuntimeError(f"Failed to initialize video writer for {output_path}")
            print(f"   ✅ Video writer initialized: {w}x{h} @ {out_fps:.2f}fps")
            _highlight_stats["processing"]["output_fps"] = out_fps
            if ENABLE_HIGHLIGHT_LOGGING:
                _highlight_log(f"Video writer initialized: {w}x{h} @ {out_fps:.2f}fps", "INFO")

        # ---- Write frame ----
        try:
            writer.write(vis)
            written_frames += 1
        except Exception as e:
            errors += 1
            if ENABLE_HIGHLIGHT_LOGGING and errors <= 5:
                _highlight_log(f"Error writing frame: {e}", "ERROR")

        # ---- Progress logging ----
        current_time = time.time()
        if current_time - last_progress_log >= PROGRESS_LOG_EVERY_N_SEC:
            progress_pct = (elapsed / MAX_DUR * 100) if MAX_DUR > 0 else 0
            progress_info = (
                f"⏱️  {elapsed:6.1f}s / {MAX_DUR:.0f}s ({progress_pct:5.1f}%) | "
                f"frames={global_frame_idx} | written={written_frames} | "
                f"cam={active_cam} ({CAMERA_NAMES[active_cam]}) | "
                f"switches={_highlight_stats['processing']['switches']}"
            )
            print(progress_info)

            if ENABLE_HIGHLIGHT_LOGGING:
                _highlight_log(f"Progress: {elapsed:.1f}s/{MAX_DUR:.0f}s, {global_frame_idx} frames, "
                             f"{written_frames} written, {_highlight_stats['processing']['switches']} switches", "INFO")

            last_progress_log = current_time

except KeyboardInterrupt:
    print(f"\n⚠️  Processing interrupted by user at frame {global_frame_idx}")
    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log(f"Processing interrupted at frame {global_frame_idx}", "WARNING")
except Exception as e:
    errors += 1
    error_msg = f"❌ Error during highlight generation: {e}"
    print(f"\n{error_msg}")
    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log(error_msg, "ERROR")
    raise
finally:
    processing_time = time.time() - start_time
    _highlight_stats["processing"]["written_frames"] = written_frames
    _highlight_stats["processing"]["errors"] = errors
    _highlight_stats["processing"]["processing_time_sec"] = processing_time
    _highlight_stats["processing"]["end_time"] = datetime.now().isoformat()

# ------------------------------
# CLEANUP
# ------------------------------
print(f"\n🧹 Cleaning up resources...")

for cam_id, cap in caps_out.items():
    try:
        cap.release()
    except Exception as e:
        print(f"⚠️  Warning: Error releasing camera {cam_id}: {e}")

if writer:
    try:
        writer.release()
        print(f"   ✅ Video writer closed")
    except Exception as e:
        print(f"⚠️  Warning: Error closing video writer: {e}")

if _highlight_log_file is not None:
    try:
        _highlight_log(f"Highlight generation complete: {written_frames} frames in {processing_time:.1f}s", "INFO")
        _highlight_log(f"Results: switches={_highlight_stats['processing']['switches']}, errors={errors}", "INFO")
        _highlight_log_file.close()
        if _highlight_log_file_path:
            print(f"📝 Log saved: {_highlight_log_file_path}")
    except Exception:
        pass

# ------------------------------
# FINAL STATISTICS & SUMMARY
# ------------------------------
print("\n" + "=" * 50)
print("✅ HIGHLIGHT GENERATION COMPLETE")
print("=" * 50)

print(f"\n📈 Processing Summary:")
print(f"   Frames written: {written_frames}")
print(f"   Processing time: {processing_time:.1f} seconds ({processing_time/60:.1f} minutes)")
if processing_time > 0:
    output_fps = written_frames / processing_time
    print(f"   Output speed: {output_fps:.1f} fps")
print(f"   Camera switches: {_highlight_stats['processing']['switches']}")
if errors > 0:
    print(f"   Errors: {errors}")

print(f"\n📹 Camera Usage:")
for cam_id, usage in sorted(_highlight_stats["processing"].get("camera_usage", {}).items()):
    pct = (usage / written_frames * 100) if written_frames > 0 else 0
    print(f"   Camera {cam_id} ({CAMERA_NAMES.get(cam_id, 'Unknown')}): {usage} frames ({pct:.1f}%)")

# ==============================
# IMPROVEMENT: Camera Dominance Warnings
# ==============================
CAMERA_DOMINANCE_THRESHOLD = 90.0  # Warn if any camera > 90% usage
if written_frames > 0:
    for cam_id, usage in _highlight_stats["processing"].get("camera_usage", {}).items():
        pct = (usage / written_frames * 100)
        if pct >= CAMERA_DOMINANCE_THRESHOLD:
            print(f"\n⚠️  WARNING: Camera {cam_id} ({CAMERA_NAMES.get(cam_id, 'Unknown')}) dominates output: {pct:.1f}%")
            print(f"   This may indicate switching logic issues or camera availability problems.")
            print(f"   Consider checking:")
            print(f"   - Switching thresholds (may be too conservative)")
            print(f"   - Camera synchronization (other cameras may be out of sync)")
            print(f"   - Ball detection quality (may be failing on other cameras)")

if _highlight_stats["processing"].get("switch_events"):
    print(f"\n🔄 Switch Events (last 5):")
    for event in _highlight_stats["processing"]["switch_events"][-5:]:
        print(f"   {event.get('time_sec', 0):6.1f}s (frame {event.get('frame', 0)}): "
              f"{event.get('from_cam', '?')} -> {event.get('to_cam', '?')} "
              f"(zone={event.get('zone', 'NONE')}, prob={event.get('exit_prob', 0.0):.2f})")

# Get final statistics from other components
try:
    _highlight_stats["detection_stats"] = get_detection_stats()
    _highlight_stats["sticky_stats"] = get_sticky_stats()
    _highlight_stats["switcher_stats"] = get_switcher_stats()
except Exception as e:
    if ENABLE_HIGHLIGHT_LOGGING:
        _highlight_log(f"Warning: Could not get component stats: {e}", "WARNING")

# Save statistics
if SAVE_HIGHLIGHT_STATS:
    try:
        stats_file = DEBUG_DIR / f"highlight_stats_{timestamp}.json"
        with open(stats_file, 'w', encoding='utf-8') as f:
            json.dump(_highlight_stats, f, indent=2, default=str)
        print(f"\n📊 Statistics saved: {stats_file}")
    except Exception as e:
        print(f"⚠️  Warning: Could not save statistics: {e}")

print(f"\n📁 Output:")
print(f"   Video: {output_path}")
if ENABLE_HIGHLIGHT_LOGGING and _highlight_log_file_path:
    print(f"   Log file: {_highlight_log_file_path}")

# Check output file size
try:
    if output_path.exists():
        file_size_mb = output_path.stat().st_size / (1024 * 1024)
        print(f"   File size: {file_size_mb:.2f} MB")
except Exception as e:
    print(f"   ⚠️  Could not check file size: {e}")

if ENABLE_HIGHLIGHT_LOGGING:
    _highlight_log(f"Highlight generation complete: {written_frames} frames in {processing_time:.1f}s", "INFO")
    _highlight_log(f"Results: switches={_highlight_stats['processing']['switches']}, errors={errors}", "INFO")

print("\n" + "=" * 50)

FINAL HIGHLIGHT OUTPUT GENERATION

📹 Output Configuration:
   Output path: /content/drive/MyDrive/football/final/output/Highlight_20260115_190345.mp4
   Skip seconds: 0 (0.0 minutes)
   Max duration: 1000 seconds (16.7 minutes)

📝 Highlight logging enabled: /content/drive/MyDrive/football/final/debug/highlight_output_20260115_190345.log

📥 Opening camera streams for output...
   ✅ Camera 0 (RIGHT_CAM)
      File: /content/drive/MyDrive/football/final/input/IMG_2789_synced.mp4
      FPS: 30.00, Frames: 102273, Size: 1920x1080, Dur: 3409.4s
      Skip: 0s (0 frames)
   ✅ Camera 1 (LEFT_CAM)
      File: /content/drive/MyDrive/football/final/input/IMG_0689 2_synced.mp4
      FPS: 30.00, Frames: 102260, Size: 1920x1080, Dur: 3409.0s
      Skip: 0s (0 frames)

⏩ Seeking to skip point (0s)...
   ✅ Skipped first 0 seconds (~0.0 min) for all cameras

🔍 Verifying camera synchronization after skip...
   ✅ All cameras appear synchronized after skip.

🔄 Resetting trackers and statistics...
   ✅ Tra